In [ ]:
import os
import csv

SIG = "data/train/signatures"

with open("signatures.tsv", "w", encoding="utf-8", newline="") as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')

    for dir in os.listdir(SIG):
        for file in os.listdir(os.path.join(SIG, dir)):
            if not file.endswith(".txt"):
                continue

            filename = file[:-4]
            if not os.path.exists(os.path.join(SIG, dir, f"{filename}.jpg")):
                continue

            # If jpg exists, read txt
            with open(os.path.join(SIG, dir, file), "r", encoding="utf-8") as txtfile:
                label = txtfile.read()
                writer.writerow([f"{SIG[5:]}/{dir}/{filename}.jpg", label])

In [3]:
def is_chinese(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False


def is_number(uchar):
    """判断一个unicode是否是半角数字"""
    if uchar >= u'\u0030' and uchar <= u'\u0039':
        return True
    else:
        return False


def is_Qnumber(uchar):
    """判断一个unicode是否是全角数字"""
    if uchar >= u'\uff10' and uchar <= u'\uff19':
        return True
    else:
        return False


def is_alphabet(uchar):
    """判断一个unicode是否是半角英文字母"""
    if (uchar >= u'\u0041' and uchar <= u'\u005a') or (uchar >= u'\u0061'
                                                       and uchar <= u'\u007a'):
        return True
    else:
        return False


def is_Qalphabet(uchar):
    """判断一个unicode是否是全角英文字母"""
    if (uchar >= u'\uff21' and uchar <= u'\uff3a') or (uchar >= u'\uff41'
                                                       and uchar <= u'\uff5a'):
        return True
    else:
        return False


def is_other(uchar):
    """判断是否非汉字，数字和英文字符"""
    if not (is_chinese(uchar) or is_number(uchar) or is_alphabet(uchar)):
        return True
    else:
        return False


def B2Q(uchar):
    """单个字符 半角转全角"""
    inside_code = ord(uchar)
    if inside_code < 0x0020 or inside_code > 0x7e:  # 不是半角字符就返回原来的字符
        return uchar
    if inside_code == 0x0020:  # 除了空格其他的全角半角的公式为: 半角 = 全角 - 0xfee0
        inside_code = 0x3000
    else:
        inside_code += 0xfee0
    return chr(inside_code)


def Q2B(uchar):
    """单个字符 全角转半角"""
    inside_code = ord(uchar)
    if inside_code == 0x3000:
        inside_code = 0x0020
    else:
        inside_code -= 0xfee0
    if inside_code < 0x0020 or inside_code > 0x7e:  # 转完之后不是半角字符返回原来的字符
        return uchar
    return chr(inside_code)


def stringQ2B(ustring):
    """把字符串全角转半角"""
    return "".join([Q2B(uchar) for uchar in ustring])


def stringpartQ2B(ustring):
    """把字符串中数字和字母全角转半角"""
    ustring = ustring.replace("％", "%").replace(
        "／", "/").replace("－", "-").replace("．", ".").replace("＂", "“")
    return "".join([
        Q2B(uchar) if is_Qnumber(uchar) or is_Qalphabet(uchar) else uchar
        for uchar in ustring
    ])

In [4]:
def punc_to_chinese(string):
    return string.replace(",", "，").replace(";", "；").replace(":", "：").replace("!", "！").replace("?", "？").replace(")", "）").replace("(", "（")


def pubc_to_english(string):
    return string.replace("，", ",").replace("；", ";").replace("：", ":").replace("！", "!").replace("？", "?").replace("）", ")").replace("（", "(")

In [5]:
import os

opts = {"punc": False, "q2b": False}
file_opts = {}

for root, dirs, files in os.walk("labels-src"):
    for file in files:
        if not file.endswith(".tsv"):
            continue

        file_opts[file] = opts

print(file_opts)

{'digit_95k.tsv': {'punc': False, 'q2b': False}, 'hand_line_all_548k.tsv': {'punc': False, 'q2b': False}, 'tang_syn_1577k.tsv': {'punc': False, 'q2b': False}, 'web_line_238k.tsv': {'punc': False, 'q2b': False}, 'hw_chinese_240k.tsv': {'punc': False, 'q2b': False}, 'hwdb_ic13_47k.tsv': {'punc': False, 'q2b': False}, 'hwdb2.1_13k.tsv': {'punc': False, 'q2b': False}, 'hwdb2.2_12k.tsv': {'punc': False, 'q2b': False}, 'hwdb2.0_16k.tsv': {'punc': False, 'q2b': False}, 'signatures_472k.tsv': {'punc': False, 'q2b': False}, 'hwdb2.0_4k.tsv': {'punc': False, 'q2b': False}, 'hwdb_ic13_3k.tsv': {'punc': False, 'q2b': False}, 'hw_chinese_17k.tsv': {'punc': False, 'q2b': False}, 'hwdb2.2_3k.tsv': {'punc': False, 'q2b': False}, 'hwdb2.1_3k.tsv': {'punc': False, 'q2b': False}, 'hwdb_ic13_val_5k.tsv': {'punc': False, 'q2b': False}}


In [6]:
import os

file_opts = {
    'labels-src/train/digit_95k.tsv': {
        'punc': False,
        'q2b': False
    },
    'labels-src/train/hand_line_all_548k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/train/tang_syn_1577k.tsv': {
        'punc': False,
        'q2b': False
    },
    'labels-src/train/web_line_238k.tsv': {
        'punc': False,
        'q2b': False
    },
    'labels-src/train/hw_chinese_240k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/train/hwdb_ic13_47k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/train/hwdb2.1_13k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/train/hwdb2.2_12k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/train/hwdb2.0_16k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/test/hwdb2.0_4k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/test/hwdb_ic13_3k.tsv': {
        'punc': True,
        'q2b': True
    },
    'labels-src/test/hw_chinese_17k.tsv': {
        'punc': False,
        'q2b': False
    },
    'labels-src/test/hwdb2.2_3k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/test/hwdb2.1_3k.tsv': {
        'punc': True,
        'q2b': False
    },
    'labels-src/test/hwdb_ic13_val_5k.tsv': {
        'punc': True,
        'q2b': False
    },
    "labels-src/train/signatures_472k.tsv": {
        "punc": False,
        "q2b": False
    }
}

OUTPUT_DIR = "clean_labels"

for root, dirs, files in os.walk("labels-src"):
    for file in files:
        if not file.endswith(".tsv"):
            continue

        output_dir = os.path.join(OUTPUT_DIR, root)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        opts = file_opts[f"{root}/{file}"]

        with open(os.path.join(root, file), "r", encoding="utf-8") as input:
            with open(os.path.join(output_dir, file), "w", encoding="utf-8") as output:
                text = input.read()
                if opts["q2b"]:
                    text = stringpartQ2B(text)
                if opts["punc"]:
                    text = punc_to_chinese(text)

                output.write(text)

## Zhwiki

In [23]:
import re
import os
import shutil
from os import path
import json
import csv
from tqdm import tqdm


def keep_chinese_english_punctuations(text):
    # Regular expression pattern
    pattern = re.compile(
        r'[^'  # Start of the negated character class
        '\u2000-\u206F'  # General Punctuation
        '\u3000-\u30ff'  # CJK Symbols and Punctuation, Hiragana, Katakana, Bopomofo, Hangul Compatibility Jamo, Kanbun, Halfwidth Katakana
        '\u3400-\u4dbf'  # CJK Unified Ideographs Extension A
        '\u4e00-\u9fff'  # CJK Unified Ideographs
        '\uFF00-\uFFEF'  # Full-width punctuation, Full-width letters and numbers
        '\uf900-\ufaff'  # CJK Compatibility Ideographs
        '\uff01-\uff9f'  # Halfwidth Katakana and Hangul, Full-width punctuation, Full-width letters and numbers
        '\U00020000-\U0002A6DF'  # CJK Unified Ideographs Extension B
        'a-zA-Z'  # English characters
        '\u0020-\u007F'  # Basic Latin characters and common English punctuation
        '\u0080-\u00ff'  # Latin-1 supplements, including signs like ¥
        '\n\r'  # New lines and carriage returns
        ']'  # End of the negated character class
    )
    # Substituting characters not matching the pattern with empty string
    new_text = re.sub(pattern, '', text)
    return new_text


text_labels_path = "./zhwiki/zhwiki-braces-opencc"

OUTPUT_DIR = path.join("zhwiki", "labels")
INDEX_FILE = path.join("zhwiki", "label_index.tsv")

if path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

if path.exists(INDEX_FILE):
    os.remove(INDEX_FILE)

with open(INDEX_FILE, "w", encoding="utf-8") as index_file:
    writer = csv.writer(index_file, delimiter='\t', lineterminator='\n')

    for root, _dirs, files in os.walk(text_labels_path):
        print(root)
        for file in files:  # Loop through the dataset tsvfiles
            if not file.startswith("wiki"):
                continue

            lines = []  # lines for a single wiki file

            file_path = path.join(root, file)
            output_dir = path.join(OUTPUT_DIR, path.basename(root))
            if not path.exists(output_dir):
                os.makedirs(output_dir)

            output_path = path.join(output_dir, f"{file}.txt")
            relative_output_path = path.join(
                path.basename(OUTPUT_DIR), path.basename(root), f"{file}.txt")
            with open(file_path, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    doc = json.loads(line)
                    text = doc["text"]

                    text = keep_chinese_english_punctuations(text)

                    for text_line in text.split("\n"):
                        text_line = text_line.strip()
                        if not text_line:
                            continue

                        text_line = re.sub(r"(《》|（）|\}\}|\{\{)", "", text_line)
                        text_line = re.sub(
                            r"&lt;.*&gt;(.*?)&lt;.*&gt;", r"\1", text_line)
                        text_line = re.sub(r"(.{1,10})\.", r"\1", text_line)

                        if not text_line:
                            continue
                        lines.append(text_line)

            if lines:
                with open(output_path, "w", encoding="utf-8") as f:
                    f.write("\n".join(lines))

                writer.writerow([relative_output_path, len(lines)])

./zhwiki/zhwiki-braces-opencc
./zhwiki/zhwiki-braces-opencc/AA
./zhwiki/zhwiki-braces-opencc/AB
./zhwiki/zhwiki-braces-opencc/AC
./zhwiki/zhwiki-braces-opencc/AD
./zhwiki/zhwiki-braces-opencc/AE
./zhwiki/zhwiki-braces-opencc/AF
./zhwiki/zhwiki-braces-opencc/AG
./zhwiki/zhwiki-braces-opencc/AH
./zhwiki/zhwiki-braces-opencc/AI
./zhwiki/zhwiki-braces-opencc/AJ
./zhwiki/zhwiki-braces-opencc/AK
./zhwiki/zhwiki-braces-opencc/AL
./zhwiki/zhwiki-braces-opencc/AM
./zhwiki/zhwiki-braces-opencc/AN
./zhwiki/zhwiki-braces-opencc/AO
./zhwiki/zhwiki-braces-opencc/AP
./zhwiki/zhwiki-braces-opencc/AQ
./zhwiki/zhwiki-braces-opencc/AR
./zhwiki/zhwiki-braces-opencc/AS
./zhwiki/zhwiki-braces-opencc/AT
./zhwiki/zhwiki-braces-opencc/AU
./zhwiki/zhwiki-braces-opencc/AV
./zhwiki/zhwiki-braces-opencc/AW
./zhwiki/zhwiki-braces-opencc/AX
./zhwiki/zhwiki-braces-opencc/AY
./zhwiki/zhwiki-braces-opencc/AZ
./zhwiki/zhwiki-braces-opencc/BA
./zhwiki/zhwiki-braces-opencc/BB
./zhwiki/zhwiki-braces-opencc/BC
./zhwiki/zhwi

In [22]:
import re

wikitext = """ASMPT（、，前称ASM太平洋）是一家在香港交易所上市的工业公司。主要业务是设计、制造及销售半导体工业所用之器材、工具及物料。ASM太平洋在开曼群岛注册。
2010年，ASM太平洋收购西门子旗下的Siemens Electronics Assembly Systems（简称：SEAS），拓展电子装嵌系统市场。收购价为1欧元，ASM并承诺会向SEAS增资202亿元，并在3年内再提供202亿元贷款融资。
2022年8月，ASM太平洋（ASM Pacific Technology）为避免与ASM国际（ASM International）混淆而改称ASMPT。
国浩集团有限公司是一家在香港交易所上市的金融公司。主要业务是物业发展及投资、证券及期货经纪业务、保险及基金管理服务，以及投资及财资管理。公司在百慕达注册，主席为郭令灿。
从道亨集团换取上市首日在1991年1月22日。
2011年5月丰隆金融集团持有644%股权的丰隆银行完成以50亿6000万令吉全现金收购国贸银行（EON Bank）
跃居马来西亚第四大银行集团
2012年5月13日，旗下公司Rank以现金205英镑（约256亿港元），收购英国第三大赌场营运商Gala Casinos全部已发行股本
在伦敦经营4间赌场，以及于英国各省地区经营19间赌场。
于完成交易后，Rank将成为英国最大赌场营运商，拥有58间赌场。
2013年12月1日，国浩集团已获大股东GuoLine Overseas Limited（GOL）知会，已于11月29日通过证券经纪售7291万股，相当于已发行股本约222%，买家为独立第三方。出售上述股份后GOL持股降至7188%，故GOL及关连人士之总持股恢复公众持股量要求。
合和实业有限公司（Hopewell Holdings Limited，港交所除牌前：0054 HK），主要业务是基建项目投资，物业租赁、代理及管理，酒店营运及管理，餐馆营运及食品经营，建筑及项目管理。
公司在香港注册。虽然公司是上市公司，但同时亦是家族式企业：创办人、董事局主席为胡应湘，胡应湘夫人郭秀萍担任其中一位非运行董事，而其中一名独立非运行董事胡文佳则是胡应湘的侄儿。1972年8月21日，合和实业正式上市。
由2018年2月，胡文佳被重新归类为非运行董事；董事会共有15人，其中5人为独立非运行董事。
截至2018年3月31日，市值为港币26052 亿元。
合和实业有限公司的总部位于湾仔的合和中心，合和中心隔邻是胡应湘为纪念父亲胡忠而建造的胡忠大厦。
2019年3月21日合和实业（054）大股东胡应湘联同亲友提出以每股388元私有化，在法院会议上获963%独立股东投票赞成，在股东特别大会获979%股东投票赞成，符合75%出席股东会并投票的独立股东赞成以及在全部独立股东中少于10%反对的私有化要求，将于同年5月3日撤回上市地位。
合和实业（00054）公布，该股于港交所买卖最后时间为同年4月17日下午4时，因股份不再纳入收市竞价交易时段的上市证券。
历史
合和实业的创办，可追溯到胡氏家族的中央建业有限公司。创办人胡应湘，祖籍广东花县，1935年在香港出生，其父胡忠是香港著名的的士商，胡应湘早年赴美国普林斯顿大学攻读土木工程，1958年学成返港，曾任职政府工务局助理工程师。1962年胡获得建筑师执照后，即创立家族的中央建业有限公司。当时，港府为香港的士不足，因而增发牌照，其父胡忠多购了数十部的士，使车库不敷应用，于是便在铜锣湾海旁购地建造大厦，底层作停车场，上盖作住宅出售。这次交易令胡氏家族赚取了可观利润，并开始兼营地产。
1965年香港爆发银行危机，地产市道萧条，中央建业业务大受影响，胡应湘遂决定另组公司自行发展。1972年，胡在父亲协助下向汇丰银行贷款1,500万元，创办合和实业有限公司。同年8月，合和在香港挂牌上市。当时，合和实收资本2亿元，分为10,000股，每股面价2元，以每股5元公开发售2,500万股新股，集资125亿元，用作减轻债务及地产发展。当时，香港地产市道蓬勃，股市飙升，合和的股价也于1973年3月冲上每股30元水平，比认股价急升5倍，整间公司市值高达36亿元，成为规模最大的华资地产上市公司。上市首年，合和纯利达6,070万元，比预测利润高出四成半。
可惜，好景不常。1973年3月恒生指数攀升至177496的历史性高位后即急转直下，当时股市早已脱离客观经济情况，潜伏严重危机，股市暴跌的导火线是发现假股票，而最早发现的假股票就是合和。合和即被证券交易所勒令停牌以便调查。期间，合和持有25％股权的保利建筑公司宣布破产清盘，令合和遭受损失，故合和复牌后股价一泻不止。1974年底，合和股价跌至065元，比最高位时下跌97％。遭此打击，1974年度合和纯利仅2,179万元，大幅下跌64%，以后各年度虽逐步回升，但直至1979年才超越1973年度水平。这一时期，合和侧重于地产发展，相继完成了协威园、康丽园、东威大厦、山光苑、云景台、荃威花园、健威花园、德福花园等多处物业。
1970年代合和最重大的物业发展项目是位于湾仔皇后大道东的合和中心。合和中心既是身为建筑师的胡应湘引以为傲的得意之作，也是合和实业的招牌大厦。合和中心在设计上颇具特色，它包括两个内筒墙和一个直径15呎的外筒墙，第一个内筒墙内是电梯信道，内筒墙之间作洗手间、储物室及管理信道之用，第二个内筒墙和外筒墙之间，是写字楼和商场。合和中心的图筒形设计，据说可抵受时速194哩的负载风速，这一极限承载的风力，是香港有史以来所能遇到的最大风暴。胡应湘设计的合和中心原计划于1976年完成，后因建筑图则批出的拖延，至1980年才完工。该中心楼高66层，是当时香港最高的建筑物，比康乐大厦还高出139呎，可供出租面积达80万平方呎，顶层是旋转餐厅，中低层是写字楼，底层和地库是商场和停车场。合和中心的建成，为合和实业在香港地产界奠下重要的基础。
1993年合和分拆旗下印尼、菲律宾及部分中国内地的发电业务，以亚洲电力发展的名义上市，其后却因亚洲金融风暴及泰国业务受到挫折而向出售亚洲电力发展。
2018年4月4日，合和实业完成交易，将合和公路基建（现名：湾区发展）约6669%已发行股份出售予深圳市投资控股有限公司。交易产生税后之出售净收益约港币51亿元。深圳市投资控股有限公司成为湾区发展的最终母公司。
2018年12月2日合和实业，获持有的3693%的公司大股东兼主席胡应湘及其妻持有的Petrus HK Co Ltd）以每股388港元向包括胡应湘儿子胡文新在内的非财团要约人一致行动方等其余股东，收购拥有的547亿股，占已发行股本约6307%，涉及金额21256亿元，完成私有化后将申请撤销上市地位。
戏院
合和实业有限公司全资附属机构，以「星影汇」为其品牌名称经营。座落于东九龙地标九龙湾国际展贸中心（合和实业有限公司之投资物业），拥有9个影院的多银幕戏院。
裕元工业（集团）有限公司是一家在香港交易所上市的台资工业集团公司，其母公司为宝成工业。
裕元工业为全球最具规模之品牌运动鞋和休闲鞋制造商，负责为国际品牌代工制造、代工设计制造，具备多品牌服务灵活的超卓生产能力，以鞋履年产量计，在运动鞋及休闲鞋市场的占有率为全球第一。
裕元工业于中国﹑越南﹑印尼﹑孟加拉、柬埔寨、缅甸等国家均设有厂房。业务范畴以制造运动、户外鞋﹑休闲鞋﹑多功能鞋﹑运动凉鞋﹑鞋底及配件为主。它是全球最大主要国际品牌运动鞋、户外鞋及休闲鞋制造商，旗下全球鞋类品牌客户包括Nike、Adidas、Reebok、Converse、Salomon、New Balance、Timberland、Puma及Asics等。
浙江沪杭甬高速公路股份有限公司 （Zhejiang Expressway Company Limited，港交所：&amp;sc_lang=zh-HK 0576 ，，） 是中国浙江省一家公路投资、开发和经营的基建公司，主要从事开发和经营高速公路，包括沪杭甬高速公路及上三高速公路，以及公路沿线汽车维修、加油站和广告牌等配套业务。
在1997年，由浙江省政府组成，到了2001年，浙江省交通投资集团有限公司正式成立，故此占4508%股权，其股份自1997年5月15日在香港交易所主板上市，而2000年5月5日则在伦敦证券交易所作第二上市，而最后是以美国场外交易市场，则在2002年2月14日美国纽约实施。
2019年6月5日浙江沪杭甬以101亿人民币(约115亿港元)现金向控股股东浙江省交通投资集团收购浙江大酒店全部股权，浙江大酒店位于杭州市下城区武林商圈延安路595号，占地4956平方米、总楼面面积55000平方米，主要从事酒店经营、零售、租赁及饮食业务
中国东方集团控股有限公司是一家在香港交易所上市的工业公司。主要业务是生产及销售钢坯和带钢。
公司在百慕达注册，主席为 韩敬远。
2007年，当时的中国东方集团第二大股东陈宁宁因与第一大股东韩敬远有不满，向中国东方集团提出敌意收购，但有关收购最终失败。其后，欧洲钢铁企业安赛乐米塔尔收购了中国东方集团73%的股权。
长城环亚控股有限公司（，前称南华早报集团有限公司、南潮控股有限公司及SCMP集团有限公司）现为中国四大国有金融资产管理公司之一的中国长城资产管理股份有限公司 (「中国长城资产」) 的首家境外上市控股平台。
公司在百慕达注册，集团主席兼行政总裁为欧鹏。2006年全职雇员1,016人，资产净值为1883亿港元，纯利为339亿港元。
上市历史
1971年11月，南华早报集团的前身第一次上市。1987年被新闻集团私有化，1990年6月26日重新上市，曾经被纳入为恒生指数成份股。
2015年12月，南华早报集团宣布将《南华早报》等媒体业务出售予阿里巴巴集团，代价为20606亿港元，交易于2016年4月5日完成，集团名称改为「南潮控股有限公司」。
2016年，南潮控股将清水湾电视城重建项目售予复星国际。
2016年10月30日，南潮控股董事会获买方Great Wall Pan Asia (BVI) Holding Limited及卖方嘉里集团通知，完成已于二零一六年十月三十一日根据买卖协议的条款及条文进行。于完成时，要约人（买方的一家全资附属公司）持有1,163,151,308股股份（于本联合公告日期约占已发行股份总数之7419%）。完成后，卖方不再为本公司之主要股东。
2017年11月28日，长城环亚控股全资附属公司GWPA Property将于合营投资项目注资最多3192亿港元，认购合营公司A类股份299%，合营公司为合营集团的控股公司，组合资产包括领展(00823)早前宣布出售的旗下17个商业物业及购物中心。
2018年5月，长城环亚控股收购葵芳广场，是领展出售的17个商场之一。
中国外运股份有限公司（简称“中国外运”）（ 港交所：598 、上交所：601598 ）是招商局集团下属的物流企业，在上交所和港交所挂牌上市。
成立于2002年11月20日，并于2003年2月13日在香港成功上市，是中国外运长航集团有限公司（简称“集团”）控股的H股公司。主要业务是货运代理、快递服务、船务代理、仓储和码头服务、海运和汽车运输。公司在中华人民共和国注册，主席为赵沪湘。2004年资产净值为67,809,290,000元，纯利为802,819,000元。截止到2015年12月，公司拥有从业员工27150名，总资产3508亿元人民币，净资产1846亿元人民币。
中国外运由中国外运长航集团有限公司占股579%。
历史
前身
中外运是中华人民共和国交通部下属的央企。中外运事业开创于1946年11月26日，根据中共中央东北局决定开辟满洲里口岸的指示，开创对苏贸易：“打通国际线路，同苏联取得联系，派干部去解放满洲里，建设满洲里，巩固后方，支持前线，为解放东北和全中国做物资准备”，东北民主联军西满军区驻满洲里办事处成立。人员包括处长贾石（后任国家外经贸部副部长、国家贸促会会长）、副处长常彦卿（后任中国银行副行长）、科长郑拓彬（后任国家外经贸部部长）等七人。同期，还在绥芬河、图们、安东、辑安开设口岸办事处。1947年西满军区驻满洲里办事处改称“东北贸易总局驻满洲里办事处”，受东北贸易管理总局运输部指导。1949年改称“东北对外贸易部满洲里办事处”，1950年改称“中央贸易部满洲里口岸管理局（1950年）”。
1953年1月1日，中国陆运公司成立，统一办理进出口货物在港口和口岸的交接和储运工作。1955年，中国陆运公司与中国海运公司合并为中国对外贸易运输公司，隶属于国家外贸部。
的办事处
大众金融控股有限公司（原名为「日本信用保证集团」）于1991年8月16日根据百慕达1981年公司法于百慕达注册成立之投资控股公司，是一家在香港交易所上市的金融公司，并为恒生香港小型股指数成份股之一。主要业务为接受客户存款、提供个人与商业贷款、信用卡业务、按揭贷款、证券经纪服务、投资物业租赁、向的士及公共小型巴士之买家提供融资及买卖的士车辆与的士牌照及出租的士。
公司在百慕达注册，主席为 丹斯里拿督郑鸿标。2005年资产净值为HKD 2,393,434,000 ，纯利为HKD 446,297,000 。
大众财务
大众金融控股旗下的大众财务有限公司（Public Finance Limited）是香港的一间财务机构，亦是香港的一家接受存款公司，其前身是日本信用保证财务有限公司（JCG），1977年10月20日由日籍台裔商人长原彰弘博士根据香港《公司条例》在香港注册成立，1978年1月31日根据银行业条例注册为接受存款公司。
1990年，日本信用保证财务有限公司因美国投资失利，被马来西亚的大众银行收购，成为大众金融集团的成员之一，2006年1月16日起改名为现时名称。
大众财务以「大众财务，大众首选」为口号。
日本信用保证财务有限公司的原股东在卖出公司之后，于1991年1月29日创立了另外一间财务机构 — 亚洲联合财务有限公司（简称UA），继续由长原彰弘主理，注册资本4000万港元。1993年获联合集团及个别日资财务公司入股，再把部份日本财团股权售予新鸿基金融集团，再于2007年元旦成为新鸿基金融集团成员。
创科实业有限公司（TECHTRONIC INDUSTRIES CO LTD ，）(ADR:TTNDY)，是一家主要从事电动工具产品业务的投资控股公司，于香港交易所上市。业务包括制造及贸易再充电式电动工具、配件、手动工具、户外园艺工具及地板护理产品，为DIY用家、专业人士及工业用家提供家居装修、维修、保养、建造业及基建业解决方案。
Horst Julius Pudwill及钟志平 GBS BBS JP 于1985年创办创科实业。公司旗下品牌包括Milwaukee、AEG、Ryobi、Homelite、Empire、Stiletto、Hoover US、Hart、Oreck、Vax及Dirt Devil。于2021年，创科实业的员工人数超过51,000名，全球销售总额达132亿美元
历史
TTI 于 1985 年由 Horst Julius Pudwill 和钟志平教授在香港创立。TTI 最初是海外品牌的原始设备制造商。1987 年，Sears 与 TTI 合作生产了 Craftsman 品牌的无绳电动工具。TTI 在开始为 Bissell 品牌生产无绳手持式真空吸尘器时进入了地板护理行业。1990年，TTI通过首次公开募股的方式在香港联交所上市，股票代码0669。TTI 在美国股票市场的股票代码为 TTNDY。
TTI 的收入从 1990 年的 6266 万美元增长到 2001 年的 7786 亿美元。从 2010 年到 2019 年，TTI 的销售额以 10% 的复合年增长率增长。TTI 于 2019 年 3 月 11 日被列为基准恒生指数的成分股。列入名单的公司“构成了该市股票市场总市值的前 90 个百分点”。TTI 报告的 2021 年收入为 13203 亿美元。
香港铁路有限公司（简称：港铁公司；MTR Corporation Limited；企业标志为MTR；港交所：&amp;sc_lang=zh-HK 0066 ，）于1975年9月26日创立，当时名为地下铁路公司（Mass Transit Railway Corporation），于2000年宣布私有化，同年4月26日注册成为有限公司，更名为地铁有限公司（MTR Corporation Limited）。2007年两铁合并，同年12月2日起通过服务经营权营运九广铁路公司网络及业务而改为现时名称。
香港铁路有限公司总部位于香港九龙湾伟业街33号德福广场港铁总部大楼及国际金融中心二期若干楼层（包括45楼董事局主席办公室），主要业务是在香港经营垄断型的综合铁路网络——港铁，当中包括由其建造及拥有的系统及服务经营权下的九广铁路公司系统。
简介
举行的历史展览
「地下铁路公司」于1975年根据《地下铁路公司条例》（《香港法例》第270章）成立，法定资本为20亿港元，作为一家法定法团在香港建造及经营一个集体运输铁路系统，并由财政司法团以信托形式代表当时的香港政府全资拥有，而且采取审慎商业原则运作，配合本地的公共交通运输需求。于1979年，地下铁路首段落成激活。政府在1981至1995年期间多次增加该公司的法定资本至327亿港元使地下铁路公司可应用资本以兴建包括机场铁路在内的铁路。而其他行车线相继于1982年及1986年激活。截至1997年3月31日止，经财务委员会批准，已向地下铁路公司注资合共约321亿港元。1998年，一条特别为香港国际机场而建造的铁路机场快线落成激活。
私有化
1999年3月3日，财政司司长曾荫权的财政预算案演词中宣布，香港特区政府建议将地下铁路公司私有化：财政司司长法团代表政府在2000年6月公开招股，出售地下铁路公司23%的股份；地下铁路公司成为「地铁有限公司」，并于2000年10月5日在香港联合交易所上市。2000年11月1日，财政司司长法团（政府）根据授予最初股份发售包销商的超额配股权，完成出售额外15亿股地铁公司股份；政府现持有4,434,552,207股股份、占全部已发行股本约7509%，政府虽然仍为大股东，但该次公开招股向香港市民等投资者出售了近10亿股份，地铁公司成为香港上市公司中股票持有人最多的公司。政府承诺会由上市日起计最少20年内维持为地铁有限公司的主要股东，并继续在法律及实益上持有不少于地铁有限公司普通股本的50%及持有不少于地铁有限公司股东大会上投票权的50%。上市后，地铁公司的盈余持续增长。
发展地产
港铁公司成立初期已经与地产发展商合作，在车站与车厂上盖及周边从事住宅及商业物业发展及销售；而且拥有一个投资物业组合作长期投资，并提供物业管理服务。其发展的沿线物业及物业管理业务于2000年代持续发展。较知名的香港物业业务包括：君临天下、凯旋门、日出康城等住宅项目，及圆方、国际金融中心一、二期／国际金融中心商场、环球贸易广场（已连同管理权售予合作发展商新鸿基地产）等商业项目。截至2011年12月31日，港铁公司管理住宅单位数目86,162个单位和管理商用及写字楼面积744,725平方米。2007年，地铁公司录得151亿港元盈利，比1997年的278亿港元为高。
其他业务
随着业务持续发展，港铁公司亦从有关铁路的其他业务中赚取收入，包括经营广告服务、车站小商店租赁及提供电讯服务；而且亦积极在香港以外跟其他机构合作，提供铁路顾问服务，兴建及营运伦敦地上铁、深圳地铁、北京地铁等城市的铁路系统。近年，港铁更成为「国际都市铁路联会（CoMET）」的创会会员之一，及成为「国际公共交通联合会（UITP）」的会员之一。2008年8月，港铁公司成为唯一提供「北京奥运马术比赛铁路运输服务」的交通运输机构。在电子票务方面，港铁公司成立了附属公司「八达通卡有限公司」从事智能卡系统的营运，港铁及其他主要交通营运商均利用八达通系统收取大部份车费，八达通卡公司更将其业务拓展至非运输行业。
庞大盈利
港铁公司不但在公共交通系统企业中以拥有庞大盈利而闻名于世，在2008年度，其经营利润增长577%至9325亿港元，是历年最多。翌年，港铁更在《福布斯》全球上市公司2000强（Forbes Global 2000）排行榜中再次进入全球850强，较2008年前进91席，名列第721位，在多家上榜的香港企业中，排名第13，位列运输企业第2名；而且在英国《金融时报》公布的2009年全球市值500强企业排行榜（FT Global 500），港铁公司在全球企业排名中占第367位，在全球旅游休闲行业排名第4。同时，港铁公司在香港交易所中的总市值超过1,500亿港元，成为总市值最大的基建类公司。目前为止，港铁公司一直维持超过50%之一的营运盈余，得以收回投资资本，令其成为全球少数能在营运公共交通系统中有盈利的公司，而且是世界唯一不再需要依赖政府补贴（如铁路上盖物业等）营运的城市轨道交通系统运营商。2008年，港铁在香港的专利公共交通工具市场占有率为427%。2013年，港铁在香港的专利公共交通工具市场占有率为469%。
两铁合并
2007年12月2日，九广铁路公司（由香港特区政府全资拥有）所经营的网络「合并」由地铁有限公司营运。地铁有限公司的中文名称改为「香港铁路有限公司」（港铁公司）。这也标志着香港铁路发展的一个新里程。
年表
急速扩张（成立至1990年代）
」路线图
附近的翻新列车）
发展新方向（2000年至2007年两铁合并）
前的路线图
是全球首条专为迪士尼乐园而设的铁路（图为摄于欣澳站的主题列车）
迈进新纪元（2007年两铁合并至现今）
生效，九铁公司网络交由地铁公司管理及营运（图为2007年12月2日凌晨，尖东站举行的合并仪式）
通车，令西区有铁路服务
通车，象征港铁公司网络遍布全港十八区（图为摄于香港仔海峡大桥）
通车，令港铁首次参与高速铁路营运
通车，标志着沙田区有多一条铁路出市区
全线通车，成为全港最长铁路线
过海段通车，成为香港第四条过海铁路
香港铁路业务
」（图为东涌线韩制列车）
港铁公司目前在香港经营主要以铁路为基础的运输系统──综合港铁系统，包括由九条本地城市轨道路线组成的港铁系统及过境服务、高速的机场快线和轻铁系统。整个系统总长约2309公里，有93个车站和68个轻铁站。综合港铁系统是全球最繁忙的铁路系统之一。港铁公司亦提供前往中国大陆（北京、上海和广东省）的城际客运服务，以及为乘客提供本地接驳巴士服务。
港铁
港铁系统于香港运作，由观塘线、荃湾线、港岛线、东涌线、将军澳线、东铁线、屯马线、迪士尼线及南港岛线共9条港铁路线组成，总长度超过2850公里，共有98个车站，当中有4个车站和轻铁系统连接、1个车站和缆车系统连接、3个车站和机场铁路系统连接。
港铁在大部份日子提供每日约19小时的服务，其中东铁线及屯马线为195小时，大约由香港时间上午5时28分至翌日凌晨1时30分（若适逢大节日，则会通宵服务）。于早上繁忙时间，港岛线的列车班次最短可达120秒一班，载客量最多的港铁伊藤忠近畿川崎列车，每小时载客量可达120,000人次。港铁列车数目由地下铁路公司成立时的140卡车厢，增加至1863卡。
轻铁
市场的主要营运商地位（图为轻铁751线车辆正在兆康站中停站）
轻铁系统（亦称为西北铁路）于1988年9月18日投入运作，长362公里，双轨行车，设有68个车站，当中有4个车站可转乘屯马线。轻铁系统于新界西北部的屯门、元朗及天水围等地区运行。轻铁是一个地区集体运输系统，其使用的车辆与在公共道路上的路轨行走的电车相若。轻铁是新界西北的综合运输系统的一部分，为屯马线提供接驳服务。港铁公司现时共有140部轻铁车辆，每部可载客200余人。近年，轻铁的行车正点率及票务设施正常操作率均有99%以上。
轻铁于2011年底录得每日平均45万人次的乘客量，是全球最繁忙的轻铁系统之一。
机场快线
机场快线连接中西区中环的香港站，与离岛区赤𫚭角亚洲国际博览馆的博览馆站之间，属于机场联系轨道系统。此线连接了赤𫚭角香港国际机场，以特快形式运行，故得此名。
过境及城际客运服务
穗港高速动车组列车
广州的城际客运服务（图为行走T821次（现为Z821次）的「Ktt」列车正通过大围站）
港铁公司负责以东铁线营运罗湖及落马洲至红磡的香港及过境客运服务。同时，港铁公司亦以双层「Ktt」列车（「广九直通车」）营运往返中国广州的城际客运服务，让中国铁路公司营运来往香港和内地六个城市之间的直通车，这些城市分别为东莞、广州、佛山、肇庆、北京及上海。
港铁公司以动感号列车营运由香港西九龙站至广州南站沿途各站之间的城际高速铁路，让中国铁路公司营运由香港西九龙站至内地58个车站的高速铁路。
港铁公司营运香港唯一连接中国大陆铁路网络的铁路。为充分开拓中国内地「港澳个人游」这个市场的庞大潜力，港铁公司与商业伙伴合作，为乘搭广九直通车来港的内地旅客提供特别优惠。与其他商业伙伴合办的推广计划，还包括酒店套票、娱乐及旅游景点优惠、零售商户折扣优惠等。由于罗湖为接近中港边境的禁区，而港铁公司拥有香港境内的独家进入权，因此于过去，港铁公司一直受惠于跨境交通量的增长。
巴士服务
」单层巴士已投入服务（图为摄于元朗大马路的港铁巴士）
港铁公司营办巴士营运业务以为轻铁系统、东铁线及机场快线提供有效率的接驳巴士服务。
不计九巴经营的东铁线接驳巴士服务，港铁公司共经营14条接驳巴士路线。
港铁公司无权就新界西北的铁路服务范围以外的接驳巴士服务收取车费。因此，港铁公司与九龙巴士公司订立商业协议，经营前往大埔的东铁线(接驳巴士路线（K12、K14、K17及K18）。港铁公司一方面可从使用巴士服务但并无转乘东铁线的乘客身上获得收入，另一方面亦可通过九巴等专营巴士经营商所享有的燃油税优惠来降低成本。
港铁公司的免费机场快线穿梭巴士主要服务乘搭机场快线乘客及机场快线香港站及九龙站附近主要酒店及交通交汇点。穿梭巴士开办共9条路线。其中4条（H1，H2，H3，H4）往来香港站，5条（K1，K2，K3，K4，K5）往来九龙站。唯所有路线因2019冠状病毒病疫情的原因，已于2020年6月起停办。
每年平均每日巴士乘客量超过82,400人次，此数字不包括九巴经营的4条接驳东铁线的路线。
香港网络拓展
修正早期系统观塘线于1979年10月1日正式投入服务，市区铁路网络随即迅速发展。一年后，首条连接香港的港岛及九龙的铁路线亦投入服务，为香港当时的交通服务带来重大的改变。荃湾支线和港岛线相继于1982年和1985年落成。1989年，东区海底隧道竣工，观塘线伸延至香港岛，成为第二条过海铁路。机场快线及东涌线的机场铁路于1998年开始营运，连接香港市区与赤腊角新机场及东涌新市镇。将军澳线于2002年通车，并于2009年延伸至日出康城。2009年，九龙南线竣工，西铁线伸延至尖沙咀，红磡站成为东铁线和西铁线的终点站。近年，在香港的5个新铁路项目正处于不同的规划和建造阶段，而落成后将使港铁公司的网络增加约60公里。
另外，港铁公司未来将会发展其他新支线，以便应付香港长远的发展，当中包括南港岛线（西段）、北环线及北港岛线等。港铁公司亦有意参与探讨港深机场联系线（香港段）及启德环保运输系统项目的兴建及营运。
车站商务及铁路相关业务
现时，港铁公司除营运铁路外，亦从事多元业务，通过利用其铁路资产及专业技术拓展至其他业务领域（包括出租列车与车站广告媒体、车站内的零售商舖），并通过在港铁公司铁路范围内可使用个人电讯网络服务设施获得收入。港铁公司还有铁路相关顾问服务，集团亦不时从其他乘客服务获取收益，例如出售港铁纪念车票。
广告
内的环回电视屏幕是其中一个崭新广告形式的例子
港铁公司的广告媒体业务已成立超过25年，为集团带来非车费收入。
港铁通过利用设于车站大堂及轨道旁的等离子电视屏幕、车站柱位和墙身作广告用途、于列车及车站内分别安装「港铁资趣台」电子广告板或提供「新闻直线」多媒体广播服务及乘客信息显示系统、港铁立体展览车、即时信息投影区及环回电视屏幕，以及引入免费报章。
港铁公司更推出多个崭新的广告形式（包括于2008年5月在铜锣湾站低层信道设立经翻新的「广告廊」），而且车站还设有「交互电子广告屏幕」、「消费地图王」及「十二封转动灯箱」作产品展示或宣传海报用途，以及为车站采用分区设计以增加收入及广告效力。截至2020年12月31日整个港铁设有总数49519个广告板。
现时港铁车站广告及活动推广由德高贝登（JCDecaux）（代理所有前地铁路线及南港岛线）及雅仕维（Asiaray）（代理东铁线、屯马线及轻铁）两间公司代理。
电讯服务
设于国际金融中心二期
港铁公司与香港流动电话网络营运商（例如：和记电讯、CSL等）订立协议，于铁路范围内提供流动通信覆盖，为集团带来持续的收益。
乘客可通过其流动电话进行电话通信、上网及使用其他数据服务。合并前的地铁公司铁路范围内每日约有90万个电话呼出，每日的总通话时间约为1,900,000分钟。除语音通话外，数据服务的使用率亦不断增加，例如短讯服务。
港铁公司于2002年成立全资附属公司TraxComm Limited，TraxComm并于2003年6月2日获电讯管理局发出固网电讯牌照，以在集团现有的2100公里光纤基础设施上发展批发固网电讯业务。于2003年，TraxComm兴建并开始营运经加强的网络，有关网络与现有的光纤网络同时运作，令整体网络速度提升至100千兆（Gbps）。
尽管其经营历史较短，但TraxComm已向若干客户出售了逾180千兆（Gbps）的带宽服务。这些年来，TraxComm不断将其光纤基础设施扩展至多个数据中心及商业楼宇。于2007年，TraxComm已经在某些港铁站引入Wi-Fi无线上网。
于2009年，TraxComm伙拍和记电讯于将军澳86区完成了首项建筑物内部铺线服务合作项目，优化住宅用户电讯服务上的选择权；并计划将此服务推广致车公庙、黄竹坑及青衣商场等港铁物业。
车站商用设施
内的商舖）
港铁站内共设有1529间不同类别的零售店舖提供多种货品及服务，饮食、健康及美容、饼店、便利店、时装、化妆品、饰物及精品店、凉茶及健康饮品、糖果、日用品、干洗服务、补鞋店、书店、诊所、旅行社以及银行及理财服务等均一应俱全。多款自助服务设施现已遍布铁路沿线车站，包括自动柜员机、自助照相亭、其他自助售卖机及各种电子服务等。往返中国大陆时，可顺道前往位于红磡站及罗湖站的免税品商店享受免税购物。新免税品商店设于新落成的落马洲站，亦已提供服务。
港铁公司亦提供车站展览场地，现时市区线及机场快线车站，共有51个展览场地，加上在东铁线车站的14个展览场地，为各参展商提供大小不同的场地，作为展出商品及推广服务。
港铁集团于2001年开始推行大型翻新计划，到2007年为止，已于40车站进行翻新。现正计划在更多车站进行商舖翻新工程。于2020年12月31日，零售设施总面积为为67746平方米，车站商店数目为1529间
现时在港铁网络内，各线车站商舖是一体化管理的，都是由港铁公司直接租赁车站商舖予各商户。
顾问服务
（图为一列台湾高铁列车）
港铁公司近年不断集中于中国大陆或欧洲主要城市的铁路相关项目提供全方位顾问服务，为未来投资创造条件。
现时，港铁公司的主要服务对象为全球各政府、铁路营运商、铁路策划师、设备生产商、承包商和其他顾问。
截至2008年底，港铁公司为中国、亚洲、澳洲、中东以及欧洲的铁路项目提供铁路定线和信号系统等的顾问服务。港铁公司还提供其他顾问服务，范围包括铁路策划、施工项目管理、建造、营运、维修、资产更新、物业发展和其他非车费收入的顾问服务。当中包括上海地铁九号线（申松线）、台湾高铁及高雄捷运等。至2010年，港铁已经为24个国家提供铁路服务顾问。
另外，香港国际机场旅客捷运系统的维修合约是港铁公司为期最长的顾问服务，为港铁公司带来每年约2亿港元的收入。
物业及其他业务
港铁公司与不同地产发展商合作，进行以住宅为主的物业发展。港铁公司拥有的投资物业主要为商场及写字楼，并管理旗下物业及其他业主的物业（除前九铁银禧花园及文礼阁以实益拥有权出让方式发展，其余均以发展权出让方式发展）。港铁公司的投资物业组合包括12个商场及国际金融中心二期18层写字楼。
物业发展
物业发展是港铁公司业务的重要组成部分，并提供一个重要的收入来源，为铁路项目的兴建成本提供资金，同时亦因在物业发展项目附近产生的载客范围而有助增加日后的铁路路线提供大量的乘客量。
在兴建铁路的同时，港铁公司根据与多家物业发展商订立协议，参与发展港铁站及车厂上盖及附近的住宅及商用为主的物业。港铁公司从这些发展项目所得的利润补充相关铁路收益，从而提高兴建新铁路线的投资回报率。
香港特区政府根据按十足市值评估的地价（并无考虑被估价的土地位于铁路沿线上）授予港铁公司用作发展物业的土地的发展权。港铁公司在物业发展方面的做法是安排第三方发展商根据港铁公司的要求进行实际发展工程。一般而言，发展商负责支付所有发展成本（包括政府地价、建造及准备工程成本、市场推广及销售开支、专业费用、财务费用及其他开支），并须承担一切发展风险。港铁公司通过与发展商按协定比例摊分销售或租赁物业的利润（经扣除发展成本）、摊分实物资产或通过发展商支付的预付款项从物业发展中获得利益。
公司近5年来自物业发展利润收益累积达171亿元，占每年经营利润普遍达2至3成，显示物业发展已经成为港铁公司主要收入来源之一。
批评
落成后与「名城」发展协调一致，造成屏风楼效应
Palace by the Sea会所）
落成后阻挡天线信号，令大围村的电视频号接收不佳，情况长达5年，严重影响日常生活，惟港铁一直没有理会
物业规划透明度不足
港铁沿在线盖物业项目招标，有别于政府勾地和推地形式。当港铁决定推出旗下项目，会公开邀请财团交意向书，及后港铁会向有意参加的发展商，发出招标文档，包括上盖物业建筑物位置、布局、规划数据、初步补地价和详细发展限制等条款。由于招标文档不公开，因此吸引不少发展商蜂拥提交意向书，是为取得文档了解内容。当发展商收到招标文档所提供入场费和补地价等数据，发展商提出的分红等条件，成为中标的关键，若没有标书合乎港铁心目中条件，则会流标。
港铁一直将公众可关注的城市规划视为智识产权及商业利益一部份。物业发展所公开的数据只有一堆数字。上盖物业建筑物位置、布局同规划数据等数据，公众无法得悉。以日出康城第4期发展为例，虽然市民可以由「总纲发展蓝图」查得其具体发展计划，唯港铁可随意更改发展规模及位置，与过去市场传言的位置并不相同，早前市场一直认为第4期邻近日出康城3期致蓝天，又或位于日后商场部分的上盖，但实际位置其实在日出康城第2期领都旁边，毗邻日后拟建的中小学，位置属临海地段。有媒体问到第4期的具体位置及发展详情，港铁发言人只表示要等待发展商在数年后项目推出时再交代。有居民批评港铁的规划改动重大，完全没有经过任何公众咨询，不符合公众的合理期望。
设计及休憩空间只限住客享用
不过近年亦有民间批评指港铁发展的物业设计完全以豪宅定位，并采用巨型平台及屏风楼设计。基座均设豪华大型会所、园艺平台及巨型入口大堂，使项目发水比例较一般住宅楼宇为高。地区人士批评港铁发展的物业形成屏风楼效应、只为物业住客缔造优质生活居所及环境，而且物业发展设计是「为钱而非为人」，大部份绿化、康乐设施及休憩空间只限住客享用，公众只可享用商场及面积细小的地面广场，比东京、新加坡及上海等地落后得多，公众并没有因新物业发展而得到改善。但政府最终没有理会区议会以至地区人士反对声音，如期让港铁为物业发展项目进行招标工作。而本地主流传媒亦对民间反对港铁发展的物业声音并不重视，难以让公众关注。
即使港铁在2012年6月表示新发展的住宅物业会依据「可持续建筑设计指引」实施项目，必须达到香港绿色建筑议会制定的绿色建筑指针，并且取得绿建环评金级认证（BEAM Plus Gold Certification）。不过由于宽免面积不纳入发水上限一成内，因此将来发展的住宅物业只需缩减露台、会所及大堂设施面积即可达标，对于高度及体积，影响不大。
外判管理及赚取佣金
本身管理多个大型屋苑的港铁，在项目入伙后，会将保安及会所服务外判管理，并压低做价从中抽取行政费用，不断增加开支以赚取佣金。又指名工程商揾钱，使业主白交管理费而得不到应有的服务，港铁便坐享其成。如有住客投诉，港铁压抑保安公司扣钱，并无正视问题，纵容无理租客生事及怒骂员工。有住客批评物业管理费昂贵以及管理不善的情况，被迫加管理费和动用屋苑储备。港铁将军澳站上盖天晋屋苑管理费每平方呎建筑面积为275元，较将军澳区其他屋苑昂贵。
近年有业主透露因来年开支增加，要增管理费9%至18%不等。葵芳新葵芳花园业主质疑，港铁管理楼宇时涉不必要开支，例如每年更换垃圾桶、更换防烟门。另外会聘请额外员工增加支出。由于经理人酬金与屋苑管理支出挂钩，结果港铁所享酬金亦水涨船高，屋苑储备至年底只剩84万。
过度迁就居民，限制铁路车速
东铁线自100年以来均以路面行驶，随人口上升，越来越多居民住在铁路旁边。但因为嘈音问题而向港铁及环保署投诉，以东铁线为最严重。东铁线一共有4个限速段，包括大埔康乐园到太和（110公里限制至60公里车速）、大围到显径（90公里限制至70公里车速）、九龙塘到旺角东段（南行：70公里限制至50公里车速；北行：70公里限制至40公里车速）以及旺角东到红磡段（80公里限制至40公里车速）。每晚11点过后由九龙塘到红磡段全线限速30公里。港铁被批评过度软弱，纵容居民到铁路作随便限制，令悉乘客乘车时间拉长5-10分钟，同时减少载客达7,500人。
停车场车位炒卖问题
2013年起，港铁亦为了维持高收益，以高价拆售港铁沿线车位，包括沙田溱岸8号、奥运站柏景湾及帝柏海湾的全数车位，其中溱岸8号车位最高造价更达188万元，创新界车位售价新高；7月中更有车位以月租4,500元租出，创新界最高车位租金，结果使区内停车场车位亦被炒卖，居民泊车成问题。
物业租务、管理及其他业务
现时，港铁公司拥有主要为商场及写字楼的投资物业组合不断增加。该等投资物业带来经常性收益及加强集团的资产基础。截至2020年12月31日合计，港铁公司于整个投资物业组合中的应占份额包括可出租面积达257692平方米的零售物业、可出租面积达39410平方米的写字楼及可出租面积达18905平方米的其他用途物业。港铁公司的投资物业组合当中包括13个商场及国际金融中心二期18层写字楼等。港铁公司还提供港铁物业管理、代理、港铁八达通管制系统及楼宇环境卫生等服务。于2020年12月底，港铁公司于香港拥有多个物业管理合约，合共管理逾111000个住宅单位以及逾772000平方米的写字楼及零售空间。
于2000年，港铁公司创建了一个新的服务品牌Premier Management Services，以管理选定的高档物业，包括国际金融中心二期、九龙站上盖豪宅(包括凯旋门、天玺、君临天下等)
2020年香港物业管理收入上元，较2019年减少220%至237亿港元。
中国内地物业发展
港铁公司现正于中国内地发展物业顾问及管理业务，现有7份物业管理合约，并就此订立了新合约，为中国国内不同地区的高档物业发展项目提供管理及顾问服务。于2006年，SOHO中国有限公司订立了委托港铁公司管理豪华写字楼／商业发展项目的3份新的物业管理合约，有关物业均位于北京的中心商业区，包括SOHO尚都、建外SOHO第七期及朝外SOHO。于2007年上半年已就北京2个办公室及商业发展项目签署一份谅解备忘录，一个位于光华路，另一个位于西单。港铁公司亦就北京银座Mall的经营及物业管理订立了长期的租约，银座Mall为一可租用面积达19,295平方米的购物中心，已于2007年1月激活。2011年8月19日港铁首次夺得中国铁路上盖物业、以底价1977亿元（人民币．下同），平均每平方米楼面地价9590元，投得的深圳龙华线车厂一期，为深圳地铁龙华车辆段上盖港铁．天颂，总楼面面积约2219万平方呎，其中住宅占2063万平方呎，涉及1,700伙，在2015年4月开售，预计在2016年落成，地价约1977亿元人民币。
至于与天津市地下铁道集团合资发展的天津地铁6号线北运河站项目，亦会作综合商住发展，涉及总楼面近300万平方呎，较龙华线车厂1期规模更大。
港铁副行政总裁梁国权接受访问就曾指出，计划以香港的铁路加物业模式进军内地，除了北京、深圳及杭州之外，亦会在其他一线城市物色拓展机会，将海外净资产比例由现时的10%提升至15%。
高端商场：
地区商场：
普通商场：
内地商场：
昂坪360
系统之一（图为昂坪360其中一个缆车车厢）
由港铁公司拥有的东涌缆车项目名为「昂坪360」，是香港一条连接大屿山东涌及昂坪的缆车（吊车）路线，系统并由附属于港铁公司的昂坪360有限公司管理及营运。「昂坪360」于2006年9月18日激活。
于2008年全年昂坪360缆车及昂坪市集，为港铁公司带来156亿港元的收入，旅客人数逾160万人次。港铁公司还推出私人包厢服务，并与圆方、青衣城及港铁友礼会联合推出推广活动以吸引旅客。
八达通
港铁公司于1997年推出八达通卡。自此，免触碰的八达通卡在香港大行其道，截至2020年12月31日，在香港采用八达通的商户逾31000家。八达通卡及其他储值八达通产品的流通量达3410万张，而每日平均交易量及交易金额分别为1160万宗及1937亿港元。。八达通卡不仅获各类公共交通工具广泛接受，同时亦获若干大型连锁便利店及超级市场以及快餐店广泛接受，而且通过便携式八达通处理器，将业务扩展至中小型零售市场，吸纳更多零售商户采用八达通。
八达通卡由八达通卡有限公司拥有，八达通卡有限公司为港铁公司的非受控制、间接附属公司，而港铁公司为该公司的最大股东。港铁公司目前拥有八达通控股有限公司574%的已发行股本，而八达通控股有限公司则拥有八达通卡有限公司全部已发行股本，八达通控股有限公司余下426%已发行股本由九广铁路公司221%、九龙公共巴士控股有限公司124%、城巴有限公司及新世界第一巴士服务有限公司拥有。尽管港铁公司持有八达通控股有限公司574%已发行股份，但本公司于八达通控股有限公司董事局会议的投票权以49%为限，而概无八达通控股有限公司的股东可能够单方面控制八达通控股有限公司的董事局。而于两铁合并后，港铁公司和九广铁路公司持有的股份则维持不变。
八达通卡有限公司在香港以外，亦赢得多项合约以扩大其海外业务，包括在荷兰和中国长沙。
去中心化游戏虚拟平台
2022年4月21日，港铁宣布与去中心化游戏虚拟世界The Sandbox成为合作伙伴，成为全球首个进驻The Sandbox元宇宙的交通营运商。公司表示已经购入虚拟土地兴建虚拟车站、铁路博物馆及推动STEM教育，期望通过把握Web3及元宇宙的大趋势构建一个崭新平台。不过网民批评港铁利用政府的金钱投资去中心化的NFT，是不能接受。
零售品牌
2022年7月，港铁宣布设立O2O (Online-to-Offline)零售品牌「LOUDER」，为本地年轻艺术家及创业家提供自家设计及手作，通过平台进行较低成本的推广及宣传。同时会于2022年8月在将军澳The LOHAS康城设立实体店，其后亦将在围方及The Southside商场开设门市。
MTR Lab
针对不同行业和需要，参考港铁多项业务的方案提供度身定制的产品，当中包括综合车速及位置监督系统和V Sensor。2022年6月，港铁与香港数码港管理有限公司(「数码港」)订立策略联盟，其后亦投资支持多个初创企业和设立子公司，包括物流科技alfred24和一站式减碳奖赏平台Carbon Wallet。
港外业务
拓展香港以外的市场是港铁公司发展策略的一部分。而为中国及国际市场引进铁路发展和物业管理专业知识，亦是港铁公司持续增长策略的内核元素。港铁公司继续投资中国大陆的城市铁路网络，以及在欧洲私有化或对外开放的市场中，寻求毋需投放大量资金的铁路营运专营权，以实现公司的海外拓展策略，港铁公司在4个国家，营运铁路线总长度2,183公里，2020年铁路业务总载客量138亿人次。
中国大陆
中国大陆为港铁公司业务发展策略的重点，主要集中在城市铁路的开发及投资，并为中国大陆提供铁路营运、物业管理及顾问服务。目前，北京港铁4号线、14号线项目进展畅顺，港铁公司亦为深圳市轨道交通四号线项目获得审批作好准备，并继续在杭州、苏州、天津及武汉等主要城市以及北京及深圳寻找更多类似投资项目机会。
上海地铁
早于2002年，港铁公司在上海创建了名为「上海港铁建设管理有限公司」的合资企业，负责上海铁路建设的项目管理。该公司为上海地铁九号线第一期的项目管理及业主代表，负责九号线的设计及施工，并为八号线及十一号线作设计咨询。九号线第一期由松江新城站至桂林路站已于2007年12月底通车，而桂林路站至宜山路站的一段已经于2008年底完成。
北京地铁
的分离式月台）
港铁公司与其公私合营伙伴合组的公司，在2006年4月12日与北京市政府签署了北京地铁4号线项目的《特许经营协议》，项目总投资金额约153亿元人民币，是集团海外业务发展上的重要里程碑。北京地铁四号线不但是京港合作其中一个最大的投资项目，也是港铁公司在中国国内首个以公私合营模式合作投资、开通和运营的铁路项目。
约46亿元人民币或成本总额的30%由港铁公司、北京市基础设施投资有限公司及北京首都创业集团有限公司联合组成的公私合营公司北京京港地铁有限公司承担，三方在合营公司中分别拥有49%、2%及49%的权益。其余建设成本由北京市政府提供。合营公司将负责北京四号线的机电铁路系统及列车，并获得特许经营权经营该铁路30年。北京京港地铁有限公司亦于2008年11月27日与北京市政府全资拥有的北京轨道交通大兴线投资有限责任公司，就全长22公里的北京地铁大兴线的营运及维修项目签署了谅解备忘录。由于大兴线为四号线的伸延，合营公司于大兴线落成后可营运大兴线直至特许经营完结。北京四号线全长29公里，共24个站，南起丰台区南四环路的公益西桥站，北至海淀区的安河桥北站，是北京南北交通的主要铁路干道。建造工程已于2009年9月落成；而大兴线则为2010年落成。于2009年9月28日正式通车，及成为首个香港公司营运的中国铁路。
深圳地铁
港铁公司在2005年5月跟中国深圳市政府签署了深圳市轨道交通龙华线二期的兴建及全线30年的特许经营协议，采用一种建基于「铁路和物业」模式（即由港铁公司负责龙华线沿线的物业发展项目）的融资安排，全资以建设、营运、转移模式兴建及营运，并成立了全资拥有的子公司「港铁轨道交通（深圳）有限公司（简称为港铁（深圳）公司）」管理整个总投资58亿元人民币的项目。
于2009年3月，深圳龙华线整个项目已获国家发展和改革委员会批准，相关公用设施迁移及收地工作已经开始，前期准备工作已接近完成，主要土木工程可随即展开。于2009年3月18日，港铁（深圳）公司与深圳市政府就龙华线项目正式签署了《特许经营协议》。根据是项《特许经营协议》，港铁公司负责投资及建设龙华线二期，并需负责龙华线所有运营、维修及资产管理等方面的开支，而深圳市政府会提供适当财务支持，并营运全线30年。港铁公司将同时着手安排接管龙华线一期的有关工作。于2009年6月9日，国家工商总局授权深圳市工商局正式签发了企业法人营运执照给港铁（深圳）公司，标志龙华线全线逐渐转入运营阶段。在龙华线全线通车前约12个月，港铁（深圳）公司会着手营运龙华线一期，为全线通车做好准备工作。港铁（深圳）公司于2010年7月1日接管龙华线。整条龙华线工程已于2011年6月16日，深圳举办第二十六届世界大学生运动会前竣工通车。
深圳地铁4号线全长313公里，共23个站，从福田口岸作起点直达龙华新区。龙华新区是中国高速铁路深圳北站的所在。龙华线连接港铁落马洲站，将深港两地的铁路服务紧密连接起来。龙华线一期从福田口岸站至少年宫站，一期长46公里，于2004年12月28日通车，二期长16公里，设有10个车站，连接少年宫站至清湖站，于2011年6月16日开通。三期长107公里，设有8个车站，连接清湖站至牛湖站，于2020年10月28日开通。
2020年7月23日，港铁（深圳）与中铁电气化局合营公司投得深圳地铁13号线建设及经营权。
杭州地铁
港铁公司于2008年5月向杭州市改革及发展委员会提交杭州地铁一号线的提资建议书，于2008年7月获选为项目的「优先谈判对象」后，并于2009年1月16日与杭州市政府及杭州市地铁集团有限责任公司签定原则性协议，就杭州一号线项目以公私合营模式投资、建设及为期25年的营运。该项目的总投资金额达220亿元人民币。杭州一号线已于2012年通车。港铁公司同时与杭州市政府签订策略性协议，探讨杭州地铁沿线的物业发展机会。
港铁公司于2017年6月26日与杭州市人民政府签订杭州地铁5号线PPP项目特许协议。根据协议，杭港地铁将负责5号线的机电设备工程部分的投资以及线路落成通车后25年的营运管理和维护。
沈阳地铁
港铁公司再于2008年11月跟沈阳市政府及沈阳地铁集团有限公司签订协议，成立特许经营公司营运全长50公里的沈阳地铁一号线及二号线。并和沈阳市政府成立联合合作小组，研究沈阳地铁第四、九、十号线，及一、二号线延线的发展。沈阳一、二号线分别将于2010及2012年投入服务。此外，港铁公司签订了另一份协议，就沈阳一、二号线的沿线物业发展机会展开规划和探讨。
于2009年5月18日，港铁公司与沈阳市政府及沈阳地铁集团有限公司就沈阳地铁一、二号线的营运及维修项目，签订《沈阳市地铁一、二号线特许运营承包协议》。根据协议，港铁公司与沈阳市政府全资拥有的沈阳地铁集团有限公司将成立特许经营公司，双方分别持有49%及51%的权益。特许经营公司负责就沈阳地铁一、二号线进行为期30年特许经营的营运及维修，包括运营前准备工作、列车及车站运作及维修等，并收取特许经营费用。沈阳地铁一、二号线整项工程耗资约200亿元人民币。此外，沈阳市政府亦与港铁公司签署另一份有关探讨沈阳市地铁沿线及站点周边房地产的合作开发协议，当中包括位处「金廊」的物业发展项目。与此同时，沈阳市政府与港铁公司已成立联合工作小组，现正探讨未来沈阳地铁四号线及二号线北延的发展。
港铁于2010年的年报表示，由于沈阳地铁项目出现不确定性，无法履行合约，故已经从沈阳撤离并注销所有于沈阳项目相关开支。
澳门
2018年4月11日，港铁公司全资附属公司港铁 (澳门)获委托承担澳门轻轨氹仔线通车前的预备工作、首5年的营运以及基础支持维修保养的服务，合约总值571亿港元。
欧洲
港铁公司秉持以低投资额的业务发展策略，在欧洲私有化的市场上争取资金需求较低的铁路及地铁运营专营权合约。
伦敦铁路（伦敦交通局辖下）
在2007年1月，港铁公司与Laing Rail联合竞投伦敦铁路营运权，成为两个进入最后遴选进程的投标人之一，并于2007年3月提交了最佳及最后报价。在2007年6月19日，伦敦交通局向港铁公司与Laing Rail的合营公司──伦敦地上铁铁路营运有限公司（London Overground Rail Operations Ltd）授予伦敦铁路营运权（后来改名为伦敦地上铁，Overground），并于2007年11月11日成功地接管该营运权。伦敦地上铁是伦敦一个重要的铁路网络，服务伦敦西部、北部及东部，并是2012年奥运会举行期间的重要运输系统。根据这项营运权，LOROL将会在大伦敦地区经营列车服务，为期七年，并可由伦敦交通局酌情延长两年。2016年11月，营运权到期，伦敦地上铁并交给 Arriva UK 营运。
2014年7月18日，港铁公司赢得伦敦伊利沙伯线（Elizabeth line）的营运权，为期8年及可选择延长多两年。合约由2015年至伊利沙伯线通车前以临时品牌「伦敦交通局铁路」（TfL Rail）营运旧有接替路线，2022年5月24日以代号为横贯铁路（Crossrail）的基建项目中贯通伦敦市中心的新建隧道和车站投入服务起全线包含接替路线都一并改名为伊利沙伯线继续服务。
英国国家铁路
2017年3月27日，英国运输部公布，港铁公司与第一集团（FirstGroup plc，）联营First MTR South Western Trains Limited 获英国西南铁路专营权，新专营权于2017年8月20日生效，为期7年。其后英国运输部可行使选择权，延长专营权11个月，First Group及港铁各占该公司70%及30%股权
斯德哥尔摩地铁
港铁公司于2008年8月入标竞投瑞典首都斯德哥尔摩地铁的专营权，并于2009年1月21日，港铁公司获批这个全长108公里、共100个车站的铁路系统为期八年的专营权，并成立子公司MTR Stockholm AB（MTRS）。港铁公司于2009年11月2日开始营运及维修该铁路。
在这项目，港铁公司伙拍来自挪威的Mantena公司负责列车维修。
瑞典铁路运输
港铁公司也营运由斯德哥尔摩至哥特堡的城际客运铁路「MTR express」，使用X74型动车组运行。
都柏林地铁
港铁公司于2009年2月入标竞投爱尔兰首都都柏林地铁北线（Dublin Metro North）的营运专营权。
澳洲
港铁公司在澳洲的市场上亦争取到铁路运营专营权合约。
墨尔本轨道交通
港铁公司于2009年4月入标竞投澳洲墨尔本地铁的专营权。港铁公司与澳洲当地两间公司（John Holland Pty Ltd及United Group Rail Services Ltd）组成合营公司Metro Trains Melbourne（MTM），当中港铁公司拥有6成股权，而合营公司在2009年4月7日把完成好的标书递交给澳洲维多利亚州政府的交通部，于2009年6月25日获批这个专营权合约，并于2009年9月1日与维州政府正式签署专营权协议。合营公司于2009年11月30日接管墨尔本铁路的营运及维修工作，首期为期八年，并将该铁路系统命名为「Metro」。
在专营权期内，MTM会提升铁路的可靠性及安全性以及扩大网络的容量。MTM会增聘人手在网络为乘客提供服务。
墨尔本铁路网络共有15条铁路线，全长372公里，每星期有340万人次使用；为全球最为庞大的城市轨道交通系统之一。
悉尼地铁
港铁公司(60%)与John Holland(20%)与UGL Rail(20%)(与墨尔本地铁专营权相同)组成合营公司 Metro Trains Sydney，于2019年营运新开通的悉尼西北铁路线(Sydney Metro North West)，是澳洲首个全自动高速铁路系统。因为悉尼西北铁路线与悉尼市区与西南铁路线 (Sydney Metro City &amp; South West)是相连的，Metro Trains Sydney 也会于2024年营运该条线，整个系统称为悉尼地铁(Sydney Metro)。
经营特色
港铁公司的业务发展模式为「铁路和物业综合发展经营」模式。港铁公司与发展商在车站和车厂上盖及邻近地方兴建住宅及商业物业。
为了与乘客打开沟通渠道，港铁于2000年成立「地铁友礼会」，为乘客送上最新的港铁优惠和信息。两铁合并后，「地铁友礼会」改名为「港铁友礼会」，并扩展至东铁线、西铁线、马鞍山线及轻铁。直至现在，会员人数经已超过74万。
港铁公司亦通过电邮，向会员发送载有港铁最新信息的eNews，并定期进行问卷调查收集会员意见。
环保及社会责任
与中环站之间的行人隧道）
前港铁公司行政总裁周松岗称：
港铁车站艺术
自1998年起，港铁公司的「车站艺术」计划随着「机场快线艺术品计划」正式展开。除了在港铁车站及机场快线车站设备不同艺术品外，在兴建新站或翻新现有车站时，港铁公司会把艺术元素融入车站建筑之中。
鼓励健康生活
的扶手电梯及楼梯）
2005年起，港铁公司与香港业余田径总会携手举办全港最大型的竞步比赛「港铁竞步赛」，鼓励市民多做运动。除本地的参赛者之外，比赛更吸引来自世界各地的运动员参加。
港铁公司与香港的衞生署携手合作举办「多行楼梯、多点健康」活动，推广行楼梯为健康带来的好处。乘客途经车站时，会看见健康标贴张贴于站内楼梯旁。标粘贴会显示站内楼梯的级数，并列有行楼梯的好处，以鼓励乘客利用楼梯代替扶手电梯或升降机上落。
义工服务
1993年起，港铁公司的员工会利用公余时间贡献社会，自发举办及参与义工活动。多年来，义工定期为不同社群提供服务。于2005年，港铁公司更成立了义工计划，为员工提供支持。港铁公司于「铁路人‧铁路心」义工计划中共参与86个社区项目，参与义工达1,800人而受助者则有12,000人。计划的服务对象包括长者、低收入家庭、身体残障人士、智障人士及其他有需要人士。
此外，港铁公司在环保及社会责任方面所作的工夫亦包含：
（图为合并当日，车务总监麦国琛于尖东站向员工发表讲话）
公司总部
（图为港铁总部大楼的外观）
1975年成立时，地下铁路公司的临时总部曾设于中环和记大厦，直至1979年才迁入第一代永久总部—「香港地下铁路公司行政大厦」，位于九龙湾伟业街33号，总部旁另设九龙湾车厂存放列车及进行维修、清洗等工作，及1980年正式开幕的车厂上盖物业—德福广场。
1993年7月，地下铁路公司与新世界发展有限公司的附属公司签订合作协议，重新发展地下铁路公司的行政大厦原址及毗邻的地盘。
1997年，「地铁行政大厦」原址重建为德福广场第二期。当中包括31,200平方米的商场及两幢共52,500平方米的写字楼大厦。地下铁路公司保留其中一幢大厦，约33,500平方米的写字楼楼面成为地下铁路公司的新总部—「地铁大厦」，作为地下铁路公司车务和行政等工作的办公室。而在重建期间（约1994年至1997年），地下铁路公司的总办事处就部份迁到「九龙湾宏开道8号其士商业中心十七楼」办公，其后迁回原址。
随着两铁合并，香港铁路有限公司于2007年12月2日将总部改名为「港铁总部大楼」。
公司标志
香港铁路有限公司采用的标志，乃于1975年地下铁路公司创立时所设计，公司标志最初并没有颜色，只有上下两个相反半圆加上一条直线中间穿过的徽号，及旁边的「地下铁路公司」中文或英文本样，其徽号构思源自汉字的「木」字，相传地铁在五行中为金，但地在五行中为土，地铁在地下行走等同被土给克制，所以特别选用了克制土的木来作为标志以达到平衡，蕴含力量、稳健与增长的意思，象征着当时地下铁路贯通一水之隔的港岛与九龙（与此同时，亦包含抽象的地铁英文缩写「MTR」）。为配合香港社会的不断进步，这标志于1996年亦换上新面貌，除保留原有徽号外，更改为加上「地铁公司」的中英文本样，充分体现香港作为一个国际性都市的坚毅进取特质。
港铁公司为配合两铁合并，于2007年尾将公司徽号旁边的「地铁公司」的中英文本样则改为「MTR港铁」，并是沿用至今。
另外，港铁的标志与提非纳字母ⵣ高度相似，但并没有直接关系。
管治丑闻
大堂示威，抗议港铁禁止携带大型乐器及纵容水货客，对港铁执法不一表示不满
港铁声称为乘客提供优质服务，早年更以「香港人的骄傲」自居甚至作为广告标语，唯近年无论在服务质素及多项业务方面备受各界批评。在两铁合并后多项新铁路工程发生的种种问题，亦让港铁的管治失控渐见现形。
高层涉贪被免职
港铁一名负责内地事务的港铁高层2018年7月25日突然离职，并在保安人员陪同下「执包袱」离开港铁九龙湾总部办公室。消息人士向媒体透露是一名负责内地业务的高层怀疑卷入贪污调查，被港铁即时解雇。涉事港铁高层负责中国内地业务，曾参与港铁物业发展的首个内地楼盘项目—深圳龙华车辆段天颂，疑因售楼过程中涉贪而卷入贪污调查。
廉政公署表示不评论个别事件，若接获贪污投诉并获足够数据起诉，廉署会按既定进程依法跟进。
事故应对争议
广深港高铁试运事故
广深港高速铁路试营运近2个月以来事故频生，港铁被批评每次都是被传媒揭露后才承认事件。作为港铁董事局主席的马时亨要求公众相信港铁管理层，「我哋话畀你听OK就得㗎啦（我告诉你OK就行了）」。他更质疑记者和传媒不是专业人士，追问数据并无意义「畀咗（数据）你又点？（给了数据给你又如何？）」更谓港铁日日有事故如每宗都要公布「我哋管理层唔使瞓觉（我们管理层不用睡了）」。
马时亨的言论惹来公众反弹，被质疑态度傲慢。他在商业电台节目《在晴朗的一天出发》访问称因可能「天气热」、「当日没祈祷」为由解释自己的失言，但没有就失言致歉。
沙中线工程造假的应对
在沙中线连串工程问题的应对上，马时亨和一众高层应对公众的态度和透明度也大有问题。
《苹果》于2018年5月揭发红磡站沙中线月台剪钢筋事件后，港铁曾发表声明指报道以误导的标题及内容引起公众恐慌，质疑传媒报道危言耸听。到工程造假丑闻闹大，随着更多爆料曝光，港铁才承认有「剪钢筋」事件，但一直只以以「唧牙膏」式披露工程不合格记录详情，又声称未有为部分工序保留书面记录，一直避答记者问题，都对公众感觉港铁的管治崩坏。
登报声明反击公众「单方面揣测」
面对社会各界指摘，港铁在2018年6月27日于多份报章刊登声明强调工程符合质量安全，「确定有关工程符合相关规格，已批准的图则及法定要求」。并称「只不断提出单方面的指控或揣测对事件毫无帮助，亦不必要地影响公众对铁路基建的信心。」惟声明未有正面回应，会否凿开月台层石屎抽验钢筋。
对于港铁在报章刊登广告，称有人单方提控揣测会影响公众对铁路基建的信心，最初揭发事件的工程分判商中科兴业董事总经理潘焯鸿批评港铁浪费金钱，「玻璃心」且「小家」，强调中科声明都是建基于事实，并非危言耸听，认为港铁要检讨监管工程制度。潘又表示，若立法会传召他到场交代事件他定会出席，惟希望有《立法会（权力及特权）条例》保障，方可「讲得舒畅」；潘又不点名批评工程承建商礼顿「连政府都不怕」，是「香港建筑界的毒瘤」。
对于有人向田北辰报料指「见到工人扭钢筋入连续墙的螺丝帽」，被港铁质疑纯属揣测，田北辰强调消息只是「听回来」而自己不掌握任何证据，称「若有证据就一定会报警举报」。田指出若要公众挽回信心，港铁必须立即抽检钢筋。又指礼顿同样「一直未提交任何证据显示全部26万支钢筋都无剪短」，港铁反而要求公众「相信她们」，并不公道。
中科直指亲见有人剪钢港铁坦护礼顿
翌日，中科兴业董事总经理潘焯鸿于不同媒体指出，自己曾与礼顿高层视察工地时亲眼目睹剪钢筋过程，估计数目过千，并在当时拍下照片视频，指这是经过策划而非单纯前线人员施工失误造成，潘焯鸿更透露，在与承办商礼顿建筑再度签署保密协议前，曾用手机展示在工地拍摄钢筋被剪短的视频，惟被对方当场要求删除视频。其后港铁更在去年工程尾段终止中科余下工程，涉及金额约7,000万元。潘指港铁高层包括工程项目总监黄唯铭不可能不知情。
傍晚，潘亦到苹果日报直播交代细节，
潘焯鸿批评事件发生后港铁不讨论事件，但又出广告质疑别人批评片面。他又指港铁曾指「中科要胁要钱」，企图抹黑中科，「将中科在建筑界推埋死角」，潘谓港铁刊登广告声明不尽不实，驱使他站出来说清楚。有读者呼吁全港市民应罢坐港铁。
港铁非执董论坛言「你哋无资格讲」
身兼港铁非运行董事及测量师的刘炳章，于7月1日出席香港电台电视部节目《城市论坛》时，被热血公民主席、立法会议员郑松泰质问有关沙中线工程丑闻；刘反指郑「挑动矛盾」，更大声强调称「讲港铁我最有资格讲，你哋完全冇资格讲」。
肥上瘦下 高层酬金与服务质素相违背
港铁近年在铁路，物业和其他业务都以以利益最大化，管理层声称要顾存股东利益，但香港政府实际才是港铁的最大股东；另外，港铁近年全力开拓非车务收入和非本地业务，同时又压缩成本，牺牲了维修的人力物力投放，间接影响列车的安全可靠性和服务质素。可是董事和管理层提供劣质服务却可获得丰厚酬金，加上高层的言论和态度，都令外界有感港铁欠缺社会道德和责任。不少市民要求港铁管理层薪酬花红需要与服务水平挂勾，但政府作为监管机构及港铁大股东并没有理会。
工程主管内部评核试疑大规模作弊
2018年传出港铁专责维修安全的工程主管考试，可能大规模造假。有员工指，在参与内部工程主管的考试前，有考生事先获发全卷答案，现场考官亦会「提水」，确保考生答对才准交卷，相信有关做法与港铁近年人手短缺有关。有议员批评靠出猫「上位」的工程主管严重威胁前线人员的性命安全，事件更可能违法；港铁回复指，调查显示所有员工均按照指引进行考核。
高铁建设严重超支延误
港铁主导香港高铁建设，但在工作中出现了严重问题，造成了大量的超支和严重的延误，引发了香港社会的广泛不满，政府亦成立专门独立调查委员会调查该事件内情，委员会在调查报告中指出了港铁内部管理等方面存在的严重缺陷。
港铁的独立调查报告炮轰管理层传讯策略和危机处理拙劣，点名批评时任主席钱果丰未有召开紧急会议、行政总裁韦达诚欠缺「专业的怀疑态度」，以及工程总监周大沧「判断力欠佳」，最终导致三人先后黯然下台，然而却仍享有巨额酬金。
沙中线造价高昂
2012年3月，香港政府公布沙中线的以付款当日计算价格798亿港元。传媒人林芸生评论指，兴建沙中线的所须开支比起成本约100亿美元（780亿港元）的太空电梯还要高昂，造成「出中环」比「上太空」花费更多的荒谬现象。同月27日，行政长官会同行政会议批准进行沙田至中环线铁路项目，预计项目工程于同年中动工，大围至红磡段预期在2018年完成，红磡至金钟段预期在2020年竣工，
港铁公布于2017年12月5日已向政府汇报，估计沙中线项目总成本将由70827亿元，增加16501亿元至87328亿元，包括备用资金及可能上调的项目管理费而需支付的款项。加上沙中线其他项目造价8987亿，以及金钟站扩建工程超支848亿，令沙中线所涉工程总造价增至97163亿元，每公里沙中线造价逾57亿元，较广深港高铁每公里造价33亿元贵72%，
成为香港铁路史上造价最贵的工程，堪称「千亿铁路」几乎刷新高铁超支的造价。
沙中线连番被揭偷工减料
红磡站地底隧道出现渗水 需拆石屎重新施工
2018年3月中，《东方日报》揭发红磡世界殡仪馆对出的隧道自2017年8月已出现渗水问题，工程人员用近半年时间作补救措施，但成效欠佳。港铁到2018年2月发现隧道的接驳工程出问题，需凿开其中一个驳口石屎表面调查，发现内部的钢筋疑未有完全接驳好和未有灌满石屎。其后重新施工，需临时拆除一段长约10米的路轨和三处有同类问题的石屎，并重造驳口。港铁预计4月完成，并将向承建商索偿。
红磡站地底新建月台工程造假
2018年5月30日，苹果日报报道红磡站地底新建月台两幅主要墙身，涉及26万个螺丝帽接驳位近两成螺丝头破损或移位，不能与月台层的钢筋接驳扭紧，而负责承建商礼顿不但未有更换问题部件，反伪装成功接驳，安排工人将钢筋剪短。工程师指有关做法令钢筋拉力大减，影响结构承重能力，最严重可致全层倒塌。而下层南北走廊（东铁线）月台层板搭建后，连续墙的接驳位在2016年6月拍下的相片显示多处出现渗水。港铁一度否认墙身渗水，到媒体公开渗水的视频及相片后，港铁才改口承认。
港铁承认早在2015年12月已发现钢筋遭剪短及造假，但苹果日报取得的工程时间表显示，部份涉造假钢筋早在2015年9月已落石屎。港铁董事局主席马时亨拒绝回应事件。到5月31日，港铁再度发新闻稿，改口暗示在2015年12月前已得悉有「施工质量问题」，不过未有交代发现问题的确实时间、数量及位置。路政署发新闻稿指港铁有责任确保工程质量符合标准，要求港铁安排独立第三方专家进行负重荷载测试，并于一星期内提交报告。
负责层板混凝土工程的分判商中科兴业有限公司在6月1日发新闻稿，证实其前线人员曾最少两次向总承建商礼顿通报造假情况，但与承建商礼顿签有保密协议，不能披露工程内容。
港铁5月31日首次公开回应事件的新闻稿中，没有交代被剪钢筋的数量。工程总监黄唯铭6月1日出席立法会会议时，终承认在2015年8月已发现钢筋被剪痕迹，且总共发现5条钢筋被人剪短，不过仍没有交代有否其他地方出事。
港铁高层在6月6日再召开记者会，表示在5次巡查发现钢筋有问题，其中2015年12月发现有5条钢筋被剪短，但另外4次没有完整纪录，只提到「少于5支」。港铁行政总裁梁国权6月7日出席电台节目时称，按港铁监督人员的汇报和书面纪录，「推算」被剪短的钢筋少于20多支。港铁再在6月15日公开红磡站报告，将问题钢筋由不多少20多支收窄至不多于17支。
不过，田北辰于6月24日于电台节目引述消息指，涉事钢筋多达两成，即有5,000多支被剪短，建议拆除混凝土抽检钢筋。《香港01》访问了两名工程师，二人均指，如要得知钢筋「真相」，除了移除石屎，别无他法。
港铁于6月中旬向政府提交港铁沙中线红磡站的报告，但中科的证供不被公开，港铁当时指因涉及刑责问题；中科于6月26日回复传媒称已就相关事宜先后3次联系港铁采证负责人，及1次联系港铁工程总监黄唯铭，但却全数被拒听。中科继而改为联系路政署，并已初步获署方回复正处理会面要求。中科又表示，在向港铁作供期间，多次被礼顿的法律代表，以超越临时宽免保密协议范围为由打断，当时礼顿并无反对中科已作出的证供，而港铁及港铁律师亦并无任何表示。中科再次重申，既然就连礼顿代表都已当场知悉证供，港铁根本没有合理理据隐藏该等数据，而港铁律师一直保持沉默，有容许礼顿影响报告(向路政署提交)内容之嫌。
港铁回复表示，倘若任何人有数据可证明事件涉及不法行为，应立即向政府或执法部门提供数据。
苹果访问资深建筑业界人士指，铁路月台结构必须有足够承托力，港铁沙中线红磡站月台石屎层有3米厚、面积逾5,000平方米，按一立方米石屎重23吨计算，整个月台层自重就有34,500吨重。该业界人士指，一般承建商有做ISO认证，应一套严格标准施工，质疑承建商礼顿未有跟随，自从沙中线爆出钢筋被剪短丑闻后，自己亦告诫亲友日后避免前往该月台搭车。
石硖尾站墙壁被偷薄少放钢筋
2018年6月，石硖尾站月台近电梯墙身被揭发被偷薄墙身，结构墙九层钢筋的其中八层被剪走。
土瓜湾站疑以建筑废料建墙
2018年6月29日，议会阵线议员毛孟静收到一名自称土瓜湾站地盘前员工的匿名信并附有3张相片，指疑有人在该站月台其中一条楼梯，以建筑废料填充墙身。从相片可见有人将多袋建筑废料整齐堆放墙边。报料者指有驻地盘经理安排管工将垃圾放至该空间，惟他重回该处已发现墙身注满石屎。
毛孟静指她征询过专业人士意见，包括一名沙中线前工程人员，对方表示如是准备丢弃的垃圾理应不会堆放得如此整齐，而且该些废料上亦支撑着喉管，并不寻常。另一土木工程师则认为用这种方式建墙是「百份之百违规」。不过亦有专业人士认为，单凭这几张图片难以猜测是否真有人用废料填充墙身。
港铁疑串通公证行 阻挠居民申索
苹果日报报道指出，港铁为处理土瓜湾一带居民因楼宇结构索偿，所安排的第三方公证行，当每完成调查完成都会先将报告草稿电邮予港铁高层人员审批，经修改后才发给居民，有律师质疑事件涉及诈骗，有该区区议员考虑报警。
亦有居民表示，港铁开展挖掘工程后单位经常震动，屋内有墙身和地板出现裂痕。虽然港铁曾派员检查，但事后认为符合安全标准，又认为裂痕是楼宇本身老旧所致，并无证据显示与工程有关，因此拒绝负责。而当居民从媒体得知港铁疑与公证行「打龙通」后，他质疑港铁当日说谎，要求对方正视工程影响及作出赔偿。
立法会议员林卓廷批评港铁欺骗无企业道德，林港铁审批公证行文档等同港铁附属组别，「非常可耻」。林又称，如港铁和公证行明知投诉个案中的损坏是与工程有关却故意隐瞒，有串谋诈骗之嫌。
法政汇思召集人、律师文浩正表示，若港铁私下与公证行沟通并修改报告，会损害居民索偿法律权利，港铁有触犯诈骗罪之嫌。
会展站工程漏放支撑洞壁工字铁
2018年6月，立法会铁路事宜小组委员会主席田北辰表示，收到消息，会展站的承建商在挖掘工程中，未有按进程放置足够的横向支撑，被港铁发现并勒令停工，现未知工程状况如何。&lt;br&gt;
会展站工程由「礼顿—中建」联营承建，田北辰说，挖掘工程的进程是先建造两幅垂直墙身，再向下挖走墙身之间的泥土，同时会在墙身之间加上横向工字铁支撑，但在施工期间，港铁发现横向支撑物数量不足。&lt;br&gt;田北辰关注港铁高层是否得悉事件，内部沟通有否问题。
多名高层下台
运输及房屋局在8月7日公布发现港铁新近提交的图则与之前的版本，红磡站月台的钢筋连接螺丝帽减少达2000颗，事态极为严重，需要转交警方刑事调查，港府亦明言对港铁的管理失去信心，要求港铁董事局解除工程管理人员职务。同日，港铁董事局主席马时亨代表港铁向公众道歉，同时宣布多名管理层人员离职，当中包括工程总监黄唯铭，即时生效，而行政总裁梁国权则提早退休，马时亨则曾两度请辞但被留任，以于全球招聘并甄选因沙中线偷工丑闻而出缺的新管理层人选。
「可加可减机制」名存实亡
港铁一直按「可加可减机制」制定票价，原目的是借以限制公营机构加价的幅度，并要求在通缩时下调收费。但此机制的缺陷就是容易受外围经济影响而引发通胀，而导致经常加价，加上港铁每年盈余过百亿仍要加价，令市民更加不满，但港铁表示不能冻结票价，因为必须保持财政稳健，才能改善服务及维持维修水平。
到2017年7月，港铁主席马时亨出席商台节目，被主持问到港铁票价问题时表示，「（港铁）票价相比全世界不高，但永远都有人不满意」。
节省成本工序外判 保养粗疏致故障频密
港铁开业至今多年，系统严重老化（以东铁线为甚），同时港铁公司为求利润最大化，将保养工序外判节省成本，维修检查质素每下愈况。无论列车，信号系统及路轨结构等，都经常出现严重问题，最终列车班次出现严重延误，甚至发生持续数个小时的局部瘫痪。
港铁故障频繁，时至2014年8月最少每两天就会出现信号故障列车延误，港铁亦承认问题严重，更预告未来延误将会更严重，甚至有增无减。
隐瞒意外延迟通报 受运输署谴责
港铁在处理事故的态度亦屡受各界批评。早于2010年，运输署已经严正要求港铁认真对待延迟20分钟通报故障，批评港铁在警报机制、接驳巴士安排、与公众的沟通，以及员工的应变准备，都明显不足。而港铁承诺按既定机制适时向运输署通报以及跟警方、运输署，全面检讨接驳巴士的上落点安排。但时至2014年，港铁仍然依旧延迟二十分钟后通布政府。港铁出现事故时至今仍然不会向乘客公布确实原因，更由于没有确切延误通报罚则，港铁通报政府监管机构也严重延迟，更甚出现传媒追问下才公布事件。
工作外判 影响列车安全稳定和服务质素
近年港铁将更多任务作和职位外判，当中包括列车保养以及客户服务，间接影响列车安全稳定和服务质素，更招致恶性遁环。
2013年将军澳线高压电缆事故更突显港铁事故处理能力差劣，无论在交通安排，信息发布上都不能令人满意。例如当时港铁职员派发的「乘车应变锦囊」小册子列出已取消的过海隧巴692线。又如2014年9月3日，钻石山站路轨出现裂缝事例亦说明港铁误导乘客，称服务只会延误五分钟。港铁指由钻石山去黄大仙站的行车时间增加5分钟，其后才公布观塘线由调景岭至油麻地行车时间增加15至20分钟，但实际乘客延误时间远超半小时。
当出现严重故障事件时，港铁在迫不得以情况下才会安排接驳巴士疏导，每次安排仍然相当混乱。大部份市民对港铁故障安排相当不满。
站长心情差 请勿打扰
有市民向传媒提供一张照片，显示港铁深水埗站张贴了一张告示，指「站长心情极度欠佳，敬请保持安静，如没必要，请勿打扰，多谢合作」。而告示下方以站长署名和深水埗站的印章。
港铁指7月30日下午一名职员在车站控制室内张贴告示，但指该告示对象是同站职员而非乘客，告示也非张贴在站内公众地方。港铁指车站经理知悉事件后，已即时提醒该职员移除告示。
使用劣质零件致电缆爆炸
港铁于2014年初多次出现电缆绝缘体故障，被揭发使用中国制劣质零件，最终导致电缆爆炸初时港铁拒绝向传媒公布供应商详情，仅称货源是欧洲名牌，直至传媒掌握近乎确实数据，才承认零件是中国制造。
港铁于同年5月再被揭发南港岛线及高铁使用中国制造劣质螺丝，螺丝比正常拉力大减七成，澳洲当局警告使用该批螺丝对乘客构成严重安全风险，可能造成严重受伤甚至死亡的风险，呼吁立即停用并退回产品 ，港铁承认部分工程用次货螺丝，称有关螺丝已进行检验及在安装过程中抽查作测试并确认品质符合标准。
港铁购入中国制列车的质量亦被受广泛质疑。新西兰铁路公司揭发「中国北车集团」子公司生产的40辆列车含致癌石棉物料，需紧急停用。而港铁亦有向该集团另一子公司，购买22辆列车，首批2011年已投入服务。港铁事后委托独立顾问进行抽查长春轨道客车股份公司生产列车，最后未能于被抽查之样本上发现石棉物料。
低估铁路承载力和挤迫情况
运输及房屋局呈交立法会的最新数据显示，港铁一直低估载客率。根据运输及房屋局2014年最新的标准计算，现时将军澳线，东铁线，西铁线，观塘线及荃湾线的载客率于繁忙时间早已全部达到100%，即完全没有任何剩余容量吸纳更多乘客；港岛线于繁忙时间亦达93%。而港铁过去公开的数据一直坚持，繁忙时段的载客率最高仅为七成左右。
营运深圳地铁评价不佳
由港铁负责营运的深圳地铁4号线(龙华线)亦评价不佳。由于4号线承担了龙华区来往深圳市区的运输，再加上深圳北站开通后带来了额外乘客，4号线的客流量早已超出了设计容量，又由于事故频发，早期更只有4节列车载客、班次疏落，即使现在全线改以6节车厢列车，客流量依然拥挤，令不少乘客都抱怨出行不便。4号线的乘客满意度也因此是深圳地铁各线中排名最低的。
疑恐吓媒体阻报负面新闻
2011年4月，港铁通过其合作逾7年的媒体策划广告代理浩腾媒体（OMD）向15份报章发出信件，要求传媒不得报道任何涉及港铁列车、路轨、系统和网络事故的负面新闻，会保留抽起广告的权利，又要求广告部与记者及编辑沟通。港铁其后解释，是OMD错误阐释港铁公司立场，港铁从没有企图或意图干预新闻自由，又指「经一事，长一智」，就事件向公众及传媒机构致歉。
惟南区区议会区诺轩曾与都市日报记者合作，报导港铁黄竹坑车厂上盖发展项目的消息，但因该报在港铁派发，有关对港铁的负面报导都一律被禁。这样媒体变相局限读者接收的信息，不能做到为民监察的效果。
广告过份覆盖月台 影响乘客办识
港铁尖沙咀站曾被媒体报道站内墙壁粘贴大量蓝色的巨型广告画布，只露出原本深色底色的车站名，被质疑为了「赚到尽」以广告客户为先，未有考虑颜色与本来车站的主题颜色变化过大，漠视乘客办识或引致下错车问题。
当时，尖沙咀站月台所张照的广告画布所采用蓝色与金钟站为的色系甚为相似。实际上，港铁首席建筑师文若德过去接受访问时就说过车站选色原则：会尽量与相邻车站不同和有较大的对比，重要车站会采用鲜色使乘客易于辨别，不同车站会以不同颜色代表，以帮助市民即使听不到广播也能凭颜色判断列车抵达哪一个车站。，
营利不择手段 出售乘客私隐
港铁作为八达通控股有限公司大股东，于2010年中被质疑出售八达通持卡人数据至第三方作商业用途。同年六月有市民发现八达通卡的「个人数据声明」中，列明该公司在卡主不反对下可把卡主个人数据用作推广及直销用途。八达通行政总裁前陈碧铧当时强调，八达通公司并没有向任何机构出售客户个人数据，但有参与日日赏计划的商户信诺环球前雇员指控陈碧铧说谎，指八达通早已将会员身份证等个人数据，转交信诺作电话推销保险用途。
最后私隐专员公署主动介入调查事件，陈碧铧被传召至个人数据私隐专员公署聆讯，承认旗下两间公司，用4年半时间出售197万客户数据作推销用途并获利4400万元，占期间总收入31%。
对意外伤者凉薄拒负责不赔偿
5名在2017年港铁纵火案中受伤的市民向港铁及保险公司索偿无果，有伤者需住院半年，手脚等身体多处造成残障，又失去工作能力，更指港铁「除送上水果篮外便再无下文」，更有指第三者责任保险不会为索偿的伤者赔偿一分一毫。港铁回复重申事件非港铁的责任，港铁亦是受害者，案发当天港铁迅速处理事件，而收到乘客提出书面索偿后已按既定进程即时转交保险公司处理。
纵容水客运货 苛对港客
2015年9月16日，facebook「大围联盟」群组于15日晚上载一张照片，一名身穿校服的女学生，背着一个与她身高相约的古筝，站在港铁大围站。当时，她正被3名港铁职员包围，疑因携带体积大的深啡色古筝盒，而遭截停查问。有网友指，事件发生于傍晚6时许，该名女生被指古筝太大，不获进入月台。该相片在网上疯传，网民纷纷「洗版式」抨击港铁处理乘客携带大型对象的条例时，「双重标准」及「执法不公」。他们以水货客事件作比较，指港铁对水货客宽容，却严禁乘客携带大型乐器上车。当中更有不少音乐爱好者坦言「身同感受」，促港铁检讨现行条例。有网民亦随即在facebook上载多张摄于港铁车厢的相片，相中可见乘客带着关刀、长枪及洗衣机等上车，但未见有职员阻止，有网民质疑港铁双重标准，欺善怕恶。
公司管理每况愈下 铁路安全无保证 撞车、出轨严重事故频生
当中号称的世界级服务亦具有讽刺性
奖项
港铁公司历年来获奖无数，例如于丹麦哥本哈根举行的「2008年度铁路大奖」中被评选为「亚太区最佳铁路」、于英国伦敦举行的「2009年度铁路大奖」中被评选为「亚太区最佳铁路」最后五强等。
近年奖项及殊荣清单
超大现代农业（控股）有限公司是一家在香港交易所上市的工业公司。主要业务是农产品种植及销售，牧畜繁殖及销售及副食品销售，出口贸易及经营连锁超级市场业务。总部位于中国福建福州。公司在开曼群岛注册，主席为郭浩。
历史
2004年资产净值为4,613,044,000人民币，纯利为996,754,000人民币。截至2015年6月30日止财政年度亏损超过66亿人民币。
嘉里建设有限公司，成立于1978年，由郭鹤年家族控制，现在香港交易所上市。
嘉里建设专注于豪宅市场，亦投资、经营及拥有物流及货仓业务、进行与基建有关之投资、及拥有及经营酒店业务、包括香格里拉大酒店。集团已发展成香港最庞大的货仓业主及营运商，亦正致力扩展旗下的物流业务。
公司在百慕达注册，主席为黄小抗。2004年资产净值为23,025,315,000港元，纯利为1,955,560,000港元。
投资物业
2019年5月17日以6814亿人民币元（约为7768亿港元）投得,占地约98053平方米，规划总建筑面积229613平方米。
2019年11月嘉里建设以4997亿港元购入黄竹坑道42号利美中心一篮子物业，包括地厂部分、10楼、17至18楼、22楼顶层共5层工业楼层连天台、19个车位、11楼隔火层、地下大堂及外墙以总楼面约538万方呎计，呎价约9293元。
住宅项目
屯门满名山
地皮位于青山公路第48区，占地约7228万方呎，可建楼面面积9396万方呎，呎价2915元，项目需兴建不少于1100个住宅单位。
2013年3月13日以1168788亿元，批予嘉里建设旗下Classic Gold Holdings Limited。地皮的地盘面积约为24,077平方米，楼面呎价约10,233元成功投得何文田常乐街与常盛街交界、九龙内地段第11227号的住宅用地，
九龙城
获批契约修订，作住宅乙类地盘发展，但需补价37988亿元，以补价所涉及60,870方呎计算，每呎补地价6,241元
笔架山
新九龙内地段第6532号，以23亿8,980万元批予嘉里建设旗下的NMC 8 Limited。地皮的地盘面积约为10,642平方米，楼面呎20,500元
长沙湾嘉里鸿基货仓和润发仓库重建项目
黄竹坑站第二期发展项目
西环山道83号
黄竹坑站第四期发展项目
苏州工业园区开发的项目中海国际社区二区双子座
中国海外发展有限公司是中国海外集团旗下的地产开发商。主要业务是物业发展及投资、楼宇及土木建筑、基础工程及项目管理、房地产代理及管理，以及基建项目投资。其内地子公司中海企业发展集团以「中海地产」品牌在中国内地进行房地产开发，在业界有「利润王」之称。
历史
1979年，公司前身中国海外工程有限公司在香港成立。
1985年，进军房地产领域，首个项目为香港大埔4号地块的「海宝花园」。至1987年，拥有超过10个小型地产项目，均以「海」打头：海峦苑、海祥大厦、海兴大厦、海荣大厦、海景山庄、海茂苑、海景花园、海景豪园等。
1992年，中国海外集团将旗下房地产业务以「中国海外发展」在香港联交所上市。
2002年，业务重心向中国内地转移，以「中海地产」品牌在国内大中城市进行地产开发。
2015年，中国海外发展将物业管理业务分拆，以中海物业在港交所上市。
交易明细
2007年11月7日，恒指服务公司宣布中国海外加入恒指成份股（蓝筹股），2007年12月10日起生效。
2011年4月中国海外发展以292888亿元向经营保龄球场生意的杨家树、李旺卿及杨敏健购入西贡「飞鹅山道5号」，地盘32077方呎，总楼面20862方呎，
鸭脷洲海旁道及鸭脷洲径交界豪宅项目，屋宇署批准兴建2幢32层高分层豪宅，另有2层地库，总楼面大约23万方呎。该项目已命名为南区左岸，已于2016年第四季落成。
2013年6月，中国海外发展以227亿元投得启德第1H区1号港人港地，物业位于沐宁街2号，以可建最高楼面约418243平方呎计，平均楼面呎价5428元，物业命名为启德1号（I），由两座高座及6座低座大楼组成，提供545伙单位，预计2017年10月底入伙。
2013年6月中国海外发展以227亿元投得启德第1H区2号港人港地，物业位于沐宁街8号，以可建最高楼面约462053平方呎计，平均楼面呎价4913元，物业命名为启德一号（II），由两座高座及7座低座大楼组成，提供624伙单位，预计2017年10月底入伙。
西营盘第三街重建项目：为市建局重建项目，于2010年9月中标。该项目已命名为星钻，预计于2016年中入伙。
土瓜湾北帝街重建项目：为市建局重建项目，于2012年6月中标。该项目已命名为喜点，预计于2016年中入伙。
2015年3月24日，中国海外发展将以代价1824亿元人民币（2309亿港元）购入母公司内地多个省市以至英国伦敦物业，总建筑楼面约1,091万平米。，并承担目标集团欠付中建股份的31993亿元人民币（40497亿港元）股东贷款，总额相当于33817亿元人民币（42806亿港元）。就有关项目注入，中建股份同意按每股2538元，即中海外前日收市价229元溢价108%，认购1686亿股中海外新股，占已扩大后股本171%，认购代价42806亿元，约相当于代价及中建股份股东贷款总额；完成有关认购后，母公司的持股将由逾53%，上调到6118%。
2015年7月初，当时控股公司中国海外发展有限公司正计划分拆香港和中国内地经营物业管理行业（中海物业集团有限公司）上市，预计上市日期为2015年10月23日，由于该计划不涉及发行新股配售和集资，因此只作为介绍形式上市。
2016年2月12日，由中国海外、中国建筑国际各占80%和20%股份的国万投资有限公司以213亿港元投得大埔市地段第221号大埔荔枝山山塘路地皮，楼面呎价仅1,848元，低于市场估值42%。地皮地盘面积约405760平方呎，最高可建楼面面积约1152824平方呎，项目将会兴建17幢10至19层高的分层住宅大厦，总楼面达1149万平方呎，提供1620伙。
2016年3月11日，中国海外以310亿元（人民币．下同）（约3708亿港元），收购中信集团住宅地产业务，目标公司之物业组合包括121个物业项目，截至2016年4月30日，总建筑面积约为3,155万平方米，总权益建筑面积约2,352万平方米（合营项目按持股比例计算），分布于中国内地25个主要经济地区城市，代价支付包括中国海外将向中信卖方以每股2713港元发行约1096亿股，占公司现有已发行股本的1111%，并占扩大股本后的10%。交易完成后，中信将成为中国海外第二大股东。
2017年5月26日，港铁公司公布，由信和置业、嘉华国际及中国海外发展各占3333%股份的宏键有限公司以833亿元，即每呎楼面地价6735元投得西铁或屯马线锦上路站1期项目，项目地皮面积约448,719平方呎，可建楼面面积达1,236,741平方呎，项目将分二期发展期提供2113个住宅单位，第一及第二期住宅单位数目分别约有1423个及690个，预期于2021至24年取得「入伙纸」及「满意纸」。
2018年9月3日，大埔市地段第221号大埔荔枝山山塘路地皮已命名为天钻，合共1,620伙，项目预计2021年1月份落成。
2018年12月27日，中国海外以8033亿港元投得启德4B区2号地皮，楼面呎价13,523元。地皮地盘面积约974万平方呎，最高可建楼面面积约594万平方呎。
2019年3月27日，地政总署公布会德丰地产新世界发展恒基地产及中国海外发展30%合组的财团Infinite Sun Limited以9893亿港元夺得启德第4B区1号地盘的新九龙内地段第6576号的用地，地盘面积约103,151平方呎，以最高楼面面积为722,060平方呎计，平均楼面呎价约13,702元。
2019年5月7日，地政总署公布会德丰地产，新世界发展，恒基地产及中国海外发展占股18%及华懋集团合组的财团Marble Edge Investments Limited以1259亿元投得启德第4C区2号商住临海地皮的新九龙内地段第6552号用地，地盘面积约105110平方呎，以最高楼面面积为641168平方呎计，平均楼面呎价约19636元，成为启德最贵地皮。
2019年7月24日，地政总署公布会德丰地产，中国海外发展及嘉华国际的合资公司星龙香港投资有限公司以127398亿元投得启德第4A区1号地盘的（新九龙内地段第6577号用地），地盘面积约176,368平方呎，以最高楼面面积为1075831平方呎计，平均楼面呎价约11842元，发展商需负责兴建安老院舍同幼儿中心等社会福利设施，有关面积将纳入总楼面；发展商亦要修建海滨长廊。
2019年11月13日，地政总署公布九龙仓，中国海外发展，及嘉华国际，恒基地产的合资公司瑞建控股有限公司以159529亿元投得启德第4A区2号地盘的（新九龙内地段第6554号用地），地盘面积约197552平方呎，以最高楼面面积为1205062平方呎计，平均楼面呎价约13238元，发展商需预留8%楼面面积作社福用途。
子公司
2010年正式完成收购蚬壳电器，并易名为中国海外宏洋集团；而中国海外宏洋集团主要持有及发展中国内地地产项目，包括广州中信广场。
外部链接
&lt;/noinclude&gt;
大众食品控股有限公司，简称大众食品控股，以及大众食品（People's Food Holdings Limited，、前港交所上市编号：0708），于1994年由明金星（主席）创立，名为「临沂新程金锣肉类制品有限公司」。由是一家在香港交易所前上市的工业公司。
大众食品总部位于山东临沂，公司在百慕达注册。主要业务是生产「金锣」品牌肉制品。2004年资产净值为RMB 3,208,645,000 ，纯利为RMB 616,243,000 。
2014年有传闻报道有国资背景的“中粮集团”与明金星洽谈收购事宜，尚不明朗。2014年8月，公司在没有任何公告的情况下，默默成立了新公司，集团公司更名为临沂金锣投资有限公司，子公司更名为临沂金锣文瑞食品有限公司。
青衣城的佐丹奴分店
佐丹奴（Giordano，）是一家在香港交易所上市的综合企业公司，在百慕达注册，主要业务是零售及分销时装，现任董事会主席兼行政总裁为刘国权。
历史
1981年由黎智英于香港创办，以崭新服务理念，例如不需理由任意试穿及更换，大受欢迎。
1989年，中国六四事件期间，黎智英通过佐丹奴捐赠了20万件恤衫给参加游行的香港民众支持民运。1990年，黎智英称因其于六四事件的致力参与，特别在一封公开信尖锐批评国务院总理李鹏之后，佐丹奴所有位于内地的业务遭北京政府封杀，黎智英被逼将手头上所有股份全部低价售出，自此专注发展传媒事业，后逐渐把持有的佐丹奴股票出售，在2000年已经完全退出佐丹奴集团。
2012年5月6日，佐丹奴决定大举进军中东市场，宣布与ETA签订谅解备忘录，增持佐丹奴中东公司（包括杜拜、KSA、阿曼、科威特及多哈公司）之权益，持股由20%增至84%，涉资6700万美元，即约52亿港元。
ETA将出售佐丹奴澳洲公司734%，作价为15万美元，于完成收购后，佐丹奴持有的权益将由目前的86%增至9334%。
2017年，佐丹奴于中国的实现销售额2618亿港元，集团表示其利润上涨因其商品组合得宜，定价合当，是个中因素之一。
品牌
佐丹奴公司有以下旗下品牌：
中泛控股，前名和记港陆，是一家在香港交易所上市的综合企业公司，由「玩具博士」陆宗霖创办。于2002年后隶属于和记黄埔。集团在中国内地拥有物业投资，并赚取租金收入。公司在百慕达注册，2014年8月被泛海控股收购，现主席为韩晓生，副主席为秦定国及郑东，副总裁为武晨及张克明，财务总监为刘国升
2008年，和记港陆作以4438亿元人民币向亚太置地出售世纪商贸广场，获利21亿港元。
2014年8月，和黄宣布以总代价最多3829亿元向泛海控股出售和记港陆7136%股权。
2015年8月5日，中泛控股通过旗下中泛房地产开发第三有限公司，以39亿美元（约3042亿港元），收购位于美国纽约31及13号地段72区1段（根据数据显示，地址位于80 South Street, New York, the US及163 Front Street, New York, the US）。
2015年8月20日，中泛控股以215亿美元向泛海控股子公司泛海建设国际投资有限公司收购美国洛杉矶土地。同日，中泛控股以3709万美元收购中国泛海电力有限公司，中国泛海电力持有位于印尼棉兰电厂项目60%股权。
2019年9月23日，中泛控股公布，全资附属获OCM Harbour Investments Pte Ltd提供为期三年贷款融资，总额为14亿美元(相当于约1098亿港元)。惟须待融资协议所载先决条件达成后方可作实。该贷款将用作借款人及其附属公司的一般企业及营运资金用途及偿还贷款。
业务
和记港陆经营以下有关业务：
胜狮货柜企业有限公司是一家在香港交易所上市的工业公司。主要业务是货柜制造、货柜场、码头及中流作业等。公司在香港注册，主席为张松声。2004年资产净值为USD 180,737,000 ，纯利为USD 39,636,000 。
因全球供应链扰乱导致集装箱需求增长以及干集装箱平均售价上升，2021年度集团盈利不少于18亿美元，按年增383倍。由于经营业绩明显改善，销售量及毛利率均有显著上升；定制集装箱之需求亦录得迅速增长，特别是有关可再生能源及环保的定制集装箱，为此利润改善的部分因素。
美丽华酒店企业有限公司（简称「美丽华集团」，港交所：71 ）创立于1957年，总部设于香港。集团拥有多个特色服务业品牌，旗下业务发展多元化， 包括时尚酒店及服务式公寓、物业租赁、餐饮及旅游，遍布香港及中国。美丽华集团于1970年正式在香港上市（股份代号：71），是恒基兆业地产集团成员。
2010年11月28日斥资6400万元购入赤柱大街88号
楼面1589方呎计，平均每平方呎楼面成交价40,277元
获城规会批准可重建为一幢10层、总楼面约8355方呎小型酒店，提供不超过6个客房，不过需要修改地契及补地价。
信利国际有限公司 ()是一家在香港交易所上市的工业公司。在1980年代由林伟华创立。主要业务是制造及销售液晶体显示器，电子消费产品，包括计算机、传呼机、MP3播放机及电子零件。公司在香港注册，主席为 林伟华。
深圳投控湾区发展有限公司（港交所：&amp;sc_lang=zh-HK 0737 、港交所：80737 （人民币结算），；Shenzhen Investment Holdings Bay Area Development Company Limited）是一家在香港交易所上市的工业公司。主要业务是于中国倡议、推动、发展及经营策略性重点高速公路、隧道、桥梁及相关基建项目。公司在开曼群岛注册，主席为刘征宇，前身为「合和公路基建有限公司」。
现时合和公路基建营运的收费高速公路包括广深高速公路和广珠西线高速公路一期及二期。另外，合和公路基建于2007年9月以17255亿元人民币出售其于广州东南西环高速公路项目的45%权益予广州市通达高速公路有限公司。
而在2012年10月24日，合和公路基建计划以每股315元至329元人民币，配售12亿股新股，集资2205亿元至2303亿元人民币，成为汇贤产业信托后，再有企业以双币方式上市。
2017年12月29日合和实业以每股48港元出售所持合和基建全部约2055亿股，占已发行股本约6669%，涉资9865亿港元，深圳市投资控股有限公司自此控股基建。
创维集团有限公司（英文Skyworth，简称创维）创立于1988年，总部设立于中国广东省深圳市高新科技园区，是一家跨越粤港两地，生产消费类电子、网络及通讯产品的大型高科技上市公司。香港创维数码控股有限公司于2000年在香港主板上市，目前是创维集团的控股公司。目前创维是中国三大彩电龙头企业之一
，同时也是世界十大彩电品牌，特别是室外大型显示屏，是中国显示行业领导品牌和中国电子百强第十三位。此外还有照明灯具事业和汽车制造产业。2021创维集团实现营收50928亿元。2021年8月31日，创维集团专利突破1万件，已申请专利10510余件。
历史
1988年，创维由黄宏生等人在香港成立，初期名称为“创维实业有限公司”。
1990年，创维实业有限公司正式进入制造业，初期主要代工生产电视遥控器。
1991年，收购濒临破产的香港电视机制造商———讯科国际的大多数生产性资产，开始转向彩色电视机生产。
1992年，“创维集团”于香港成立，主要生产彩电、VCD、DVD，家庭影院和卫星接收机等产品。
1994年，创维与深圳RGB电子公司成功合资，成立“创维-RGB电子公司”，此后逐步将经营重心向深圳转移。
1996年，“创维电子城”在深圳宝安区建成投产
2000年4月，“创维数码控股有限公司”在香港主板上市，集资数十亿港元。
2002年，创维投入使用位于深圳市高新科技园区的“创维（深圳）数字研究中心”。
2003年，“SKYWORTH创维”荣获广东省著名商标称号，董事局主席黄宏生当选中华人民共和国全国政协委员及深圳民营企业家商会会长，同年，创维集团总部迁入位于广东深圳高新科技园新落成的创维大厦。
2006年，启用全新企业标识。同年10月14日，创维和朗科科技共同出资成立深圳酷开网络科技有限公司。
2007-01-18 创维集团建设发展有限公司成立，是集团投资地产的子公司，据其2021中报数据，截至今年上半年，其物业存货约5011亿元。其地产投资分布在广州、宁波、滁州、呼和浩特等城市
2009年，创维与第26届世界大学生运动会组委会执行局签约，成为中国2011年深圳第26届世界大学生夏季运动会的赞助商。
2013年，创维推出全球首款智能电视操作系统“天赐系统”（随后2014年8月并更名为酷开系统）。
2014年9月，创维集团下属的创维数字股份有限公司在深交所借壳“华润锦华”上市，并更改股票简称为“创维数字”。
2015年4月22日，创维集团与德国高端电视机品牌美兹（)
破产管理人在深圳正式签订收购合同，创维用自有资金成功收购美兹电视业务相关的资产和Metz品牌。
2015年7月，创维数字收购欧洲著名机顶盒品牌企业Strong集团，提升创维在欧洲、中亚和北非市场的占有率。
2018年4月1日，创维旗下互联网公司酷开网络剥离电视业务，专注构建开放统一的超级智能系统生态，挖掘大屏商业价值。原酷开电视产品、供应链、业务、运营、市场、服务团队全部平移集成至创维全球电商中心，实行“创维+酷开”双品牌运作。
2020年12月，创维旗下酷开网络科技有限公司更名为酷开网络科技股份有限公司。
截至目前，创维集团旗下设深圳创维-RGB电子公司、海外发展公司、数字技术公司、电器科技公司、群欣安防科技公司、液晶器件公司、酷开网络科技公司等十多家产业公司。拥有成都、南京、宜春三大物流中心；深圳石岩、深圳龙岗、广州、南京、内蒙古呼和浩特等生产基地；香港、深圳、北京、广州、南京等科研机构。
2021年9月，创维以底价559亿元竞得，楼面价5973元/平方米拿下南京溧水区永阳街道G18地块
前董事长黄宏生入狱事件
创维创始人兼前主席是身兼全国政协委员的黄宏生，其胞弟是黄培升。 2005年，黄宏生被控于2001年1月至6月间，串谋其母罗玉英，从创维数码全资子公司创维集团有限公司（现更名创维电视控股有限公司）账户中盗窃公司50万港元。另外，黄宏生兄弟被控于2000年11月至2004年10月间，与罗玉英及王鹏串谋，以“佣金”名义分别窃取上市公司资金221万港元和4838万港元。2006年7月13日，黄宏生及其胞弟、创维前运行董事黄培升因串谋盗窃及串谋诈骗创维数码5000多万港元被判监禁6年。其出狱后蜕变为懂得授权、善于分享、热衷激励的“隐形船长”，在2010年出狱首年，在全球彩电产量萎缩的背景下，创维营收却大涨34%；他更是通过分拆上市的方式（拟将一家上市公司拆为三家），给团队内部创业的机会。2013年8月6日，外界猜测良久的创维机顶盒业务分拆上市方案出炉 。
被指监控用户
2021年4月，有用户称创维电视依托一项名为“勾正数据服务”的后台服务，实时搜集用户联网设备敏感数据，包括主机名称、IP地址、MAC地址、网络延迟时间等资料，乃至附近的Wi-Fi名称，所有数据全部上传至勾正数据的服务器。勾正数据在声明中表示所采集用户数据的相关信息用于研究收视情况，并非收集用户隐私，而创维电视则宣布终止与勾正的合作关系。
理文造纸有限公司是一家在香港交易所上市的工业公司，主要业务为生产牛皮箱板纸及瓦楞芯纸，分别在广东东莞、江苏常熟、重庆永川、广西梧州、越南设有造纸厂。公司在1994年成立，在2003年9月26日于香港交易所上市，当时招股定价为每股417港元。
2009年宣布1股分拆为4股。
大股东李运强通过其控制之法团持股5409%（2012年12月31日）。
理文造纸在开曼群岛注册，董事会主席为李文俊，行政总裁为李文斌。
理文造纸在2005年赞助香港贺岁烟花汇演，10年后再次赞助
招标，是一种市场交易、搜索商业对象的行为。 招标指在一定范围内公开货物、工程或服务、采购的条件和要求，并接受或邀请众多投标人参加竞标，之后按照原先规定进程，从中选择合适者作为交易对象。
中国平安保险（集团）股份有限公司（Ping An Insurance (Group) Company of China, Ltd），简称中国平安、平安保险、平保（港交所：2318、上交所：601318、OTCBB：PNGAY），是中国第一家股份制保险企业，于1988年在深圳蛇口成立，于2004年香港交易所及2007年上海证券交易所上市。该公司业务主要涉及保险、银行、资产管理、金融服务、医疗健康、汽车服务、智能城市等领域，现任董事长为马明哲，现任联席首席运行官分别为谢永林、陈心颖及姚波。2022年8月3日发布的《财富》世界500强排行榜中，其以1,996亿百万美元营业收入位列第25位，全球金融企业排名第4位。 平安同时名列《福布斯》2022年「全球上市公司2000强」排行榜第17位。平安亦名列2021年BrandZ TM 全球品牌价值100强第49位，首次位列全球银行保险机构第1位，六度蝉联全球保险品牌第1位。
业务
集团业务分主要分为以下部分：
ESG
2019年，中国平安正式签署联合国支持的「负责任投资原则」（UN Supported Principles for Responsible Investment），成为中国第一家加入该组织的资产所有者。PRI首席运行官菲奥娜·雷诺兹曾表示「希望平安的领导力能够鼓励中国的其他公司将ESG因素集成到他们的投资策略中」。 平安亦是中国大陆首家签署《联合国环境规划署金融倡议可持续保险原则》的公司。
同年，平安被正式纳入2019道琼斯可持续发展新兴市场指数(Dow Jones Sustainability Emerging Markets Index)中国平安签署国内首家《可持续保险原则》，成为中国大陆首个入围保险企业。
中国平安签署国内首家《可持续保险原则》
平安在国际专业财经杂志《财资(The Asset)》举办的2020年「三星可持续投资大奖 - 机构投资者，ETF及资产服务」中，获评为「年度中国最佳ESG投资机构-保险类」企业。
2021年，中国平安宣布于2030年实现运营“碳中和”，未来将聚焦绿色运营、绿色资产、绿色保险、绿色公益和绿色科技五大层面明确方向助力社会绿色转型，推动可持续发展。同时承诺绿色投资规模和绿色贷款余额每年增速不低于20%，绿色保险保费每年增速不低于70%。到2025年，力争实现绿色投资+贷款规模4000亿元、绿色保险保费总额2500亿元的总体目标。。
在第52个世界地球日，中国平安发表「致地球的一封信」，提出全方位升级绿色金融行动，量化制定绿色金融工作目标，通过金融产品和服务创新，响应中国2060年碳中和目标，参与应对全球气候变化，助力社会可持续发展。
2021年7月, 河南暴雨灾情后, 中国平安启动I级重大灾害回应, 迅速开展相关客户排查, 创建绿色理赔信道流程, 预估总体赔付金额超过人民币10亿元。
2021年12月，中国平安发布《中国平安2020气候风险管理报告》，依据金融稳定理事会气候相关财务信息披露工作组(TCFD)的框架建议，从治理、战略、风险管理、指针和目标四个方面入手，披露了平安绿色金融战略布局。
2022年，中国平安和其他数十家中国公司宣布帮助制定了中国首个ESG披露标准，即中国企业改革与发展协会（CERDS）发布的「企业ESG披露指南」，并于2022年6月1日起正式实施 。同年，平安获Sustainalytics 评为「低风险」评级，并给予183的评分，为中国内地保险企业中的最佳评分 ; 获标普全球纳入《2022年可持续发展年鉴》，该《年鉴》为最具国际影响力与公信力的企业可持续发展评估之一 。
平安银行个人碳账户平台「低碳家园」正式上线，由平安银行携手中国银联、上海环境能源交易所共同打造，是中国首个覆盖银联信用卡和扣账卡的碳账户。
中国平安升级其专利环境、社会和治理评价体系，体系涵盖21个一级指针、90个二级指针及278个数据点，用以量度37个行业的ESG风险及机遇，该体系亦会基于新闻及舆论情况作即时调整 。
同年7月，平安观澜3号数据中心建设完成并投入试运，是中国内地首个电力使用效率（PUE）低于125的金融数据中心，预计每年可节约用电约1,050万度，减少6,406吨二氧化碳排放 。
同年12月，MSCI上调中国平安ESG评级至A级，位列综合保险业亚太地区第一位。
外资控股
2010年，汇丰通过旗下的全资子公司控制了平安的大部分流通股份，其中汇丰保险控股有限公司持有81%股份、香港上海汇丰银行有限公司持有803%股份，在2010年中期报告中，汇丰共持1623%股份。
2011年3月14日，郑裕彤及旗下周大福代理人有限公司附属公司金骏有限公司斥资19448亿港元场外购入平安保险272亿H股，每股均价715港元占平安保险扩大已发行 H股 87%或全部扩大已发行股份 344%。
2011年8月23日，郑裕彤及旗下周大福代理人有限公司附属公司金骏有限公司以每股6225元，在场外沽出076亿股平保，以及在8月24日，以每股6485元，沽出一亿二千万股，两次合共减持一亿九千六百万股，套现一百二十五亿元。而周大福所持平保只余下7600万股。
2012年12月5日汇丰控股以72736亿元港币（约合583亿人民币）（约9385亿美元），相当于每股59元全部出售所持的1557%中国平安股权给泰国正大集团旗下的间接全资附属公司：同盈贸易有限公司、隆福集团有限公司、商发控股有限公司及易盛发展有限公司。
此外，正大集团通过全资子公司林芝正大环球投资有限公司持有工布江达江南实业发展有限公司8981%的股份，工布江达江南实业发展有限公司持有平安集团176%股份。
2013年11月8日，正大集团对所持中国平安H股进行持股调整：易盛发展有限公司将其持有的022%股份转让给隆福集团。易盛发展有限公司将其持有平保股份中的382%以结构性金融衍生产品封顶保底式期权交易（Collar）的融资方式，作为抵押品交付与瑞银。
截至2022年6月30日，卜蜂集团是平安的最大投资者，持有其68%的股份。
业务发展
根据报告，在2006年12月31日，平安保险拥有的人寿保险顾客已经超过了3200万名和财产保险客户超过了790万名。
2007年11月29日旗下平安人寿入股荷兰-比利时金融服务公司富通集团Fortis 9501万股股份，占其已发行股本的418％，成为这家荷兰－比利时合资公司的最大单一股东。这一交易的总金额为181亿欧元（约合209亿港元，27亿美元）。
2008年10月3日荷兰政府于将以168亿欧元收购富通集团（Fortis NV）在荷兰的业务。
2009年6月12日平安保险旗下平安寿险拟以每股1826元人民币（下同），购入深圳发展银行至少370亿但不超过585亿股新股 最多支付现金106821亿元；同时又以平保299亿新H股（相当于已发行股本的4%），向深发展第一大股东新桥（NEWBRIDGE）购入深发展52亿股股份占1676%的股权。 平保是次交易作价最多达到22131亿元。
2010年5月13日新桥亚洲通过高盛配售16亿股平保H股，占平保已发行股份209%，占H股已发行股份约56%。新桥减持平保之后，在H股的持股量将下降至486%
每股配售价：606元，套现9696亿元。
交易完成后，平安将持有深圳发展银行达30%股权，成为最大股东。
2004年新桥以约1235亿元收购约348亿股深发展股份，约占深圳发展银行总股本的1789%，成为其第一大股东。
其后又以每股成本约 19元人民币，在两次深发展股改中行使认股权证，以及2008年中期送红股，新桥所持股份遂增至约52亿股并维持至今。以此计算，新桥入股深发展总成本约 4503亿元人民币（5133亿港元）。
2010年8月24日，平安保险宣布引入南非最大健康保险公司Discovery，作为旗下平安健康保险的战略投资者。双方签署合作协定，根据协定，Discovery初期将购买平安健康险20％股份。2013年2月22日，Discovery增加对中国平安旗下子公司平安健康保险股份有限公司的持股，持股比例将从现在的20%增加至2499%。
2010年9月1日，平安保险以持有的平安银行9075%或7825亿股股份，加上现金2692亿元，足额认购深圳发展银行非公开发行股份，涉及1639亿股，认购价为每股1775元，令平安保险持有深圳发展银行由目前2999%增至约5239%。是次交易总对价为291亿元（人民币）。
2011年8月16日，深圳发展银行将以每股人民币1681元的认购价向中国平安定向增发至多119亿股股票﹐占扩大后发行股本的19%，最多融资人民币200亿元（合31亿美元）。预计交易后，平保于深发行的持股量介乎5944%至6136%。
2011年11月8日，平安信托旗下上海平浦投资有限公司成功以不低于人民币51亿元（合797亿美元）收购化妆品生产商上海家化（集团）有限公司（Shanghai Jahwa Group）100%股权。上海家化拥有LVMH旗下中国Sephora化妆品连锁店19%的股权。另外，上海家化的佰草集（Herborist）系列化妆品也在Sephora海外化妆品连锁店有售。上海家化（集团）有限公司持有上市公司上海家化(600315) 28382%股权。
2012年1月19日，平安保险通过旗下深圳发展银行吸收平安银行的方式，将旗下该两间银行合并，深发展将以每股337元（人民币，下同）购入平安银行股东股份，料是次吸收涉资269亿元。深发展吸收平安银行后，将改名为平安银行股份，并完全继承平安银行的资产。
2012年9月22日，中国平安宣布投资170亿元建设总建筑面积达150万平方米的浙江桐乡养生养老地产项目。
2013年7月7日，中国平安斥资26亿英镑（约30亿港元）购买英国伦敦的地标劳埃德大厦（The Lloyd's Building），是中国保险业首次投资于英国伦敦的房地产市场。
2013年8月26日，平安不动产以1097亿元竞得广州国际金融城AT0909035地块。
2013年9月8日，中国平安保险斥148亿元（人民币，下同），增持旗下平安银行非公开发行的1323384994亿股A股，每股认购价为1117元，以补充平安银行的内核资本，满足资本监管要求。目前，平保直接及间接持有平安银行约4294亿股股票，约占平安银行总股本的5238%。发行完成后平保持股比例将由目前的5238%提高至59%。
2014年11月30日，中国平安保险已根据股东大会授权，以每股62元，向不超过10名股东，定向增发594亿股H股股票，预计集资约368315亿港元，成为港交所历史上最大规模的定向增发之一。
2015年4月1日，碧桂园以毎股2816元向中国平安附属平安人寿保险配售2236亿股新股，相当于扩大股本990%，成为碧桂园第二大股东。集资总额6297亿元，所得净额6295亿元，拟用作公司发展及一般营运资金，交易完成后，碧桂园大股东兼主席杨国强女儿杨惠妍持股量将由5948%降至536%。
2016年5月，平安好医生完成5亿美元A轮融资，估值达到30亿美元 。同年6月，平安以16亿美元收购汽车之家474%的股权，正式成为汽车之家最大股东。
2018年，中国平安旗下三家科技公司平安好医生、平安医保科技和金融壹账通顺利完成定向私募融资，以软银愿景基金(投资平安好医生及平安医保科技)、IDG、SBI集团等境外投资者为主 。 11月，平安管理层在投资者开放日上表示，平安未来十年的科研投入预计将达1,000亿元，以巩固其在金融服务行业的领导地位 。
2018年5月，平安好医生在香港联交所主板挂牌交易，引发市场对互联网医疗新一轮关注。业内认为，作为全球AI医疗科技第一股，平安好医生的上市或将开启互联网医疗新时代 。
2019年，平安旗下平安壹账通银行获香港金管局颁发虚拟银行牌照，并于6月正式开始营运 。
2019年12月，金融壹账通成功在纽约交易所上市。该公司定位为面向金融机构的商业科技云服务平台（Technology-as-a-Service），承载了平安集团金融科技输出的重要责任 。
2020年9月，平安首次全面披露医疗生态圈战略，表示将从政府、用户、服务方、支付方、科技等5个方面发力构建医疗生态闭环，抓住医疗健康市场巨大发展机遇，打造公司未来价值增长新引擎 。
2020年10月，平安集团成员公司网上财富管理平台陆金所美国纽交所上市。
2021年5月， 中国平安发布「平安臻颐年」康养品牌及首个高端产品线「颐年城」，集成集团在金融、医疗与科技领域的优势资源，布局康养产业，打造有品质的康养服务。
2021年7月，由中国平安与日本盐野义合资成立的平安盐野义有限公司正式在上海开业。平安盐野义将打造全面解决患者需求及社会难题一体化医疗健康平台，即「ＨａａＳ企业」。
2021年10月，平安银行「跨境理财通」业务于10月18日获得中国人民银行和香港金融管理局批准，成为内地首批试点机构之一。
2022年1月，中国平安旗下平安人寿参与方正集团重整投资获批，7家新方正集团主体注册成立，将进一步深化医疗健康产业战略布局、积极打造医疗健康生态圈。
2022年3月，平安旗下平安人寿推出居家养老服务，进一步推动「综合金融+医疗健康」战略升级，积极布局「金融+养老」、「金融+健康」等产业生态，以客户健康和养老新需求为导向，推出「保险+健康管理」、「保险+高端养老」、「保险+居家养老」产品和服务。
2022年7月，面向金融机构的商业科技服务提供者金融壹账通以介绍形式在港双重第一上市。
2022年8月，平安首颗对地遥感成像卫星「平安３号（又名：泰景一号０１星）」在酒泉卫星发射中心成功升空，与此前发射的物联网卫星「平安１号」、「平安２号」协同合作，将共同支持平安银行的供应链金融服务，赋能普惠金融发展。
截至2022年6月30日，集团个人客户数225亿。客均合同数稳步增长至295个，较年初增长14%。其中集团互联网用户量达66,845万人，较2021年末增长33%。 同时，得益于平安对互联网用户的高效经营，用户活跃度逐步提升，用户黏性持续增强，年活跃用户量达342亿。
中国人民财产保险股份有限公司，简称「人保财险」、「中财」或「财险」，原身为“中国人民保险公司的财产保险部”，是中国一家保险业上市公司。
历史
中国人民保险公司成立于1949年，后改名中国人民保险集团公司。2003年7月，中国人民保险集团新成立中国人民财产保险股份有限公司，注册资本1225598亿元人民币。公司在中华人民共和国注册，继承了原中国人民保险公司商业业务的全部资产、负债以及分支机构，经营除人身保险以外的一切保险业务。
2003年11月6日在香港交易所上市，是目前中国内地最大的非寿险公司。主要业务是为中国各界客户提供多种财险产品。它是香港中资金融股的八行五保之一。
人保财险相继成为北京2008年奥运会、2010年上海世博会保险合作伙伴。2008年，人保财险保费收入突破1000亿元人民币，成为中国内地第一家年度保费突破千亿元大关的财产保险公司，进入全球财产保险业务前十强。2012年，人保财险保费收入超过1900亿元人民币，亚洲排名稳居第一，在全球单一品牌财险公司中位列第二。
2011年纯利增长五成一，赚八十亿二千七百万元人民币，不派息，业绩逊预期。营业额增加一成二，升至一千七百三十九亿元人民币。
中国财险宣布（A+H）大折让供股，将发行41817亿股H股（92998亿A股），占经扩大后H股股本99%，每10股供11股，H股供股价538元（43元人民币），较该股2013年5月20日收市价102元折让47%；连同内资股供股，将合共筹7257亿元（5763亿元人民币），供股所得将用于补充其资本金，以改善财险偿付能力。AIG 承诺供股，现时AIG持有H股3192%1213亿股H股、公司已发行股本约99%。中国人保，承诺参与子公司中国财险的供股计划，将出资3999亿元人民币，全数认购获配发的近93亿股内资股供股股份。
2015年12月29日中国人民财产保险斥资不多于257亿人民币现金，向德意志银行购买877亿股、萨尔．奥彭海姆购买267亿股及德银卢森堡购买922亿股，合共购买2136亿股华夏银行股份，约占华夏银行全部已发行股份的1999%
2017年1月9日，前董事长王银成被有关部门带走调查，3月6日人保财险公司宣布王银成辞去公司董事长职务，2018年5月24日，王银成被一审判处有期徒刑十一年。
和记亚洲电讯有限公司，前称和记电讯国际有限公司（简称和电国际，前上市代号：2332及HTX），是长江和记实业有限公司旗下的前上市企业，主要业务是提供流动电话及固网电讯服务，在泰国、斯里兰卡、印尼及越南等地区经营。
和电国际在开曼群岛注册，于2004年10月在香港交易所及以美国存托股份（ADS）形式在纽约证券交易所上市。
年表
2009年5月8日，和电国际以介绍形式分拆和记电讯香港控股上市，和电国际的香港及澳门业务转移至该公司。
2010年1月，和记黄埔提出以每股22港元私有化和电国际。和电国际于2010年5月25日及纽约时间2010年6月4日分别自港交所和纽约证券交易所除牌。和电国际于同年10月11日易名为和记亚洲电讯。
2012年6月5日，和记亚洲电讯旗下子公司Persall Pte Ltd，与李嘉诚基金会旗下的Kelburgh Pte Ltd，已签订有条件协议，分别向Suny Electronics Ltd购入Partner大股东Scailex Corp50%及25%股权，涉资125亿美元，负债76亿美元（59亿港元）连同债务总代价为69亿港元，由于Scailex现持有Partner445%控股权。Scailex将以1亿美元，把在当地进口及销售三星流动电话业务，售予Suny，让Scailex因此获1亿美元现金，以进一步减债。
2012年9月4日，和记黄埔以8710万美元（约677亿港元）代价，出售印度客务中心─和记环球服务予当地上市公司Tech Mahindra。该业务主要为和黄在英国、澳洲及爱尔兰电讯业务提供客户服务，和黄又与买家签订协议，买家继续成为和黄个别业务独家服务供应商，并承诺在未来5年，使用相等于出售价10倍，约657亿港元的服务合约。
现有经营地区、业务及品牌
印尼流动电话业务
于2014年年底，印尼之活跃客户总人数较去年增加35%至约4,310万名
越南流动电话业务
于2014年年底，越南之活跃客户总人数较去年减少5%至约940万名
斯里兰卡流动电话业务
于2014年年底，斯里兰卡之活跃客户总人数较去年增加25%至约200万名
过往经营地区、业务及品牌
以色列流动电话及固网业务
和电国际持有约5146%股权的附属公司Partner Communications Company Ltd于1997年成立，并于1999年1月以「Orange」为品牌推出GSM流动电讯服务，于2004年12月推出3G流动视像通信与互联网服务，于2006年3月推出HSDPA (35G)流动宽带服务，下载速度达 72 Mbps 。
2007年年月，Partner Communications 的全资公司获以色列电讯局颁发为期二十年的当地固网通信服务牌照，并于2008年9月开始试行推出固网和互联网供应商服务。
Partner Communications 在特拉维夫证券交易所（股份代号：PTNR）和美国纳斯达克交易所（NASDAQ:PTNR）上市，并获列入纳斯达克全球经精选市场级别。
2009年8月，和电国际出售 Partner Communications 的百分之五十一股权予Scailex，总作价约为一百零七亿港元。
巴拉圭业务
2000年，和记电讯进军巴拉圭，并于2005年把巴拉圭业务出售。
印度业务
1992年和记电讯与Max合资进军印度电讯市场，并于1995年11月推出「Hutchison Max」。2005年，Hutchison Essar Telecom Limited、Hutchison Telecom East Limited、Fascel Limited、Hutchison Essar South Limited 与 Aircel Digilink India Limited 已把全数股份转让予 Hutchison Max Telecom Private Limited，并易名为 Hutchison Essar Limited ，该公司因而成为印度最大的流动电讯经营商之一，履盖十三个牌照地区。2006年1月，Hutchison Essar收购 BPL Mobile Cellular Limited (「BPL Cellular」)，其经营版图增加马哈拉施特拉邦 (Maharashtra) ，泰米尔纳德邦 (Tamil Nadu) 与喀拉拉邦 (Kerala)三个牌照区，至共十六个牌照区。同年10月，Hutchison Essar收购 Essar Group 的 Essar Spacetel Limited，经营版覆盖全印度当中，包括所有都会及A区牌照区。
2007年，和电国际出售 Hutchison Essar Limited 的百分之六十七直接及间接股权及贷款权益予Vodafone Group Plc ，总作价约为一百一十一亿美元或八百七十亿港元。
越南CDMA流动电话业务
和电国际旗下全资拥有附属公司Hutchison Telecommunications (Vietnam) Sa r.l（「越南和记电讯」），与Hanoi Telecom Joint Stock Company合作创建使用CDMA 2000-1X 技术的流动电话网络，并于2007年1月推出「HT Mobile」。但由于手机生产商陆续缩减为越南市场研发可显示越南语的CDMA手机，市场欠缺并且价格相宜的CDMA手机令不少消费者对HT Mobile的服务却步，越南和记电讯于2008年3月获当地政府批准将旗下CDMA技术转换为GSM后，将原有客户迁移至第三方网络。
泰国流动电话业务
和电国际旗下公司和黄无线多媒体服务（HWMH）在泰国与CAT Telecom合作提供使用CDMA技术的电讯服务。2010年12月，和电国际以435万泰铢（约 112万港元）造价，出售和黄无线多媒体服务925%股权予泰国电信集团True Corp PCL，撤出泰国电讯市场。
加纳业务
1998年，和记电讯投资于加纳的Kasapa Telecom Limited。2008年，和电国际出售加纳业务。
香港
天地线（CT2）
太平洋航运集团有限公司 ()是一家在香港交易所上市的航运公司，提供干散货运输及物流支持，主要在亚太地区营运。太平洋航运首于1987年于香港成立，并以小灵便型干散货船的船东及营运商为主，期后于1996年公司被收购并进行私有化。在1998年，集团始创人重组团队及业务，公司重新成立，并命名为太平洋航运集团有限公司。太平洋航运于2004年7月14日于香港交易所上市。截止2017年，太平洋航运于全球六大洲设有12个办事处：奥克兰，大连，德班，香港（总部），伦敦，马尼拉，墨尔本，圣地亚哥，斯坦福德，里约热内卢，东京和温哥华。
大新银行集团有限公司（Dah Sing Banking Group Limited，）是大新金融集团有限公司专责在香港以「大新银行」的名义提供个人银行、商业银行及财资业务的附属机构。于1947年5月1日在香港设立，公司在香港注册，董事会主席为王守业，董事总经理兼行政总裁为王祖兴。
大新银行集团持有三间银行附属机构（大新银行有限公司、澳门商业银行股份有限公司及大新银行（中国）有限公司），通过分布于香港、澳门及中国内地合共约70间分行之分行网络提供银行及金融服务，及一所证券交易公司。
截至2019年12月31日，大新银行集团录得总资产总值2434亿港元，股东权益总值273亿港元，客户贷款总值1359亿港元，纯利达22亿港元。
历史
1947年大新银行由曾在上海成立永大银行的杨元龙家族创立。
50年代，王时新开始入股大新，至1972年，其家族成为大新银行的大股东。
1987年大新银行收购香港工商银行，并以大新金融集团有限公司（大新金融）名义于香港联合交易所上市。
1990年大新人寿保险有限公司成立。
1993年大新金融收购永安银行，其资产及业务均移往大新金融旗下之大新银行有限公司。
1994年大新银行有限公司出售于香港工商银行有限公司之40%权益予中国建设银行。
1997年大新金融与艾比国民银行和亨宝银行订立合营协议，将永安银行改为以香港为基地之合营私人银行，名为安新私人银行有限公司（DAHP）。
1998年大新银行有限公司出售于建新银行（前称香港工商银行有限公司）之另外30%权益予中国建设银行。
2000年大新金融收购艾比国民有限公司之代名人及SG Hambros （前称Hambros Bank Limited）于驻港安新私人银行之权益，遂成为大新金融集团全资附属公司。其香港私人银行业务、贷款及存款转让予大新银行有限公司。
2002年中国建设银行购入大新银行有限公司于建新银行之余下全数股权。
2004年大新金融集团进行业务重组，成立大新银行集团有限公司，并为大新银行有限公司之控股公司，于香港联合交易所上市。
2005年大新银行集团有限公司斥资936亿港元收购在港经营信贷业务的怡泰富财务（香港）。同年，大新银行有限公司以1669亿港元收购澳门商业银行以及获得澳门商业银行保险子公司96%的股权。
2007年，大新银行购入重庆银行17%的股权，重庆银行遂成为大新银行联营公司。
2010年大新银行与丰明银行（前身为安新私人银行）进行品牌集成。
2011年安基财务有限公司成立，为大新银行有限公司之附属公司及大新金融集团有限公司之成员。
2014年3月26日大新银行供股集资。大新银行建议折让3399%，每100股供12股，供股价8元，筹最多1205亿元，以增资满足资本要求。为支付大新银行供股的资金需求，其母公司大新金融（440）也折让34%，每100供13筹最多903亿元。
2017年大新金融集团向泰禾投资集团出售旗下大新人寿保险有限公司、大新保险服务有限公司及澳门人寿保险股份有限公司之全部已发行股本。
香石竹（学名："Dianthus caryophyllus"）又称剪绒花、洛阳花、康乃馨、狮头石竹、麝香石竹、大花石竹、荷兰石竹，为石竹科石竹属的植物，分布于欧洲温带以及中国大陆的福建、湖北等地，原产于地中海地区，是目前世界上应用最普遍的花卉之一。康乃馨包括许多变种与杂交种，在温室里几乎可以连续不断开花。1907年起，开始以粉红色康乃馨作为母亲节的象征，故今常被作为献给母亲的花。在2000多年前希腊语文献中就在曾提到这种花，古希腊科学家泰奥弗拉斯托斯创造了现在的属名“Dianthus”，这个词来自希腊语“神圣”（dios）和“花”（anthos）。
康乃馨是肯尼亚的主要出口种类，也是美洲哥伦比亚最大的出口花卉品种，在亚洲的日本、韩国、马来西亚等国都有大量栽培。在欧洲，德国、匈牙利、意大利、波兰、西班牙、土耳其、英国和荷兰等国栽培的规模也很大。
形态特征
康乃馨茎秆硬脆，节很大。花单生或2－3朵簇生，花萼长筒状，花瓣扇状。盆栽用品种有纯黄色、白色、紫色具白边、淡橙色、鲜红色、红色、粉红色、粉红色具橙红深条纹，以及自然矮生种的花淡紫色边缘深紫色。
品种
常见的有极爱（Adorable）系列的金块（Nugget）、雪花（Snowflake）、甜头（Sugarplum）、板球（Cricket）、标枪（ Dart）、壁画（Fresco）、小精灵（EIf）、丘比特（Cupid）、蒂斯凯特（Tisket），罗曼史（Romence）系列的CFPC梦（CFPCDream）自然矮生种， 黛安特（Dianette）系列， 阿维（Avi）自然矮生种，戴奥吉恩斯（Diogenes）大花种。
文化象征
康乃馨是中国婚礼仪式中最常用的花。
在西方国家的一些特殊场合或节日，人们会佩戴康乃馨，其中尤以母亲节和婚礼上为常见。1907年，美国的母亲节创始人安娜·贾维斯选择康乃馨为母亲节象征，原因是此花为其母最爱，同时她选择的康乃馨为白色，象征了母爱的纯洁。随着时代变迁，母亲节如果母亲尚在人世，便应佩戴红色康乃馨，反之才为白色。
奥地利和意大利等一些国家的人们在国际劳动节时会佩戴红色康乃馨，以之为社会主义和工人运动的象征。葡萄牙康乃馨革命的象征也是红色康乃馨。
圣帕特里克节时有佩戴绿色康乃馨的习俗，而又因为被爱尔兰作家、著名同性恋者奥斯卡·王尔德佩戴而出名，所以也成了同性恋的象征。在牛津大学，参加考试的学生在传统上应该佩戴康乃馨，例如第一场考试佩白色、期中考试佩粉色、期末则佩红色。
栽培
理想的栽培是夏季凉爽、湿度低，冬季温暖的地区。在14℃以下不大开花；25℃以上的生长快，但茎秆细弱而花很小。适宜在空气相对干燥、通风的环境中生长 。需要选择阳光充足、通风的场所；生长在疏松肥沃、含丰富腐殖质。土壤酸碱度在pH60－75为宜。
种苗定植用12厘米或15厘米盆，浅栽，不要伤根，栽后浇水。栽后20天摘心，促使基部萌发侧枝。 保持盆土湿润，每半月施肥1次。
繁殖
扦插、播种和组培繁殖。温室以1－3月和9－10月为宜，露地以4－6月和9－10月为好。插床用苗床或塑料箱，插条基部2片叶去掉，留上部2－4片叶。生根适温为10－13℃，插后20－25天生根。以秋播为好，播种适温为18－20℃。
广告媒体是用于向公众发布广告的传播载体。传统的“四大广告媒体”为家内电视、电台、报纸、杂志。在广告行业把电视媒体和电台媒体称为电波媒体；把报纸和杂志媒体称为平面媒体，以此区分。
除了传统四大广告媒体外，可供广告发布的媒体还包括传统家外媒体、数字家外电视(Digital Out-of-home)互联网媒体等。
中国电力国际发展有限公司，简称中国电力，是一家在香港交易所上市的公共事业公司，主要业务是在中国开发、建设、拥有、管理及经营大型发电厂。公司在香港注册，在2004年3月24日成立，是中华人民共和国五家发电集团中，唯一一家红筹股上市公司，母公司为中国五大发电集团之一的中国国家电力投资集团公司。2004年10月15日在香港交易所上市，当时招股定价为每股253港元。
截至2004年底，公司拥有其三座已投入商业运营的大容量燃煤发电厂中的两座发电厂（分别是平圩电厂和姚孟电厂）之100%权益，并拥有余下一座发电厂（即常熟电厂）之50%权益。这三座电厂的总装机容量为3,610兆瓦，而公司在这三座电厂的应占权益装机容量为3,010兆瓦。
2009年6月8日收购母公司中国电力投资集团持有的湖南五凌电力有限公司63%的股权，代价4465亿元人民币。五凌电力总资产以水电为主，并持有湖南核电有限公司50%的股权。收购完成后，公司权益装机容量将由2008年12月31日的约9,037兆瓦增加约2,245兆瓦而至11,282兆瓦。
2017年10月9日收购母公司中国电力投资集团及国家电投持有清洁能源项目公司100%的股权，代价4969亿元人民币，大部分代价拟以供股方式筹集资金支付。清洁能源项目公司资产包括水电、天然气发电、风电及光伏发电。收购完成后，公司权益装机容量能够增加约3,078兆瓦。中国电力公布按于记录日期每持有当时三股现有股份获配发一股供股股份的基准以每股供股股份182港元的认购价进行供股。中国电力在2017年12月配发及发行2,451,721,580股供股股份，使中国电力已发行股份数目增加至9,806,886,321股。
截至2017年12月31日，国家电投通过子公司「中国电力国际」（中电国际）、「中国电力发展」持有中国电力国际发展5561%股权。
TOM集团有限公司（股份代号：2383）为香港联合交易所主板上市的科技与媒体公司。 TOM集团科技相关的营运业务包括社交网络、移动互联网；其策略投资领域涵盖电子商贸、金融科技和先进的大数据分析。此外，其媒体业务覆盖经营出版和广告。集团总部设于香港，地区总部分别设于北京及台北，TOM集团为长江和记实业有限公司成员。公司在开曼群岛注册，主席为陆法兰。
TOM集团（前称TOMCOM）2000年3月1日在香港创业板上市，当时上市编号为8001HK。TOMCOM当时上市招股价为178元，上市首日收报775元，较招股价上升335倍。
TOM集团2001年底收购台湾出版集团城邦文化。2006年1月25日集团宣布：首席运行官王兟以希望投入更多时间于私人股权投资为由向董事局请辞，并于当年1月27日生效，财务总监汤美娟继任。王兟在辞去TOM集团首席执行官后，随即加入美国私人股权投资公司德州太平洋集团，并担任投资合伙人。TOM集团现任首席运行官兼运行董事为杨国猛。
TOM集团于2004年分拆TOM Online在香港创业板上市，然后再在2007年私有化TOM Online。
2006年12月20日TOM和eBay同时宣布创建合资公司，这也成为Tom集团的一个里程碑。
2010年8月10日，中国邮政集团公司与TOM集团有限公司合资建设的B2C网上购物平台《邮乐网》正式上线，由中国邮政和TOM集团分别持股51%和49%的合资公司“北京邮乐电子商务有限公司”负责运营。
现时TOM集团的主要股东是周凯旋。
售票处（民航营业大厦）
中国国际航空（Air China，IATA：CA，ICAO：CCA，、、上交所：601111 ），简称国航、中国国航，是中国大陆三大国有航空公司之一，是中华人民共和国的载旗航空公司，总部设在北京，以北京首都国际机场和成都双流国际机场为枢纽机场。
中国国际航空公司成立于1988年。2002年10月，中国国际航空公司联合中国航空总公司和中国西南航空公司，成立了中国航空集团公司，并以联合三方的航空运输资源为基础，组建新的中国国际航空公司。中航集团（中国国际航空股份公司母公司）与东航集团、南航集团合称中国三大航空集团，经营业务包括航空客运、航空货运及物流两大内核产业。国航是中国唯一载国旗飞行的民用航空公司以及世界最大的航空联盟星空联盟成员，并负责了中国国家领导人出国访问的专机任务，也负责许多外国元首和政府首脑在国内的专包机任务。
中国国际航空股份公司2017年营业总额1,21363亿元人民币，全年净利润724亿元人民币。
历史
国航的历史最早可追溯到1955年的中国民航北京管理局飞行总队，1988年中国民航被分为多家航空公司，分别为中国东方航空、中国南方航空、中国北方航空、中国西南航空、中国西北航空和专门负责国际航线的航空公司中国国际航空，但国航在后期也负责国内航线。
1990年代，中国政府大幅调整和改革民航事业。1994年，中国国务院批准国外资金投资中国的机场建设和民航业，放宽外国制造的民航机输入中国，使航空公司的机队急速现代化。90年代后期，国航的营运状况并不理想，更出现亏损情况。
2002年10月，以原先之中国国际航空公司为基础，联合中国航空总公司和中国西南航空公司组成中国航空集团公司，并以联合三方的航空运输资源为基础，组建新的中国国际航空公司，仍为中国载国旗和担任中国领导人专机任务的航空公司。
2004年9月30日，经国务院国有资产监督管理委员会批准，作为中国航空集团公司控股的航空运输主业公司，中国国际航空股份公司（以下简称国航）在北京正式成立，进行了股份制改革，继续保留原中国国际航空公司的名称，并使用中国国际航空公司的标志。同年12月，国航进行首次境外发售股票；12月15日，公司在香港和伦敦证交所挂牌上市。
2006年8月18日，公司在上海证券交易所上市。2006年5月，国航与德国汉莎航空签署同意书，于2007年12月12日正式加入星空联盟（Star Alliance），成为国际航空联盟成员的大型国家航空公司之一，也是联盟中首位来自中国大陆的成员航空公司；而旗下子公司深圳航空也将在2012年加入联盟。2006年9月28日，国航把港龙航空的股份全部售于国泰航空，使港龙航空成为国泰航空的全资公司。而国航亦入股国泰航空使两者成为互控公司及重要合作伙伴。
2007年度，国航入选了世界品牌500强，为中国民航唯一入选公司；2007年国航被世界品牌实验室评为中国500最具价值品牌；美国评级机构标准普尔评为中国大陆上市公司百强；国航品牌被英国《金融时报》和美国麦肯锡管理咨询公司联合评定为中国大陆十大国际品牌之一，是中国大陆航空公司第一的品牌。2008年，国航获得世界品牌价值实验室颁发的“中国最佳信誉品牌”奖项。
2008年，国航成为了北京奥运会航空客运合作伙伴。
2009年8月17日，中国国际航空斥资6335亿元向中信泰富购入49亿股国泰航空的股权，相等国泰125%已发行股本，而太古则斥资1213亿元，购入2%国泰股权。完成交易后，国航于国泰的持股量再由1749%升至2999%，而太古的持股量则由3997%升至4197%。中信泰富于国泰的持股量则减至298%。
2010年3月22日，中国国际航空出资682亿元人民币增持深圳航空股权，由原先25%增至51%，成为其控股股东。
2011年，公司总营业额93343亿元，纯利约7476亿元人民币，同比跌3875%。
2011年6月，中国国际航空首度获得Skytrax授予四星级航空公司之评级。
2011年4月11日，以公务专、包机飞行为主要业务的北京航空责任公司获准营业，中国国际航空控股51%。
2012年3月27日，北京航空责任公司与瑞士商务机公司VistaJe联合组建一支以公务机为主的商业喷射机队。
2013年7月3日，国航在一架编号为B-6525的空中客车A330-300上首度试行空中无线上网服务，这是中国首次全球卫星通讯互联网航班飞行。但国航并没有公布正式提供服务时如何收取费用，有部分人士表示为免费上网服务。空中无线上网服务启用后，旅客可以开启使用笔记本电脑、平板电脑等电子设备，登录飞行上网界面就可以实现空中上网，但手机仍被禁止。而空中上网服务带宽有限，机上系统能提供的上网服务并不多，仅支持微博和邮件，而且都内置在机上系统内。
2018年8月2日，中国国际航空广东分公司于广州市正式成立。
2018年8月30日，中国国际航空以2439亿人民币出售中国国际货运航空51%股权给控股股东中航集团，预期出售收益411亿人民币。
2021年9月10日，中国国际航空新疆分公司于乌鲁木齐正式挂牌成立。首批将投入3架B737-800飞机在乌鲁木齐运营，主要执飞北京首都、成都航线。
标志
国航的企业标识由一只艺术化的凤凰和邓小平书写的“中国国际航空公司”以及英文“AIR CHINA”构成。凤凰是中华民族古代传说中的神鸟，也是中华民族自古以来所崇拜的吉祥鸟。据《山海经》中记述：凤凰出于东方君子国，飞跃巍峨的昆仑山，翱翔于四海之外，飞到哪里就给哪里带来吉祥和安宁。国航航徽标志是凤凰，同时又是英文“VIP”（尊贵客人）的艺术变形，颜色为中国传统的大红，具有吉祥、圆满、祥和、幸福的寓意，正是希望这神圣的生灵及其有关它的美丽的传说带给朋友们吉祥和幸福。代表国航视每一位乘客及货主为上宾看待。
自国航开航以来至今，除了垂直尾翼的中国国旗换成凤凰，机身上的CAAC标志和“中国民航”字样改为“国旗+AIR CHINA+中国国际航空公司”的排列外，其余部分涂装设计均沿用自CAAC时期的经典腰线涂装。
另外，国航是中国大陆的航空公司中唯一在机身涂有国旗的航空公司，但飞往台湾的航班，因涉及两岸问题，机身上的国旗都会遮盖掉，或是派出没有国旗的彩绘机飞航台湾航线。
A330-300在松山机场，目前只有飞往台湾的机身上不载国旗
营运
国航总部设在北京，辖有西南、浙江、重庆、天津、上海、湖北、新疆、广东、贵州、西藏和温州分公司，国航主要控股子公司有深圳航空责任公司（含昆明航空公司）、澳门航空股份公司、北京航空责任公司、大连航空责任公司、中国国际航空内蒙古公司、北京飞机维修工程公司、国航进出口公司、成都富凯飞机工程服务公司、中国国际航空汕头实业发展公司等；合营公司主要有北京集安航空资产管理公司、四川国际航空发动机维修公司等；另外，中国国航参股国泰航空公司、山东航空股份公司、西藏航空公司等，且为山东航空集团公司的第一大股东。曾为国航控股、现同为中航集团所属的北京航空食品公司于1980年5月1日在北京成立，是为《中华人民共和国中外合资经营企业法》颁布后的第一家中外合资企业。中国国际航空的主要枢纽机场为北京首都国际机场和成都双流国际机场。此外，国航在广州、乌鲁木齐、杭州、重庆等地设有分公司。
机队
中国国际航空早期机队由波音707，波音737-200和三叉戟，BAe-146等老旧机队组成。1994年，国务院批准放宽外国制造的民航机输入中国，使国航的机队急速现代化。目前国航的机队主要由波音及空中巴士飞机所组成。除此之外，国航直属的分公司——北京航空也拥有多架公务机。
2011年3月，中国国际航空正式向波音公司订购7架747-8客机，并在2012年9月11日获得确认。首架飞机已于2014年10月1日飞抵北京首都机场。
现役
截至2020年6月，中国国际航空的机队平均机龄71年，详情如下：
货运机队
-4J6F（74F）
目的地
中国国际航空共开通了200多条国内外航线，国内通航城市超过80个，国际及地区际通航城市超过40个，以欧亚大陆和北美航线为主，南半球的航线有印度尼西亚、澳大利亚、新西兰、南非、埃塞俄比亚、巴西等国。在2019冠状病毒病疫情爆发前，是世界民航业少有的拥有飞往所有人类常驻六大洲客运航线的航司。
另：中国国际航空是除高丽航空外唯一可于朝鲜平壤顺安国际机场通航的航空公司（航班编号为CA121/122，只限星期一和五，和只使用B-5043~5045，最常用为B-5044），即唯一可在朝鲜民主主义人民共和国内固定通航的外国航空公司。但曾经于2017年11月22日至2018年6月5日期间因客流原因及中国配合联合国制裁朝鲜而停航。）
代码共享
中国国际航空已与以下航空公司签署代码共享协议（截至2019年11月）：
星空联盟成员
非星空联盟成员
贵宾休息室
中国国际航空在特定机场设有专属贵宾休息室，为国航头等舱、商务舱乘客、凤凰知音白金卡、金卡、银卡会员提供服务。详情如下：
常旅客计划
凤凰知音（Phoenix Miles，2013年前称国航知音）是中国国际航空的飞行常客奖励计划，是中国大陆最早创立的的飞行常客奖励计划，乘客拥有知音卡片后，可通过搭乘国航及下属公司以及星空联盟成员等伙伴航空公司航班累积里程，来换取免费机票及升舱等多项待遇。凤凰知音亦与酒店、汽车租赁公司、银行等商家合作，提供里程积累、兑换及各类增值服务。
合生创展集团有限公司 () 是中华人民共和国广东省五大地产发展商（碧桂园、恒大、富力、合生创展、雅居乐）之一，由民营企业家朱孟依先生于1992年成立，1998年在香港联合交易所有限公司主板上市， 是以发展中高档大型住宅地产物业为主的大型地产集团。 主要业务是投资控股、物业发展、物业投资及物业管理业务。
2010年11月3日以总代价6875亿元人民币向主席朱孟依儿子朱一航收购日佳投资全部股权，日佳主要在北京市通州区马驹桥镇环保产业园区发展 38幢总建筑面积 625万平方米的办公楼宇，完成后将纳入为集团的租赁物业组合。合生将以现金23亿人民币及每股作价8806元的新股支付收购，朱一航将占扩大后股本 1849%
2012年4月17日以293亿人民币认购北京农村商业银行499%权益
2018年10月，合生创展向张顺宜购入中环中心49楼全层，作价为1118亿港元。
2020年10月，合生创展向陈秉志购入中环中心48楼全层，作价为98亿港元。
2020年11月4日合生创展集团向最终控股股东朱孟依认购内地在线交易配对平台「优选好生活」- i元知集团20%股权，现金代价182亿美元，将以内部资源拨付。
主要物业
五角场的合生汇为合生创展集团属下项目
广州
惠州
北京
上海
天津
环球全域电讯有限公司（HGC Global Communications Limited），前称和记环球电讯有限公司，由I Squared Capital全资拥有，主要业务是在香港提供话音及数据通信服务，中型电脑系统集成，软件开发及提供相关服务、分销电脑产品，是香港持牌固网电讯商之一，并提供光纤上网服务。
上市及私有化
和记环球电讯，正式把中联系统控股有限公司借壳上市，曾是长江和记实业的联营公司。2004年1月，中联系统以港币71亿元收购和记环球电讯全部股份，其中港币39亿元以中联系统新股（发行价每股港币八角），而另外的港币32亿元则以可换股票据（换股价每股港币九角六仙）支付；另外，中联系统同时向长江实业和中电控股以4亿港元收购电联网络香港有限公司全部股权。
2004年4月，中联系统更名为和记环球电讯，有投资者憧憬和记环球电讯会注入更多电讯业务，但有关憧憬最终没有实现。其后，和记环球电讯曾以每股9角配股，集得18亿港元。和记电讯国际最终以每股65仙私有化和记环球电讯。
2009年，和记电讯国际分拆香港及澳门电讯业务重新上市，新公司名为和记电讯香港控股，和记环球电讯的拥有权亦由和记电讯国际转移至和记电讯香港控股。
2017年7月30日，和记电讯香港控股以14497亿港元现金代价，出售固网电讯业务和记环球电讯给I Squared Capital全资拥有的基金公司Asia Cube GlobalCommunications；交易于同年10月3日完成。其后和记环球电讯改名为环球全域电讯。
2019年8月22日，环球全域电讯收购数码技术解决方案供应商高威电信，管理层未有透露作价。
服务内容
环球全域电讯提供各类的电讯业务，包括 HGC 家居宽带、IDD 国际长途电话、商业宽带和电话线等。
竞争对手
香港电讯 HKT、香港宽带 HKBN、有线宽带 i-Cable 等。
富豪酒店国际控股有限公司（Regal Hotels International，）是一家在香港交易所上市的酒店公司。主要业务是持有与管理酒店、物业发展及投资，及其他投资（包括金融票据及有价证券之投资及买卖）。
公司于1979年由鹰君集团有限公司分拆成立，现在于百慕达注册，属世纪城市国际控股有限公司旗下公司，主席为罗旭瑞。
发展
2017年2月3日，富豪酒店以21亿8890万元中标香港国际机场 SKYCITY 航天城第A1a号，占地7万1580平方呎，许可建筑面积36万2746平方呎，将发展成一座提供超过1000间客房及套房的酒店项目。
肥皂剧（soap opera）是从英文传至中文的外来辞汇，通常是指一出连续很长时间的、虚构的电视剧节目，每周安排为多集连续播出，因此又称为系列电视连续剧、连续剧（Serial TV series，但“连续剧”一词又可以泛指一切剧情连续的电视节目），另有新兴词狗血剧代指（韩国以形容狗血剧），拉丁美洲的Telenovela即以此类型的电视剧为大宗。
来源
肥皂剧于1920年代在美国的收音机广播节目中兴起，1940年代移师电视屏幕，大多数肥皂剧都是白天播岀。
「肥皂剧」（soap opera）一词，起源于当年在收音机广播时于节目间隔时段播放的广告。因为白天收听连续剧的听众绝大多数都是家庭主妇，所以这些广告通常都是和洗涤用品有关。当时最主要的广告赞助厂商就是P&amp;G（宝洁公司），由于这类广告总是在这类连续剧的间隔播放，于是就慢慢生成“肥皂剧”这个词，意为“播放肥皂等洗涤产品广告的情节剧”。今日在美国，肥皂剧的间隔时段仍然播放P&amp;G等的广告，不过因为收视群扩大，也会放一些诸如皮肤药、避孕器具等广告。
亚洲地区方面，台湾、香港、中国大陆、日本、韩国等地亦有肥皂剧的拍摄。在台湾，部分晚间八点播出的长寿台语连续剧会被视为肥皂剧，也就是所谓的「八点档」，或因剧情常涉及家庭斗争，而被戏称为「家庭伦理剧」。日本则因于午间时段播出的电视剧内容，与英文指称的肥皂剧相近，因此通称为「午间剧」（昼ドラマ）。在韩国，多指平日播出的日日剧（일일극）与周末连续剧（주말 연속극），日日剧又可分为晨间与晚间两个时段。
概述
肥皂剧聚焦于某个特定社群中多个角色之间的关系。剧中的人物和现实生活的人一样，不过通常会比爱看此类节目的观众更俊美、性感、富有一些。剧中发生的故事也是日常所见所闻的，但通常要更戏剧化。
浪漫、外遇、阴谋、真爱等等情节是肥皂剧的主线，离不开一见钟情、三角恋爱、多角关系、神秘情人、私生子、外遇、家族斗争、企业争权、婆媳问题等等线索。
人物角色可在无伏笔下随意出现，也可突然消失；剧情只求有戏剧性，可以不合情理，甚至出现前后矛盾。故经常会岀现意想不到的突发事件，比如多年前的一次外遇情人在结婚那天岀现，邪恶的双胞胎是常用手法，喜事通常都会被意外事故打断等等。而且如果角色死亡却不播放尸体镜头，那有很大可能根本没有死。有的时候甚至观众见到尸体，编剧也会编个理由让角色复生（例如安排同一演员扮演一个与之前死亡角色样貌相似的角色，例如张慧仪在《真情》分饰李彩瑶和吴芳贵）。或者根据故事情节需要调整角色（例如隐姓埋名或改名换姓），方便延伸剧情发展（以寻秦记为例，郭羡妮分饰「秦青」与「琴清」，林峰饰演「赵盘」与「嬴政」。宣萱饰演「乌廷芳」与「凤菲」，江华则饰演「连晋」与「嫪毐」两个角色）。
肥皂剧拍摄不一定按照播放顺序，演员可能无法掌握拍摄内容，甚至剧本在摄影现场才会拿到。出场镜头多的主要演员需要比一般戏剧更长时间投入拍摄工作。
大成生化科技集团有限公司 ()是一家在香港交易所上市的工业公司。主要业务是在中国研究、开发及制造玉米提炼产品及以玉米为原料的生化产品。公司是在1994年成立，并在2001年于香港交易所上市，总部设香港。它的母公司是长春大成集团，而旗下的大成糖业是最大制造玉米提炼产品之一。
大成生化曾于2021年入围全球食用增稠剂企业榜，在变性淀粉榜单中位列第二十。
华润电力控股有限公司，简称华润电力、润电，是一家在香港交易所上市的公共事业公司。主要业务是在中国北京、河北、河南、辽宁、山东、江苏、浙江、安徽、湖北、湖南、广东、云南等省份投资、开发、拥有和经营大型燃煤发电厂。
历史
华润电力2001年8月27日在香港注册成立，法定股本100亿港币。
2003年11月12日在香港交易所上市，当时招股定价为每股28港元。
截止2008年12月31日，运营权益装机容量达12,981兆瓦。
华润电力将会在2009年6月8日，被纳入为恒生指数成份股（红筹股），取代被剔出的裕元工业，恒指成份股数目维持在42只。恒生指数公司的季度调整落实后，华润电力将占恒生指数比重054%，是第四只公用股。
2009年6月由华润电力与山西联盛集团共同出资设立的有限责任公司山西华润联盛能源投资有限公司，注册资本为38亿元，旗下拥有山西兴县、中阳县、临县、石楼县、柳林县、交口县六家分公司，主营业务为煤业、煤化工、水泥等，2010年，企业煤炭年产能将达1350余万吨，2015年，企业煤炭终端产能将超过5000万吨。
太原华润为华润电力控股联营公司，旗下山西华润联盛拥有49%股份、中信信托持31%股份、金业集团持20%股份。
2011年3月，华润电力旗下华润煤业，斥资669亿美元收购亚美香港。亚美大陆最主要资产是负责山西省晋城潘庄区块煤层气项目的「亚美大宁」。晋城市政府1999年赴美国招商投资有关项目，获美资CBM能源集团设立离岸公司亚美大陆，再加入多家美国能源基金，之后泰国万浦入股亚美大陆。亚美大陆〇七年在港注册成立亚美大陆(香港)，将所持亚美大宁56%股权注入。
2013年3月6日华润电力以384亿元(人民币，下同)(约478亿港元)出售山西亚美大宁能源5%股权。
2013年5月10日华润电力斥资42868亿港元向大股东华润集团收购电风电资产英飞有限公司全部已发行股本。
2013年5月10日提出华润电力与华润燃气以换股方式，每100股华润燃气股份可换取97股华润电力股份，按停牌前收市价计，即每股华润燃气收购价为2464元，合并后华润燃气将成为华润电力全资附属公司，并会撤销上市地位，合并成华润能源控股有限公司(China Resources Energy Holdings Ltd)。当时华润集团分别持有华润电力及华润燃气64％及6395％股权。股权架构方面，华润燃气小股东将持有日后的华润能源112%股权，华润电力小股东持股比例会由3645%摊薄至2511%，华润集团及一致行动人士持股比例，由6355%微升至6369%。
华润电力以换股方式合并华润燃气的建议，2013年7月22日遭润电独立股东以大比数否决，1年内更不可重提并购建议。
2014年8月26日，华润电力公布，行政总裁王玉军因被江苏省检察机关立案调查，故暂停他担任相关职责，由主席周俊卿兼任。
相关
信和置业有限公司（Sino Land Company Limited；港交所：&amp;sc_lang=zh-HK 0083 、）是一家在香港交易所上市的地产公司。主要业务是物业租赁，物业销售，酒店经营，管理服务，股票投资及买卖，与财务。
公司在香港注册，董事局主席黄志祥为新加坡远东机构主席黄廷芳家族成员。
根据信和官方网页指出，亚洲最大饮料生产商之一杨协成亦是信和置业的姊妹公司。
历史
2017年9月7日信和置业及尖沙咀置业联合公布旗下附属信和置业中国投资集团以87675亿元人民币（约1051亿港元）现金出售两幅位于中国四川省成都市成华区御风二路9号宗地之住宅、商业及酒店发展项目，地盘面积共约244,3593平方米，面积约1,330万平方呎，项目名为信和‧御龙山的80%股权予深圳华侨城股份有限公司附属群升发展，交易完成后，信置将持有信和置业（成都）余下20%股权。
2017年5月26日，港铁公司公布，由信和置业、嘉华国际及中国海外发展各占3333%股份的宏键有限公司以833亿元，即每呎楼面地价6735元投得西铁锦上路站1期项目，项目地皮面积约448,719平方呎，可建楼面面积达1,236,741平方呎，项目将分二期发展期提供2113个住宅单位，第—及第二期住宅单位数目分别约有1423个及690个，预期于2021至24年取得「入伙纸」及「满意纸」
2017年11月15日信和置业（占225%权益），会德丰地产（占225%权益），世茂房地产（占225%权益）、嘉华国际（占225%权益）及爪哇控股（占10%权益）合组的财团天基置业有限公司以17288亿元夺得长沙湾兴华街西对出的新九龙内地段第6549号的用地 地盘面积约为208262平方呎，以最高可建楼面约987812平方呎计算，每呎楼面地价约17501元，成为香港「地王之王」。
2017年12月5日嘉里建设及信和置业各占50%股权的合资公司High Crown Holdings Limited，以补地价约521亿元，以项目可建楼面面积约493万方呎计算，每方呎楼面补地价约10,576元成功投得港铁黄竹坑站第二期发展项目
2017年12月15日中华电力公布，将于信和置业 共同发展，位于何文田亚皆老街139至147号中华电力总办事处重建项目，地盘面积约61万方呎，住用楼面309707平方呎，获批建3幢25层高（连5层平台）的住宅楼宇，提供175伙，最底5层为会设会所、大堂及停车场等设施。该项目以暗标形式招标，估值料超过100亿，并会保留钟楼建筑作保育用途。
2017年12月18日市建局公布信和置业有限公司及庄士机构组成的财团－泰康发展有限公司，合作发展旺角新填地街╱山东街项目，地盘面积14961平方呎。项目完成后，预计可提供112204平方呎住宅楼面面积（约304个住宅单位），以及约2,085平方米商业楼面面积。
2018年4月6日中华电力总办事处重建项目重建计划，完成202099亿元补地价，每平方呎楼面地价约6,525元
2018年5月8日港铁公司公布信和置业和资本策略地产各占80%和20%股份的的腾洋有限公司成功投得油塘通风楼住宅发展项目，项目地盘面积434万方呎，可建楼面3253万方呎，补地价151499亿元，每方呎补地价约4,657元，固定分红比例为25%，中标财团需负责拆卸现有的政府抽水站、重建一道公共行人路及斜坡工程，项目可兴建一幢楼宇,提供约500个住宅单位，预期2025年落成
2018年12月5日地政总署公布信和置业以203889亿港元，每方呎楼面地价约17604港元投得大屿山长沙屿南道用地属于丈量约份第332约地段第765号，属于屿南道43及44号，邻近大屿山南分区警署，地盘面积约28977方呎，可建楼面面积上限约11582方呎，建筑物限建最多两层
2019年4月25日：信和置业40%权益、嘉华国际30%权益及招商局置地30%权益合组公司Sky Castle Limited成功投得将军澳日出康城第十—期住宅发展项目，项目可建住宅楼面9565万方呎，可建单位最多1850伙，补地价金额为30549亿元，折合每方呎补价3194元，固定分红比例为20%，预计于2025年6月30日前落成
2019年10月28日嘉里建设50%权益、太古地产25%权益及信和置业25%权益合组财团WCH Property Development Company Limited 以固定分红比率25%，补地价675774亿港元成功投得黄竹坑站第4期纯住宅项目，以可建楼面面积638305平方呎计，每平方呎补地价10587元，由2幢住宅楼宇组成，共提供800伙中大型单位，预计2025年落成。
2022年10月27日信和置业及招商局置地合组公司富利恒有限公司成功投得市区重建局土瓜湾荣光街/崇安街发展项目，项目可建住宅楼面279万方呎，可建单位560伙，投标金额2388亿元，折合每方呎8571元，项目内的商业部分，落成后由市建局及财团共同持有和管理，市建局将拥有当中三成的权益
业绩
其股价成长亦相当惊人，2004年底~2005年初，其股价尚未满10港元，至07年底已高达27港元，可见其经营绩效。
如今(2014年2月股价又回落10块多，和公司绩效关系不大。）
中国石油天然气股份有限公司（PetroChina Company Limited，上交所：601857 ，，）是一家在中华人民共和国注册的上市公司，简称中国石油，于1999年11月5日由中国石油天然气集团公司发起成立，主要从事与原油和天然气相关的各项业务。公司董事长王宜林，总裁汪东进。
2022年，中国石油在福布斯全球2000大上市公司排行榜中位列第21名。
历史
中国石油天然气股份有限公司于1999年11月5日由中国石油天然气集团公司发起成立后，2000年4月6日在美国纽约证券交易所发行托管股份，2000年4月7日在香港交易所以127元发行价挂牌上市，2007年11月5日在上海证券交易所以人民币167元发行价挂牌上市。在上证所上市当天即超越艾克森美孚成为全球最大市值的上市公司。不到半年，中石油便跌破发行价。
2007年11月7日，恒指服务公司宣布中国石油天然气股份加入恒生指数成份股（蓝筹股），指数比重是第三大，仅次于汇丰和中国移动，2007年12月10日起生效。
巴西2006年发现深海盐下油田引起世界注目，巴西有潜力成为世界前三大产油国，2018年中石油、中海油等联合竞标，得到巴西东部海域的利贝拉油田区块35年开采权。
争议
东城区东直门的地标型建筑——中国石油总部大楼图二
中国石油一直被国内舆论所诟病。其股票价格暴跌、油价居高不下都是被大众舆论征讨的焦点问题。
华能国际电力股份有限公司简称华能国际，华能是一间在纽约证券交易所、香港交易所和上海证券交易所上市的国有控股火力发电公司。
华能国际的最终母公司是亚洲最大的独立发电企业，也是中华人民共和国五大国有电力公司之首中国华能集团。
概况
华能国际电力股份有限公司成立于1994年，是中国华能集团公司的控股子公司。华能集团的前身是1985年成立的华能精煤公司和华能国际电力开发公司，它依托国家以煤代油专项资金，与地方政府合作，利用外资，兴建电厂。到1990年代初，华能集团已通过这种方式建设了多座现代化大型发电厂。
1994年6月，华能集团对旗下的数座主要发电厂进行重组，联合多家地方政府投资公司共同成立了华能国际电力股份有限公司和山东华能发电股份有限公司，并于当年在纽约证券交易所分别发行上市。1998年华能国际以介绍方式在香港交易所挂牌上市，随后即发行了H股。2000年在华能集团的重组过程中，华能国际合并了山东华能，从此成为华能集团旗下唯一的内核企业，这也是中国大陆迄今为止最大的一宗上市公司之间购并重组的案例。2001年，为降低因合并山东华能而形成的较高负债率，华能国际再于上海证券交易所发行股票，募集资金归还贷款。
华能国际的内核业务是经营火力发电，早期的发展策略以沿海地区为主，收购与开发并重。华能国际的电厂多位于经济发达、用电需求旺盛的大城市附近，而且设备先进、管理优秀，但用地，用人成本较高。其中旗下的大连发电厂是中国第一家获得“一流”荣誉称号的电力企业，位于上海的石洞口第二发电厂是中国第一座使用超临界机组的电厂，位于重庆的珞璜发电厂是中国第一家引进全套脱硫环保设备的电厂，也是华能旗下最大的电厂。华能国际自上市后，又陆续从华能集团处收购了大量发电厂，将母公司的优质资产几乎全部囊括。同时，华能国际也大力推动旗下电厂的新建和扩建，从而使公司的主营业务稳步增长，其中重庆珞璜电厂扩容完成后将成为东亚最大的火力发电厂。而近年来，由于华能集团的资产已大都注入了华能国际，且煤炭价格的大幅飙升降低了火电厂的利润，因此华能国际的发展策略也开始转向经营中西部的坑口电厂和水力发电业务。
2021年3月，公司发表公告称，截至2020年年底，华能国际拥有可控发电装机容量1134万兆瓦；境内电厂全年发电量404016亿千瓦时，居中国行业可比公司第一。
业务
华能国际电力股份有限公司的主要业务是发电，它拥有和控股了分布于中国全境的多座大中型发电厂。
华能国际也参股了一些大中型发电厂。
此外，华能国际还持有深圳能源集团股份有限公司和华能四川水电有限公司的部分股权。
冠捷科技有限公司，（港交所除牌前0903HK、新交所除牌前T18SG），是电脑显示器品牌AOC的母公司。曾在香港交易所与新加坡交易所两地上市，目前正在进行私有化进程。主要业务是各式各样电脑监视器、CRT、LCD显示器、LCD TV、PDP等离子电视、及彩色扫描仪之制造、设计及销售业务。旗下品牌除了AOC之外，另包含Envision及一些获许可的的飞利浦产品。
冠捷科技在中国大陆创建了福建、武汉、北京、苏州、厦门五座工厂的「五角星型」的生产供应链，并于巴西、墨西哥、波兰、俄罗斯、及阿根廷拥有海外生产基地。公司于1998年在百慕达注册为冠捷控股有限公司，并在2000年易名为冠捷科技有限公司。主席为宣建生，2017年员工总数近2万9千人。
历史
冠捷早期为AOC（Admiral Overseas Corporation）为了在中国创建生产工厂而在福建设立的公司，因此冠捷发展与AOC密不可分。AOC当时为(Admiral)在台湾的子公司，艾德蒙为美国曾经的电视制造商。后随着艾德蒙的分崩离析，以及冠捷扩张成为具有中国央企背景的集团，AOC成为冠捷旗下全资控股公司。
新世界中国地产有限公司（英文：New World China Land），简称「新世界中国地产」，为香港上市公司新世界发展有限公司(香港股份代号：00017)旗下内地物业旗舰，是内地最具规模的房地产发展商之一。作为最早进入中国内地房地产的香港公司之一，新世界中国地产秉承可持续发展之城市社区建设愿景，发展各类优质住宅社区、大型综合商业地标、商场、写字楼及酒店，业务多元化。
2011年6月18日，新世界中国地产旗下间接全资附属新世界酒店管理以代价2295亿美元(约179亿港元)，向Hotel Value Fund LP及瑰丽酒店和度假村有限公司收购管理公司全部成员权益。瑰丽酒店和度假村公司于8个国家管理共19所物业。
2011年7月31日，新世界中国地产旗下间接全资附属新世界酒店管理斥资57亿美元（约4446亿港元）向瑰丽酒店集团及Maritz, Wolff &amp; Co合共购入5间位于美国的豪华酒店。
2013年12月15日，新世界中国地产旗下间接全资附属新世界酒店管理，斥资1,3536万欧元（约144亿港元）向控股公司周大福企业购入PHHL集团（主要在欧洲从事酒店管理业务）PHHL主要资产为德国成立的PHGG，以「Penta」品牌于欧洲管理24间酒店。
2016年8月4日，新世界中国地产撤销其于香港联交所之上市地位。
2021年1月13日，新世界发展宣布将新世界中国总部从香港迁至广州。
新世界中国地产有限公司2020年，1月27日宣布捐款1000万元，成为首家捐资抗击疫情港企。
1月28日，郑家纯副主席旗下K11集团宣布成立武汉抗疫专项基金，总金额为500万元人民币，基金将用于购买医疗用品给武汉协和医院，并捐款给武汉同济医院。同时，基金会已紧急运送两批合计20万个外科口罩前往武汉。
荃湾沙咀道251号的招商永隆银行（时为永隆银行）荃湾分行
招商永隆银行是香港一间华资银行，前身为1933年创立的永隆银号，后更名为永隆银行。永隆银行于2008年被招商银行并购，成为招商银行全资附属子公司。2018年10月1日，永隆银行正式更名为招商永隆银行。
历史
永隆于1933年2月25日在香港文咸东街37号开业，定名永隆银号，资本港币44,500港元，经营找换、汇兑、存款、各端口来往、代客买卖股票、黄金及国内公债等业务，成绩按年递增。
1937年，为扩充营业，迁至香港皇后大道中112号。
1941年，太平洋战争爆发，香港沦陷。永隆转往澳门继续经营银业，及在广西柳州设立「永隆金号」。
1945年，战事结束，回港复业。
1952年，开始经营押汇业务；同年旧总行行址重建落成。
1956年，注册为有限公司。由于业务发展，职员增多，原有行址已不敷用，乃将预置的右邻和店后舖位重建，扩大营业。
1960年，中文名称由「永隆银号有限公司」改称「永隆银行有限公司」。
1965年，为配合业务发展，将原总行行址作第3次重建。
1973年，位于德辅道中连贯干诺道中之总行新大厦落成，适值该行成立40周年。总行大厦楼高300呎，共25层，总面积145,000余呎。是年改为公共有限公司。渣打银行加入为该行成为股东。
1975年，另一物业——旺角弥敦道之银行中心亦告落成，大厦楼高300呎，共25层，总面积245,000余呎，同时设「旺角分行」于该大厦。该行亦于是年开始使用电脑处理存款业务，初期主机附搭电脑公司，至1981年自行设立电脑中心，陆续将各项业务电脑化。
1980年初，本行股份上市（上市编号096），将25%股份公开发售与社会人士，并在香港证券交易所挂牌买卖。
1982年，联同另外4间华资银行组成银联通宝有限公司，向客户提供自动柜员机服务。
1984年，在美国洛杉矶开设首家海外分行。
1986年，全资附属之永隆保险有限公司获政府授权经营保险业务。
1987年，渣打银行将所持10%的永隆银行股权转让与伍氏家族控制之公司。同年成立康令有限公司（其后改名为永隆证券有限公司），向客户提供证券投资服务。
1992年，新加坡发展银行从伍氏家族控制之公司承让百分之十本行股权，加入为股东。
1994年，在广州市设立代表处，为永隆银行在中国内地首个业务据点。
1995年，开始经营信用卡。
1996年，成立服务后勤中心，将各部及各分行多项业务工序集中在该中心处理。同年增设开曼群岛分行，向客户提供离岸银行服务。
1998年，通过全资附属之永隆财务有限公司经营租购贷款业务。同年推出「永隆网上银行」，利用互联网向客户提供自动化服务。
1999年，在上海市设立代表处。同年，永隆银行联同多家银行组成银联信托有限公司，开拓强制性公积金服务。并推出电子证券交易系统「永隆电话买卖证券」及「永隆网上买卖证券」服务。
2000年，与同业组成银和再保险有限公司及香港人寿保险有限公司，进一步拓展保险业务。
2004年，永隆银行成为首家获中国银行业监督管理委员会批准在内地设立分行的香港银行。同年，永隆银行深圳分行成立，开办人民币业务，成为该行首家中国内地分行。主要股东伍絜宜有限公司购回星展银行所持10%本行股权。
2008年，大股东伍宜孙家族向招商银行以193亿港元出售所持有的5312%永隆银行权益。伍氏家族伍步高、伍步刚及伍步谦分别辞任公司董事长、副董事长及行政总裁职务，但续留任为非运行董事。永隆银行委任朱琦、马蔚华及张光华为新行政总裁、董事长及副董事长。
2009年，招商银行完成强制性收购永隆银行余下股份后，永隆银行撤销上市地位。
2011年4月，永隆银行推出「红酒贷款」服务，以配合当时香港成为全球红酒交易和拍卖中心的发展。
2011年10月，永隆银行推出小微企业金融服务。
2018年10月1日，永隆银行正式更新名字为招商永隆银行。
营业点
至2022年9月，招商永隆银行在港共设有30家分行和私人银行中心，并于中国内地、澳门及海外等地设立了机搆网点，拥有员工逾2000人。
大唐国际发电股份有限公司(、上交所：601991 、)，简称大唐发电，是中国独立发电及供电企业，也是中国五大国有电力集团之一。在伦敦和香港的证券交易所上市，是中国大唐集团公司的内核子公司。公司旗下的电厂遍布中国各地，而其中在华北地区更具有举足轻重的地位。
概况
大唐国际发电股份有限公司原名北京大唐发电股份有限公司，早期曾由中国华北电力集团公司控股，而在2002年中国电力体制改革后，公司成为了中国大唐集团公司的内核子公司。公司于1994年成立，拥有主要向京津唐电网（范围包括首都北京市、天津市、河北省北部）供电的几座大中型发电厂。1997年，公司在香港交易所和伦敦证券交易所发行了H股，募集资金用以建设新电厂。2005年，公司决定未来在上海证券交易所增发A股筹集资金，同样用于兴建电厂。
由于大股东的关系，公司早期集中在京津唐电网及华北电网发展。在电力体制改革后，以公司为内核，组建了中国大唐集团公司。此后，公司开始向全国范围内拓展，水电与火电并举，陆续获得华东、南方等多个地方的电厂发展权，甚至还参与了煤矿和大秦铁路的建设。由于华北电力集团曾将公司的股权作为筹码，与北京、天津、河北等地方政府拥有的电网资产进行交换，导致了大股东对公司的控股权被大量稀释，使得如今的大唐集团没有将资产注入公司的意愿，公司只能依靠自己的力量发展。目前大唐集团主要致力于自有电厂的改建和扩建，新建电厂则一般交由公司发展。
2017年12月6日大唐发电以181275亿元现金向控股股东中国大唐集团公司收购其拥有的河北公司、黑龙江公司及安徽公司的全部股权。 该3家公司主要从事电力生产及销售业务。
业务
公司主要经营发电业务，以火电为主，同时亦新建了不少水电站，在全国范围内拥有和控股了多座大中型发电厂。
杰里·阿伦·韦斯特（Jerry Alan West，），美国职业篮球选手，教练和经理。生于西弗吉尼亚州。是NBA历史上唯一一位以败者身份获得NBA总决赛MVP的球员（1969年）。他运球时的剪影是NBA标志的原型，因此也被称为“标志”（The Logo）。他在比赛关键时刻发挥出色，为他赢得了“关键先生”的美名。
简介
球员时代
与大多数NBA球员一样，韦斯特在高中和西弗吉尼亚大学时期就表现得很突出，获得过1959年美国大学篮球锦标赛（NCAA）的冠军，之后他加入了洛杉矶湖人队，并为之服役了14年之久。他也代表美国国家队夺得了1960年罗马奥运会的金牌。
在他的职业生涯中，韦斯特共得25,192分，平均每场比赛得27分，罚球7160次，助攻6238次。
韦斯特曾四次入选NBA最佳防守阵容，10次入选NBA最佳阵容，14次入选全明星赛。在1980年NBA35周年的纪念活动中，他入选。1996年NBA50周年的时候，又入选NBA50周年纪念队的名单。
作为一名完美的射手，他在153场季后赛中平均每场得291分，其中在1965年度的11场季后赛中平均每场得406分，在1970年冠军争夺赛中，他在最后时刻从60英尺远处投篮命中（史上最远两分球），创造了NBA历史上经典的一幕。但是，韦斯特只在1972年赢得过唯一的一次NBA总冠军。1974年退役后依然在俱乐部中担任各种职务，之后又作为湖人队教练，在1976年至1979年三次打入季后赛。
总经理时代
1980年，湖人队在魔术师约翰逊的带领下夺得了NBA总冠军。1982年，杰里·韦斯特被任命为湖人队总经理，他通过精明的交易和技巧性的选秀，使湖人队在80年代一直处于NBA的领先球队位置。当时湖人队的内核球员是约翰逊、贾巴尔、沃西，湖人队也是继波士顿凯尔特人队（1968年－1969年）之后第一支连冠的球队。
经过90年代初期的一阵沉寂之后，韦斯特带领湖人队又进入了季后赛，他也获得了1995年度NBA最佳经理的荣誉。同年，韦斯特被任命为球队篮球事务副总裁。
1996年，独具慧眼的韦斯特在夏季交易中用中锋弗拉德·迪瓦茨从夏洛特黄蜂队换来当年的第13顺位选秀权，选择了高中生新秀科比·布莱恩特 (Kobe Bryant)，又卖掉George Lynch和Anthony Peeler以清理薪金空间，然后用7年1亿2300万的价格签下了已成为自由球员的奥兰多魔术队超级中锋沙奎尔·奥尼尔 (Shaquille O'Neal)，后来两人作为内核带领湖人在2000年－2002年实现继公牛王朝之后NBA的第一个三连冠。不过在1999年，球队老板老巴斯为了加速球队复兴，将前公牛队的冠军教头菲尔·杰克逊招来，但杰克逊的执教理念与韦斯特格格不入，两人在如何使用奥尼尔、是否应该用科比交易来贾森·基德、该不该留下训练懒散的丹尼斯·罗德曼等问题上发生冲突，最终在湖人2000年夺冠之前韦斯特主动辞职，寻求新的挑战。
2002年，韦斯特被刚刚搬家的孟菲斯灰熊队聘为篮球总监。尽管当时这支队伍的成绩处于NBA底部，韦斯特成功地进行了改造。2003-04赛季，灰熊队首次在一个赛季赢得了50场比赛并首次进入季后赛，他本人也再次被选为NBA最佳经理。随后两个赛季，灰熊队亦顺利打进季后赛，并逐渐成为西区强队之一。可惜2006-07赛季，球队主将保罗·加索尔 (Pau Gasol) 受伤缺阵，影响球队战绩，使灰熊队再次成为联盟最差球队（22胜60负）。球队状况混乱，加上球队老板一直想出售球队，韦斯特遂以「不再年轻，身心俱疲」为理由，于2007年夏天离开灰熊队。
并受到联盟内所有人的尊敬。而其最为人所称道的，是他担任湖人队和灰熊队管理层时所作出的选秀及球员交易，韦斯特往往能替球队找到才华洋溢的球员，当中尤以「科比与欧尼尔」的交易最为著名；此外，在灰熊队时以加索尔为中心打造强队，使球队成功打进季后赛，亦常为人赞赏。但他在2006至07年赛季前，把队上重要人物沙恩·巴蒂尔 (Shane Battier) 交易至休斯敦火箭队以换取新秀鲁迪·盖伊 (Rudy Gay)，被认为是过于冒险的一着，此外，又季中把另一能攻擅守的明星球员埃迪·琼斯 (Eddie Jones) 放弃，均被视为导致灰熊队战绩差劣的原因。而灰熊队连续三年于季后赛第一轮被横扫出局，无法取得季后赛一胜的成绩，也是韦斯特为人诟病的地方。
NBA生涯数据
NBA常规赛数据统计
!赛季
!球队
!出赛
!先发
!平均上场时间(分钟)
!投篮命中率(%)
!三分球命中率(%)
!罚球命中率(%)
!平均篮板
!平均助攻
!平均抄截
!平均阻攻
!平均得分
NBA季后赛数据统计
!季后赛
!球队
!出赛
!先发
!平均上场时间(分钟)
!投篮命中率(%)
!三分球命中率(%)
!罚球命中率(%)
!平均篮板
!平均助攻
!平均抄截
!平均阻攻
!平均得分
†代表该年度拿下总冠军
竣工决算是指建设项目全部竣工验收合格后编制的实际工程造价的经济文档。竣工决算可以反映该建设项目交付使用的固定资产及流动资产的详细情况，可以作为财产交接、考核建设项目使用成本及添加资产价值的依据，也是对该建设项目进行清产核资和后评估的依据。
蒋孝章（英文：Chiang Hsiao-chang，），籍贯浙江奉化，中白混血，蒋经国与妻蒋方良的第二个孩子，也是蒋经国唯一的女儿，蒋中正唯一的孙女，自幼万千宠爱集一身。
出身
蒋孝章为华裔与俄罗斯混血。就读北一女时，蒋经国打算找侍卫和司机接送她上下学，但她都拒绝不愿意给人接送，她喜欢一个人骑自行车到学校上课。蒋经国不放心让蒋孝章吃外面的午餐，叫司机开着小型座车，中午为孝章送便当，孝章站在校门口，对司机怒目相视，待司机走过去把便当交给孝章之后，她说：「你们走远一点好不好？」司机可以理解她的用意，是不希望被同学视为特权阶级。所以，司机之后就将便当放在学校门房，在角落等着看她拿走后，才打道回府。
蒋孝文曾回忆表示，每当他挨父亲打时，蒋孝章就是唯一的救星，只要蒋孝章对蒋经国撒娇跟劝解，多半能让蒋经国消气。
蒋孝章18岁时进入东海大学就读，因为东海大学独创的劳作教育，蒋孝章被分配至洗碗工作，蒋宋美龄得知宝贝孙女的情形后心疼不已，更通过总统府函信至东海大学，希望蒋孝章可免除劳作教育，但校方仍坚持所有学生皆需实施劳作教育。不久，蒋孝章还没读完大一上学期的课程便转学离开东海大学。
家庭
蒋孝章身为第一千金，这样的家世背景，让蒋孝章自小能接触的对象很有限。1957年赴美留学，蒋经国托付俞大维之子俞扬和代为照顾，俞扬和为有妇之夫，却与蒋孝章陷入热恋，引起一阵喧然大波，最后终于说服蒋经国接受。1960年8月在美国旧金山两人低调完成婚礼，1961年5月生有一子俞祖声。当年两人恋爱，蒋经国十分恼火，曾冲到国防部部长办公室，气急败坏向俞大维发飙掀桌砸场。后来幸有俞扬和的努力以及宋美龄的耐心说服调解，蒋经国才慢慢接受。蒋经国极为宠爱外孙俞祖声。
婚后定居美国，殊少露面，蒋经国两次就任中华民国总统时，均邀请蒋孝章夫妇回台参加就职典礼，但是他们两次都没有回台，据传是由于其夫俞扬和与蒋家三个兄弟关系不和。1988年蒋经国逝世时，蒋孝章曾回台参加告别式。2003年宋美龄逝世，蒋孝章出席参加告别式。2004年母亲蒋方良女士病逝，蒋孝章却因病重无法返台奔丧，由夫俞扬和代表出席。
蒋孝章是蒋经国与蒋方良唯一仍在世的子女，近年来为妇科癌症所苦，目前定居美国加州奥克兰山麓。
日本学（英语：Japanology 或 Japanese studies）是研究日本语言、文化、历史、政治乃至于经济等等内容之学科。日本学的源头可以追溯到江户时代在长崎出岛的荷兰人。萨道义和弗雷得里克·迪金斯等人在1872年于横滨成立日本亚洲学会是一项重要的推进，令这门学科发展成受国际重视的学科。
单位工程是指具备独立施工条件并能形成独立使用功能的建筑物及构筑物。从施工的角度看，单位工程就是一个独立的交工系统，有自身的项目管理方案和目标，按业主的投资及质量要求下，如期建成交付生产和使用。
电导（Electrical conductance）是表示一个物体或电路，从某一点到另外一点，传输电流能力强弱的一种测量值，与物体的电导率和几何形状和尺寸有关。
现在国际单位制对这个数值的单位为西门子（Siemens，缩写“S”，国际电导单位；等于欧姆的倒数）。在过去，电导的单位为「姆欧」（Mho，由Ohm即欧姆这个词的字母顺序颠倒而得，或以℧来表示）。
与其它物理量的关系
对于纯电阻线路，电导formula_1与电阻formula_2的关系方程序为
formula_3。
欧姆定律是
formula_4；
其中，formula_5是电压，formula_6是电流。
所以，可以得到欧姆电导定律的关系方程序：
formula_7。
请注意，当阻抗是复值时，这些关系方程序不成立。这时，电导与电纳formula_8和导纳formula_9的关系方程序为
formula_10，
或者，
formula_11，
其中，formula_12是虚数单位。
一个截面面积为formula_13，长度为formula_14的物体，其电导formula_1可以由电导率formula_16求得：
formula_17。
电路等效电导的运算
从克希荷夫电路定律，我们可以演绎电导组件的综合法则。
并联电路
给予两个并联的电导组件formula_18、formula_19。这两个电导组件两端的电压必相等。按照克希荷夫电流定律，总电流formula_20是
formula_21 ;
其中，formula_22、formula_23分别为通过电导组件formula_18、formula_19的电流。
将欧姆电导定律的方程序代入，可以得到
formula_26。
所以，等效电导formula_27是
formula_28。
串联电路
给予两个串联的电导组件formula_18、formula_19。通过这两个电导组件的电流必相等。按照克希荷夫电压定律，总电压formula_31等于两个电导组件两端的电压formula_32、formula_33的总和：
formula_34。
将欧姆电导定律的方程序代入，可以得到
formula_35。
所以，等效电导formula_27是
formula_37。
重新编排，
formula_38。
小信号组件电导
我们可以应用电导于电子组件，像晶体管或二极管。通常，我们会采用小信号模型（small-signal model），在一个给定的直流操作点，称为Q-点（Q-point），相关的组件方程序会被线形化。所得到的小信号组件电阻的倒数，就是小信号组件电导。若想知道更详细数据，请参阅尔利效应。
日本时代可以指：
恒隆可能指以下公司：
现金流（英语：cash flow）是货币的真实或虚拟动作，在财务学和会计学上，是指某一特定时间以内，特定的经济单位在经济活动中为了达到特定的目的而发生的资金流入和资金流出。在会计学里，除了要量度现金流量以外，亦要为将来的现金流量作出合理的估量。现金流量的基本观念就是企业某一段时间内可以立刻拿出来使用的现金。不过，经济活动产生的现金流入和现金流出通常在时间上不匹配，而且经济活动产生的现金流的数量和时间都具有不确定性，因此企业经营者必须致力于管理现金流的缺口，确实控制好现金流量，若现金流量不足将可能很快使企业资金供应短缺而倒闭。
现金流量比率
现金流量比率（英语：cash flow ratios)，为基于企业经营活动现金流的比率，其提供的信息可以随着时间的推移进行分析，以更好地了解公司过去的业绩及其未来前景；下表呈现现金流量比率之二大分类：绩效比率与覆盖比率（前者衡量盈利能力，后者呈现偿付能力)，并总结常见的现金流量比率与其之计算方式：
九龙仓可以指：
金融学中，金钱的时间价值、货币的时间价值或资金的时间价值（time value of money，缩写为 TVM）是一种表示「金钱价值会随时间改变」的重要基本概念。在金融市场里，资金即是可供交易的商品，以「利率」为价格行贩售。当借款者与贷放者在利率上达成协议时，借贷契约则成立。同理，就整个市场而言，当所有资金供给者与需求者达成价格上的妥协后，均衡利率即被决定。而此利率的出现便会导致「今天拥有得的钱，在未来可以产生额外的价值」。而这种今日的一块钱会比明日的一块钱还要值钱，而明日的一块钱又更会比后日的一块钱还要更值钱的现象，就是金钱的时间价值。一笔资金，在现在的价值称为现值（present value），未来的价值称为终值（future value）。终值也就是复利的结果。复利是指定期以本利和计算一次利息的计息方式。
终值与现值
以储蓄为例，若今一人将他的一万元存入银行内为期一年的定期存款帐户，利率为8%，每年复利一次，则该人在一年后可以提领的金额将会是10,000×（1+8%）=10,800元 。此一万元生出了800元的利息，经济学上来说，此人一万元存款的终值在一年后就是10,800元。将以上的计算过程一般化后，即可得：
formula_1
其中P是现值，F是终值，r是每期必要报酬率（required rate of return per time period），n是期数（number of time periods）。若投资期间不变，利率越高，终值越高；在相同的利率水准下，投资期越长，息票发给次数较多，终值也会越高。如果将此式等号两侧取倒数，则可以得到：
formula_2
如上式，将终值转换为现值的过程，称为折现，其所使用的利率则称为折现率。折现的意义在于可以将未来不同时间点的货币价值换算为今日的价值，有助于财务上比较价值大小。由数学式中可以发现，在投资期间不变的情况下，折现率越高，表示未来的已知货币价值是经由较高的利率复利而来，所以其现值会较低；而在相同的利率水准下，投资期间越长，表示未来的货币价值是经过较长时间复利而来，其现值也将会较低。
年金的时间价值
年金（annuity）是指在一个特定时间内，定期支付的等额现金流量，也就是以固定的时间周期以相对固定的方式发生的现金流。例如，分期付款赊购、分期偿还贷款、发放养老金、分期支付工程款、每年相同的销售收入等，都属于年金的一种。年金随着支付时间的不同，有不同名称。其中，开始支付的时间点始于支付约定成立后的第一期期末者，称为普通年金（annuity-immediate或ordinary annuity）；而若支付始于支付约定成立后的第一期期初者，称为期初年金（annuity-due或due annuity）。要注意的是，期初年金由于几乎是在约定成立时就已经支付了第一次年金，因此总括来说，若结束的时点相同，则计算上利期初年金的复利会比普通年金还要多一次。
由于年金是一连串的定期、等额现金支付，因此年金的「年金终值」即为这一连串等额现金支付的个别终值总和。若有一个人每年都支付1,000元年金，每年年尾付款（普通年金），持续五年，利率10%，则可计算得该年金其终值会是：1,000×（1+10%）4 +1,000×（1+10%）3+1,000×（1+10%）2+1,000×（1+10%）+1,000=6,1051元。而若改为期初年金，在每年年头付款，则由于在每期期初即支付了1,000元，因此在复利过程中，每期期初年金的终值都较普通年金多复利了一次，其终值将会为1,000×（1+10%）5+1,000×（1+10%）4+1,000×（1+10%）3+1,000×（1+10%）2+1,000×（1+10%）=6,7156元。将以上算式一般化，通过等比级数公式，则可以得到第 n 个时间点为计算时点之普通年金终值为：
formula_3
其中，A是每期固定支付金额，i为利率水准；FIVFA为年金终值利率因子，可以由查表所得。而期初年金因为多复利一次，故其年金终值为：
formula_4
通过回推上述公式，则可以由年金终值推回年金现值。回推年金现值就相当于是在问「如果未来的那一天，年金内的金额被一次性地还给了付款人，则到时这笔还款的金额，就相当于是在『现在』这个时间点，一次性地还给了付款人多少钱呢？」，或者，也可以理解为是在问「如果年金付款人希望在未来的某一天，以某一笔特定的价格卖掉届时他已经付了几次的年金，那么，他未来到时会拿到的这笔钱，就相当于『现在』这个时间点，他手上握着多少的现金呢？」，又或者是在问「在未来某个时间点价值金额为某个数值的年金，在现在这个利率水准下，对我来说的价值相当于多少钱？」一个人现在这个时间点上所手握的现金，会在未来一次一次的支付中，一期一期的复利成为未来的年金终值。而一笔年金的折现即为一系列支付金额的折现值。对普通年金来说，其数学表达为：
formula_5
对期初年金来说，则为：
formula_6
永续年金
世界上大多数的年金都会有一定的支付期间，然而有一种年金的支付是没有期限的，此种年金称为永续年金。由于永续年金没有限，计算其终值可视为无意义。真正有价值者应为其之现值。当利率小于1时，也就是在绝大多数情况下，可由等比级数公式导出一个普通永续年金的现值为：
formula_7
其中，i为利率，PMT为每期支付的金额。永续年金的现值即为所有个别PMT的现值总和；然而由于是无限期支付，因此在符合收敛的条件下，无穷等比级数的应用才能得以实现。期初永续年金的现值则为普通永续年金的现值再乘上（1+i）倍。
参见
威廉·阿道夫·布格罗（William Adolphe Bouguereau，1825年11月30日 - 1905年8月19日），是19世纪末的法国学院派最重要的画家之一。布格罗的绘画常以神话为灵感，以19世纪的现实主义绘画技巧来诠释古典主义的题材，并且经常以女性的躯体作为描绘对象。布格罗在世时，荣誉众多，在法国和美国极负盛名，作品也都以高价售出。然而，作为当时沙龙画家中的佼佼者，布格罗成为新崛起的印象派等前卫艺术的首要攻击对象。到20世纪，现代主义大兴，布格罗逐渐为人遗忘。直到1980年代，艺术节重新燃起对人物画和19世纪画风的兴趣，布格罗的绘画才又开始得到重视。布格罗一生中至少创作了828幅的画作，还有许多其他作品的下落不明。
生平
1825年，布格罗生于法国比斯开湾小城拉罗歇尔。布格罗的父亲是小商人，先后经营葡萄酒和橄榄油生意。布格罗对于接手家里的生意兴趣索然，他的叔叔同时也是天主教神父的尤金于是开始教导他古典主义和圣经的题材，并且安排布格罗到波尔多的艺术学校半工半读两年，学习绘画。这段期间布格罗还帮果酱罐子公司设计商标来存钱。
不久布格罗的父亲终于被说服，让布格罗在1846年用卖出33幅画像的900法郎加上亲戚的筹款，到巴黎的法朗索瓦·爱德华·皮柯特画室学习，两个月后作为100名学生中的第99名进入法国美术学院。除了艺术上的学习外，布格罗同时也学习生物解剖、考古、和历史服装等课程。在皮柯特的教导下，布格罗继续学习学院派的画风。当时学院派将历史和神话题材作为评量艺术的最高标准，擅长这些领域的布格罗在1850年得法国国家艺术奖学金罗马大奖。获奖的待遇之一，便是与其他获奖者一起前往意大利游学，并居住在著名的美第奇别墅，布格罗游历了意大利的许多城市，模拟古代及文艺复兴时期艺术家的作品。
布格罗的画风一向坚持学院派的传统，他的画都会在每年的巴黎沙龙上的明显位置展出。在早年时一名评论家便说：「布格罗先生对于轮廓的拿捏有一种天分，他完全着迷于人体的韵律上，并且完美呈现了古代和16世纪画家们在这个题材上的画风，在这点上我们必须赞美布格罗先生的成就…拉斐尔就是从古代画家那里寻找了他的灵感…而没有人会指控拉斐尔的作品不是原创的。」
拉斐尔是布格罗最崇拜的画家，他也因此相当得意他的作品被与拉斐尔相提并论。不久后布格罗完成了模画拉斐尔的经典作品《嘉拉提亚的凯旋》，作为从法国美术学院毕业的条件之一。布格罗在诸多作品中，都已古典主义的方式来描绘结构、形状、以及题材。他对女性的描绘也在这时脱颖而出，通过他的画风，他可以将一个女人以最美丽的方式呈现在完成的作品上，同时却又保持了现实人物的长相。
1856年，布格罗与玛丽-奈莉·玛爵伯恩结婚，两人后来育有五名子女。到1850年代末，布格罗已经与当时的艺术界买卖商有密切的关系，尤其是画商保罗·丢朗-吕厄（Paul Durand-Ruel，后来成了印象派的重要支持者之一），丢朗-吕厄帮他在沙龙上卖出了大量的绘画。巴黎沙龙每年都会吸引超过30万人参观，对于当时画家的知名度宣传是极为重要的场合。布格罗的名声在1860年开始传到英国，同时他利用增加的收入在巴黎的蒙帕纳斯买下了一间房子和画室。
》，1879年。
布格罗一生中一直维持传统派的画风，作品经常以神话中的人和事为题材，不局限于基督教，对女性的裸体有特别的重视。在他的画作里，女性的形象非常恬美，有妇女，仙女和农村姑娘等人物。其环境多为乡间丛林，宁逸静瑟。在他的画下，古代的神、宁芙、洗澡者、牧羊女、圣母等等都被以19世纪的现实主义画风重新诠释而在画上获得新生，呈现出高度理想化了的样貌，这也使当时有钱的艺术赞助者对他的画爱不释手。
布格罗以传统的方式来完成一幅画，包括了利用铅笔素描作为先前草稿，再利用油画作为第一步的初稿，他仔细的画风使得他的画都呈现了一种在视觉上让人满足、同时也精确描绘出人体形状的样子。布格罗在描绘皮肤、手臂、以及脚的部分尤其被尊崇。不过除了神话以外，布格罗也常使用一些之前大师所描绘的宗教和情色题材，例如隐喻了失去童贞的《破裂的水罐》。
布格罗同时也接受许多人雇请帮忙以绘画装潢居家、公共场合、教堂等等，在过程中布格罗有时候会绘画自己的风格，其他时候则遵从顾客的要求。在早年布格罗不断接到这三方面的生意，也使他获得滚滚而来的收入和声誉。除此之外他也常将他装潢画的作品重画卖给其他顾客。布格罗也是当时最高档的肖像画家之一，许多他替有钱顾客画的肖像画至今当还在私人收藏家手里。
布格罗在法国美术学院的地位不断上升，并在1876年成为终生会员，并且在1885年获得了法国荣誉军团勋章。他在1875年开始于朱利安学院授课，与法国美术学院不同的是，朱利安学院也招收女学生，同时没有入学考试，而且学费也较为便宜。
1877年，布格罗的妻子因难产而死，他后来（1896年）与一名他教导出的女画家伊丽莎白·珍·加德纳结婚。由于他的第二个夫人也是他的学生，所以当时受夫人的影响，极力帮助女性画家举办画展。尽管在画风上布格罗是标准的传统主义者，然而他在世时却利用他的影响力使许多法国的艺术机构开始开放给女性学生就读，包括了法兰西学术院在内。
晚年的布格罗曾这样描述他对于艺术的热爱：「每天早上到画室时我都充满了喜悦，到了晚上因为黑暗而不得不停工时，我都迫不及待明天早晨的到来…如果没办法将我献身于我热爱的绘画上，我将是多么的悲惨。」
1905年春天，布格罗在巴黎的住所兼画室遭窃。同年8月19日，因心脏病发死于他的出生地拉罗歇尔，享年79岁。
作为艺术大师
从1860年代开始布格罗开始于朱利安美术学院授课，在那里他将他绘画的技巧和心得分享给来自世界各地的艺术学生，包括女画家在内。在他几十年的教学生涯里他指导了数百名甚至上千名的学生，许多人会在他们各自的国家成为知名的画家，有些人会延续布格罗的画风，而其他人例如亨利·马蒂斯则反叛发展出了不同的画风。
名声和20世纪的忽略
》，1885年。
布格罗在世时被当时世界上的艺术界认为是最伟大的画家之一，同时也因此被前卫艺术流派大肆攻击。除了在他的母国法国外，他在比利时、荷兰、西班牙、美国等地都享有崇高的名声，他的画也都以市场上的超高价卖出。
布格罗的画家生涯几乎是一帆风顺，从没遭遇什么挫折。对当时许多人而言，布格罗的画呈现了最高尚的品味和细致，也完全符合艺术界的传统。而对反对他的前卫画家和20世纪的现代主义者而言，他则是过去时代的画家，有着高等的绘画技巧，但却不合新时代潮流。爱德加·窦加和他的同僚便以「布格罗派」来贬称任何他们认为只是依靠「华而不实而人工的表面」的艺术风格。尽管如此，窦加在1872年一封信中称他努力模仿布格罗在绘画上有条理和纪律的绘画过程，虽然以窦加的个性来看，这句话可能只是一种嘲讽。
布格罗的画也被许多美国的百万富翁拼命抢购，他们认为布格罗代表了当时法国最重要的画家。到了1920年之后，布格罗和其他学院派的画风开始走下坡，现代主义和抽象艺术开始崛起，在接下来几十年里现代主义会将布格罗等19世纪画家的作品打入冷宫。
后续影响
从1970年代到1980年代开始，19世纪画风开始重新受到重视。在1974年纽约文化中心（New York Cultural Center，也就是现在的艺术和设计博物馆）出于好奇，决定举办一场布格罗的作品展览。当时美国的艺术收藏家Robert Isaacson已经开始了一场要替19世纪画风恢复名誉的运动。在1984年马克·保季画廊展出了23福布格罗的油画和一幅素描，并在后来的1997年举办了一个早期的互联网展览。1984年加拿大的蒙特娄美术馆也对布格罗作品进行了一次大型巡回画展，这次展览从从法国的小皇宫开始，经过沃兹沃思学会，最后到达蒙特娄。在2005年加州的保罗·盖蒂博物馆展出了三幅布格罗的作品，迅速成为了当时博物馆里最受欢迎的展览，吸引了数万名访客挤在大厅里要观赏展出。
学生
布格罗曾在巴黎国家高等美术学院及朱利安美术学院任教，学生众多。他的学生包括：
清朝官职表枚举出清代乾隆以后的官职，分为文官、武官，官阶高低。
说明
下表基本体制为乾隆皇朝以降之吏部官制，于此之前官制变动太大，难以笔墨概之。
参见
布莱恩特·塔克曼（Bryant Tuckerman，），美国数学家，生于内布拉斯加州林肯市。
他在普林斯顿大学修读拓扑学期间研究出将的一面全部转换的方法，这个方法称为。
1971年3月4日，他靠计算机发现了第24个梅森质数formula_1。
他亦是DES的开发小组成员之一。
赖瑞·乔·柏德（Larry Joe Bird，），出生于印第安纳州西贝顿春谷。昵称大鸟和人猿泰山，为前美国职业篮球运动员，曾担任印第安那溜马总教练与总管。他被公认为有史以来最伟大的篮球运动员之一。
大鸟伯德与魔术师约翰逊两人是80年代的NBA的领袖人物，被称黑白双煞、西魔东鸟，也是NBA历史上最优秀的球员之一，被誉为“史上篮球智商最高的人”。他的12年职业球员生涯全部在波士顿凯尔特人（Boston Celtics）度过，其位置是小前锋或大前锋。
早期生活
柏德生于印第安纳州西贝顿春谷，后在该州的弗兰奇利克附近的一个小镇上长大。在高中时期，他就是当地的明星球员。他曾经加入了印第安纳大学的校队，但却在该队的首次比赛之前离开了。后来他在印第安纳州立大学完成了学业。1979年，他带领该校无花果树队进入了NCAA联赛的决赛，却败给了由魔术强森带领的密歇根州立大学的斯巴达人队。这场比赛在之后的25年中一直是收视率最高的大学篮球比赛。柏德赢得了年度最佳大学篮球球员的奈史密斯奖以及约翰·伍登奖，被评为当年的最佳大学篮球球员。
职业生涯数据
NBA例行赛数据统计
!赛季
!球队
!出赛
!先发
!平均上场时间(分钟)
!投篮命中率(%)
!三分球命中率(%)
!罚球命中率(%)
!平均篮板
!平均助攻
!平均抄截
!平均阻攻
!平均得分
NBA季后赛数据统计
!季后赛
!球队
!出赛
!先发
!平均上场时间(分钟)
!投篮命中率(%)
!三分球命中率(%)
!罚球命中率(%)
!平均篮板
!平均助攻
!平均抄截
!平均阻攻
!平均得分
†代表该年度拿下总冠军
莱尔富便利商店，简称莱尔富，是由光泉牧场投资成立的台湾第一家本土便利商店品牌。1988年6月，正式成立董事会，展开筹备工作。1989年4月，莱尔富国际股份有限公司正式成立于台湾台北市大同区迪化街。同年9月正式开立9家分店（延吉、安东、民族、北安、木新、民权、延平、南港、南港二）。
莱尔富成长快速，于1994年7月开设了第100家分店。2004年6月，开设了第1,000家分店。至2010年2月为止，店数已经有1,250家。至2018年12月为止，店数已经有1,380家，2018年营收220亿元，员工1,800人，若加入加盟店员工合并计算后为9,400人。
莱尔富属于光泉牧场关系企业，总经理为光泉创办人汪金来三个儿子之一的汪裕丰。成立之时不被看好，后来改为自愿加盟形式后，连锁商店数冲破50家。目前采用RC直营、FC委托加盟、FVC特许加盟三种模式经营。
莱尔富除了零售商品外，更致力于电子商务。除代收以外，更支持以塑料货币付款，全国各门市均可使用Mondex、MasterCard Cash等电子钱包，2004年更成为全台湾第一个全面导入多媒体终端机（Multi-Media Kiosk）的便利商店通路，莱尔富的多媒体终端机称为「Life-ET」。2009年设立店内网络购物——hishopping，提供网络购物，到店付费取货服务。
莱尔富的门市遍布全台，在部分学校、政府机关、医院内亦设有门市。但截至2021年5月，却仍未踏足花莲县、台东县、澎湖县及连江县。莱尔富的市场占有率已经排名全台第三大便利商店，仅次于统一超商和全家便利商店。
分店概况
自1989年至今，莱尔富于全台的门市数已突破1,200家，于超商业者中排名第三名。截至2021年5月份，目前开业最久的门市为北市延平店（公司设立登记日期为1989年8月8日），最新开幕的门市为鼓山龙德店（公司设立登记日期为2021年5月5日）。
门市命名规则
也因此命名逻辑，时常造成同一个区域会有两种不同格式店名之区别，例如同样位于北投行义路的不同分店（北市纱帽店、北投行义店），因开幕时间之差异，就造成位于同一条路上之分店命名方式不同的景象。
另外，有时位于同一条路、同一区相邻之分店，分店可能不会另外取名，而是直接以二店、三店之方式命名，如归仁航特店、归仁航特二店。
更有一特殊情况为同一店址之分店，因来客数较多而必须增设机器分流结帐，亦有可能会开设归属于虚拟店之机器编号来分流结帐，主因为早期莱尔富POS系统无法完全支持超过2台以上之收银机进行后端结帐处理，2013年起莱尔富已全面更新POS系统，迄今已不见由此方式而成立之虚拟分店。
特殊门市
莱尔富在部分学校、政府机关、医院、风景区、国道休息站、转运站等设有门市，这些门市因为商圈属性，可能并非24小时营业，与一般的门市有些许差异。
门市风格
自1989年开业以来，莱尔富于各年份的门市装潢风格统一变化不大，但于2017年起，店铺开始陆续更新风格，位于柜台后方的墙面上开始添加Hi-Life英文本样Logo，主因为竞业统一超商、全家便利商店陆续更新店铺风格，莱尔富亦同时跟进。
2020年7月，莱尔富进行商标异动，由原本的爱心外有框线+爱心内Hi-Life字样，改为无框线字样。2020年7月前发包重新装潢的门市Logo维持不变，2020年7月后重新装潢的门市Logo则已更新成新版Logo。
自有品牌及服务
Hi Cafè
Hi Cafè为莱尔富现煮咖啡品牌，与伯朗咖啡合作，采用伯朗咖啡豆及光泉牛乳，尚未成立品牌时曾以「伯朗严选」为名，于特定店铺进行试卖，现已有九成以上门市贩售现煮咖啡。
2010年更曾邀请艺人莎莎代言。
Hi Cafè产品多元，曾与泰山企业御奉合作于2010年推出现泡茶饮，亦曾贩售利用微波冷冻黑糖珍珠制作而成的「黑糖珍珠鲜奶」，还有与阿华田合作推出「冻粒阿华田」等饮品，是国内第一家贩售现泡茶类、珍珠茶饮的便利商店业者。
Hi-Life Original
Hi-Life Original为莱尔富自有品牌，于2016年创立，商品种类包含食物、饮料、日用品等。
Life-ET
莱尔富的店内多媒体终端机，是全台首家于店内架设多媒体终端机的便利商店业者，该机器可打印代收费用、购买游戏点数、寄件、购物、信用卡红利兑换商品等功能。
HiCard
莱尔富的会员卡，消费可累积HiPOINT，HiPOINT可折抵莱尔富门市消费。
HiPay行动支付
莱尔富的行动支付App，可绑定台新银行、国泰世华银行、联邦银行发行的信用卡于莱尔富进行行动支付，亦可于乐法门市使用(限绑定台新银行信用卡)。
Hi立即冲印
与立可得合作，于店内架设可立即拍摄证件照之机台，提供相片冲洗服务。
云端超商
云端超商是莱尔富第一个设立的手机购物App，提供一次买足，多次取货，可跨店取货之服务，于2022年3月15日停止服务，原服务整并至Hi-Life VIP App。
面包优先
面包优先为莱尔富的现烤面包品牌，主打于门市架设烘焙厨房，提供顾客新鲜现烤面包选购，每日出炉两个时段，于2014年结束。
教宗保禄三世（Paulus PP III；）原名亚历山大·法尔内塞（Alessandro Farnese），1534年10月13日当选罗马主教（教宗），同年11月3日即位至1549年11月10日为止。他将英格兰国王亨利八世逐出教会，推动反宗教改革运动，创立耶稣会，召开特伦特宗教会议。
教宗亚历山大六世的情妇朱莉娅·法尔内塞是他的妹妹，首任帕尔马公爵皮埃尔·路易吉·法尔内塞则是他的长子。
参考文献
钟楚红（Cherie Chung，），暱称「红姑」，出生于香港，香港电影女演员及环保人士。她是已故经理人陈自强的义女，于1984年及1987年两度荣获亚太影展最佳女主角殊荣。
背景
钟楚红早年在九龙尖沙咀重庆大厦生活，有三名弟妹（胞妹钟楚仪曾是一名电影演员），父亲经营时装及装修生意。她就读圣玛加利女书院，在学期间曾经任职珠宝店。后来于1979年参选香港小姐出道，当时只得第四名。赛后，她在无线电视试镜失败，却得到电影公司垂青，因而踏足影坛。她的第一部大屏幕作品是杜琪峰的导演处女作《碧水寒山夺命金》，饰演运金镖头的女儿尤佩佩。另一方面，无线电视安排她在《欢乐今宵》演出，因表现不错而有意邀请她签下长合约，同期仍在珠宝店任职，负责鉴别珠宝。1984年，钟楚红凭《男与女》首次获得第3届香港电影金像奖及第20届金马奖最佳女主角提名；之后在1987年与周润发合演《秋天的童话》再度获得提名，可惜仍未能获奖，但她在片中的演出广获好评，同时让她第二次荣获亚太影展影后殊荣，又获得香港导演评选大赏最佳女主角奖。
参与环保工作方面，钟楚红为香港演艺界的环保人士，不时推广环保习惯，于1994年被委任为地球之友社区活动统筹。另外又曾经公开反对明日大屿发展计划，批评明日大屿为有违天意的大白象工程，会令香港独特面貌消失，政府必须停止，香港的整体生态健康才能安全。
个人生活
1984年，广告界才子朱家鼎（Mike Chu）邀请钟楚红拍摄运动产品广告，两人开始交往。1991年12月10日，钟楚红与朱家鼎在美国罗德岛州圣百兰天主堂举行婚礼，花费200多万港币，婚前协议不生育。1994年，钟楚红正式息影。2007年8月24日，朱家鼎因大肠癌在香港养和医院病逝。2009年，钟楚红为圆曾是虔诚天主教徒的亡夫遗愿，在一年多前开始上教堂、报读「慕道班」，在4月11日正式接受领洗成为天主教徒（公教基督徒）。
钟楚红人缘甚佳，圈中好友甚多，尤其是周润发、张曼玉、叶蒨文、张国荣、黄耀明、张学友、陈百强、毛舜筠、郑秀文。
地质作用是促使地表形态发生变化的作用。按照其能量来源，地质作用可分为外力作用和内力作用两种。
外力作用是来自地表以外的作用，主要有流水，冰川，海浪，潮汐，生物，风力等等形式。其能源主要来自于太阳能，但是地球重力作用下的地质作用也属于外力作用。
内力作用是来自地表以内的作用，主要有岩浆活动，板块运动，变质作用等等形式。其能源主要是地球内能，地球内能的积累和地球内部的放射性物质衰变有关。
外力作用对地表形态总体的效果是使之更平坦，而内力作用的总体效果是使地表更加不平坦。目前以地质作用主要以内力作用为主。
高 （学名：Himantopus himantopus），又名黑翅长脚、红腿娘子，为长脚鹬科长脚鹬属下的一个种。
分类
黑翅长脚鹬的分类一直有争议，有人认为可以分为五个种，也有人认为这些种有的是亚种。北美的黑翅长脚鹬在眼后有一块白斑，在鸟类DNA分类系统中，认为是一个种"Himantopus mexicanus"。
生态环境
黑翅长脚鹬常结群活动，喜沿海浅水及淡水沼泽地。可见于湖泊、沼泽、鱼塘、虾池以及河流平缓水面开阔处，栖息于近水的草地农田等处。由于腿较长可在水位较深的池塘种涉水觅食。
分布地域
高跷鸻分布在欧洲东南及南部、印度、中国及东南亚、北非、澳大利亚、加拉帕格斯群岛、夏威夷、加拿大中部和南部、美国西部和东南部。在中国新疆西部、青海东部及内蒙古西北部繁殖，越冬于广东及香港等地，全境有过境纪录。台湾有冬候鸟族群，但亦有许多留鸟纪录。
特征
高跷鸻体态修长，体长越35厘米，通体黑白分明，一双红腿很容易辨认。黑翅长脚鹬的喙黑色细长，雄鸟繁殖羽的从眼后到头顶和颈后黑色，背部和翅为黑色，与颈后的黑色不相连，其他部位为白色；雌鸟的繁殖羽在头和颈后没有黑色，眼后有灰色斑。亚成体的鸟色块分布接近成鸟，但颜色较浅，为灰褐色。黑翅长脚鹬的腿和跗趾修长，为红色，亚成体鸟的腿颜色略浅，显橘红色；飞行时长腿拖于尾后，是重要的辨识特征。虹膜粉红色。
叫声：
食物
黑翅长脚鹬以昆虫、蝌蚪等为食。
繁殖与保护
常集群筑巢于水边土堆或稻田的田埂上，与燕鸻、凤头麦鸡、泽鹬等灰群营巢。巢材采用小石子、泥土、芦苇、枯草等。
嫦娥一号是中国的首颗月球探测器，其以中国古代神话传说人物嫦娥命名。嫦娥一号于2007年10月24日（UTC+8，下同）在西昌卫星发射中心搭乘长征三号甲运载火箭顺利发射升空。嫦娥一号的总重量为2350千克左右，尺寸为2000毫米×1720毫米×2200毫米，太阳能电池帆板展开长度18米，寿命大于1年。该探测器的主要目的是获取月球表面的3D立体影像；分析月球表面有用元素的含量和物质类型的分布特点；探测月壤厚度和地球至月亮的空间环境。嫦娥一号是中国嫦娥工程的第一阶段任务，自2004年1月立项，第一阶段耗资14亿人民币。2009年3月1日完成使命，硬着陆于月球预定地点。
研制
嫦娥一号工程由五大系统组成，分别为卫星、运载火箭、测控、地面应用、发射场。
嫦娥一号卫星由中国空间技术研究院负责研制，卫星总设计师是叶培建。卫星以东方红三号通讯卫星平台为基础，采用三轴稳定姿态控制，具有自身变轨能力。
仪器
科学仪器
嫦娥一号一共带有130公斤有效载荷，其中包括9件主要仪器：：
地面仪器
测量嫦娥一号位置的科学仪器包括：
租用了其他国家设备有：
主要任务
嫦娥一号主要有四个任务：
后期
由于减少了变轨次数，嫦娥一号省下的燃料足够其工作一年有余。在预计工作期满之时，专家预计可以将卫星轨道调整为100公里，以获得更清晰的月面图像。另有计划使嫦娥一号主动撞月。
出于避免突发事故的考虑，嫦娥一号一共制造了两颗，而由于嫦娥一号正式星的成功发射，其备份星失去了备份功能，中国探月工程首席科学家欧阳自远表示，嫦娥一号备份星（即嫦娥二号）将于2010年10月发射升空。备份星将不再进行多次变轨，而是直接进入地月转移轨道。而且，备份星的月球轨道高度也会有变化，星载设备会有更换。
相关事件
现场参观
2007年10月初，西昌卫星发射中心开始对外出售“嫦娥一号发射现场观看”的门票，定价每人800人民币，总额在2000个席位左右。且只向中国公民出售。
歌曲搭载
鉴于之前有东方红一号向地面传回歌曲的先例，此次「嫦娥一号」也将搭载32首歌曲，并向地面传回。
2006年7月7日，国防科工委决定选择30首歌曲搭载，并随后与中央电视台、中国音乐家协会共同举行了全民投票活动。 10月，在科工委提出的150余首备选曲目基础上，选出以下30首歌曲搭载，名单如下：
此外，还特别增加了《东方红》和《义勇军进行曲》（国歌），由于地位特殊，没有经过投票挑选直接入选。
卫星失踪质疑
11月13日，一些网友开始质疑嫦娥一号是否已经失踪，其理由是12日新华社曾发表过写有《嫦娥一号曾与地面失去联系45分钟》的文章。随后相关的质疑帖在网上流传开来。17日，探月工程负责人龙乐豪出面辟谣，指出：由于月球遮挡，地面测控只能覆盖92%至98%的区域，暂时失去联系在设计考虑范围之内，卫星可以自主工作。
传回数据
2007年11月20日下午4:48分，由CCD相机拍摄的第一轨月球图像开始传回。
2007年11月23日嫦娥一号卫星获取的第一批原始图像数据（共19轨）传回地面，并开展整理图像的工作。
2007年11月26日，国家航天局正式发布经处理后的第一幅由嫦娥一号卫星传回月面图像。该图像是由嫦娥一号卫星上的CCD立体相机以线阵推扫的方式获得的，轨道高度约200公里，每一轨的月面幅宽60公里，像素分辨率120米。此共由19轨图像制作而成，位于月球表面东经83度到东经57度，南纬70度到南纬54度，图幅宽约280公里，长约460公里。在图中右侧60公里宽的条带是CCD相机开机后获得的第一轨图像。
2007年12月11日，国家航天局发布嫦娥一号的CCD立体相机所拍取的月球背面部份区域的照片，该图是由5轨图像合并制作而成。
2008年1月31日，国防科工委公布了首副月球极区图像。
2008年11月12日，依据嫦娥一号卫星拍摄数据制作的中国第一幅全月球影像图正式公布。
进驻后改名为「中影八德大楼」
《中央日报》，为中国国民党创办之中文机关报，由何浩若于1928年2月创刊于上海，彭学沛为首任总编辑。1949年随中华民国政府迁至台湾。2006年实体报停刊，实体报总计发行28,356号，之后改为网络报形式发行直到2018年。
中央日报为中华民国国民政府在大陆时期之代表性的官方媒体。 1949年中央政府撤退到台北市后接续在台湾发行五十余年，为国民党党营文化事业成员。报禁开放前销量很高，1960年代至1980年代间与《中国时报》和《联合报》并称为「三大报」。第一次政党轮替以前与《青年战士报》（今名《青年日报》，目前为国防部机关报纸）同为宣导官方立场的主要媒体。
香港《大公报》与马来西亚《光华日报》是少数历史比《中央日报》长的中文报纸。
历史沿革
中国大陆时期
《中央日报》创办的情况比较复杂。1926年12月，随着北伐战争的顺利推进，广州国民政府迁到武汉。1927年3月22日，武汉国民政府在汉口创办《中央日报》，社长由国民党中央宣传部部长顾孟余兼任，总编辑为陈启修。陈启修为中国共产党党员，所以有一批共产党员和左翼人士参加编辑，如沈雁冰、孙伏园等。所以在“四一二事变”后，有不少反蒋中正的文章发表在《中央日报》上。7月15日，“宁汉合流”，武汉《中央日报》于9月15日停刊，共出版176号。台湾新闻史学界为《中央日报》修史时，一般不算入此段时期的《中央日报》。
1927年4月，南京国民政府成立时，就有设立机关报的筹议。正逢上海《商报》停刊，于是国民党接收了其一切设备，利用这些设备，于1928年2月1日在上海创办了《中央日报》，孙科任董事长，国民党中央宣传部部长丁惟汾任社长，国民革命军东路军前敌总指挥部政治部主任潘宜之兼任总经理，彭学沛任总编辑。编辑委员会由各方代表人物组成：胡汉民、邵力子、罗家伦、傅斯年、邵元冲、唐有壬、马寅初、王云五、潘公展、郑伯奇等。2月10日，《中央日报》发刊词中公开表示：“本报为代表本党之言论机关报，一切言论自以本党之主义政策为依据。” 1928年6月，国民党中央政治会议通过了《设置党报条例草案》等三文档。《中央日报》于1929年2月1日迁址南京。《中央日报》迁址南京后，由国民党中央宣传部党报委员会领导，中宣部部长叶楚伧兼任该委员会主席，下设经理部、编辑部；迁址南京后首任总编辑为严慎于，后由鲁荡平、赖琏相继接任。南京《中央日报》特别强调以“拥护中央，消除反侧；巩固党基，维护国本” 为职责。
“九一八事件”之后，面对新局势，国民党在第四次全国代表大会举行前夕，特地召开中央执行委员会临时会议，通过关于《改进宣传方略案》和《改进中央党部组织案》的决议，对改进和加强新闻宣传提出了很多有益意见。据此，《中央日报》率先改组。《中央日报》社原实行总编辑制，社长由中国国民党中央宣传部部长兼任，1932年3月1日改行社长制，脱离中国国民党中央宣传部，改为党营媒体。1932年5月，37岁的程沧波被任命为改组之后的《中央日报》第一任社长，言论报道上直接对国民党中央负责，行政上独立，内部管理实行社长领导下的总编、总经理负责制。1932年，《中央日报》先后增出《中央夜报》、《中央时事周报》。
1935年，《中央日报》开始使用轮转印报机，日出3大张，每日销售数由8,000份增加到30,000份。同年10月，在南京市中心新街口建成了中央日报大楼。由于庐山开办“军官训练馆”，军政人员一时云集，于是《中央日报》在1937年6月开办庐山版。
对日抗战爆发后，《中央日报》先迁至长沙，后到重庆，于1938年9月在重庆复刊，此后又增设长沙版、昆明版、成都版、西康版、贵阳版、屯溪版、桂林版、福建版。《中央日报》西迁重庆7年，报社经历5次改组，社长、总编辑、总主笔、总经理等,均先后更换5任。
1947年，《中央日报》股份改造，成立“中央日报社股份有限公司”。
1948年《中央日报》社开始筹办台北版，做应变准备。1949年，南京总社社长马星野带着总社的大批工作人员和大部分器材、设备到了台北，于1949年3月12日出版台湾《中央日报》。
台湾三大报时代
台湾戒严时期，《中央日报》身为中国国民党党营媒体，享有特殊地位。所有政府机关、学校，奉国民党各地方党部指示，一律订阅《中央日报》，为《中央日报》提供稳定收入来源。1984年10月8日，《中央日报》在台北市八德路二段256号举行总社新厦「中央日报大楼」奠基典礼，《中央日报》董事长曹圣芬、《中央日报》社长姚朋、中国国民党秘书长蒋彦士、中国国民党中央委员会文化工作会主任宋楚瑜、中国国民党中央委员会考纪委员会主任周应龙共同奠基，共有地上11层楼与地下3层楼，基地一千七百余坪，总楼地板面积八千余坪，建成后将取代位于台北市忠孝西路一段83号且已有20年历史的《中央日报》总社。台北市八德路「中央日报大楼」土地为报社于1982年以新台币39亿元向台湾铁路管理局购得，后于1999年该大楼连同土地以新台币2758亿元售予国民党另一党营企业中央电影公司。关于政府机关统一订阅，被部分人士的批评为「以国库资助《中央日报》营运」；关于八德路大楼低价买入的批评，则为「国有财产直接变为党产，不当利益输送的典型案例」之嫌。
1983年4月26日11时许，位于台北市忠孝西路一段83号的《中央日报》总社一楼发生爆炸案，12人被炸伤。法务部调查局证实本案与22分钟前发生的《联合报》总社9楼电机房爆炸案（两案合称「两报爆炸案」）是同一批人发动的炸弹恐怖攻击，放置炸弹者是黄世宗、策划者是台湾独立建国联盟主席张灿鍙、指挥者是台湾独立建国联盟美国本部副主席陈南天、运行指挥与接应者是人在巴西的李朝望，但黄世宗早已偷渡离开台湾且至今未曾被逮捕。
1985年8月16日，宋美龄函电蒋经国：
&lt;templatestyles src="Template:Quote/blockquotecss" /&gt;
「『匪』『人民日报』已在美出版，其排字方式标题醒目，字体有三、四种大小不同者，利于阅读。我《中央日报》皆数十年前之老排版，较长报导因囿于规格，使续读费时寻找，失去传递消息之初衷。但我亦不必抄袭『匪』格式为要。辱暑中，特别注意休息、饮食。母八月十六日」
1985年8月21日，蒋经国函电宋美龄：
&lt;templatestyles src="Template:Quote/blockquotecss" /&gt;
「『匪』『人民日报』在美出版，目的在分化侨社对国家之向心，极其险恶。《中央日报》航空版，正在切实检讨调整版面、充实内容、增加发行量之中。又筹划由《英文中国日报》编印每周画页，通过卫星发送世界日报分赠海外读者。儿近尚身心如常，唯恳母亲倍加珍护。肃叩 福安。儿 经国跪禀八月二十一日」
衰落
解严后，报禁解除，《中央日报》民营化，以国民党为最大股东，实际编辑团队亦以国民党干部为主。《中央日报》与同业一样，每日出刊的总张数增加为六大张。但《中央日报》因传统官营色彩浓厚，编辑版面保守，不及其它报纸灵活生动，发行量每况愈下。当时网络媒体开始盛行，读者阅读习惯改变，实体报纸发行量普遍下滑，各报均进入衰退期，《中央日报》亦无法幸免。
1994年10月，《中央日报》非法解雇记者苏诤，苏诤向「901新闻自主推动小组」（台湾新闻记者协会前身）申诉；1994年11月，901新闻自主推动小组调查本案，认定《中央日报》不合理解雇，要求《中央日报》让苏诤复职，是901新闻自主推动小组第一个保护记者工作权的个案。
1995年，国民党内延续「主流、非主流」斗争，出现党员要求分党产的声浪。当时的国民党秘书长许水德有鉴于此，要求文工会研究是否以免费方式发送小组长一份《中央日报》，以平息党内要求分党产的声音。当时的《中央日报》社长唐盼盼认为，如能争取到党中央免费发送《中央日报》，一方面可以增加《中央日报》的报份，另方面也可以强化他与许水德的关系，于是承接此一任务。不过，由于国民党小组长与《中央日报》订户高度重叠，小组长收到免费的《中央日报》后就把原先出钱订阅的《中央日报》退掉，造成《中央日报》有费报份大幅滑落，损失三分之二左右的报份。《中央日报》因而元气大伤，无法与《中国时报》、《联合报》并列三大报，此后被《自由时报》追过。
1996年6月26日，民进党籍台北市长陈水扁在台北市政府市政会议中指示，禁止台北市政府所属各机关学校刊登广告与统一订阅《中央日报》，理由是：⑴《中央日报》要求台北市政府刊登招标或采购公告，强迫机关学校订报；⑵他不满意《中央日报》报导台北市政方式。此举遭到外界的批评和质疑。
1996年7月，针对陈水扁指示禁止台北市政府所属各机关学校刊登广告与统一订阅《中央日报》案，《中央月刊》报导，「《中央日报》是国民党党营的报纸，陈水扁是民进党籍市长；不知是否党籍情结作祟，陈水扁总把登在《中央日报》上有关市政报导当成国民党在修理他的文章。……后来记者们追踪报导，市府各机关学校订《中央日报》的单位也同时订其他报纸，《中央日报》未享受任何特权；而在民政局赠区委、里/邻长报纸统计中，《中央日报》所占比例极少，也证明该报并未运用特权强迫市府单位订阅；所以陈水扁『打倒特权』选《中央日报》发作，也选错了对象」，而陈水扁发动拒订《中央日报》的可能原因是该报批到了陈水扁的逆麟：
2000年政党轮替后，由于过多政治版面、内容与社会脱节，《中央日报》除传统国民党员及图书馆外，一般民众罕有阅读甚至订阅，零售销售据点也大量减少。《中央日报》开始出现赤字，平均每月亏损新台币844万元，累积亏损新台币8亿余元。
解严后，不断有「党政军退出媒体」的声音；特定政党持股过重或由政治人物经营的媒体，如《中央日报》、《自由时报》、中国电视公司、民间全民电视公司等，均饱受抨击。而2000年政党轮替后，过去政府财政文件公开，中央日报大楼等来源有问题的资产亦成为抨击的目标。舆论压力下，《中央日报》的主要经营者国民党亦开始寻求出脱持股。
实体报停刊
2002年4月，中国国民党主席连战裁示推动《中央日报》人事精简案，《中央日报》编制由326人调整为90人。马英九接任国民党主席后，积极寻求买家接手《中央日报》。2005年国民党有意出售《中央日报》及《中华日报》时，因《中央日报》的负债大于资产，无人有意承接；此时又提出「转赠连战」一案（当时连战已经转任国民党荣誉主席），部分干部如总编辑江伟硕、副总编辑樊祥麟等，都被交代准备因应新局，但悬宕未决，不了了之。同时有媒体报导，国民党大老陈立夫的家属有意承接《中央日报》，但国民党中央对《中央日报》的态度暧昧不明，导致最后议案无疾而终。
2006年5月24日，国民党中常会通过《中央日报社股份有限公司股权处分案》，停止挹注《中央日报》每年新台币9,000万元的补助经费，避免实体报纸的出刊继续造成《中央日报》财政负担。2006年6月1日，《中央日报》停刊。《中央日报》停刊的消息相当突然，未依《劳动基准法》向员工预告停业，所有员工突然一起失业，引起《中央日报》产业工会激烈反弹及抗争。
2006年5月24日，《苹果日报》根据《中央日报》员工爆料内容报导，黄辉珍在《中央日报》社长任内「为当时的总统李登辉出言论集，花掉（新台币）十五亿元」；同日，黄辉珍否认。
网络报停止更新
据中央社报导，因为投资方「伟业投资股份有限公司」资金不足，《中央日报网络报》14名员工于2018年6月接获离职通知，离职时间为2018年5月31日。《中央日报网络报》页面停止更新。当前无法连上。
原来与YAHOO奇摩、PCHOME等门户网站新闻合作的商标超链接，现在被导引到另一个网站《大华网络报》，而《大华网络报》的网站架构与新闻、评论等内容，基本上与停刊之前的《中央日报网络报》几乎相同，但没有信息显示营运者是否相同。
据询问《中央网络报》官方粉丝专页，目前暂停一段时间再出发，现处解散状态，预计2020大选之后重振旗鼓。当前建议关注言论方向一致之《大华网络报》。但截至2021年7月，该报脸书粉专仍处在停止更新中。
特色内容
《中央日报》的副刊，名为「中央副刊」，简称「中副」，与《中国时报》「人间副刊」及《联合报》「联合副刊」同为1970年代至1980年代的主要文学发表媒介，许多作家都在中央副刊上贡献过一方笔墨，中央副刊也成就了许多的作家。由于《中央日报》有发行航空海外版，中副除了反共文学与怀乡文学之外，也经常刊登海外留学生作品。中副的特征之一是版面下方的读者投稿笑话「趣谭」以及美国漫画《白朗黛》（"Blondie"）。
「地图周刊」也是《中央日报》的重要特征。
二代目光生会（こうせいかい）本部事务所位于福冈县福冈市博多区博多駅南5-22-36 的设置；日本指定暴力团之一・六代目山口组二代目伊豆组的三次团体。
最高干部
（2016年）
下部团体
二代目高桥组、池组、加纳组、栗秋组、健诚会、极龙会、张本组、八诚会、极诚会、浦山组、诚勇会、克诚会、极真会、永木会、田中会、田城组、马场组、诚仁会、中野组、八重组、诚道会、竹松组、加来组、高桥会、藤本组、松中组等。
在柏林
张申府，原名张崧年，字申甫，男，直隶献县人，中国哲学家，中国共产党创始人之一。
生平
早年在献县读私塾，1907年被父亲送进北京的小学念书。1909年入读北平顺天高等学堂中学班，1911年，辛亥革命时曾在天津《民国报》上刊登《零金碎玉》等文宣传革命，革命后，学校停课就返回家乡结婚。1912年夏转入北京高等师范学校附属中学，1913年考入北京大学预科。1914年考入北京大学文科哲学系，后转入理科数学系。1917年毕业后留校任助教，后任讲师。
1919年，五四运动时期，和陈独秀、李大钊创办《每周评论》，参加新潮社，同时也担任过《新青年》的编委。1920年春，共产国际的代表维经斯基到北京来，经过北大俄籍教授柏烈伟的介绍认识了李大钊和张申府，又由李大钊介绍认识了陈独秀。9月中旬，张申府因罗素来华讲学去上海迎接，在上海时住在陈独秀家就筹组党组织事宜多次交换意见。10月，回京后和李大钊成立了北京共产主义小组，后来发展了张国焘加入，张也是北京共产主义小组的第三个成员。1920年12月，和蔡元培、陈大齐、刘清扬、郑毓秀几个人同船赴法，在里昂中法大学教逻辑学，后在巴黎和女友刘清扬一同介绍周恩来加入中国共产党，同赵世炎，陈公培一起在巴黎成立了共产党小组。1921年9月，爆发了里昂大学事件，张为此联系吴稚晖协调未果便离开了中法大学。1922年2月，与刘，周二人转至德国，创建柏林小组。在德国时常去哥廷根游学1923年2月30日比扬古会议上陈延年和尹宽指责张干预少年共产党的工作故将他开除出党。少年共产党后被共产国际更名为中国共产党旅欧支部，恢复了张的通信员地位。7月，返回巴黎，年底，取道苏联回中国。1924年2月，抵广州，经李大钊介绍在广州大学任教。5月，被孙中山任命为黄埔军校政治部副主任（主任为戴季陶），6月辞职到北京度暑假。1925年1月在上海中共四大上，关于与国民党合作问题，与张太雷冲撞，后退出中国共产党。1928年在暨南大学、大陆大学、大夏大学任教。1929年返回北京在中国大学、北京大学任教。
9号
1931年5月应冯友兰邀请赴清华大学哲学系任教。1935年参与一二九运动，1936年1月，北平文化界救国会成立，张被推选为大会主席团成员和救国会的执行委员，后又担任华北各界救国会的负责人。2月29日被捕，5月被冯玉祥保释出狱，但被清华解聘。抗日战争期间参与创建民盟。1946年1月，作为国民参政会参政员参加“旧政协”。1948年秋天，因发表《呼吁和平》一文，公开承认国民政府的宪政，拥护蒋中正的戡乱政策，被民盟开除。12月16日，《人民日报》发表文章，“痛斥叛徒张申府的卖身投靠”。妻子刘清扬也登报宣布跟他离婚。1949年后在北京图书馆任研究员，娶满族关女士为妻。1957年春，整风运动开始，4月27日，张申府也在《光明日报》上发文主张百家争鸣，思想解放。6月8日，反右开始，罗隆基，章伯钧，储安平纷纷被划为右派。张申府在批斗章伯钧的大会上发言：“章伯钧的立场不够百分之百坚定，但也够百分之九十五坚定“关于章伯钧所说的组织‘政治设计院’的意见，还是值得考虑的。”张申府也因此被划为右派。
晚年少问政事，曾接受美国犹太裔历史学家舒衡哲女士的采访，披露了一些关于中共早期组织和领导层的细节史料。其访谈被舒整理为《Time for Telling Truth is Running Out》，中文版题为《张申府访谈录》，有删节）。1986年6月20日在北大医院去世，死于肺功能衰竭。
家庭
父亲张濂，母亲赵氏。张有三个弟弟，二弟张崇年，三弟张岱年，小弟在五岁的时候溺水死了；还有两个妹妹，都没有念书。张道兴是他的族侄。
第一任妻子：朱德侬（1914年在分娩后去世）
哲学思想
张申府热衷于数学哲学，是中国积极译介英国哲学家伯特兰·罗素的第一人，也是中文界翻译维特根斯坦的第一人。在罗素访华时，二人有过学术交流。中华人民共和国建国后，罗素失去了和张的联系，还曾专门写信询问张的下落。
大理宣宗段智兴，又名段易长兴，后大理国君主，在位时间自1172年至1200年，共28年。在位期间大修佛寺，建了60寺院，对国力造成损害，他本身也终日崇佛，不理国事。死后谥功极皇帝，庙号宣宗。
金庸作品《射雕英雄传》及《神雕侠侣》中的段智兴（法号「一灯」）即以他为原型人物。
优胜美地瀑布（Yosemite falls），是北美洲落差最大的瀑布，位于美国加州内华达山脉，属于优胜美地国家公园，其最壮观的季节在春末，水量充沛，气势惊人。又译为约塞米蒂瀑布。
优胜美地瀑布全高为739公尺（2425呎），一共可分为三段，瀑布的落差距离排名世界第六，虽然在外观看来仅有上优胜美地瀑布与下优胜美地瀑布两段，但事实上包括了中间的落差地带，这三段分别是：
在降雪量比较少的年份，优胜美地瀑布在夏末或秋初就因水量减少而逐渐消失，此时往往有攀岩者趁机挑战从岩面攀登而上，此举在突如其来的午后雷阵雨中，是相当冒险的，瀑布可能突然出现，水量足以将攀岩者冲落。
下优胜美地瀑布可从优胜美地旅馆旁的步道到达，上优胜美地瀑布则需由向阳营地Sunnyside Campground的入口进入，此步道相当陡峭、险峻、不少路段亦相当粗糙、容易滑倒，来回行程长度是，落差达，预计来回行程要花6～8小时。其他通往上优胜美地的步道，主要是由北方的Tioga Road前来。
关于其北美最高瀑布的争议、有一说是华盛顿州的殖民地溪瀑布（Colonial Creek Falls）新量测出来的结果可能高于此瀑布有150英呎。然而此说法尚未被证实。
zh-tw:杰弗里·威廉·「杰夫」·胡恩;zh-hk:禤智辉;zh-cn:杰弗里·‘杰夫’·威廉·胡恩（Geoffrey William "Geoff" Hoon，），英国政治人物，前任国防大臣、下议院领袖兼掌玺大臣、、工党党鞭兼财政部政务次官、运输大臣；2010年卸任下议院议员。
口簧琴，又称口簧、口弦、嘴琴等，是一种薄片（英语：Lamellophone）乐器，由连接在框架上的柔性金属或竹舌或簧片组成。 口簧琴很可能起源于西伯利亚，特别是在阿尔泰山脉或其周围。 它们是一种古老的民间乐器，广泛使用于亚洲（特别是东南亚、南亚以及北亚）、美洲 、大洋洲以及欧洲，非洲的记录则相对较少。
口簧琴的乐器主体是一个一端固定于框式琴台上，另一端可自由振动的簧片，乐器演奏者将琴台置于双唇之间，以拨动簧片或拉扯一根系附于琴台上细绳的方式，使簧片振动发声，并以演奏者的口腔共鸣产生乐音。
口簧琴可以被归类为 idioglot 或 heteroglot（无论框架和音叉是否是一体的）； 根据框架的形状（杆或饰板）； 根据尖齿的数量，以及尖齿是弹拨的、联合敲击的还是拉弦的。
乐器的形态概说
在乐器分类上的位置
由于口簧琴的声响来源是簧片的振动，因此若依照一般常用，将乐器分为弦乐器、管乐器、打击乐器的分类方式，则口簧难以归类。而根据现今音乐学上所通用之霍恩博斯特尔-萨克斯分类法（Hornbostel-Sachs，简称H-S分类法），口簧乃属于体鸣乐器（分类编号 1）下，弹拨体鸣乐器（plucked idiophone，分类编号 12），其分类编号为 1212。
兹将H-S分类法中，关于口簧的部份节录于下：
H-S分类法中所谓的「弹拨体鸣乐器」，是指该乐器的最主要发声体为一或多个一端固定之弹性簧片，然而所用「弹拨」一词，并不适用于以拉动细绳致使簧片振动产生声响的口簧种类。
有些音乐学者如1943年曾至台湾本岛进行音乐调查的日本民族音乐学家黑泽隆朝认为，H-S分类法中，将口簧琴从形态学观点区分为自体口簧及异体口簧的方式并不十分恰当，而另采演奏方法为标准，划分做弹口簧以及绳口簧（黑泽使用的日语原文做「弹き口琴」与「纽口琴」）两大类。
中国大陆的口簧琴
民族音乐学家从传说中推测认为新石器时代晚期出现。先秦时期称“簧”，《诗经·小雅》：“巧言如簧。”约在唐宋时期有“竹簧”、“铁簧”、“铁叶簧”、“雅簧”等名，明清时期主要称“口琴”，现在通常称“口弦”。在中国少数民族地区流传广泛，各民族称呼不一。与古代汉文化的名称有密切关系。在流传有口弦的38个少数民族语言的汉译名多达100个以上。
簧在少数民族中是谈情说爱的乐器之一，还常被用来作为青年男女定情的信物。多片制的簧往往配有一个小竹筒，竹筒表面刻有精致的花纹，随身携带，以便在需要的场合演奏。
结构形制
一般在一根长薄竹片上刻出一簧舌，舌根扁薄舌端细长的尖，是发音部位。外缘为簧框，簧框在近舌根外的一端为股部；簧框在近舌尖外的一端为首部，是手的执持部位。
框的形状主要有竹制的长薄形和金属制的马蹄形两种，可以弹拨股部发音，可以弹拨舌尖发音，也可以拉拽股部的线来发音。
多片口弦组合的形制。
演奏与调音
以左手执首部，框横置于口，用两唇轻含，用右手拨簧的股部，使簧舌振动，同时以口腔为共鸣器，不断改变口腔大小，可奏出一系列自然泛音。常见形制的簧舌为长三角形和长条形，舌尖凸出一尖刺。簧也常用来与其它乐器合奏或为歌舞伴奏。
为了同其他乐器合奏，可调整簧的基音。竹簧的调音是以切削股部的方法来进行的，股部的长度决定基音音高，也有在簧舌粘蜡珠的方法来调音的。
台湾原住民的口簧琴
台湾原住民几乎各族都有口簧琴的使用。由于平埔族汉化日早，目前已无确实的口簧实物留下，但根据明清时代的文献来看，平埔族群的原住民确有口簧琴的使用。至于目前官方承认的16个原住民族中，只有达悟族人不使用口簧琴，其他各族都有口簧琴，又以泰雅族、太鲁阁族及赛德克族使用最为有名，其次为阿美族、布农族等。民族学、人类学家以及私人爱好者有收藏各族口簧琴实物，并有诸多音乐学者对上述提到的几个族群的口簧琴音乐留下录音。
泰雅族
泰雅族口簧琴泰雅语称为lubuw，用于泰雅人的自娱、男女诉情等场合，老幼皆宜，一般在出草、狩猎、丧事期间禁止使用。泰雅口簧以其多簧口簧著名，传说可至8簧之多，然文献至多仅记录到6簧，现代泰雅族人已尝试制作全套包括竹台单竹簧、竹台簧金属簧、2簧、3簧、4簧等到8簧的口簧组。多簧乐器的簧片音高，调成与泰雅传统音乐音阶的排列相同，即「大二度－小三度－大二度」。
除乐器实体的多簧构造十分特殊外，泰雅族口簧亦以其口簧舞著称，男女相对，一面演奏着口簧琴，一面左右擡脚随着音乐节奏踏跳。泰雅人也可以使用口簧代替实际语言来传递消息，即将原本用「口说」的内容通过口簧，表达给对方知道。有一说法是青年男女用这种方式说悄悄话不让外人得知，然而今天已经越来越少人还具备这种能力了。
太鲁阁族
由于太鲁阁族过去被视为泰雅族的亚族，因此文献记录常常没有分辨出来，我们只能进一步从收集或录音的地点来判断是否为太鲁阁族口簧。太鲁阁族口簧琴较为常见的有lubug qawqaw（竹口簧）、brux/lubug kingal（单铜片口簧）、lubug dha（2铜片口簧）、lubug spat（4铜片口簧）、lubug lima（5铜片口簧琴），其它如6、7、8铜片口簧较为少见，比较不常见到3簧的口簧琴，传说3在太鲁阁族里是不吉祥的数字，所以3簧口簧琴就不被族人制作。另有一种说法是，据太鲁阁族Hayu Yudaw（哈尤．尤道）牧师说法，太鲁阁族少见３音律的歌，３簧口簧琴不易伴奏与诠译太鲁阁族传统的歌谣，因此太鲁阁族的社群中没有３簧的口簧琴。
2015年，太鲁阁族人Lahang Lowsing(拉罕罗幸)为了更积极推广口簧琴，引入３Ｄ打印及激光切割技术来降低口簧琴制作的门槛，开创了台湾原住民族口簧琴新兴制程。(详见口簧工竂)
文化霸权（Egemonia culturale，Cultural hegemony）也称为文化领导权、领导权，是意大利马克思主义者安东尼奥·葛兰西所提出的哲学和社会学理论。它指的是：一个社会阶层可以通过操纵社会文化（信仰、解释、认知、价值观等），支配或统治整个多元文化社会；统治阶级的世界观会被强制作为唯一的社会规范，并被认为是有利于全社会的普遍有效的思想，但实际上只有统治阶级受益。阶级都要通过其知识分子在公民社会争取到知性与道德上的领导权。
历史追溯
文化领导权最初由格奥尔基·瓦连京诺维奇·普列汉诺夫在1883年-1884年间作为推翻沙皇的策略之一提出的。其后，列宁使用这个概念指出，无产阶级不要躲避资产阶级的革命，而要领导社会各阶级去争取推翻沙皇的胜利，同时不能放弃对资产阶级革命的领导权。之后，经过葛兰西的完善，才成为了一个概念。
葛兰西的文化霸权理论
葛兰西在二十世纪初期，见到十月革命的成功，以及西方资本主义国家并未发生如马克思所预期的发生革命，因此提出了文化霸权的概念来作进一步的探讨，希望能够对资本主义社会作更深入的了解，并依此来订立更有效的革命方针。
根据马克思与恩格斯的理论，资本主义将不可避免地导向无产阶级革命，而且必然先在工业化的欧洲爆发；而由于资本主义生产方式的本质，在这过程中将产生与革命运动相应的文化与政治上层建筑。虽然马克思早在1848年就如此预言，百年后的工业化国家工人仍然没能达成这个任务。
葛兰西针对马克思主要强调社会经济结构、而对思想文化等上层建筑的影响以及个人的主动性较不重视的情形加以修正。他认为下层结构是「制约」而非「决定」上层结构。他提出的文化霸权理论，指出了一个政权的维持，是需要政治的强制力加上霸权文化的力量配合；而后者来自于在市民社会的配合之下，以包括如教育、大众传播媒体等对于大众的潜移默化，造成了工人阶级的虚假意识，使此一霸权得以维持。在所谓先进工业化国家中，文化霸权的力量尤其隐密而强大。因此，要霸权有所转变，必须由知识分子先以所谓的阵地战进行长期准备，与社会上的许多集团合作以及并逐渐促使其意识形态的转化，取得了此一基础之后，方能进一步使霸权转移。
葛兰西认为在朝与在野阶级通过各自的知识份子，必须争取到对公民社会在心智上与道德上的领导权，他甚至认为一定先要有霸权，才能进一步取得政权，也就是在西方政治思想中合法垄断武力的权利。每个利益团体(意识形态)要通过争霸来让他人认肯。
尽管葛兰西的文化霸权论起源于对阶级斗争状况的分析，这套理论可以应用得更广。根据葛兰西的理论，当前盛行的文化范式既非自然的，也非必然的；相反地，我们必须从包括机构、实践及信念等等的文化范式中分析其支配结构。
文化霸权并不是单独的或统一的，它是社会结构的分层展现。其中的每一种结构都有各自的「使命」与内在逻辑。然而一如军事组织，每个结构都假设了其它结构的存在，并彼此结合以达成更高的全局使命。霸权便是如此运作。社会中的每个人都有各自的生活方式，对于个人，社会的其它部份貌似无关紧要。然而整体而言，每个人的生活都构成霸权结构的一环。在日常生活的琐碎活动中，多样性与自由意志看似存在；而正是借着无数这样的情境，巨观的霸权结构得以维持，而不为生活在其中的人们所察觉（见葛兰西《狱中札记》，页233-38）
葛兰西的这种理论能更进一步解释社会文化与政权变迁的过程，并且更强调人尤其是知识分子的主动性作用，可以弥补马克思过于重视社会经济结构的不足之处。
影响
文化霸权理论对欧洲共产主义、社会科学与社会运动有深远影响。在社会科学领域，米歇尔·福柯等人应用霸权的概念，深入研究了主流论述。在教育学中，霸权理论导向了批判性教学。
此后，后马克思主义的代表人物厄尼斯特·拉克劳和尚塔尔·墨菲对葛兰西的这种理论重新进行了思考，批评葛兰西的理论中存在本质主义问题，坚持领导权的阶级主体性，而忽视了非阶级力量之间的斗争，过分强调领导权在社会斗争中的中心性和单一性。
大日本帝国，又称日本帝国，是日本在1936年至1947年间所使用的国号。在法律使用上最早出现于1889年公布的《大日本帝国宪法》，在1936年后才成为日本官方文档上的通用国号，一般是指1868年－1947年帝国主义时期的日本。
日本经过王政复古大号令及戊辰战争后，政治权力从德川幕府武家重回天皇手中，随后聘请外国人积极引入西方技术，开始了明治维新现代化进程。《大日本帝国宪法》于1889年发布，至1890年11月29日召集了第1回帝国议会。由明治时代开始，日本积极向外扩展势力范围，国力逐渐壮大，成为当时的列强之一。这场成功的改革对日本的政治及文化影响深远，也为后来在欧洲萌芽的图兰主义提供了理论和实践的基础。从十九世纪末到二十世纪中叶，日本曾经在中国的五个城市中开辟过专管租界，有天津日租界、苏州日租界、杭州日租界、重庆日租界和汉口日租界。1902年1月30日，日本与大英帝国结成同盟，双方的友好关系一直延续到1923年盟约失效。二十世纪初，加藤高明内阁于1925年3月公布《普通选举法》，同年4月22日为抑制国内共产主义运动，公布了《治安维持法》，于5月12日起施行。《普通选举法》在1928年2月20日举行的第16回众议院议员总选举得到实行，自加藤高明以后的六代内阁皆为众议院多数党交替组成的政党内阁，由立宪政友会及立宪民政党两大政党轮替执政，开始了「宪政之常道」的惯例。
1930年代中期后，自1912年第一次护宪运动以来的政党政治日渐衰落。早年军部兴起法西斯主义，在1929年经济危机爆发后，军部便乘机制造了一连串政变。1936年，日本爆发二二六政变事件，事后军部扶植了傀儡广田弘毅内阁掌政，广田弘毅按军部的意思改组政府，恢复了在1913年被废除的军部大臣现役武官制，使内阁成为了军部的傀儡。日本自始确立了其军国体制，统制派与大政翼赞会掌控了政治权力。1936年11月与德国签定《反共产国际协定》，次年意大利加入，后有更多反共主义国家加入。
1936年12月，日本在《伦敦海军条约》有效期结束后，不再续签《第二次伦敦海军条约》，同时《华盛顿海军条约》（五大国条约）的有效期亦终结，日本其后重启海军军备竞赛。1937年7月7日卢沟桥事变后，日本与中国全面爆发战争。1940年9月27日，大政翼赞会促成了《三国同盟条约》的签定，日本成为轴心国成员之一。1941年12月7日珍珠港事件后，太平洋战争爆发，日本至此完全涉入第二次世界大战。日本于1945年8月15日向同盟国无条件投降，9月2日正式向盟军签署及递交投降文档，成为二战轴心国阵营最后一个投降的国家，日本被击败后丧失所有在海外的占领地和属地，至此大日本帝国已名存实亡。随后同盟国对日本实施军事占领，同时修改宪法与改造政治制度。1947年《日本国宪法》颁布后，日本官方不再以此为正式国号，大日本帝国结束。1951年9月8日，《旧金山和约》签定，日本才结束长达七年的盟军占领时期，并恢复正常国家地位。
大日本帝国是日本历史上的一个重要时期。自1868年明治天皇发表《五条御誓文》至1947年5月3日施行新宪法为止，大日本帝国存在79年。期间，日本在经济、文化、政治上的成就都达到了很高的水平。在科学方面，大日本帝国时期的日本经历了迅速的现代化，出现空前的工业化发展和技术创新，对后世的发展有着深远影响。国际上这段时期的日本是亚洲唯一的列强，走上了当时流行的帝国主义道路，积极对周边国家进行军事扩张，先后经历中日甲午战争、日俄战争、日韩并合，以及两次世界大战，塑造了近现代东亚的政治局势；侵略战争同时在各国遗留不少历史争端，至今仍未能平息。
名称
1867年11月9日（庆应3年10月14日），德川幕府第15代将军德川庆喜向登基不久的天皇归还政权，称为大政奉还。1868年1月3日（庆应3年12月9日），日本改元明治，朝廷同时发布“王政复古”大号令。根据大号令，以讨幕派的萨摩藩和长州藩的领导班子为中心组成了新的政府，天皇正式重新掌握政治实权。1889年（明治22年）2月11日，《大日本帝国宪法》生效，天皇定为国家元首，政体为二元君主制（君主立宪制的分支），内阁及国会也渐次设置。
之后的时间里，“日本”、“日本国”、“大日本”、“日本帝国”、“大日本帝国”等名称都曾被日本政府使用过。受到西方影响崩溃的藩幕体制，自明治维新以来借由再造天皇制的方式，试图创造日本人现代的国家认同。直到1934年发表：「国体明徵声名」，日本为由天皇统治的国家，首度确立现代国体意义的“日本”主干。1936年（昭和11年），国号正式统一为「大日本帝国」，通称“帝国”。第二次世界大战结束后，败战的日本在盟军占领当局的指导下形成新的国家体系；1947年（昭和22年），《日本国宪法》颁布施行以取代《大日本帝国宪法》，官方文档也改用「日本国」国号，「大日本帝国」一词走入历史。
历史
早期历史
的最后阶段，帝国海军正在扫荡德川幕府的残余军力。图中的两艘军舰为帝国军的东舰与春日丸
的众著名成员。从左至右：木户孝允、山口尚芳、岩仓具视、伊藤博文、大久保利通
幕末时日本诸藩当中，以萨摩藩和长州藩最具政治上的影响力高喊高举「攘夷」大旗，两藩对于「开国」的幕府不满，与朝廷联手逼迫幕府转向「攘夷」，继而不惜主动挑起与西洋诸国的事端。萨摩与幕府对长州作战，种下传统武士的不满加深日后武力讨伐幕府的种子。1866年1月21日，在坂本龙马的中介下，萨摩藩与长州藩缔结同盟，决定在倒幕上合作。1867年11月9日，幕府将军德川庆喜眼见萨长同盟的威胁，决定先行上奏明治天皇，提出大政奉还，翌日获得敕许，将260年来由德川幕府掌握的政权交还天皇，并创设以德川家为中心的新政府，德川家将作为政治中枢继续掌握实权。公家的岩仓具视和萨摩藩的大久保利通等讨幕派见此，策划政变以阻止亲德川派创建以德川家为中心的新政府。在1868年1月3日明治天皇召开的最后一次朝议后，萨摩等五个藩的军队封锁了京都御所的九门，禁止亲德川派的摄政二条齐敬和贺阳宫朝彦亲王进宫。随后，明治天皇在京都御所内学问所发布了「王政复古大号令」，并在当天进行的小御所会议上，决定了要幕府将军德川庆喜向天皇交还内大臣官职和领地。此事刺激了德川庆喜，更激化了德川庆喜与讨幕派的对立。1868年1月中，亲德川派的会津藩、桑名藩以谒见天皇为名，从大坂向京都进军。同时，以萨摩和长州两藩军队为主力的讨幕军亦在京都重兵集结，双方在京都南郊的鸟羽、伏见爆发武装冲突，即鸟羽伏见之战，内战戊辰战争爆发。讨幕派的萨长军获得鸟羽伏见之战的胜利。德川庆喜战后于庆应4年1月6日放弃大阪城，并由海路逃往江户。1月31日，天皇发布讨伐德川庆喜的命令，并在2月3日剥夺了德川的官职。以萨摩、长州等藩兵为主力的讨幕军，相继从京都出发。3月12至13日，讨幕军队到达江户城外。经过一连场谈判后，讨幕军统帅西乡隆盛成功劝服德川庆喜投降，于4月21日和平接收江户。事后德川庆喜遭到软禁，江户时代结束。
明治天皇于1868年4月6日发表《五条御誓文》，开启了明治维新的历史序幕。自1868年开始日本进行了多项改革。1868年，明治新政府宣布设立后，便将北海道纳入新政府统治之内。
在德川庆喜被迫使交还权力后，讨幕军征讨由仙台藩的伊达氏作为盟主的奥羽越列藩同盟等残余幕府势力。幕府军在接连败仗后，在1869年1月北上占领五棱郭，将领榎本武扬宣布成立虾夷共和国，并和讨幕军之间爆发箱馆战争。1869年6月27日，榎本武扬宣布投降，随着幕府势力的抵抗消失，日本全境统一。
在幕府军投降之后，新政府废藩置县及废除大名，并宣布在北海道全境设11国86郡。1869年天皇和新政府迁都，从京都移驻江户，并把江户改称为东京。同年，新政府创建了开拓使，北海道开始进入大规模开发时代。除了政府动员的移民之外，亦有自发性的大规模移民，其代表例为带广的晩成社。为了加强国防，还设置了屯田兵制度。1882年，开拓使被废止，政府改设函馆县、札幌县、根室县三个县来管理北海道（三县一局时代）。1886年，政府决定废除三县，改设。
日本后来有出兵台湾之议。1871年10月，一艘琉球国船只于台风后漂流至台湾东南部，其中54名登岛乘客被原住民出草杀害，史称八瑶湾事件。由于当时琉球国同时向日本和清朝朝贡，而清廷表示将事件交由日本处理，日本因而在1874年5月攻打台湾南部排湾族部落，是为「牡丹社事件」。在驻华英国公使调停下，日清两国签订《北京专约》。1871年12月23日，岩仓使节团出使外国。1873年，日本政府取消了自丰臣秀吉起，基督教传教士在日本国内传教的禁令。
君主宪政
明治初期，政府内长州与萨摩两派摩擦越来越烈，不久政治家板垣退助提出的征韩论引致明治政府的内部冲突，于1873年10月爆发明治六年政变，事件中大久保利通等人成功说服明治天皇搁置征韩。事后征韩论领导者板垣退助宣布下野，连同六百多名军人及官员离开政府。直至1874年，已经下野的板垣退助、后藤象二郎等人撰写民撰议院设立建白书，向明治政府表达开设民选议会的期望，是一份肇始日本自由民权运动的文书。由于大量士族在明治维新的过程中失业，旧藩士在新政下生活困苦，西乡隆盛引发1877年的内战西南战争。
1875年，与俄国签订《桦太千岛交换条约》，日本获得堪察加半岛以南的整个千岛群岛的主权、鄂霍次克海的捕鱼权和其周边俄国港口十年的免费使用权，条件为让出整个桦太的主权予俄国。板垣退助虽然于明治六年政变失败，但是征韩论始终未有失势。1875年9月，发生了江华岛事件，最终使日朝双方签订江华条约，日本最终透过条约打开朝鲜国门，取得了自由勘测朝鲜海口、领事裁判权、贸易等权利，并承认朝鲜为独立国。1877年，西乡隆盛以「清君侧」为名，发起西南战争讨伐明治政府，最终萨摩兵败。
1878年，实行地方三新法。引发各地不安，1878年5月14日，明治维新元勋大久保利通在东京的纪尾井町清水谷遭到暗杀。1878年8月23日，发生竹桥事件，驻扎在竹桥附近的帝国陆军近卫军发起武装叛乱。1879年，日本政府将《北京专约》中的「琉球人」诠释为「日本国属民」，染指中国自明清两朝以来的朝贡国琉球国，借此机会在1879年8月31日吞并了琉球，琉球王国灭亡，并改设为冲绳县。1880年，国会期成同盟结成，是自爱国社等组织改称发展而成的组织。借着这个组织，国会开设运动得以发展成全国性的运动。1881年，发生自由民权运动，自1874年民撰议院设立建白书递交以来，该运动的分子就向明治政府表达一系列开设议会、保障自由等期望。1880年，元老院按明治天皇的要求制定了《日本国国宪按》，并将此作为草案提交。此外，时任大藏卿的大隈重信与自由民权运动组织互相呼应，也提出了自己的《宪法意见》，这使得他与伊藤博文不和。而《日本国国宪按》规定了天皇需遵守宪法，并给予议会很大权力，亦遭到了伊藤博文、岩仓具视等人的反对，与大隈的意见一样，未能获得最终通过。在明治十四年政变中，岩仓具视最终以财政政策失政为由褫夺大隈重信参议一职。大隈被罢免后，天皇随即颁布开设国会的敕谕。大隈垮台后，松方正义接任大藏大臣和内务大臣，上任后推行松方通货紧缩，削减政府开支，改革地税改革、设置有权发行可兑换钞票的央行日本银行和兑换制度。三年后，通货膨胀得到控制，政府的财政状况有所好转。
的山县有朋，被普遍认为是日本军国主义之父。在山县任内颁布的《教育敕语》，其宗旨成为该时期日本教育的主轴
，是明治维新时期日本脱离亚洲、全面西化的象征。俄国在十七世纪末进行的西化改革，成为日本西化的参照。
1882年8月，山县有朋在一份意见书中认为，目前不存在欧洲各国入侵日本的可能性。就在这一时期，沙俄将其对外政策的重心从西方移到远东，准备修建一条横贯西伯利亚的铁路，企图控制太平洋上的一切国际商业活动，这引起了日本的警觉。1882年，朝鲜发生壬午事变，云岘君（兴宣大院君）兵变成功掌握政权后不久，即被清朝朝鲜事务大臣吴长庆、帮办大臣袁世凯平定，清方将云岘君监禁于直隶保定。事件中，当地日本公使馆被焚，多名日本侨民遇害，日本公使花房义质亦经济物浦乘船返回日本求援。同年8月12日，花房义质带着日军一千五百人和四艘军舰回到汉城，要求朝鲜政府赔偿损失，以及同意日军驻扎朝鲜。8月30日，日朝双方就上述内容签订了《济物浦条约》，日本获得了在朝鲜的驻兵权。1884年12月4日，朝鲜发生甲申政变，开化党人挟持朝鲜高宗李㷩，矫诏称清军作乱，请求日方派兵支持，日军借此占领朝鲜王宫。至12月6日清军入宫，击败开化党人和日军。政变失败后，日方于1885年派井上馨与载着军队的兵舰六艘登陆朝鲜济物浦，要求朝鲜赔偿损失，朝鲜在日本威脋下签署了《汉城条约》，同意日本在汉城增置戍兵。1885年4月，日本与清方签订《朝鲜撤兵条约》，双方皆从朝鲜半岛撤军。日本虽然撤走了驻朝军队，但保留随时派兵到朝鲜半岛的权利，为以后的中日甲午战争埋下伏笔。
1885年12月22日，伊藤博文基于内阁职权组成第一任内阁，并担任首任内阁总理大臣。1889年，《大日本帝国宪法》发布。同年12月，山县有朋组阁，山县出任内阁总理大臣不久，向内阁提出了《外交政略论》和在1888年写成的《军事意见书》。《外交政略论》以明治政权海外扩张的基本国策为基础，将当时视清帝国为其宗主国的朝鲜、以及满洲划入其利益线之内。为了防止俄国染指这些地区，山县有朋大肆扩充日本海陆军；还出台了《出师储备物资管理委员会条例》、《战时大本营条例》和《海军军令部条例》。1890年7月1日，日本举行第1届日本众议院议员总选举。同年10月30日颁布《教育敕语》，其宗旨成为战前日本教育的主轴，该文档由山县有朋内阁的内阁法制局长官井上毅等人负责起草。1890年11月29日召集第1回帝国议会。1891年，军事工程师下濑雅允研制出下濑火药，后来被使用在中日甲午战争及日俄战争中。
甲午战争
1894年1月11日至1895年3月29日，朝鲜发生东学党起义，成为中日甲午战争的导火线。东学党起义中，日清按1885年《朝鲜撤兵条约》同时出兵朝鲜半岛，起义平息后日本要求继续驻兵被拒，其后扣留朝鲜高宗李㷩，任命大院君李昰应主理朝鲜国事。1894年7月16日，日本与英国签署《日英通商航海条约》，条约由日方代表青木周藏与英方代表约翰·伍德霍斯在伦敦签署。促成条约签订的官员，包括日方的陆奥宗光，英方的休·弗雷泽 、约翰·哈林顿·歌宾斯。条约规定由1899年7月17日开始，在日本的英国国民会受到日本法律的限制，而非英国法律与领事法庭。以往根据德川幕府与外国人的相关协定，在条约港居住的外国人，由该人的所属国家制定的法例管制。该条约的签订促成了1902年缔结的日英同盟。1893年，化学家长井长义自麻黄碱合成出甲基苯丙胺，甲基苯丙胺后来被广泛作军事用途。
1894年8月1日，中日甲午战争爆发。1895年2月，清军战败后，派李鸿章代表清廷在日本签订了《马关条约》，日本在此条约中获得了台湾和澎湖群岛的主权以及二亿两白银的巨额赔款。台湾住民因不满《马关条约》的签订，于1895年5月29日爆发乙未战争。至1895年11月18日起，经过理蕃政策的运行，台湾原住民各政权亦纷纷因武力的征伐而纳入日本殖民地。
，曾三度出任大日本帝国阁揆。早年辅佐其导师山县有朋对日本的军事改革
1900年6月中国爆发义和团事件，日本派出陆军第5师团参与了八国联军之役。1900年，俄国趁八国联军之役，出兵占据中国东北地区，在清廷与各国议和后，俄国仍然不撤兵。1901年，高峰让吉首次分离出肾上腺素。1901年4月29日，皇太孙迪宫裕仁亲王（昭和天皇）出生。1902年1月30日，日英双方结成同盟，日英两国希望藉盟约钳制俄国和德国在远东的发展。1904年，长冈半太郎提出了早期的原子行星模型，称为，建议电子的轨道就像土星环。，该模型后来被玻尔模型取代。
日俄战争
中的众日本陆军步枪兵
1904年日俄战争爆发，日本取得决定性胜利，但由于战争导致财政恶化，而俄国亦爆发血腥星期日革命事件，两国要继续战争都很困难。1905年1月日露战争期间，夏目漱石的长篇小说《吾辈乃猫也》开始在《子规》杂志上连载，成为日本文学史上的一部名著。1905年9月5日，日俄两国在美国总统西奥多·罗斯福的中介下签署《朴次茅斯和约》，结束了战争。日本战后取得桦太北纬50度以南的部分，这样的战争赔偿与日本国民的期望有落差，最终引发1905年9月5日的日比谷纵火事件。日本在《朴次茅斯和约》中取得俄国东清铁路长春至旅顺段的控制权，及后改称该铁路为南满铁路。1906年，南满州铁道设立，旨在垄断满州地区的铁道运输。
吞并朝鲜
举行的东京劝业博览会
1907年，大韩帝国高宗派遣至海牙的密使任务失败，事后日本藉事件逼使韩国高宗让位。1908年，日本无政府主义者引发赤旗事件，与警察发生冲突，最终演变成械斗事件。
，日本驻英国大使。在1902年为日本政府签署日本与英国之间的盟约
日本在甲午战争之后，将原本朝鲜王国的宗主国大清帝国逼退，朝鲜王国也改为大韩帝国，而后成为日本的保护国；日本政治界对韩国问题颇多关注，认为应该要立即性的合并，唯有元老伊藤博文因为国际观感因素，强力反对。1909年10月26日伊藤博文被韩国刺客安重根暗杀之后，主张「立刻合并派」成为日本主流。虽然韩国出现如李容九、宋秉畯等人为首的「一进会」主张日韩「平等合并」，共同创造新的联邦政府，但已经无力回天，日本宣布吞并朝鲜。日韩合并的结果导致大韩帝国（朝鲜王国）的灭亡，朝鲜半岛成为日本领土的一部份，朝鲜总督府成为日本在朝鲜的最高统治机关。1910年8月22日大日本帝国基于《日韩合并条约》，将朝鲜并入版图；此举象征朝鲜历史上长久做为独立国家地位的中断，以及朝鲜日据时期的开始。
1910年，化学家铃木梅太郎从米糠中提取出了抗脚气病酸，将它命名为硫胺（维生素B1）。1912年7月30日，明治天皇驾崩，其子嘉仁继位，改元大正，是为大正时代。同年末，因为陆军倒阁而引起第一次护宪运动（大正デモクラシー），事情导致1913年第三次桂内阁倒阁事件，日本开始出现政党政治。1914年1月的西门子事件，最终导致执政14个月的第1次山本内阁总辞。
一战景气
1914年7月28日第一次世界大战爆发，日本以协约国成员身份在1914年8月23日向同盟国宣战。因为在第一次世界大战中欧洲战场军需激增，日本的经济从中获益很多。重工业在日本经济中的位置也因此大幅提升，当时日本开始出现自江户时代起的首次贸易顺差。
早年，高峰让吉就提出日本应该创建「国民科学研究所」的构想，这一构想后来得到了资本家涩泽荣一的响应，于1917年由日本政府补助经费，加上民间的捐款，成立了理化学研究所，其原子物理研究促进了日本后来在二战时期的核武器研制。同年，俄共推翻了克伦斯基的临时政府，夺取了彼得格勒的政权。
干涉苏俄
中的同盟国众军官
1918年3月苏维埃俄国与德国单独议和，签订了《布列斯特-立陶夫斯克条约》，苏俄退出第一次世界大战。为此，协约国决定武装干涉苏俄革命，俄国白军得到日本等协约国成员支持推翻布尔什维克政权。1918年，第一次世界大战的协约国派遣军队在西伯利亚东面海岸登陆，直接武装支持俄国白军推翻在西伯利亚的布尔什维克政权。事件最后失败告终，苏联共产政府确立。1920年协约国撤出俄国，日本军队鉴于1920年2月至3月发生庙街事件后与莫斯科布尔什维克政府交涉不果，一直驻守西伯利亚至1922年，并占领库页岛北部，直至1925年与苏联建交后才撤出该地。
一战协约国胜利后，根据在1919年6月28日签署的《凡尔赛条约》，将德国在山东半岛的权益转交日本，太平洋赤道以北的国际联盟托管地亦委任日本统治。1922年，日本在其委任统治的南洋群岛上设置的南洋厅，行政首府位于帕劳群岛的科罗尔岛。数学家高木贞治在一战期间完成了类域论中的高木存在性定理的证明，并于1920年提交到国际数学家大会，导致1920年代类域论古典理论的发展。战后日本政局出现不稳，权贵与政党不断抗争，发生第二次护宪运动。
举行，该会议的主要成果包括三个重要条约：《四国公约》，《限制海军军备条约》，及《九国公约》
大正民主
1925年实行全民普选，由唯一的元老西园寺公望推荐总理大臣，日本进入大正民主时期。日本为1920年创建之国际联盟的创始会员国及常任理事国。
1921年，大正天皇因病而由太子裕仁摄政。1923年9月，日本关东大地震，死亡人数达十三万人。1926年12月25日，大正天皇驾崩，皇太子裕仁继位，是为昭和时代。翌年，台湾总督府宣布，开始台湾日治时期极重要的水利工程——嘉南大圳，并于1930年竣工。1927年4月，由于日本政府救济台湾银行失败，陷全国金融于险地，史称「昭和金融恐慌」。1928年，八木秀次和宇田新太郎两人发明了引向天线，后来被应用在无线电测向和长距离无线电通信。1930年，世界经济陷入严重衰退，日本未能幸免。在此期间，滨口、犬养和冈田内阁为挽救经济，主张削减军费，使军部和右翼组织不满，三人亦先后被暗杀。1931年，日本发动九一八事变。1932年，日本发动一·二八事变。在1933年3月27日，日本因扶植傀儡国家满洲国而引发国际危机，于1935年3月27日退出国际联盟。1935年，汤川秀树以核作用力的理论基础预言介子的存在。
军国体制
政党政治日渐衰落（例如田中义一主张惩罚杀害张作霖的凶手而被迫下台），内阁总理大臣甚至会被军人政变刺杀（如五一五事件与二二六事件）或任意废立（总理大臣意见或政策若与军部不同调，便会遭到军方倒阁撤废，换上亲军部的人马），二二六事件之后，统制派首脑东条英机等军人集团渐渐掌握内阁与议会的控制权。1936年2月26日，发生了二二六事件，随后冈田启介内阁倒台，广田弘毅组阁，创建以天皇及军部为本的法西斯体制。广田弘毅迫于军方压力在上台后恢复了军部大臣现役武官制，此后军部的势力开始扩张。
丸之内的，原为一民营之宴会场馆，大政翼赞会成立时一度征用此地做为总部使用
入侵中国
吉隆坡
统制派军人东条英机掌权后，立刻策动了对外用兵，基于他是陆军身份调动了东北亚的部队，对中国开展大规模侵略。此时日本尚处于地区战争阶段，1937年，武见太郎建造了世界第一台便携式心电图仪，并在1939年发明了心矢量图机。1937年7月7日，日军策动卢沟桥事变，全面入侵中国，引发中日战争。
因为日本在对中国的战争中持续消耗物资，加上美英荷三国对日实施经济制裁，为了打破僵局，军部内分做两种派系：一派主张趁苏德战争苏军主力在欧洲，往满洲国北部的西伯利亚扩张，另一派则主张南下东南亚。在1941年4月13日日本与苏联签订苏日中立条约，日本签此条约之目的是确保满洲国北部边界不会爆发战争，影响到南进政策的实行。而在1941年6月22日，纳綷德国开始巴巴罗萨行动进攻苏联，希特勒随即邀请日本加入。北进派乘机要求应德国要求出兵西伯利亚背刺苏联，日本大本营亦一度动摇，命关东军在满洲国布兵演习准备入侵，史称关特演。但随着德军进攻受阻，加上日本急需资源，大本营便考虑与英美荷开战的可能性，最终决定向东南亚进发前往掠夺急需的石油资源。
中日战争时日军占领华东平原，包括首都南京，杀害中国军民，史称南京大屠杀，估计被杀人数约三十万人，蒋中正率领的国民革命军祇能退守中国西南一带。但其后战情僵持不下，令日军陷入消耗战；旷日持久，消耗日军大量人力、物力和财力。
兵下南洋
《德意日三国同盟条约》在1940年9月27日由大日本帝国与纳粹德国及意大利王国在柏林签署。此项协定正式确立了柏林-罗马-东京轴心国同盟关系，并被认为是对美国的警告：要幺保持中立，要幺腹背受敌。条约中，三国同意在未来10年他们会“支持及与对方互相合作……以期实现她们创建与维持新秩序的目标，以期提倡互惠互利与有关人民之福祉” 。三国确认对方的势力范围，并承诺“以政经与军事之力，协助任何仍未参与战争而被攻击的一方”。而条款中的攻击者不包括苏联。条约补充1936年的德日协定和反共产国际协定之不足，并消除德国与日本因1939年苏德互不侵犯条约而起的嫌隙。后来，匈牙利王国（1940年11月20日）、罗马尼亚王国、斯洛伐克共和国（1940年11月23日）加入签署三国同盟正式加入轴心国。保加利亚王国在1941年3月1日加入。
日本1940年9月派军进驻维希法国所控制的越南北部，切断了中国赖以取得同盟国军火的中越铁路。1941年4月，日本派遣外交官前往华盛顿特区谈判，但谈判破裂告终。日本遂于7月2日决定增派部队至越南。而美国则于7月25日冻结日本在美资产，并8月1日对「所有侵略国」实施石油禁运与经济制裁，英国与荷兰亦同时实施对日经济制裁。对于石油、钢铁、工具母机有70%从美国进口的日本来说，这政策极为致命，除非日本将军队撤出中南半岛，否则日本国内保存的石油资源将会耗尽。日本不接受此条件，并称这三国和中国为ABCD包围网。
、榛名、山城三艘战舰
战争的第一枪在马来亚打响。日本时间1941年12月8日凌晨00:30，日军发动马来亚战役，首先在马来半岛北端吉兰丹哥打峇鲁海岸开始炮轰岸面登陆作战，当时距离珍珠港事件发生前还有约48分钟。这场战役中，日军以难以置信的速度攻破英军防线，使得英军只能一退再退最终撤退到新加坡，最后以日军的胜利告终。日军以少许代价占领了整个马来半岛以及新加坡，远比大本营预计占领时间还要短上许多，因此该战役被称为马来亚闪击战。马来亚战役爆发后不足一小时，日军于东岸时间1941年12月7日，偷袭夏威夷的美军基地珍珠港，并正式向美国、英国和荷兰宣战。珍珠港事件后日本迅速进攻太平洋区内美国、英国、荷兰的殖民地，包括马来亚、荷属东印度、菲律宾、新几内亚、香港、所罗门群岛等地，占领及扶植多个傀儡政权，大批同盟国士官被日军俘虏。泰国在日本军事压力之下，发出通谍：要幺军事同盟让日军通过泰国、要幺被入侵，加上泰政府独裁，选择了和日本同盟，但仍拒绝签署三国同盟条约，最终加入了轴心国阵营。泰国军队参与了缅甸战役及马来亚战役并获得了大片土地。
这段时间里，日本势力向西达至英属印度、以东至新几内亚群岛，形成「绝对国防圈」，能掩护日本的海上战略资源运输。除了对东南亚英美荷殖民地的攻占外，日本接连占领太平洋的各个岛屿，由于美国太平洋舰队在珍珠港中被摧毁，无法援助其战区，且日军大规模进攻，岛屿纷纷被日军所占领。日军在各岛上修筑各个不同的防御工事和机场，使得日本的「绝对国防圈」范围不断扩增，但兵力大量分散。此外日军还有试过对澳洲的达尔文展开空袭，并在澳洲的东部海域攻击沿岸与来往的商船，但在空袭与攻击沿岸后未即时登陆，且在不久后盟军开始反攻，失去进攻澳洲的机会。在海上，日本海军局部巡弋陆续消灭盟国海军，其中最大的战果是在印度洋空袭中将英国皇家海军彻底逐出太平洋。日本海军攻击美国西海岸。
战局转折
，日本海军损失多艘主力船舰
1942年5月4日，珊瑚海海战爆发，是太平洋战争中的一场重要战役。这是历史上第一次使用航空母舰的海战，为一个月后的中途岛海战拉开了序幕。日本在1942年4月18日的杜立特空袭后，为了防范美军持续由侧翼直接攻击日本本土，日本海军大将山本五十六便积极策划对中途岛的攻占作战计划，原本预计所有参与偷袭珍珠港的南云舰队的六艘航空母舰要全部参战，为日军攻击中途岛时提供空中支持以及随时歼灭出现的美军机动部队，但由于日军在新几内亚地区发动FS作战而引发珊瑚海海战，让六艘航空母舰中的两艘「翔鹤」与「瑞鹤」航空母舰无法参战，使南云舰队减损33%的空中兵力，但日军仍按照既定计划，于1942年5月28日发动中途岛作战计划。
由于日方的密码被美军破译成功，美军在日军进攻中途岛前就已作好准备，除了强化中途岛本身的防御力量外，尼米兹也要求刚由珊瑚海海战回到珍珠港的「企业号」以及「大黄蜂号」航空母舰由斯普鲁恩斯少将领军，驶往中途岛东北海域埋伏，同时也令在珊瑚海海战中受重伤，当时在珍珠港大修中的「约克镇号」航空母舰运行紧急修理，于3日后组成另一支特混编队，由法兰克·杰克·弗莱彻少将领军，驶向中途岛东北海域与斯普鲁恩斯的舰队会合。
1942年6月4日晨南云部队开始空袭中途岛，然而因为南云舰队发生一连串的战术判断失误（其中包含侦察机未搜索到美舰队，中途岛攻击部队返航以及中途岛陆基航空部队攻击等），让埋伏在东北海域的美军特混编队有机可趁，首先对日舰队发动攻击，让南云舰队中的「赤城」、「加贺」和「苍龙」号航空母舰被美军俯冲轰炸机的炸弹击中起火，并于当晚陆续沉没。而「飞龙」号的航空机队随后对美军舰队发动反击，也让美军的「约克镇号」航空母舰受到重创，后来日军伊-168号潜舰发现了约克镇号，随即发射鱼雷击沉约克镇号。而发动攻击的「飞龙」号航空母舰也被美军攻击后沉入大海。
在南云舰队损失4艘航空母舰后，由于空中攻击力量完全丧失，山本大将不得不向全军宣布撤消战役计划并开始撤退，此战役以美军胜利而告终，也因为日军损失过于巨大（四艘航空母舰、200架以上的舰载机以及训练有素的飞行员），让日军在短期内无法再发动大型的战役，美军得以获得宝贵的喘息时间，故中途岛战役在太平洋战争中具有极重要的地位，也是整个太平洋战争的转捩点。
1938年以来，日军以「扫荡」、「三光作战」，试图稳定中国的日占区，但仍无法扭转整体局势。
最终降服
战争后期，美国海军陆战队接连攻下硫磺岛和冲绳岛，美军B-29轰炸机随即密集轰炸日本本土，许多城市遇袭，市民伤亡惨重、工业设施尽毁，并重创日本的飞机工厂和机场。1945年8月6日和9日，美军依次在广岛市和长崎市投下两枚原子弹，即广岛与长崎原子弹爆炸；同时苏联亦于8月8日对日本宣战，占领满洲及朝鲜半岛北部。8月15日，裕仁天皇以电台广播宣布向同盟国投降，时任日本内阁总理大臣铃木贯太郎也在同日下午宣布内阁总辞。9月2日，日本政府签署《降伏文书》，接受波茨坦宣言，正式投降，并将领土范围限于本州、北海道、九州、四国以及其他指定岛屿。第二次世界大战正式结束，大日本帝国实质性灭亡，但日本政府直到1947年《和平宪法》颁布后才放弃大日本帝国的称号，改称日本国。日本于明治维新后所占领，位于东亚和东南亚地区的领土，在战后则被中华民国和东南亚各国分别收回，琉球群岛和硫磺岛亦由美国军事占领，其后，美国陆续将这些地区归还日本，但美国并未归还原属日本的北马里亚纳群岛，而北方四岛则被苏联和继承国俄罗斯联邦占领至今。
在战争结束后，同盟国也对大日本帝国的战争罪行开展了清算，包括在战争期间担任首相的广田弘毅、平沼骐一郎、东条英机、小矶国昭等均被逮捕，前首相近卫文麿也因被列在战犯名单上而选择自杀以逃避审判。经过远东国际军事法庭的审理后，东条英机、松井石根、板垣征四郎、土肥原贤二、广田弘毅、木村兵太郎、武藤章等七名战犯被判处绞刑，并在1948年12月23日执行。其余战犯均被判处相应徒刑。
住民
在日本，根据大日本帝国宪法，拥有大日本帝国国籍的人被称为日本人、日本国民、日本臣民。《国籍法》（明治32年法律第66号）规定了国籍要求。根据属于以下哪一类，有时在法律制度下受到不同的对待。
内地人是适用《户籍法》（明治31年法律第12号）的公民。几乎相当于现在的日本人口。内地人分为华族、士族、平民。贵族有特殊地位，如有资格成为贵族院议员（3年），1914年（大正3年）族籍记载被废除。战后随着1947年户籍法的修订，这些区别完全消失了。
桦太人是库页岛的原住民，在桦太施行法令（明治40年法律第25号）等法律中被称为土人。他们也被称为桦太土人。有种说法认为萨哈林人没有大日本帝国国籍（百瀬后掲书），但当时的宪法文本却说他们有日本帝国国籍（美浓部后掲书）。桦太人中近80%是阿伊努人，其余还有尼夫赫人，鄂罗克人等原住民族。1932年，桦太人成为内地人（昭和7年勒令第373号），1943年桦太编入内地，实际上也成为内地人（昭和18年法律第35号）。
台湾人为台湾原居民，也被称为本岛人。1895年割让后成为大日本帝国国民，虽然不受《户籍法》的适用，但有公民身分。本岛人90%为汉族，10%为高砂族。战后，在行政上，因1952年的旧金山条约生效而被视为失去日本国籍，但司法判例表明，因《中华民国与日本国间和平条约》生效而失去日本国籍（前大日本帝国国籍），为平和条约国籍离脱者。
朝鲜人为朝鲜半岛原居民，在1910年日韩合并后，也成为大日本帝国国民。虽然不受《户籍法》的适用，但有公民身分。在朝鲜人中，前大韩帝国的皇族仅限于王族，部分在合并朝鲜有功者则仅限于朝鲜贵族。1947年，根据外国人登录令，朝鲜籍被纳入外国人，后于1952年的旧金山条约生效而正式失去日本国籍（前大日本帝国国籍），为平和条约国籍离脱者。
国民
根据法令规定，依《台湾户籍令》（明治38年）、《民籍法》（1909年，大韩帝国）及后《朝鲜户籍令》（大正12年），属于台湾户籍或朝鲜户籍的称为「帝国臣民」。内地人为「户籍法适用者」（户籍法ノ适用ヲ受クル者），而除内地人以外的日本国民统称为「户籍法不适用者」（户籍法ノ适用ヲ受ケザル者）。此外，在官方文档中，也被标记为「内地人」、「台湾人」、「朝鲜人」。
国民以外
未被正式承认为领土的统治区域居民不被赋予大日本帝国国民身份，也不被视为公民。
由国际联盟赋予委任统治的南洋群岛的居民被称为岛民，没有大日本国籍。大多数岛民是卡纳卡人，及查莫罗人。
政治
宪法及政府
1883年，伊藤博文授意井上毅着手宪法草案的起草，并设立宪法取调局，次年改称制度取调局，正式开始了制定宪法、设立国会的进程。1885年，随着太政官制的废止和内阁制度的创立，伊藤博文被任命为首任内阁总理大臣（首相），取代太政大臣三条实美成为明治政府的首脑。井上毅于1887年5月初步完成了宪法草案初稿。以该初稿为基础，伊藤博文、井上、伊东巳代治、金子坚太郎等人聚集在位于夏岛（神奈川县横须贺市）的伊藤别墅中，进行再次修改，写成了「夏岛草案」。此后，在夏岛草案的基础上再做修改，于1888年4月基本完成了定稿。随后不久，伊藤博文设置枢密院，并自任议长，对宪法草案进行了审议。审议过程持续到1889年1月方告终结。1889年2月11日，《大日本帝国宪法》正式向全体国民公布。宪法通过明治天皇向内阁总理大臣黑田清隆亲手递交的方式发布，即「钦定宪法」。同时制定的法典还有作为皇室家族法的皇室典范、议院法、贵族院令、众议院议员选举法、会计法等重要法令。大日本帝国宪法在第一届帝国议会召开当天的1890年11月29日施行。
，贵族院议员。其父益田孝是三井财阀近代化经营的主导人物
，贵族院议员。具平亲王裔孙，历任宫中顾问官、东京府知事、宗秩寮。三十六年，晋从一位
1885年12月22日，第一代内阁总理大臣伊藤博文基于内阁职权组成第一任内阁。四年后大日本帝国宪法颁布后，第五十五条规定国务大臣辅弼天皇。而宪法内文并没有「内阁总理大臣」及「内阁」名词。内阁当时依据内阁官制之法律运作，「内阁总理大臣为（国务大臣）同辈中之首席」，实际上职权并不明确，内阁总理并没有阁臣的任免权，由于任免权在天皇手上，所以当内阁意见不一时只能总辞。内阁总理大臣的任命，由天皇向总理大臣「大命降下」授命组阁。明治后期至昭和中期，总理大臣的人选由曾当过总理，被称为「元老」、「重臣」向天皇推荐。大正后期至昭和初期的政党内阁时代，是基于「宪政之常道」，由众议院的政党构成而任免政党党魁组阁（例如政友会为最大党，则任命其组阁。当政友会下台后，则任免另一个大党宪政会组阁，由大党轮流执政。）陆军大臣及海军大臣在大部份时间采用「现役武官制」，必须由现役武官出任，人选必须由陆军及海军推荐。而两大臣亦有直接向天皇上奏的权利。所以两大臣如果直接向天皇辞职，而所属军部拒绝推荐后继者则内阁倒台。而且军令方面属于天皇的统帅权，所以一般文人内阁较难干预军事政策。
1888年为了审议宪法草案，根据枢密院官制创设了枢密院。在1889年公布的大日本帝国宪法中，也明确规定其为天皇的最高谘询机关。首任议长为伊藤博文。长期以来，枢密院是掌控日本国家大政方针的显要部门，但在1931年九一八事变之后，日本国内政党势力衰退，军部逐渐强硬，枢密院的影响力也日渐式微，最后随着《日本国宪法》的施行，于1947年结束了其历史使命。枢密院设议长1名、副议长1名、顾问官24-28名。任命资格为40岁以上、国家元勋的人物。另外，各国务大臣都自然成为顾问官，可以参加表决。在东京的成年亲王也可以参加会议。枢密院建筑位于东京都皇居内，建于1921年，二战后供皇宫警察使用至今。
，北白川宫的第三代当主
大日本帝国的最高立法机关是帝国议会，是日本国会从1890年发布《大日本帝国宪法》到1947年发布《日本国宪法》期间的正式名称。帝国议会的历史从1890年11月29日的第1回议会开始，直至1947年3月31日的第92回帝国议会。1890年11月29日，第1届众议院、贵族院开议。直至1947年3月31日，第92届众议院解散、贵族院停会。帝国议会由众议院和贵族院构成，众议院议员由国民选出，贵族院议员则由不经选举的皇族、华族、勅任议员构成。贵族院不解散，所以多数议员是终身任期。两院权限对等，但是众议院拥有预算先议权。
，小仓藩藩主长男
日本二战时主导政府的大政翼赞会，是日本在第二次世界大战期间的一个政治团体，于1940年10月12日声明成立，1945年6月13日解散。其以推动政治权力集中的「新体制运动」作为主要目标，将既有的政党解散成一个全国性的政治组织，以一党专政的模式统治日本。由于当时任何形式的结社自由均被管制，大政翼赞会因而自称是一种为了公共目的而成立的「公事结社」。虽然自身认为是一个超越党派的组织，但战后被广泛被认为是与同时代之纳粹党、法西斯党相似的独裁政党，较不同之处是大政翼赞会采取多数决，与上述政党多为个人独裁相异。除了位于首都东京的中央本部外，在全国各地设有道府县支部、大都市支部、市区町村支部、町内会、部落会等分支机构。大政翼赞会成立后，曾与许多功能类似的团体合并、也成立许多外围团体，例如1942年5月26日成立的「日本文学报国会」、1942年12月23日成立的「大日本言论报国会」等。在当时太平洋战争爆发的情势下，以大政翼赞会为中心、对军部的方针给予支持的政治体制，被称为「翼赞体制」，无论是政府运作、公共政策、产业发展、文化教育等各种国政事务，均在这种体系下被大政翼赞会及其外围团体紧密掌握。1945年3月30日，其旗下肩负议会政党职责的改组为「大日本政治会」，大政翼赞会本身则在同年6月被解散并入因应本土决战而成立的「国民义勇队」。
意识形态
」是日本第二次世界大战时期的国家格言，日本政府宣传部门的解释是「天下一家、世界大同」的意思。图为1940年宫崎县八纮一宇塔的竣工仪式
明治中期以前的日本，出现以兴亚会为代表的「兴亚论」，其内容包括新日本的现代化、国家协同合作、国家合并、构筑新秩序等。日本教育家福泽谕吉在其著作《脱亚论》中，认为日本、中国之间如同近邻。而日本已经成功现代化，但中国仍然是落后地方，所以福泽认为应该想方法改善中国，不然落后的中国将会牵连日本，使日本也受到损害。为了日本的福址，日本应该不惜强占落后的中国、朝鲜，帮助其建设。而且事实上，福泽甚至还通过出资购买武器来资助过当时朝鲜的政变。这种意识形态成为日本发动大东亚战争的根源。最初亚洲主义主张日本和大陆国家创建平等的合作关系，但是后来亚洲主义得到修正，一改过往创建平等关系的主张，提倡日本成为亚洲国家发展的导师，日本应当参与东亚的政局，积极左右大陆国家的发展。根据昭和研究会的「东亚协同体论」，二战时成为构筑「大东亚共荣圈」的根基。
日本法西斯主义是指日本在上世纪三十年代政治意识形态，现代学者一般将这理解为军国主义。战前日本军部（陆军省）兴起法西斯主义，陆军省与海军省联合黑龙会之激烈夺权斗争，终于爆发1936年二二六军事政变；海军省及黑龙会政军界闻人数人遇害，事后皇道派被整肃，统制派领袖东条英机控制的陆军省自此彻底掌握军部及内阁政权，日本军国体制确立。日本的东方会是当时国内法西斯主义运动的典范。该会由中野正刚创建，主张将天皇变成纯粹象征性的国家元首。其政党在1937年选举获得21%的选票。东方会后来遭到取缔，中野正刚受到软禁，他在软禁时剖腹自杀。虽然在日本法西斯主义运动没有形成决定性的政治力量，但已经对二战期间日本军部的思想影响深远。
作家大川周明
1938年11月3日，大日本帝国总理大臣近卫文麿发表「第二次近卫声明」，号召创建「大东亚新秩序」，欲树立「日满中三国相互提携，创建政治、经济、文化等方面互助连环的关系」，以大日本帝国、东亚及东南亚「共存共荣的新秩序」为目标。1940年7月26日，第2次近卫内阁制定《基本国策纲要》（基本国策要纲），将继续扩大战争，创建「大东亚新秩序」为基本政策；「八纮一宇」被写入基本国策纲要，成为战争的方针。1940年8月，近卫文麿首度明确提出「大东亚共荣圈」的名称，及指明大日本帝国（包括其傀儡政权满洲国）、中华民国、法属中南半岛、荷属东印度、英属印度、英属马来亚（包括新加坡）、英属香港、英属婆罗洲地区（包括砂捞越与文莱）及新几内亚、澳洲、新西兰等大洋洲地区与苏联西伯利亚东部为大东亚之范围。大东亚共荣圈中，日本本国与满洲国、中华民国汪精卫政府（汪精卫政权）为经济共同体。东南亚作为资源供给地区。南太平洋为国防圈。为实现大东亚共荣圈，1942年日本内阁设立大东亚省，并于1943年11月5日由日本内阁总理大臣东条英机与满洲国（代表：国务总理大臣张景惠）、中华民国汪精卫政府（代表：行政院长汪精卫）、泰国（王子汪歪搭雅昆·瓦拉汪；唯一实质独立的参与国）、菲律宾自治邦（总统劳威尔）、缅甸国（总理巴莫）、自由印度临时政府（首席代表钱德拉·鲍斯）等共同召开大东亚会议，并在会后发表《大东亚共同宣言》揭暨其目标。
在1920年代，匈牙利民族主义者提出创建图兰社会，成立大图兰国，跨越匈牙利至日本的东北亚地区，是为图兰主义。图兰主义颇其纳粹东方主义，以及泛突厥主义的色彩，以统一阿姆河以北的图兰雅利安民族为目标。同时期土耳其正在推行世俗化，摆脱阿拉伯伊斯兰势力的控制，在急于寻求新思想的大环境下，民族主义也在土耳其萌芽。土耳其民族主义者齐亚·乔加勒甫宣称土耳其人版图由阿尔泰山至博斯普鲁斯海峡，并指土耳其人、土库曼斯坦人、鞑靼人、马札儿人、芬兰人、蒙古人、满洲人、朝鲜人和日本人也是图兰人的一分子。图兰主义于战前在东欧民族主义者中兴起，至于二战时期的日本扩张策略有否受图兰主义影响，则众说纷纭。
殖民地事务
的波利尼西亚人警察。当时楚克群岛为日本托管地
日本派往其殖民地的总督拥有行政权、立法权和兵权，掌握着殖民地所有的管治资源，并且很多时是独立于日本内阁自主运作，只对天皇负责。
台湾总督府成立之初，设民政、陆军、海军三局。民政局下置内务、殖产、财务、学务四部。此外，乙未战争期间曾短暂命高岛鞆之助为台湾副总督，高岛也是为期五十年之台湾日治时期内的唯一副总督。1896年，陆海军两局合并为军务局。民政局则在原本组织下增设总务、法务、通信共七局。之后在1898年、1901年皆曾修订总督府官制，最后在1919年的最后一次修订中，民政部（原民政局）被废除，军事部份也改由直属日本内阁的台湾军司令官负责。一般而言，台湾总督府的行政组织分为直属部局、所辖官署与地方行政三大部门。不管1896年发布的《六三法》及后来1906年《三一法》或1921年《法三号》，台湾日治时期的地方政体皆采委任立法制度，总督府为当然之中央机关。而一般政策形成过程，通常是由总督府的技术官僚制定法律政策后，即授权台湾总督以「总督府令」命总督府各级单位、所辖官署或地方政府运行政策。也因此，总督府辖下机关、尤其是地方行政机关的政策实行深具人治色彩，所有行政官员皆只以运行法律及管理行政为主要事务，各级地方行政机关缺乏自主。另外，地方行政基层以警察为中心，除了维护治安外，举凡政治思想、结婚生子、户口管理，还有鸦片管理、公共卫生、征收税金、物资分配、地政调查等，均是警察的事务。
科研及学术成果
医学院院部长
军事
军事参谋与军国体制
明治时代设立兵部省，负责管辖国家的防卫和治安维持的机関。因王政复古，征夷大将军被废除，日本朝廷其他的省决定恢复原有的权威。
1871年（明治4年）7月，兵部省设立了陆军参谋局。应萨摩阀以及长州阀的要求，次年2月，根据太政官公告，兵部省废除，拆分为海军省和陆军省，陆军参谋局改为陆军省参谋局。1873年4月1日，陆军省参谋局改称第六局，1874年2月22日又改回原状。1875年6月18日改为陆军省直属机构。
1878年12月，陆军省独立出分管军令权的参谋本部。1886年3月，参谋本部内分设海军部和陆军部，分别掌管海军和陆军的军令。1888年，海军部和陆军部分别改为海军军令部和陆军参谋本部，而陆海军全军参谋长称为「参军」。1889年，陆军参谋本部长改称「参谋总长」，设置海军参谋部负责海军军令。1893年5月制定的海军军令部条例（敕令第37号）规定，陆海军的军令部门（陆军的参谋本部和海军的军令部）在和平时期是对等的；而根据同年制定的战时大本营条例（敕令第52号），战时，海军军令权属于作为幕僚长的陆军参谋总长。1903年12月，根据修改后的战时大本营条例（敕令第293号），在战争时期，陆海军的军令部门也保持对等的地位。
1903年以勅令第294号为基础设立的军事参议院，是大日本帝国军中回复天皇重要军务咨询的国家机关。当天皇要进行军事咨询时，军事参议院会召开参议会并将意见上奏给天皇，但不允许军官独自上奏。当有海陆军不相互干涉的军务时，可只由陆军或海军的参议官进入参议会。军事参议院由议长、军事参议官、干事长，和干事组成。议长由军事参议官中一般由军事元老担任，当发生紧急事件时议长可以绕过议院回应天皇咨询。军事参议官主要是元帅、陆军大臣、海军大臣、参谋总长、军令部总长及特别的亲任官陆海军将官，亲任官作为军事参议官副官只由佐尉官1人担任。干事长由侍从武官长或其他将官担任，负责整理军事参议会的庶务。干事由侍从武官中的陆海军佐官各一人担任，负责辅助干事长的职务。另外在必要的场合中，担任要职的将官可以作为临时参议官参加参议会。但是，参议会结束后将会解任临时参议官。虽然《大日本帝国宪法》列明天皇掌握统帅权，并赋予军令长官“帷幄上奏权”，即凡有关军令事项，可以不经过内阁直接上奏天皇。然而，近代日本实行廷政分离，天皇不亲政，即使在御前会议上裁决争端，也是只听不答，所以即使陆海军名义上直辖于天皇，实际上是一个独立势力。
1921年三名日本军部军官永田铁山、小烟敏四郎、冈村宁次参与了在德国莱茵河畔的巴登温泉聚会，达成了巴登巴登密约，承诺将加强日德之间的邦交，聚会的第二天，东条英机亦刚从柏林赶到。军部也演变成一法西斯团体。1929年经济危机爆发，日本国内经济困难，军部开始猛烈评击政党政治，并制造了一连串的暗杀及政变事件。陆军省与海军省联合黑龙会之激烈夺权斗争，终于爆发1936年「二二六」东京军事政变，海军省及黑龙会政军界闻人数人遇害，事后皇道派被整肃，统制派领袖东条英机控制的陆军省自此彻底掌握军部及内阁政权。发生二二六事件之际、当时的军事参议官之中的野中四郎大尉等同情反乱军，以结束当前事态为目举行了由军事参议官参加的非正式会议。之后又陆军大臣发表告示（昭和11年2月26日）。1936年的军事政变后，由军部所扶植的傀儡广田弘毅内阁掌政，国家的决策权便从属于军部。广田弘毅按军部的意思改组国家机构，规定内阁中的陆海军大臣必须由中将以上的现役军人担任，使内阁成为军部的傀儡。广田内阁全面改组政府后，结束了「二重政府」的局面，确立了日本军国体制，即是军事独裁的政治体制。
战时大本营
甲午战争到太平洋战争期间大日本帝国陆海军的最高统帅机关是大本营，能够以大本营命令形式发布天皇敕命，是直属于天皇的最高司令部。大本营最初为临时性质，甲午战争和日俄战争期间都临时设置了大本营，战争结束后即告解散。侵华战争爆发后设置的大本营一直存在到太平洋战争结束。
1893年（明治26年）5月19日，明治天皇颁布了《战时大本营条例》（敕令第52号）；甲午战争爆发后的1894年6月5日，首次设置了大本营。根据1893年制定的《海军军令部条例》（敕令第37号），陆海军的军令部门在和平时期是对等的。而战时大本营条例规定，战时海军军令属于作为幕僚长的陆军参谋总长。同年9月15日，天皇和大本营都移到广岛，称为「广岛大本营」。1896年4月1日，大本营奉诏解散。1903年，《战时大本营条例》做了修改（敕令第293号）：在战争时期，海军军令部长和陆军的参谋总长都被视为幕僚长，陆海军的军令部门在战时也保持对等地位。日俄战争期间，大本营设置于1904年2月11日，解散于1905年12月20日。日中战争爆发初期，由于日中两国都没有正式向对方宣战，于是在1937年（昭和12年）11月18日，根据敕令第658号废止大本营只能在战时设置的规定，另颁大本营令（昭和12年军令第1号）规定大本营在发生事变时也可以设置。11月20日，随即依新令设置了大本营，这次设置的大本营一直存在到太平洋战争结束后的1945年9月13日。太平洋战争末期，曾经计划将大本营转移到长野市松代町的地下掩体中，称为「松代大本营」；但工作尚未完成，战争就结束了。
大本营下辖的机构大部分是参谋本部（陆军）和军令部（海军）的机构。大本营会议由天皇、参谋总长和次长、军令部总长和次长、参谋本部第1部长（负责作战事务）和作战课长、军令部第1部长和作战课长组成；陆军大臣和海军大臣列席会议，但没有发言权。大本营中不包含日本内阁总理大臣、外务大臣等政府文官（小矶内阁时期，总理大臣曾例外地成为大本营的成员）。大本营政府联络会议负责在大本营和政府之间协调。
陆军
和九五式轻战车组成的混成部队正在进行训练
明治时代所设立的省厅之一。负责管辖国家的防卫和治安维持的机関。功能相当于现在的防卫省。因王政复古，征夷大将军被废除，日本朝廷其他的省决定恢复原有的权威。兵部省废除之后，应萨摩阀以及长州阀的要求，成立陆军省和海军省。
1872年（明治5年）2月，陆军省从兵部省独立出来，与海军省一同被单独设置。
总军是日本陆军在第二次世界大战中部队的最高编制，总司令官为陆军大将，下辖数个方面军。在1945年8月日本无条件投降时，日本陆军计有中国派遣军、关东军、南方军、第一总军、第二总军、航空总军等6个总军。
1939年以前，日本陆军的最高编制为军，下辖数个师团，但中日战争开战后，日军大幅征召兵员，扩建陆军，方于军之上再设立方面军及总军的编制。1939年9月12日，日本陆军省为因应中日战争的需要，将华中方面军与华北方面军合并，首先于中国南京设立了第一支总军「中国派遣军」，首任总司令官为西尾寿造大将。1941年11月6日，日本于发动太平洋战争的前一个月，另编成「南方军」，以利进军南洋群岛，首任总司令官为寺内寿一大将。1942年10月1日，德苏开战，日本又将驻扎于新京的「关东军」升格为总军（总司令官梅津美治郎大将），以作为日苏之间战争之准备。1944年后，日本在战争中节节败退，军方开始有人喊出「本土决战」的口号，1945年4月7日，日本将原本防卫总司令部改制为「第一总军」（总司令官杉山元大将，驻地东京）、「第二总军」（总司令官畑俊六元帅，驻地广岛，在8月6日原子弹攻击中遭到毁灭性的打击）及航空总军（总司令官河边正三大将，驻地东京）。战后，除第一总军改制为复员司令部，专门负责部队复员工作外，其余总军均遭裁撤。
方面军是大日本帝国陆军的军事单位之一。中日战争开战之前有朝鲜军、台湾军、関东军、中国派遣军等4军存在，并没有设立称之为方面军的单位。随着1937年（昭和12年）7月7日卢沟桥事件的爆发，8月31日中国派遣军改组为第1军，在华北西部的作战地域也组建并部署了第2军，与其他部队统和编组而成的华北方面军是大日本帝国陆军最初的方面军。之后于同年11月7日设立华中方面军，1940年（昭和15年）2月9日设立华南方面军，太平洋战争开战以前撤编，太平洋战争开战时称为方面军的仅有华北方面军。但是，随着战事的激烈化，而增设方面军，终战时已组建有17支。此外，更进一步在方面军之上，设立总军作为陆军部队的最大单位。军队符号以HA为记，标记为1HA（第1方面军）、2HA（第2方面军）等与军相区别。司令官由陆军大将或中将亲补。
大日本帝国没有独立的空军，陆军与海军拥有各自的空中武力部队。
海军
进行改造空母工程的「赤城」，该舰改造后在1927年3月25日开始服役
明治政府继承了江户时代以来等组织的一些海军操练所与海军伝习所外加其他相关人员和装备。1870年（明治3年），陆海军正式分离。1872年（明治5年），海军省在川村纯义与胜海舟的指导下在东京都筑地区创建。1876年（明治9年）创立了海军兵学校。1893年（明治26年）成立了军令部。由于萨摩藩在西南战争中失去的影响力，早期的明治政府把陆军的发展重于海军。1903年（明治36年），在成立了参谋本部后，通过了两名海军大臣西乡从道和山本权兵卫的主张，开始对舰队的装备和组织结构进行了一番改革，到了中日甲午战争时，日本海军已经拥有了31艘军舰和24艘水雷艇。而到了日俄战争时，则达到了七十六艘军舰外加七十六艘水雷艇的规模。
日本海军之后在“复制·改进·创新”的政策下继续了自己的扩张道路。工厂经常会为测试和分解买入两艘相同型号的外国舰艇，随后对其进行研究和改进。这样子的作法使得最初只能依靠进口的军舰逐渐可以在国内进行组装，直到最后可以完全国产，从一八八〇年代开始的小型鱼雷艇到一九〇〇年代早期的大型战列舰。1905年建造的萨摩百分之八十的部件来自于英国，而在1910年完成的河内只有百分之二十的部件需要靠进口。日本海军最后的大型军舰进口是在1913年从维克斯购买的金刚战列巡洋舰。
」是世界上第一艘真正意义上的航空母舰
在经历了日本海海战后，等海军战略家，开始计划对策以应付美国海军的威胁，佐藤认为战列舰的数量必须达到对方的70%以上，但由于日本工业输出的局限性，战列舰数量只保持在60%。1918年，日本海军的预算达到了国家财政收入的32%，世界上第一艘正式的航空母舰凤翔竣工。410毫米口径的长门和460毫米口径的大和级战列舰后来成为了日本海军大舰巨炮主义的象征。1920年，在日俄战争结束后，海军开始了一套以美国海军为假想敌的“八八舰队”（八艘战舰加上八艘巡洋舰）增强政策。因为在1922年签订的华盛顿海军条约和1930年的伦敦海军条约的影响导致主力舰的建造被一度中断，但在伦敦海军条约破裂后则重新开始了扩充。1941年，到了太平洋战争开战前，日本海军的旗下拥有了十艘战舰，三百八十五艘舰艇和包括零战在内的3260架军用机。从日俄战争胜利以来一直到太平洋战争战败，大日本帝国海军一直和英国皇家海军与美国海军构成了全球范围内的世界三大海军。1928年，日本海军推出了吹雪型驱逐舰，开创性的127毫米口径炮和鱼雷塔成为了其它海军模仿的目标，而它610毫米的九三式鱼雷直到二次大战结束前为世界上最优秀的鱼雷之一。
1920－30年代，日本海军大概分舰队派及条约派两个对立派系，于海军假日时期的日本政坛上挑起多次冲突。条约派认识到日本经济的困难，中主张遵守伦敦海军条约，限制军备发展以充实国力的势力，以加藤友三郎元帅为代表人物，以海军省为代表。相对于此，舰队派是指大日本帝国海军中反对《伦敦海军条约》，提倡全力扩张舰队，挑战欧美列强在亚太地区权力的势力，以伏见宫博恭王、加藤寛治、山本英辅、末次信正、高桥三吉等人及军令部为中心，日后珍珠港事变率领日军舰队偷袭夏威夷的南云忠一中将亦为这一派的代表人物。虽说舰队派与条约派互为对立，然而事实上，两者所代表者仅为当时日本海军中两种对立的意见，并非有组织的两个团体，两者之间亦无明确的定义与分割。1929年，日本帝国海军成立了海军陆战队。
卫戍部队
大日本帝国陆军时期，日军将基地称为卫戍地，其部队长称为卫戍司令官，例如东京卫戍司令官（とうきょうえいじゅしれいかん）。卫戍地设有卫戍医院（卫戍病院）、卫戍监狱（卫戍刑务所、卫戍拘置所）等设施，均属卫戍司令官所管。
宪兵队是隶属于大日本帝国陆军，存在于1881年至1945年。海军有自己的宪兵队，称作海军特别警察队。
1881年3月11日，日本宪兵条例公布，5月9日宪兵队成立。宪兵条例列明宪兵是作为陆军军纪的代表，担任军内军纪的巡查和检察的任务，监督军人的日常品行。兼任行政警察和司法警察。日本陆军宪兵最初兵力1600名，在东京设有宪兵司令部。1883年在大阪创建了关西宪兵总司令部。1889年，设立六个师团后，每个师团都设置一个宪兵队。宪兵兵力2100名。甲午战争爆发，增设师团，宪兵兵力增加到5000名。第一次世界大战后，宪兵兵力裁至2000名。1923年陆军省正式规定了宪兵袖章，白棉质地的白箍宽十二公分，上面写有红色从右至左的「宪兵」二字。1931年918事变时，宪兵总数2250名。1935年9月东条英机任关东宪兵司令官后，把只有三百人的宪兵队伍迅速扩充为近千人。
大日本帝国曾设置的秘密警察组织特别高等警察，负责政治团体的调查工作。特别高等警察原隶属于内务省，二战前日本驻中华民国各领事馆均设有警察署，内设特高课。后来也被设立在占领区宪兵队内，归军部领导。
军事人员养成
1883年至1945年间，陆军大学校创设于东京都港区北青山(开校后一年迁往千代田区，在二次大战结束前疏散至山梨县甲府市)，培养参谋军官及日军将级、大佐级高端军官及军事研究为其主要任务。设置的64年间共有3,485名毕业生。报考资格为日本陆军士官学校毕业，在部队两年以上经历，未满30岁的中尉、大尉军官。教育期间为步兵、骑兵三年，砲兵、工兵为两年。日本陆军内的诸所学校通常是在教育总监部的管辖下，陆军大学校却是由参谋本部直辖的学校，陆军大学校毕业生的人事安排由参谋本部直接处理。
日本陆军士官学校是日本少尉军官的养成学校。该校于日本明治维新期间开办，前身是1868年8月开办的「京都军校」，于1874年正式成立。该校毕业生均是日本近代军队的骨干，近代日本的陆军军官无论大将还是少尉，几乎都曾是该校的毕业生。当时日本陆军士官学校和海军兵学校学生一毕业即成为奏任官。日本皇族、华族也多数进这两所学校。陆军士官学校的学制是两年预科，然后分兵科，以少尉候补生身份（佩戴上等兵军衔）进入部队实习5个月，战时缩短为3个月。部队实习结束晋升为军曹军衔，进入陆军士官学校本科或者陆军航空士官学校一年零五个月，战时缩短为一年半。本科毕业后，以见习士官身份（佩戴曹长军衔）回原部队实习数个月，实习结束后获得现役少尉的任命书。因此中学毕业生从入校到获得少尉军衔至少需要5年。
海军兵学校、海军机关学校与海军经理学校是帝国海军的三大军校。日本海军兵学校是大日本帝国海军战斗兵科军官的养成学校，设校于广岛县江田岛市。1869年，其前身「海军操练所」开设，后改称「海军兵学寮」。1876年迁往东京筑地，改称「海军兵学校」。1888年再迁往现址广岛县濑户内海安艺群岛的江田岛。1943年在岩国飞行场设立岩国分校。1944年设立大分、舞鹤分校。1945年设立针尾分校。海军兵学校的录取年龄为16-19岁的中学校毕业生。学制起初是三年制，1927年改为3年8个月，1932年时改为4年制。后来因为需要大量军事人员扩充军队，兵学校的学制由第66期开始缩短为3年8个月，再后来又缩减为3年3个月（67期）、3年4个月（68期）、3年（69期～71期）、2年4个月（72期）。海军兵学校的最高年级生称为「1号」，其下较低年级生则以「2号」、「3号」、「4号」分别称呼之。海军兵学校毕业生会以少尉候补生身份分发至练习舰队，经过远洋航行的术科讲习及实地训练后任职。
社会阶级
，1916年开始作为部落民解放运动的领袖
日本历史上存在一特殊阶级，即是部落民。部落民是过去封建时期贱民阶级的后代，他们社会阶级低下，世代住在固定的地区。主要从事被认为是宗教上「不洁」的工作，如殡仪业者、屠夫或皮革工人、拾荒者、小丑等，并传统上居住于对外隔绝的村庄或贫民区（多不适于农耕），分为非人与秽多。非人多数是乞丐、算命、监狱看守，秽多则是处理与死亡有关的工作（因神道教与佛教也认为宰杀与血不洁）。他们要住在特别区域穿特别衣服自异于其他人。居住在对外隔绝的村落，被称为「秽多」；乞丐、算命、监狱看守等则被称为「非人」，均是最低级级的人。当年「秽多」若犯罪，武士阶级可以杀了他们而不受惩罚。这些人及其子孙所聚集生活的贫困地区逐渐形成部落，故称为部落民。部落民无法参与正常的社会活动，也不能与一般人交往，只能从事粗贱的工作求生存，世袭无法翻身。
在幕府末期，当封建制度发生危机的时候，反对幕府的志士刚即提出了解放部落的主张。与此同时，幕府也从加强它的军事力量这个目的出发，企图废除部落民的身分并想把他们组成军队，做为解放他们的代价。德川政权本来从提升弹左卫及其部下的身分为平民时开始已经着手了这种工作，但它的覆灭使这种由封建统治者所进行的解放政策发生了挫折。直至1871年，日本的封建阶级制度被废除，部落民被合法的解放。明治时期并入日本与殖民地的居民，诸如阿伊努人、琉球人，这些少数族群在日本社会的地位跟和新拼入的殖民地上的原住民一样，有时甚至更低；不同的是，刚获得良民地位的部落民带有歧视意味地被称为「新平民」。
经济
明治维新以来，门户被放以后，日本政府开始推行富国强兵和殖产兴业政策。日本开始工业化，跻身列强。当时日本开始出现了数个影响日本经济深远的巨无霸公司：三井、三菱和住友。三井财团早年聘用了三野村利左卫门、益田孝、中上川彦次郎等人，掌握了近代化经营模式。1876年，三井银行和三井物产创立。此后，财团不断扩大产业领域，进入了纺织业、采矿业和机械制造业等行业。1909年，成立三井合名会社。当时已经成为日本规模最大的财阀。
1918年，米价急涨，日本农村爆发米騒动事件，并演变成武装冲突，参与者逾二百万人；事后警方逮捕超过二万五千人，其中八千二百人被判各项刑罚：轻则罚款，重则处死；此事亦使寺内内阁下台。由于当时日本内地的大米供不应求，因而增加生产朝鲜和台湾的大米，并进口更多的外地大米至内地，以解内地缺米之忧；朝鲜和台湾对内供米减少，朝鲜农民祇能从满洲进口杂粮充饥。
是当时全日本规模最大的炼钢厂
因为在第一次世界大战中欧洲战场军需激增，日本的经济从中获益很多。重工业在日本经济中的位置也因此大幅提升。当时日本开始出现自江户时代起的首次贸易顺差；而到了1920年代，制造业和矿业在全国生产总值中占的比重为23%，首度超越农业的21%，日本亦因此成为了债权国。但在第一次世界大战结束后，欧洲国家对军需品的需求下降，使依赖军需品外销的日本经济陷入低谷。1923年的关东大震灾导致日本多间银行信用不佳，1927年发生了昭和金融恐慌。1929年，更发生了全球性的经济大萧条。1930年，由于解除黄金出口禁令与世界恐慌等一连串影响，日本经济恶化。日本在第二次世界大战开始时，自由经济转换成国家统制经济（战时经济），这段时间日本制造业和矿业总值占日本全国生产总值超过30%，这是由于军事工业的发展。
货币政策
大阪分行的航拍图。银行前方的大街是御堂筋的一部份
日本银行在帝国金融体系的岗位相当于其他国家的中央银行。依据日本银行法，日本银行为财务省主管的认可法人，是独立于日本政府的法人。1882年6月帝国颁布日本银行条例。1882年10月10日日本银行正式成立。1942年2月，日本颁布日本银行法（昭和17年法律第67号）。1942年5月1日，根据旧日本银行法施行法人改组。
1871年6月27日，日本制定了货币单位「圆」（円），取代当时的四进制的二分金。当年明治政府发布《新货币条例》，规定新货币采用金本位，以圆为单位，一圆与1500毫克纯金等值，并设有辅币单位钱及厘，兑换比例为十进制，即1圆=100钱=1000厘。在日本国内，白银的价值相对较高。因而其黄金与白银比价要低于国际市场，这就使得国际投机商有利可图。他们将银两运入日本，换成黄金，再到其他国家卖出。此举扰乱了日本经济，到1874年，日本实际上转向了银本位制。虽然白银的价格相对于黄金仍然在下跌，但出于贸易结算的考虑，日本对外贸易多以银币计算。1897年，随着日本的发布，日圆的金本位制度第二次以法律形式被确立下来。该法律规定1日圆与750毫克黄金等价。1931年12月31日，日本由大藏省宣布停止黄金输出，禁止金币与钞票之间的兑换。
朝鲜半岛地区
日治朝鲜时期，日本在朝鲜半岛开展了包括兴建基础设施等的经济建设。1912至1937年间，朝鲜半岛GDP年增长42%，超过了日本列岛和西欧。其对外贸易在1911年以向英国进口最多，占3946％，其次依次为清帝国、美国、德意志帝国、荷属东印度、英属印度和俄罗斯帝国，以向清帝国出口最多，占5479％。
满洲地区
满洲当时已创建起较完备、先进的工业体系。包括铁道运输、航空、航海、钢铁、煤炭、机车、汽车制造、飞机制造和军工等工业部门。时任满洲国国务院实业部总务司长岸信介向关东军说：「要是没有财阀、资本家，就无法进行初期建国工程」，而从日本引入财阀投资。财阀的资金迅速涌入满洲国。鲇川义介的日产财阀借此进入满洲，旗下拥有日立制作所、日产汽车、日本矿业、日本化学工业等130家公司，是拥有15万名员工之大财阀。满洲国对于工业部门采取经济统制政策，推行“一业一社”的原则，每个行业都成立一个“特殊会社”——垄断性公司。满铁之所以接受改组，时任满铁总裁之松冈洋右是岸信介叔叔之妻舅，和满业之鲇川义介为姻亲。由于战争的进行导致物资奇缺，满洲国末期时国务院制定《金属类回收法》。
满洲国钢铁和化学工业主要集中在鞍山和本溪湖，煤炭工业集中于抚顺、本溪湖、阜新，油页岩和合成燃料工业集中在抚顺和吉林，菱镁矿业集中于海城和大石桥，水力发电集中于吉林和鸭绿江，机械、军火、飞机工业中心为奉天（今沈阳），纺织和和食品等轻工业则集中在大连、丹东、哈尔滨、齐齐哈尔等城市。
台湾地区
日治台湾产业发展最显著的是糖业。台湾总督府为谋求台湾产糖量的增加，进行一连串糖业改革措施，包括引进含糖量高的蔗种、改善制糖方法，奖励从事糖业者。总督府并扶植资本家投资制糖业，创建「原料采集区域制度」，规定农民只能把甘蔗卖给自己农地附近的糖厂，而且价格由糖厂决定。在这样的制度下，吸引了许多日本大财团来台设立制糖公司，在台湾众多糖公司中也有几家台湾人创办较大规模的，如林本源制糖。
1900年，三井财阀设立台湾第一家新式制糖工厂「台湾制糖株式会社」，新式糖厂至1911年已有11家、1927年增为45家，其中台湾、明治、盐水港、大日本、新港、新高等糖厂占总产量80%。然而，从台湾糖业图（1934年）可见，有部份台湾人的传统制糖事业仍以改良糖廍的方式经营。
经过总督府全力支持后，台湾糖业于1930年代产量达到高点。以1931年为例，台湾全年总共产出24亿3834万4890斤（台斤），是整个帝国产糖量最多的地区。日本稻作专家矶永吉在任教台北帝国大学期间，以日本种稻米改良培植出蓬莱米。日本农学家新渡户稻造任台湾总督府殖产局长时，于1901年提出《糖业改良意见书》，对台湾糖业有重大影响。总督府又大力推广阿萨姆红茶的种植，让福尔摩沙茶外销欧洲与美洲市场。
文化
文学
，摄于1912年9月13日明治天皇国丧
明治时代的日本经历迅速现代化，大量外来文化也在这段时期被引进日本。自由诗就是随着欧洲文学引进日本的诗歌题栽之一，这种体裁主要是一些长篇的及体现新学术理念的作品。年轻日本散文家及剧作家争相猎取新思想，但小说家还是最早把这些理念融入其作品之中。
当时新出现的通俗文学集中于“私小说”这一种体裁的创作以及一些不寻常的重要作家，比如夏目漱石，就写下有名的小说《我是猫》。除此之外，他还有代表作《少爷》及《心》。被誉为“小说之神”的志贺直哉及森鸥外就致力于运用西方的文学风格及技巧。芥川龙之介也写下不少有名的历史短篇小说。尾崎红叶、泉镜花及樋口一叶等作家的风格则保留了近代日本文学早期的面貌。
在明治世代早期（1868－1880年代），福泽谕吉创作启蒙文学，是近世描画激烈转变的社会的普及读物。接着，在明治中期（1880年代后期至1890年代早期），坪内逍遥及二叶亭四迷开始引入现实主义，但在同时，古典主义的尾崎红叶，山田美妙及幸田成行也大受欢迎。樋口一叶，是在当时少数的女性作家，以口语及文学语言用简单的手法写下一些关于该世代女人无力的短篇故事。泉镜花，尾崎的追随者，追求典雅的风格，其早期的小说如《外科室》（1895年）就用文学的语言写成；后来的小说，比如《高野圣》（1900年）就用口语创作。
，因区内置筑大量使用烧制砖瓦而得名。1872年银座大火后，日本政府按「都市防火」原则于原火除地修筑而成，但因砖瓦建筑抗震能力不佳，当时又没有抗震设计，区内置筑大多在1923年关东大地震期间被震毁
日本纹身热
1868年以后，日本数百年来第一次向西方打开了贸易大门。欧洲几乎同时出现了对日本商品和文化迅速膨胀的需求。富有的欧洲贵族们浑身上下满戴着各色日本饰品回到国内。
1881年，16岁的乔治五世在横滨让日本刺青师雕千代在他手臂上纹上了一条红蓝双色的龙。在当时，未来英国国王的纹身是一起知名事件，当时还出现了一幅乔治在1893年婚礼上展现纹身的假想图。王子纹身的新闻在英国、法国乃至美国迅速掀起了纹身热潮：纹身已经成为社会地位，以及对日本商品购买力的标志。埃塞克斯大学当代艺术讲师马特·洛德（Matt Lodder）说：「当时人所共知的是，如果你很有钱并且去过日本，那么最好在回国前给自己纹身。」。1891年，俄国的沙皇尼古拉二世在外访日本时，也让雕千代在右臂纹上了一条五彩斑斓的龙。另外还有希腊奥尔嘉王后等多位显赫的欧洲贵族身上也有日本纹身。
国土范围
大日本帝国的领土分为「内地」与「外地」，两者的区别在于后者具有属地的性质；广义上的领土也包括在中国境内的数个租借地。此外，大日本帝国发动大东亚战争后，在其占领的部分外国领土上树立多个傀儡政权，如在中国东北扶持满洲国、南京国民政府（汪精卫政权），在菲律宾创建第二共和国等。(有人将泰国视作日本傀儡、占领地，泰国并不是日本的傀儡，日本视泰国为盟友，加上泰政府亲日和泰日军事同盟，泰国是日本在亚洲唯一的盟友。)
明治天皇在1867年即位后，隔年即以「行幸」（天皇出巡）的名义，将皇室驻地由京都迁至江户（日本历史上称为「」），以之为行在，并以「在东方的京都」之名将江户改名为「东京」。在此之后，日本朝廷及政府也随之迁移至东京。明治天皇以江户时代的江户城为新住所，这个新皇宫曾使用「东京城」、「宫城」等名，二战后更名为皇居至今。东京则因为是「帝国的首都」而常被称为「帝都」。
日本法律并没有规定首都位置，仅有在1923年（大正12年）9月12日发布的《关东大地震发生后之诏书》（関东大震灾直后ノ诏书）提到「东京为帝国之首都」（东京ハ帝国ノ首都）。事实上，象征天皇驻地所在的天皇宝座「高御座」并没有随着朝廷迁至东京，而是继续留在京都旧皇宫（至今仍是如此），大日本帝国时期在位的明治、大正、昭和等3位天皇也都是在京都即位，这使得京都仍然具有部分的首都机能。
内地
与现在日本的领土范围大致相同，即北海道、本州、四国、九州等日本列岛四大岛，加上奄美群岛、琉球群岛、小笠原群岛，以及这些群岛的附属岛屿。
内地的范围中，曾有重大主权变更的区域有：
第二次世界大战结束前，南桦太与整个千岛群岛被苏联占领；日本虽在《旧金山和约》中放弃这两地的主权，但没有涵盖持续被苏联管治的南千岛群岛。此争议延续至苏联解体后继承其政治地位的俄罗斯联邦，双方至今尚未解决。
外地
有别于内地的称呼。性质上等同于属地，因此适用日本本土的部分法律。在1945年日本投降后，日本被迫放弃所有外地的统治权。
原为一清朝省份，1895年通过《马关条约》割让，以及数个原住民政权，自1895年后通过武力征伐逐一占领，并于1933年大分社投降后正式掌控全岛，战后由中华民国政府接管。参见：台湾日治时期、台湾总督府。
原为清朝附属独立国，1910年通过《日韩合并条约》正式吞并，战后由美苏两国接管，后分裂为朝鲜民主主义人民共和国和大韩民国。参见：朝鲜日治时期、朝鲜总督府。
帝国遗产及影响
鄂温克族的族旗，旗上的最上层是一面日本国旗
日本于1945年8月15日终战时，仍然控制着中国东部及东南亚的大片领土，当地的日本资产，包括工厂、基础建设，有很多都由当地在战后成立的政府所接收。中国的鞍山钢铁集团的前身就是南满洲铁道株式会社在1916年成立的「鞍山制铁所」和「昭和制钢所」。国民政府收复中国东北后，负责掌管全国重工业的资源委员会利用剩余练钢设备改组成鞍山钢铁公司。中国共产党占领中国东北后鞍山钢铁公司仍维持既往编制，并借由苏援重建成为中华人民共和国规模最大且最先进的钢铁生产公司。
屯门天主教中学（Tuen Mun Catholic Secondary School）位于香港新界屯门建生邨，是天主教香港教区23间中学之一。学校于1987年成立，现任校监为陈振光博士，现任校长为萧思铨。该校部分班别以英语授课，除中文科、中国历史、普通话、通识与社会、宗教及伦理外，其余科目均以英语教学。
校训
真谦恒诚
校舍
此校为一座连环型校舍，面积达4000平方米，设施包括球场、礼堂、多媒体室、电脑室、语言实验室、实验室、中国语文活动室、英语自学中心、数学角、通识资源中心、家政室和设计与科技室等。另设有宗教室和祈祷室，而天台设置有机耕种农圃。而每层都设小书架，提供英文书借阅。
校舍在2004年曾进行维修工程，兴建新翼。到2011年暑假起，4楼课室及图书馆暂时封闭，翻新后藏书量多达五万多册。同年暑假进行维修工程，重铺篮球场。
笔记本或记事本可以指：
历史
网罟座是法国天文学家拉卡伊创造的一个星座。它的名称是为了纪念在目镜中加上了精密观测用的网，即十字丝。这个很小的远南天星座最突出的特点是几个有趣的双星和相对明亮的星系。
狐狸座（Vulpecula）是一个位于北天球银河中的模糊星座，在天鹅座南方。17世纪末波兰天文学家赫维留命名为「狐狸与鹅」。
历史
这个微弱的星座位于银河一个明亮的区域，因此更加难以观测。1660年，波兰天文学家赫维留斯感觉这里应该设立一个星座。尽管这个区域看起来很不显眼，但使用双筒望远镜或小型望远镜可以发现这是一个有趣的天区。这里有一个梅西耶天体M27。
1967年10月，位于狐狸座的脉冲星PSR B1919+21由剑桥大学卡文迪许实验室的安东尼·休伊什教授的研究生——24岁的乔丝琳·贝尔检测射电望远镜收到的信号时无意中发现。这是人类有史以来发现的第一颗脉冲星。
北冕座是北天小星座，属公元二世纪天文学家克劳狄乌斯·托勒密划分的48个星座，也属国际天文联会承认的88个现代星座。星座拉丁语原名“Corona Borealis”意为“北边的皇冠”，源自星座形状。北冕座在西方古典神话大多代表天神狄俄倪索斯授予克里特公主阿里阿德涅、后来留在天国的皇冠。其他神话把星座视为城堡、鹰巢、熊窝，甚至还有排烟孔。托勒密还划有形状类似的南天星座南冕座，与北冕座对应。
22视星等的贯索四是星座最亮恒星。北冕座R型变星是含氢极少的罕见变星，原型便是黄超巨星北冕座R，估计是两颗白矮星合并而成。北冕座T又称博雷兹星，是罕见的再发新星，通常是十等星，上一次爆发是1946年，亮度增至二等。ADS 9731与北冕座σ都是包含多颗恒星的恒星系，分别有六颗和五颗恒星。北冕座五个恒星系发现木星尺寸的系列行星。阿贝尔2065是距太阳十亿光年、有超过400个星系的星系团，属北冕座超星系团。
简介
北冕座占据179平方度夜空，占0433%，在88个现代星座排第73。星座位于北天球，南纬50度线以北才能看到完整星座。北冕座北侧和西面与牧夫座相邻，南接巨蛇座，东靠武仙座。1922年，国际天文联会确定以三字母缩写“CrB”代指北冕座。比利时天文学家尤金·德尔波特1930年正式划分星座边界，北冕座呈六条边组成的多边形（见文首信息框）。星座在赤道坐标系统的赤经位于至范围，赤纬在3971°到2554°之间。此外，北冕座还与南天球的南冕座相对。
显着特点
恒星
北冕座独特的皇冠形状由七颗恒星构成，除最亮的二等星贯索四外，另外六颗分别是贯索二、贯索三、贯索五、贯索六、贯索七、贯索八，都是四等星。日耳曼天体制图师约翰·拜耳在1603年的《测天图》用拜耳命名法为星座20颗恒星分配希腊字母“α”至“υ”。后世天文学家发现七公增七是双星，故将两星分别称为七公增七1和七公增七2。北冕座ν情况类似，拜耳以单字母命名，但约翰·佛兰斯蒂德发现该星实际上是光学双星，在他的星表上标为北冕座20和北冕座21，分别命名北冕座ν1和北冕座ν2。中国古代天文学家将上述七星与北冕座π、贯索九划为星群。天冕座范围内共有37颗恒星亮度不低于65视星等。
离地75±05光年的贯索四是22视星等蓝白星体，属大陵五型变星，亮度以174天为周期波动01视星等。其中主星是光谱等级A0V的白主序星，拥有291倍太阳质量和57倍太阳光度，周围是半径约60天文单位的岩屑盘。从星是光谱等级G5V的黄主序星，直径约太阳九成。据信贯索四是大熊座移动星群成员，与星群其他星体整体走向一致。
离地112±3光年的贯索三是光谱联星，两星体相隔十天文单位，以105年轨道周期相互围绕旋转。偏亮的主星是快速振荡Ap星，脉冲振荡周期162分钟。该星光谱等级A5V，有效温度约7980开尔文，拥有21倍太阳质量、26倍太阳半径、253倍太阳光度。光谱等级F2V的从星表面温度约6750开尔文，质量、半径、光度分别是太阳14倍、156倍、4至5倍。。贯索三附近的贯索二是联星系，合并视星等413，距地球380±20光年。偏亮的贯索二A是蓝白星，旋转速度高达每秒393公里，是岩屑盘包围的Be星。
贯索四东侧的贯索五也是联星系，其中两星体分别是光谱等级B9V和A3V的主序星，与太阳和海王星间距差不多，相互围绕旋转的轨道周期是9294年。联星系内偏亮的恒星可能是矮造父变星，但尚无定论。406视星等的贯索六离地170±2光年，是光谱等级G35III的黄巨星，质量约为太阳24倍但已膨胀到74倍太阳半径，表面温度5180开尔文。贯索六绝大多数生命周期是B型蓝白主序星，如今内核的氢已耗尽。光度和光谱数据表明该星刚刚跨越赫氏空隙，开始消耗内核周围外壳的氢。
七公增七是双星，由相隔仅63弧秒的两颗蓝白星组成，百倍望远镜可轻松分辨，主星51视星等，从星60视星等。北冕座ν是光学双星，两者与地球距离差不多但径向速度不同，估计相互没有关联。主星北冕座ν1是光谱等级M2III、亮度52视星等的红巨星，离地640±30光年。从星北冕座ν2是光谱等级K5III、亮度54视星等的橙巨星，估计离地590±30光年。 北冕座σ是多颗恒星组成的恒星系，小型业余望远镜就能分辨。恒星系内部构成非常复杂，首先是两颗质量接近太阳的恒星以114天轨道周期相互围绕旋转，接下来第三颗类太阳恒星绕前两星以726年周期旋转。第四和第五颗星体都是红矮星并组成联星系，距前三颗恒星约14万天文单位。ADS 9731是更加罕见的多成员恒星系，包含六颗恒星，其中两颗组成光谱联星。
北冕座有两颗非常特殊的变星。北冕座T又称“博雷兹星”，属激变变星。通常亮度在99至102视星等，因核连锁反应及随后的爆炸可在数小时内达到二等星亮度。北冕座T与罗盘座T、天蝎座U一样是屈指可数的已知再发新星，人类在1866年首次发现该星爆发，第二次是1946年2月。北冕座T还是红色巨星与白矮星组成的联星，以约八个月轨道周期相互围绕旋转。
北冕座R是黄色超巨变星，距地球超七千光年，是北冕座R型变星的原型。星体基准亮度六等，最低按周期跌到15等，此后几个月缓慢回归。恒星喷出尘埃导致亮度降低，哈勃空间望远镜直接成像表明广阔的尘埃云从恒星向外延伸约两千天文单位，由恒星风带动的细微尘埃流（由直径约五纳米的颗粒组成）与星体按周期喷出的粗尘（直径约014微米）组成。
北冕座有许多亮度适合业余天文学爱好者观察的变星，如三颗长周期刍藁变星：北冕座S以360天周期在58至141视星等范围闪烁，离地约1946光年，光度达太阳16643倍，表面温度3033开尔文。北冕座V是夜空非常偏红色的天体，表面温度2877开尔文，光度达太阳10万2831倍，距地球8810光年。该星以357天周期在69至126视星等变化，位置靠近北冕座、武仙座、牧夫座交汇处。北冕座τ东北15°的北冕座W以238天周期在78至143视星等范围变动。红巨星北冕座RR是M3型半规则变星，以608天周期在7.3至82视星等范围闪烁。北冕座RS也是半规则红巨变星，以332天周期在87至116视星等变化。与其他红巨星相比，北冕座RS的自行范围特别大，每年超过50毫弧秒。北冕座U是大陵五型食联星系，以345天周期在766至879视星等范围变动。
北冕座TY是鲸鱼座ZZ型脉动白矮星，质量约为太阳七成但直径仅11个百分点。1990年发现的北冕座UW是低质量X射线联星系，主星是周围形成吸积盘、向伴星吸收物质的中子星，伴星质量小于太阳。北冕座UW的亮度变化周期非常复杂，两星体以111分钟轨道周期相互围绕旋转，恒星周围的吸积盘轨道周期1126分钟。整个联星系的闪烁周期是55天，说明吸积盘不对称且绕恒星进动。
太阳系外行星系
人类已在北冕座五个恒星系发现系外行星，其中四个是用径向速度法发现。经过2005至2012年共七年的光谱分析，科学家认为贯索七拥有约67倍木星质量、公转周期418天、平均间距约13天文单位的行星。贯索七是光谱等级K2III的橙巨星，质量是太阳17倍但半径已膨胀到21倍，光度151倍。贯索增四是光谱等级K1IV的橙次巨星，质量接近太阳两倍，周围还有岩屑盘，附属行星公转周期34年。估计该行星拥有25倍木星质量，岩屑盘的尺寸表明贯索增四很可能还有第二颗亚恒星相伴。K型群巨星北冕座ο已经确认拥有083倍木星质量的行星，公转周围187天，这幺小的质量在群巨星行星里非常罕见，确知的仅有两颗。光谱等级K0III的橙巨星HD 145457所附行星质量为木星29倍，公转周期176天。XO-1是离地560光年的11等黄主序星，光谱等级G1V，质量和半径都接近太阳。2006年科学家用XO望远镜采用凌日法发现绕该星旋转的热木星XO-1b，尺寸与木星差不多但公转周期只有三天。
1997年，科学家宣布通过分析贯索九的径向速度发现木星尺寸行星。贯索九是光谱等级G0V的黄主序星，还是类太阳恒星，距地球约57光年。依巴谷卫星后来提供的测量数据更加精确，表明贯索九属约一百到两百倍木星质量的低质量恒星。计算结果表明联星贯索增三的适居带轨道范围可能存在稳定行星，联星本身由光谱等级分别是G1V与G3V的两颗黄主序星构成，质量与光谱等级都接近太阳。依巴谷卫星没有在预想位置发现行星，但2001年科学家发现距联星3640天文单位的棕矮星，质量约木星63倍，光谱等级L8。
深空天体
星系团的X射线图像
北冕座可供业余望远镜观测的星系很少，其中NGC 6085和NGC 6086是距离较近、用望远镜可在同一视野观测的螺旋星系与椭圆星系。阿贝尔2142是直径达600万光年的巨型X射线发射星系团，由尚在合并的两个星系团形成，红移00909，以每秒27250公里的速度远离太阳系，亮度只有16视星等，现距地球约12亿光年。RX J15329+3021星系团离地约39亿光年，中央椭圆星系包含的超大质量黑洞无论质量还是吸引力都位居已知黑洞前列。阿贝尔2065是非常密集的星系团，包含四百多个星系，最亮时也只有16视星等，距地球超十亿光年。阿贝尔2065、2061、2067、2079、2089、2092组成北冕座超星系团。艾伯耳2162星系团属武仙座超星系团。
神话
和北冕座
希腊神话把北冕座与忒修斯的传说联系起来，把星座视为天神狄俄倪索斯送给克里特公主、米诺斯之女阿里阿德涅的皇冠。阿里阿德涅此前被忒修斯遗弃，她戴着皇冠嫁给狄俄倪索斯，天神为庆祝婚礼把皇冠放在天国。另有说法称狄俄倪索斯一片痴心，把皇冠送给阿里阿德涅。忒修斯赶到克里特杀死克里特人命令雅典进贡供养的牛头怪弥诺陶洛斯，阿里阿德涅把皇冠转送忒修斯，后者利用皇冠的闪光逃离迷宫，狄俄倪索斯后将皇冠放在天国。拉丁作家许癸努斯笔下的北冕座代表皇冠或花环，巴克科斯首赴奥林匹斯山面见众神途中用于伪装，朱庇特也曾用来掩盖身份与凡人塞墨勒幽会。古典天文学家克劳狄乌斯·托勒密著作《天文学大成》列出的48个星座就包含北冕座。
威尔士神话把北冕座称为“Caer Arianrhod”，意为“银环城堡”，是阿丽安萝德在天国的居所。古波罗的海地区居民把北冕座视为“Darželis”，意为“花园”。
阿拉伯人曾把北冕座称为“الفكة”，意为“分离”或“破碎”，指星座恒星类似打散的珠宝，也可理解成破碎的碟子。贝都因人把北冕座视为“قصعة المساكين”，意指“穷人的碗碟”。
美洲原住民斯基迪人认为北冕座恒星代表辅佐酋长北极星的长老。星座还用于代表火炉上的排烟孔，是原住民向天神传达信息的渠道，也是召集酋长共商大计的手段。肖尼族把北冕座恒星视为“天上姐妹”，每晚从天而降在地面起舞。贯索四代表最小同时也最漂亮的妹妹，猎人变成田鼠靠近后抓住她。两人成婚但她后来返回天国，伤心的丈夫之后带着儿子追随而去。加拿大东部的米克马克人把北冕座称为“Mskegwǒm”，意为天熊的巢穴，天熊由大熊座恒星天枢、天璇、天玑、天权组成。
波利尼西亚人对北冕座各有说法，土亚莫土群岛居民称为“Na Kaua-ki-tokerau”，也可能叫“Te Hetu”。夏威夷州可能称“Kaua-mea”，新西兰称“Rangawhenua”，库克群岛普卡普卡岛叫“Te Wale-o-Awitu”，东加称“Ao-o-Uvea”或“Kau-kupenga”。
澳大利亚原住民天文学根据恒星组成的形状把北冕座称为“womera”，意为“回力镖”。新南威尔士州西北部威尔温人把北冕座视为“鹰巢”，河鼓二与织女一均代表住在巢内的老鹰。北澳大利亚的瓦达曼人把北冕座视为男子律法、女子律法、男女律法、人类存在问题的集合点。
后期发展和流行文化
科比尼亚努斯·托马斯1730年版星图为向萨尔茨堡总主教致敬把北冕座拉丁语名改为“Corona Firmiana”（“北梧桐”），后世天体制图师没有采纳。北冕座是霍华德·菲利普斯·洛夫克拉夫特1923年短篇小说《睡神》情节的重要组成，主角对之非常害怕。2002年，芬兰死亡旋律金属乐队Cadacross（“卡达克罗斯”）推出北冕座同名专辑。
英仙座是北天星座，以希腊神话英雄珀耳修斯命名，是二世纪天文学家克劳狄乌斯·托勒密所列48个星座，也属国际天文联会定义的88个现代星座。英仙座附近有许多同样以希腊神话命名的星座，如西侧的仙女座、北面的仙后座。星座南面挨着白羊座和金牛座，东靠御夫座，北临鹿豹座，西接三角座。19世纪早期星图还把英仙座画成持有美杜莎头部的珀耳修斯，两个星群一起得名“珀耳修斯与美杜莎的头”。
银河系的银河平面穿过英仙座，星座以179视星等的黄白超巨星天船三最亮。天船三与周围许多恒星都属英仙座α疏散星团，其中又以肉眼可见的食双星大陵五最富盛名，身为变星，大陵五还与代表凶险的传说联系在一起。英仙座的知名星体众多，英仙座X是拥有中子星的联星，英仙座GK是视星等在1901年达到颠峰的新星，达02。中国古代就在英仙座发现两个距离很近的疏散星团组成双星团。英仙座星系团距地球25亿光年，是规模非常庞大的星系团，因英仙座得名。一年一度的英仙座流星雨是非常显着的流星雨，辐射点就在英仙座星系团。
历史与神话
根据希腊神话所述，珀耳修斯是达那厄之子，受波吕德克特斯国王之命去取戈耳工美杜莎的头颅。美杜莎能把注视她双眼的人变成石头，珀耳修斯趁她熟睡时将她斩首，美杜莎的尸体生出珀伽索斯和克莱萨尔。珀耳修斯继续赶赴克普斯的王国，挽救向刻托献祭的国王之女安德洛墨达。他用剑杀死刻托救下安德洛墨达，用美杜莎的头把波吕得克忒斯国王及其追随者变成石头，把渔夫狄克提斯推上国王之位。珀耳修斯与安德洛墨达结婚并育有六个孩子。英仙座代表珀耳修斯，旁边是代表安德洛墨达的仙女座，代表克普斯的仙王座，代表安德洛墨达之母卡西欧佩亚的仙后座，代表刻托的鲸鱼座和代表珀伽索斯的飞马座。
公元前911至605年新亚述帝国统治巴比伦期间，英仙座人称“老人座”，此后星座又在七世纪的天文学《星表》中与东方联系起来。
非西方文明天文学
中国古代天文学将西方文明划入英仙座的恒星分为四个星官。“天船”属西方七宿“白虎”的第三宿胃宿，代表华人为应对重大洪灾季节建造的大船。以天船命名的恒星位于英仙座北部，如天船七、天船五、天船四、天船三、天船二和天船一。“积水”同属胃宿，代表八月底至九月初洪水季节开始时可能出现的特大洪灾，星官包含的英仙座恒星只有积水，天船七可能也在其中。“大陵”也是胃宿星官，代表八月埋葬大规模处决罪犯的堑壕，在英仙座拥有大陵四、大陵增十八、大陵六、英仙座24、大陵增十六和英仙座17。“积尸”代表处决犯人的尸体，在英仙座只有一颗同名恒星。
波利尼西亚文明大多不把英仙座单独视为星座，只有社会群岛居民为星座起名，发音类似“法依蒂”，意为“小山谷”。毛利人可能把大陵五称为“马多希”，意为“分裂”，但尚无定论。根据记载，毛利人声称“马多希”出现和消失的时间影响潮汐涨跌，从涨潮周期判断“马多希”可能是毕宿五附近的蓝白恒星，但潮汐回落时间与恒星消失不符。
简介
英仙座南临白羊座和金牛座，东靠御夫座，北接鹿豹座与仙后座，西接仙女座和三角座。星座覆盖615平方度的天空，在88个星座中排第24。星座每年北半球春季在北侧天空最为显着，主星群包含19颗恒星。1930年，尤金·德尔波特正式划分星座边界，英仙座呈26条边的多边形（见文首信息框）。根据赤道坐标系统，星座赤经位于和之间，赤纬在至之间。1922年，国际天文联会确定以三字母缩写“Per”代表英仙座。
显着特点
恒星
大陵五是英仙座最有名的恒星，代表希腊神话中戈耳工美杜莎的头，拜耳命名法分配的字母是“β”，埃及神话中称为“荷鲁斯”，阿拉伯语称为“رأس الغول”，代表“恶魔的头”，希伯莱语称为“Rosh ha Satan”，意为“撒但的头”。大陵五距地球928光年，视星等以2867天为周期在35至23范围变化，是大陵五型变星的原型，实际包括三颗星体，属三合星。其中最亮的是光谱等级B8V的蓝白主序星，质量是太阳的35倍，光度180倍。第二颗是光谱等级K0IV的橙次巨星，已经开始冷却并扩张，半径是太阳35倍，光度45倍，但质量只有其八成。两颗星相距仅005天文单位，即地球到太阳距离的半成，第二颗星尺寸更大，运行至挡住更明亮的第一颗星时，联星的视星等便降至低谷。第三颗星体是光谱等级A7的主序星，距另外两颗星的平均距离是269天文单位。天仙座AG也是大陵五型变星，主星是669视星等的B型主序星。天大将军二是没有相互侵蚀的双星，主星是光谱等级B05的Be星而且很可能是巨星；从星估计是恒星残留，光谱等级与O型次矮星接近。
天船三又名“米尔法克”（Mirfak，源自阿拉伯语，意为“肘”），是英仙座最亮的恒星，视星等达179。天船三是超巨星，距地球约590光年，光谱等级F5Ib，光度与直径分别是太阳的五千倍和42倍。英仙座α疏散星团（又称“Melotte 20”和“Collinder 39”）包含大量明亮星体，其中又以天船三最亮。天船三附近的英仙座明亮恒星包括30视星等的Be星天船五，43视星等的Be星天船四，40视星等的Be星天船六，29视星等的仙王座β型变星卷舌二，52视星等的恒星天船增一，55视星等的恒星大陵增廿，50视星等的恒星天船增二，47视星等的恒星天船增三。天文学家曾以为405视星等的大陵三也属英仙座α星团，但这颗与太阳非常相似的恒星离地球只有34光年，光度是太阳22倍，是光谱等级G0V的黄主序星。经过大量搜索，人类依然没有在大陵三周围发现行星系统存在的迹象。
286视星等的卷舌四亮度在英仙座排第三，距地球750光年，是太阳26至27倍半径、光度47万倍的蓝白超巨星。英仙座OB2群和卷舌四群是地球可见的明亮巨星和超巨星运动群，卷舌四是其中最亮的恒星。卷舌四还是三合星，偏亮的从星为916视星等，是距主星约3900天文单位的蓝白主序星，光谱等级B8；偏暗的从星是白主序星，距主星约五万天文单位，光谱等级A2，99视星等，可能与另外两颗星引力结合。英仙座X是运动群内的双星，其中一颗炽热明亮，另一颗是中子星，亮度只有672视星等，即便条件理想的黑夜也难以用肉眼观测。英仙座X是X射线源，目前主星质量正大量流失。英仙座ο又名“阿提克”（Atik），是385视星等的多恒星系统，人类曾将它划入英仙座OB2群。英仙座ο包含两颗蓝白色恒星，一颗是光谱等级B15的巨星，另一颗是光谱等级B3的主序星，两星围绕彼此旋转，周期45天，且因间距小扭曲成卵形。系统内还有第三颗恒星，但人类尚不了解。估计英仙座ο距地球1475光年，距卷舌四群中心太远，应该与该运动群无关。
英仙座V518也是X射线联星，其中一颗是光谱等级M45V的红矮星，围绕可能是黑洞的神秘高质量天体旋转，公转周期51小时。英仙座V518是X射线新星，电磁波谱的X射线带会以周期爆发。如果其中的神秘高质量天体不是黑洞，那幺英仙座V518就是人类所知最小的X射线新星。经过2012年的进一步分析，这个神秘天体的质量约为太阳21倍，远达不到形成黑洞所需标准，所以星体类型依然存疑。
英仙座GK又称英仙座新星1901，是苏格兰业余天文学家托马斯·大卫·安德森（Thomas David Anderson）1901年2月21日发现的明亮新星，位于大陵五和天船五之间的中途位置，最亮时达02视星等，几乎与五车二、织女一不相上下。此后亮度下降，达到最高亮度约30年后就下降到13视星等。卷舌三又称“门基布”（Menkhib），是光谱等级O7III的蓝巨星，表面温度高达375万K，在夜空所有天体中都名列前茅。卷舌三拥有26至32倍太阳质量，光度更达太阳33万倍。
大陵六的亮度也像大陵五一样变化，但不是食变星，而是脉动变星。大陵六是进入恒星进化晚期的红巨星，已经第二次扩张，半径约为太阳150倍，其中的氦已经聚合成更重的元素，内核由碳和氧组成。大陵六还是造父四型半规则变星，视星等在33至40范围变化，周期有50天、120天和250天三种。双星团拥有三颗尺寸更大的恒星，每个都超过七百倍太阳半径，分别是英仙座S、英仙座RS和英仙座SU，都是半规则脉动M型超巨星。肉眼无法看到这三颗恒星，其中最亮的英仙座SU也只有79视星等，能用双筒望远镜看到。英仙座AX也是联星，主星是进入恒星演化后期的红巨星，物质向另一颗星周围的吸积盘转移。英仙座AX是少见的共生食双星，并且更特殊的地方在于从星不是白矮星，而是A型星。英仙座DY是英仙座DY型变星的原型，属于含碳丰富的北冕座R型变星，是渐近巨星支恒星多变的明证。英仙座DY是106视星等的暗淡碳星，用双筒望远镜都无法观测。
人类已发现英仙座的七颗恒星拥有行星。英仙座V718属形成时间不长的疏散星团IC 348，每47年会被巨大的行星遮挡。估计这颗行星的质量最大可能达到木星六倍，轨道半径33天文单位。
深空天体
（NGC 869和NGC 884）
银河系的银河平面穿过英仙座，但大部分都被分子云阻挡，远不及天空其他位置明显。英仙臂是银河系的螺旋臂，在夜空穿过仙后座、英仙座、御夫座、双子座和麒麟座，朝向银河系边缘。
英仙臂内的两个疏离星团（NGC 869与NGC 884）合称双星团，用双筒望远镜或小型望远镜都很容易看到。两个疏散星团间隔数百光年，距地球均超七千光年，亮度均在4视星等左右，直径都在05度左右。两个星团按特朗普勒分类法都属I3R类，按夏普力分类法，NGC 869属f类星团，NGC 884属e类星团，表明两个星团的星体都很密集，其中又以NGC 869为甚。两个星团与周围星空差别显着，而且明显向中心靠拢，内部各有上百颗恒星，亮度范围广阔。
M34是55视星等的疏散星团，距地球约1500光年，其中散布约一百颗恒星，占据的夜空范围超过满月。视力良好可直接观测M34，但最好是用低倍率望远镜观察。IC 348是形成时间不长的疏散星团，仅约两百万年历史，周围还有形成恒星的星云，距地球约1027光年，许多恒星还有星周盘。人类已在IC 348内发现许多棕矮星，因为棕矮星会逐渐冷却，在形成时间不长的星团内更容易发现。
英仙座星云众多。小哑铃星云是行星状星云，又称M76星云，长两弧分，宽一弧分，亮度101视星等。NGC 1499又称加利福尼亚星云，是美国天文学家爱德华·爱默生·巴纳德1884至1885年发现的发射星云，因面亮度偏低显得比绝大多数发射星云暗，故而非常难以观测。NGC 1333是反射星云和恒星形成区。英仙座还包含猎户臂的巨大英仙座分子云，与其他分子云相比恒星形成率低。
英仙座包含多个知名星系。NGC 1023是1035视星等的棒旋星系，距地球约三千万光年，是同名星系群的主星系，可能还在与其他星系相互影响。NGC 1260可能是透镜状星系，也可能是缠绕非常紧密的螺旋星系，距地球约7670万光年，其中包含的超新星SN 2006gy亮度在人类观测史上名列前茅。NGC 1260属英仙座星系团，这是距地球最近（约7660万光年）的大型星系团，红移00179。同属英仙座星系团的NGC 1275是西佛星系，其中的活动星系核产生物质射流，在星系周围形成巨大气泡。这些气泡还能产生穿过星系团的声波，发出比中央C低57个八度的降B音。NGC 1275是CD型星系，形成后一直在经历星系合并，所以周围有许多小星系残留。此外，其中的活动星系核还是强劲无线电波源。3C 31也是英仙座的活动星系和无线电波源，距地球约237亿光年，红移00173。中心超大质量黑洞产生的物质射流朝反方向延伸数百万光年，形成人类已知宇宙中大小名列前茅的天体。
流星雨
一年一度的英仙座流星雨从七月中旬开始从英仙座辐射，8月9至14日最为活跃，与关联的斯威夫特－塔特尔彗星一样拥有约两千年观测史。2012年发现的英仙座ε九月流星雨源自奥尔特云，但具体源头尚无定论。
历史
尽管这个星座不是很偏南，但直到1752年才由法国天文学家拉卡伊对它进行了描述。这个星座没有星等超过+4的星，它位于波江座旁边，鲸鱼座南方，象征化学熔炉。
2014年NASA公布了这一地区的哈勃超深空照片和解说视频。在天炉座的区域的最遥远的地方包含了约1万个星系，距离地球约140亿光年。
有趣的天体
小哑铃星云，也称为M76、NGC 650/651、巴贝尔（Barbell）星云、或柯克（Cork）星云，是在英仙座的一个行星状星云。它是皮埃尔·梅尚在1780年发现，并被夏尔·梅西耶收录在他的类似彗星天体的目录，编号为M76。1918年，它首度被天文学家希伯·柯蒂斯确认为行星状星云。然而，这个说法存在一些争议，因为艾萨克·罗伯茨（Isaac Roberts）在1891年就确实曾暗示M76可能类似于M57，但是从侧面观察看到的影像。现在，依照它的结构归类为（BPNe）。
目前估计M76的距离是780秒差距，或2,500光年，依此距离推算的平均大小是0378秒差距（123光年）。
星云总光度的视星等为 +101等，它的中心恒星或行星状行星核（PNN）的目视星等为 +159等，蓝星等为 +161等。来自PNN的紫外光激发星云正在膨胀的外壳，显示整个行星状星云正以每秒191公里的速度朝向我们膨胀，并且表层的温度约为88,400K。
通用的小哑铃星云名称源于它与狐狸座的哑铃星云（M27）的相似性。它最初被认为是由两个独立的发射星云组成，因此在NGC天体表中被赋予两个目录编号：NGC 650和651。有些观测者认为这个天体是梅西耶天体中最黯淡、最难看到的天体之一。
M34 （NGC 1039）是在英仙座的一个疏散星团。它可能在1654年之前就被发现，并在1764年被夏尔·梅西耶收录在他的类似彗星天体目录中。 梅西耶描述其为"一群略低于仙女座γ，平行排列的小星星。一架3英尺长的普通望远镜就可以分辨出恒星。" 。
基于838距离模数，这个星团的距离大约是470秒差距或1,500光年 。M34大约包含400颗质量介于012至10太阳质量的恒星，在天空的跨距大约35'，这意味着真实的半径约为75光年。在远离城市的灯光，在足够黑暗的条件下，肉眼就可以看见这个星团。在光污染有限的环境下，可以使用双筒望远镜观看。
这个星团的年龄介于昴宿星团的一亿年与毕宿星团的八亿年之间。观测到的恒星光谱与恒星演化模型预测，给出M34的年龄估计值是2亿至25亿年。这大约是05太阳质量的恒星进入主序带的年龄。做为比较，像太阳这样的恒星只要3,000万年就进入主序带。
天文学家将原子序高于氦的元素都称为金属。这表现在铁元素与氢的对数比值，与太阳中相同的值来比较。对于M34，金属量的比值是[Fe/H] = +007 ± 004。这相当于铁的比例比太阳高17%。除了镍的丰度不足之外，其它元素的丰度都相似。
这个星团中，至少有19颗白矮星。这些是高达8太阳质量的祖恒星，通过主序星演化而来，不再参与核融合来产生能量的恒星残骸。and the last is a type DC。
哑铃星云（Dumbbell Nebula）又称为扯铃星云、M27、梅西耶 27或NGC 6853，是一个位于狐狸座的行星状星云，距离地球约1,360光年。
哑铃星云也是最早被人类发现的行星状星云，它于1764年被法国天文学家夏尔·梅西耶所发现。它的视星等为75等，直经约为8角分。对于使用小型望远镜或双筒望远镜的观测者，这可能是最理想的行星状星云目标。哑铃星云也是业余天文学一个受观迎的观测目标。
结构
所摄得的照片显示哑铃星云的结构与中心恒星
哑铃星云外型类似类球面，且因为酷似哑铃而得名。Moreno-Corral小组在1992年计算了它的膨胀率，发现每个世纪的膨胀率不会超过2″3。根据这个结果，决定出哑铃星云的年龄上限不会超过14,600年。Bohuski、史密斯与韦德曼等人在1970年发现它的膨胀速度为每秒31公里。从这个数据可以得知哑铃星云的半短轴为101光年，这也暗示着它的年龄为9,800年
结点
所摄得的哑铃星云结点结构
就像其他距离较近的行星状星云一样，哑铃星云也拥有结点。它的中心由黑暗的物质与明亮的结点所组成，这些结点并且链接着黑色的尾巴。这些结点较常以对称的黑尾的模样出现，而不是不规则的。就像螺旋星云与爱斯基摩星云一样，结点前的所产生的光亮是因为局部光游离的缘故。
中心恒星
哑铃星云的中心恒星是一颗白矮星，半径约为0055 ± 002R☉，也是目前已知最大的白矮星。科学家Napiwotzki在1999年则估计这个白矮星的质量是056 ± 001M☉。
渊盖苏文，又名渊盖金，扶余人，是高句丽末期极具争议的铁腕军事独裁者。中国史书《新唐书》为避唐高祖李渊名讳，记为「泉盖苏文」，《旧唐书》作「钱盖苏文」，小说《薛仁贵征东》中作「盖苏文」，《日本书纪》则为「伊梨柯须弥」。
渊盖苏文掌权期间，激起了与唐朝的战争，他虽暂时抵御了唐朝的进攻，但其跋扈统治终为高句丽招来了亡国之祸。
背景
据《三国史记》记载，渊盖苏文是渊太祚的长子，渊净土的兄长。渊太祚先后是高句丽平原王和婴阳王的莫离支。渊太祚的父亲渊子游也是高句丽的莫离支。有人说渊氏源出东扶余。
弑君摄政
渊盖苏文早年历史不详。最早的记录是他已经官拜西部大人，督造千里长城。
618年高句丽荣留王登基。642年，晚年的荣留王和他的大臣们想计划除掉一些高句丽内部颇有势力的将领，并准备第一个除掉对其王位最有威胁的渊盖苏文。渊盖苏文得知荣留王的计划后，邀请荣留王和他的大臣们视察他的军队，并设盛宴款待。在宴席上渊盖苏文杀死了荣留王的百名大臣，后又闯入王宫杀死荣留王并分尸，而且没有给荣留王举行葬礼。之后，渊盖苏文自封为“大莫离支”，立荣留王的侄子高宝藏为王并摄政。
与唐战争
唐与高句丽的战争是东亚历史上的重要事件。渊盖苏文在高句丽于唐朝的冲突中扮演着主要角色，也是战争的导火索。
渊盖苏文摄政初期曾试图与唐朝谋好。 他支持用道教取代佛教，并在643年派人到唐朝索要8部道经。一些历史学者认为渊盖苏文的这一做法是为准备日后与唐的战争拖延时间的缓兵之计，以便自己准备更充分，因为渊盖苏文要吞并新罗的企图使唐朝与高句丽的冲突不可避免。
随着高句丽对新罗发动新的战争，高句丽与唐朝的关系开始恶化。645年，唐与高句丽的战争爆发，唐太宗亲自率兵打高句丽并拿下了几座高句丽城堡。不过由于安市城主杨万春的抵抗，唐太宗受阻于安市城无法前行。唐太宗从海上派出由张亮率领攻打平壤的军队受阻于建安城（今辽宁营口）。唐太宗由于杨万春坚守以及寒冬恶劣天气被迫返回。
646年，宝藏王献给唐太宗两个美女，表示善意，唐太宗没有接受。唐太宗以弓服赐给盖苏文，盖苏文接受，但不遣使者答谢，唐太宗于是下诏削弃朝贡。
661年，唐太宗的儿子唐高宗再次攻高句丽，但没取得多大战果。662年，蛇水之战，渊盖苏文击败唐朝将军庞孝泰。不过虽然唐朝数次的出兵都没能达到灭亡高句丽的目的，但严重打击了高句丽的军力。
渊盖苏文有三子，渊男生，渊男产和渊男建。666年五月，渊盖苏文去世，668年八月，高句丽最终被唐军所灭。
评价
朝鲜之声认为荣留王等封建统治集团屈于外部势力的施压和干涉，不顾民族利益，实施让步政策。渊盖苏文发动政变是排斥荣留王腐败透顶的统治集团崇外行为、维护国家尊严的正义行动。他对外实施强硬政策，对内致力于整顿统治体制，增强国力，维护国家主权和尊严的功绩载入史册。朝鲜社会科学院历史研究所研究师黄金石：“渊盖苏文是高句丽的爱国名将，对光荣地捍卫国家主权和尊严作出了很大的贡献。当外部势力645年等经多次侵入高句丽的时候，他每次当高句丽军总指挥，以灵活的智略和战法消灭了外寇。”
参见
&lt;br&gt;618年－642年？
&lt;br&gt;642－666年
河北工程大学，位于河北省邯郸市丛台区，河北工程大学原有四个校区：校本部、中华南校区、丛台校区、洺关校区。现已合并为一个校区。
学校历史
前身为创建于1978年原国家煤炭部下属的河北煤炭学院，1981年更名为河北矿业学院。1983年，根据煤炭工业部部署，更名为河北煤炭建筑工程学院。1996年5月更名为河北建筑科技学院。于2003年4月由原河北建筑科技学院、华北水利水电学院邯郸分部、邯郸医学高等专科学校、邯郸农业高等专科学校合并组建而成；2003年7月，教育部批准河北省以“河北工程学院”为基础筹建“河北工程大学”。2006年2月14日正式由中华人民共和国教育部批准更名为河北工程大学。学校现为河北省与中华人民共和国水利部省部共建制高等院校。
合作交流
据称与清华大学、天津大学等国内大学和美国俄亥拉荷马大学、德国亚琛工业大学等国外大学均创建有一定的校际合作关系。并与美国阿拉斯加大学地质矿业工程学院、英国郎堡大学、乌克兰国立建筑大学联合创办了世界性英文版工程杂志《World Journal of Engineering》，面向全世界发行。
等周定理，又称等周不等式（isoperimetric inequality），是一个几何中的不等式定理，说明了欧几里得平面上的封闭图形的周长以及其面积之间的关系。其中的“等周”指的是周界的长度相等。等周定理说明在周界长度相等的封闭几何形状之中，以圆形的面积最大；另一个说法是面积相等的几何形状之中，以圆形的周界长度最小。这两种说法是等价的。它可以以不等式表达：若formula_1为封闭曲线的周界长，formula_2为曲线所包围的区域面积，formula_3。
虽然等周定理的结论早已为人所知，但要严格的证明这一点并不容易。首个严谨的数学证明直到19世纪才出现。之后，数学家们陆续给出了不同的证明，其中有不少是非常简单的。等周问题有许多不同的推广，例如在各种曲面而不是平面上的等周问题，以及在高维的空间中给定的“表面”或区域的最大“边界长度”问题等。
在物理中，等周问题和跟所谓的最小作用量原理有关。一个直观的表现就是水珠的形状。在没有外力的情况下（例如失重的太空舱里），水珠的形状是完全对称的球体。这是因为当水珠体积一定时，表面张力会迫使水珠的表面积达到最小值。根据等周定理，最小值是在水珠形状为球状时达到。
历史
平面上的等周问题是等周问题最经典的形式，它的出现可以追溯到很早以前。这个问题可以被表述为：在平面上所有周长一定的封闭曲线中，是否有一个围成的面积最大？如果有的话，是什幺形状？另一种等价的表述是：当平面上的封闭曲线围成的面积一定时，怎样的曲线周长最小？
虽然圆看似是问题的表面答案，但证明此事实其实不易。首个接近答案的步骤出现在1838年——以几何方法证明若答案存在，答案必然是圆形。不久之后他的证明被其他数学家完善。
其方法包括证明了不完全凸的封闭曲线的话，能以「翻折」凹的部分以成为凸的图形，以增加面积；不完全对称的封闭曲线能以倾斜来取得更多的面积。圆，是完全凸和对称的形状。可是这些并不足以作为等周定理的"严格"证明。
1901年，阿道夫·赫维兹凭傅里叶级数和格林定理给出一个纯解析的证明。
证明
初等证明
以下给出一个较初等的证明，分5步。
设一条长度为P的封闭曲线围成的区域的最大面积为A，亦以A、P来标记该区域及其边界；那么该图形应当满足如下性质：
1、A是一个凸区域。
2、凡平分周长P的弦必平分面积A。
3、凡平分A的弦，无论方向，长度相等。
4、若MN平分A，O为MN中点，那么对P上任意一点R，都有OM=ON=OR。
5、由于O到P上任意一点的距离都相等，所以P是圆。
傅里叶级数证明
不妨将封闭图形周长定为2π，选取弧长参数"t"其取值为从0到2π，有参数方程("x","y")=["x"("t"),"y"("t")]，并且根据封闭图形有["x"(0),"y"(0)]=["x"(2π),"y"(2π)]。现展开为傅里叶级数：
formula_34
以及相应导数：
formula_35
考虑帕塞瓦尔恒等式（注意这里是实数情形），可以得到：
formula_36
其中第二个等号是因为弧长参数表示的微分满足formula_37的关系。
根据格林公式，得到封闭图形面积为formula_38，因此：
formula_39
整理与联系上述等式(1)与(2)，得：
formula_40
此时可以证明"S"存在最大值（初等证明里没有证明解的存在性），即该不等式取等号时的情况，当且仅当满足以下条件：
formula_41
最终可以得到参数方程即为圆：
formula_42
证毕。
胡风，原名张光人，笔名谷非、高荒、张果等，湖北蕲春人，中国文艺理论家，文学评论家，翻译家，七月派诗人，中国左翼文化代表人之一。曾任中国左翼作家联盟宣传部部长，与鲁迅甚有交往。中华人民共和国成立后，因其文艺思想与主政者不和而遭到整肃，并掀起一场涉及面巨大的政治批判运动。改革开放后被平反。
生平
早年生活及教育
1902年11月2日生人，父亲是做豆腐的，家境不是很好。1913年，十一岁时才上村学。1920年，到武昌读中学。1923年，转入国立东南大学附属中学，与巴金等人同学，1923年加入中国社会主义青年团。1925年，五卅运动爆发，胡风积极参加，下半年到北京考入北京大学预科，1926年改入清华大学西洋文学系。
在国民革命军北伐的风潮中，胡风于1926年回到湖北家乡参加打倒军阀的斗争。中国国民党清党后他从鄂东家乡逃到武汉，正逢武汉国民政府分共，胡风对革命前途悲观。清党以后，他继续在湖北省党部和第三十一军政治部做宣传工作。1927年8月-10月末，胡风在国民党湖北省党部任宣传干事，邓初民任宣传部长，他协助邓编了两期《武汉评论》。11月，胡风到南昌，由陶希圣介绍，编《国民日报》副刊。胡风这段时间的文章，多半发表在国民党湖北省党部的机关报《武汉评论》和国民党江西省党部的机关报《国民日报》上。1928年初，胡风为江西省政府的机关报《策进周刊》撰稿。
赴日
1929年，赴日庆应大学英文科留学，加入日本普罗科学研究所艺术研究会，并在日本刊物《艺术学研究》和《普罗文学讲座》上介绍过中国左翼革命文学，和日本的左翼作家江口涣、小林多喜二交往。加入了日本共产党、日本反帝同盟和日本左翼作家组织，担任中国左翼作家联盟东京分会负责人之一。1931年，九一八事变后，由于胡风在留日中国学生中宣传马克思主义和参加反战活动，1933年3月，特高科密捕，7月初，被驱逐后回到上海，参加左翼文艺运动，任左联宣传部长、行政书记。
回国及文学活动
1935年，编辑地下丛刊《木屑文丛》发表小说，反映苏区斗争，并且介绍苏联社会主义现实主义的理论。1936年初，合作编辑《海燕》，翻译反映朝鲜、台湾人民反殖民抗争的短篇小说集《山灵》，收录台湾左翼作家用日本语创作的小说，如杨逵《送报伕》、吕赫若《牛车》等，是世界汉译日本语台湾文学的第1人。
1937年，抗战爆发后，胡风在上海编辑《七月》文学周刊，出版了《七月诗丛》和《七月文丛》丛书。1938年，当选为在武汉成立的中华全国文艺界抗敌协会常委，兼任复旦大学教授，着有诗集《为祖国而歌》、杂文集《棘原草》、文艺批评论文集《剑·文艺·人民》、《论民族形式问题》，译文集《人与文学》。1941年，皖南事变后他撤退到香港，编辑出版了文学杂志《希望》。1942年，从香港转往桂林。1943年，到重庆，以周恩来提供的中共资助筹办《希望》杂志。1945年1月，胡风在《希望》创刊号上发表舒芜的长篇哲学论文《论主观》。1946年2月回到上海，继续主办希望社，编辑和出版刊物和丛书。1948年底，应中共中央邀请经香港前往北方“解放区”，次年3月抵已“和平解放”一个多月的北平，参与筹备“中华全国文学艺术工作者代表大会”。胡风在7月召开的大会上当选为“中华全国文学艺术工作者联合会”（“文联”）全国委员会委员及随后成立的“中华全国文学工作者协会”（“文协”）常务委员，9月，出席“中国人民政治协商会议”第一届全国委员会会议，11月20日，在中央机关报《人民日报》副刊《人民文艺》上发表组诗《时间开始了》第一乐章《欢乐颂》 。
中华人民共和国建国后
1949年，中华人民共和国建国后，任中国文学艺术界联合会委员、中国作家协会理事、第一届全国人民代表大会代表。着有抒情长诗《时间开始了！》和特写集《和新人物在一起》，杂文短记《从源头到洪流》，长诗《为了朝鲜·为了人类》。
胡风反革命集团案
对其的逮捕证
胡风虽然在政治上拥护中共，但在文艺理论的主张上多次与中共在文艺界领导人（如周扬）的观点相冲突，从1945年起，他的文艺理论多次受到批评，性情直率的胡风则也毫不客气的做反批评。
1951年起，一些人写信给《文艺报》编辑部，要求再次批评胡风文艺思想。1952年初，《〈文艺报〉通讯员内部通报》陆续刊登了这些来信，从此拉开运动序幕。1955年1月中共中央批转中央宣传部《关于开展批判胡风思想的报告》。2月，中国作协主席团扩大会议决定对胡风文艺思想进行全面批判。4月13日，胡风旧友舒芜在《人民日报》上发表抨击文章《胡风文艺思想反党反人民的实质》，同时还交出了在1940年代与胡风的私人通信。5月13日至6月10日毛泽东发起批判胡适资产阶级唯心主义运动，认为胡风反对《在延安文艺座谈会上的讲话》，因此令《人民日报》在5月13日至6月10日发表《关于胡风反革命集团的材料》共三批，并亲自写编者按。在此期间，《人民日报》还在每天第三版用五分之二以上甚至全幅整版，刊登声讨文章、读者来信和漫画鼓动声势。胡风随后被判为反革命，并于同年5月18日被捕。同时还涉及2,100多人，逮捕其中92人，正式定为“胡风反革命集团分子”的78人，骨干23人。胡风入狱十年后被正式判刑14年，后文革时期重判为无期徒刑。
平反
1979年获释，1980年、1986年、1988年，经历三次平反过程，才彻底平反。其间曾出任第六届全国政协常务委员、中国文联委员、中国作家协会和文化部文学艺术研究院顾问等职。在彻底平反前的1985年因患癌症，在北京逝世，享年83岁。1980年，中共中央组织部、公安部给胡风案平反时，认定于1927年至1928年期间，在中国国民党湖北省党部宣传部任干事、在江西剿共军第三十一军政治部宣传科担任了28天科长，负责编辑湖北省党部机关刊物《武汉评论》、江西省党部机关报《国民日报》副刊，曾撰写《反共宣传大纲》等有反共内容的文章的这段历史属于政治历史问题，不应以历史反革命论罪。1985年4月，公安部对胡风历史上的问题（关于胡自动退团问题；在国民党省党部任职；在剿共军任反动职务；写“反共宣传大纲”等）又进一步复查，经研究并请示中央书记处同意，认为胡申诉的五个历史问题，有的仍然证据不足，如“撰写反共宣传大纲”仅一人检举，胡一直否认。
作品及著作
在中国文坛是一位创作颇丰、活跃的文艺理论批评家，他的理论涉及诗歌、报告文学、小说、戏剧、电影、儿童文学、小品文等，其文学理论著述辑成三卷本《胡风评论集》，由人民文学出版社出版。胡风一生主要致力于文学理论研究，着有《为祖国而歌》，杂文集《棘原草》，文艺批评论文集《剑·文艺·人民》、《论民族形式问题》、《在混乱里面》、《逆流的日子》、《为了明天》、《论现实主义的路》，散文集《人环二记》，译文集《人与文学》，特写集《和新人物在一起》，杂文短记《从源头到洪流》，《关于几年来文艺实践情况的报告》（即《三十万言书》）等。
渊男生，唐高句丽平壤城人，字符德。渊盖苏文长子，入唐后因避讳唐高祖李渊名而改称泉男生。
生平
渊男生九岁时就以父荫为郎，15岁时授中里小兄，18岁授中里大兄，23岁改任中里位头大兄，24岁兼任将军。
28岁，任莫离支兼授三军大将军。32岁(唐高宗干封元年)，嗣渊盖苏文之位，加大莫离支总录军国阿衡元首。渊男生与二弟渊男建、渊男产不睦，两方相互攻击，渊男生不敌，率所部和契丹、靺鞨仆从军退守国内城。渊男建自任莫离支，发兵攻打国内城。渊男生先派大兄（高句丽官名）弗德前往唐朝求救，由于其部属发生叛离，于是又派大兄冉有入唐，“重申诚效”，最后才派其子泉献诚正式入唐，请求内附。
唐高宗干封元年六月，唐高宗派左骁卫大将军契苾何力为辽东道安抚大使，以右金吾卫将军庞同善、营州都督高侃为行军总管，以献诚为右武卫将军，为诸军向导，前往接应泉男生。庞同善等赶至新城时，夜里遭到高句丽军偷袭，薛仁贵领军救援，斩首数百级。唐军进至金山时，为高句丽军阻击，不敌溃退，薛仁贵在高句丽军追击唐军时，领兵攻其侧翼，大败高句丽军。在金山之役中，薛仁贵斩首五万余级，并趁胜攻占南苏、木底、苍岩等三城，9月赶到国内城与渊男生会合。男生遂率哥勿、南苏、旨岩等城归降，唐授男生为特进、平壤道行军大总管、持节平壤道安抚大使。又命西台舍人李虔绎就军慰劳，赐给他袍带、金扣七事。
干封元年(666年)11月，司空、英国公李世𪟝被任命为辽东道行军大总管，率裨将郭待封等征讨高句丽。干封二年2月，李世𪟝渡过辽河进军至新城下，随后攻下新城，太大兄渊男产率首领九十八人出降。渊男生在被高宗诏令入朝后，封为辽东大都督、玄菟郡公，赐第京师。随后又被派往高句丽李世𪟝军中，招降旧部。
总章元年(668年)9月，李世𪟝移师于平壤城南，在高句丽下捉兵总管僧信诚的内应下，11月破城而入，渊男建自杀，但未死，与宝藏王一起被俘。12月，高句丽君臣被押回京师献俘于含元宫，宝藏王被授予司平太常伯；渊男产因先降，授司宰少卿；渊男建顽抗唐军，处死。但在渊男生求情下，免死配流黔州（在今重庆彭水县）；渊男生以向导有功，授右卫大将军，进封卞国公，食邑三千户，特进勋官如故，兼检校右羽林军。并赏赐宝器、宫中侍女二人、马八十匹。
仪凤二年(677年)，唐政府任命原高句丽王高藏为辽东州都督，封朝鲜王，“遣归辽东，安辑高丽余众”。但高藏到辽东后却与靺鞨勾结谋叛，被唐朝发现并召回。随后渊男生奉诏安抚辽东，改置州县，颇能安民。
仪凤四年正月二十九日(679年3月16日)，卒于安东府官舍，年四十六。高宗下诏追赠并州刺史。谥曰襄。调露元年十二月廿六日(680年2月1日)葬于洛阳北邙山。
（2007年）起使用至第71回（2020年），左上方的红色圆圈为回数「第○○回」字段
《NHK红白歌合战》，简称《红白》，台湾及香港又译为《红白歌唱大赛》，中国大陆译名为《红白歌会》，是日本放送协会（NHK）自1951年起每年播出一次的音乐特别节目，以现场直播的方式同时在NHK的电视与电台频道，向日本以及全世界播出。前3回于年初播出，第4回起固定于跨年夜播出。
节目名中的「红白」来源于日本剑道中红白对抗的概念，而红白歌合战的红白则是将女性、男性歌手分为红、白两组，并以歌曲对抗赛的形式进行。其举办初期的场地并不固定，除了在当时NHK总部所在的东京放送会馆之外，也曾在东京数个较大的表演场馆举行，例如日本剧场、东京宝冢剧场等；直到1973年起，才固定在该年落成的NHK表演厅举行。目前节目时间长逾4个小时，并分为两个部分：前半段（第1部）为19:15－20:55（日本标准时间，下同），后半段（第2部）为21:00－23:45，前后半段的中间空档插播5分钟新闻。
节目概况
NHK红白歌合战是一档将女性艺人分为红组，男性艺人分为白组，以两组互相对抗的形式演唱歌曲并决出胜负的音乐节目。该节目最初为广播节目，在第4回之后，节目加入了电视直播并改至每年12月31日播出。在日本，该节目可以被视为是“一年结束的标志”，有相当数量的家庭观看本节目。目前，这档节目从来没有被取消或延期播出过。
节目由当年日本人气歌手、当红艺人等其他社会名流参加。虽然节目播出的时段已经延伸至深夜时段，节目的平均收视率仍然位居日本当年年度收视排行的前列，并一直保持着其最高814%的平均收视率记录。公众对于本档节目的关注度也极高，从年末到新年，从报纸到杂志乃至各大商业电视台均会报道红白歌合战的相关话题。
节目一直以来均采用红组与白组对抗的形式，并在前期保持着相当可观的收视率。但自进入21世纪以来由于白组近乎压倒性的连胜，“对抗”的意味逐渐减淡，但每年仍会在正式播出时公布用于判定胜负的审查方法。
播出情况
非日本本土的观众可以在当地的有线电视频道（主要是日本相关的专门频道，例如台湾的纬来日本台）观看，针对日本山区或岛屿等有线信号难以覆盖的区域，如果当地居民的卫星天线可用的话，那幺仍可以通过NHK BS4K、NHK BS8K频道观看本节目直播。
播出时间
自（1967年）以来，红白歌合战的播出时间一度被固定在21:00（日本标准时间，下同）开始。而从1989年（平成元年）开始分为两部分播出之后，第1部播出时间往往在19:00到20:00之间变动。在第1部和第2部之间，NHK综合台会插播年末最后一档《NHK新闻》，时长五分钟。
节目结束时间自（1962年）开始，NHK综合台均于23:45结束（因转播台的不同可能有所差异）。节目结束之后，固定播出跨年特别节目。
演出人员
红白歌合战的演出人员由以下四个部分构成：主持人，表演者，嘉宾审查员与其他出场者。所有演出人员直至NHK官方宣布节目相关信息前都必须遵守严格的“缄口令”，所有消息都必须由NHK官方最先公布。
主持人
主持人（司会）通常是所有出演者中最早被公布的（一般是在10月下旬到11月之间）。正式决定后，NHK将邀请主持人一同在NHK广播电视中心召开记者会。
NHK红白歌合战主持人一般分为“红组主持”、“白组主持”和“综合主持”（総合司会；又可译为「总主持人」）。红组主持和白组主持主要负责介绍各自组内的每一首歌，而综合主持则负责串联整个节目、以及为两组主持分担部分主持内容。红组主持除了少数例外，一般均由女性负责；而白组主持则无一例外都由男性负责。红组主持和白组主持一般都是电视艺人出任，相比之下总主持则是由NHK播报员出任。由NHK播报员担任的总主持并不显眼，却是凸显红白两组主持人和出场歌手，并保证节目正常进行的重要存在。两组主持和总主持通常均由单人出任。
与其他音乐节目相比，即使没有主持经验的艺人也可能被选为红白的主持人。由于红白的台本中将直播流程安排得十分细致，并不需要主持人有过多的即兴发挥，故而NHK红白歌合战的主持人更像是一个表演者。此外，主持人同样可以作为出场歌手进行表演。
2021年的第72回NHK红白歌合战，NHK宣布该年的红白主持将不冠以「红组主持」、「白组主持」、「综合主持」的名义，通称为「主持人」（司会）。
表演嘉宾
参加节目的表演者一般会限制在红白两组各20~30组歌手之间。在对战中正式演出的歌手被称之为“出场”，节目进行时也会展示他们的“出场次数”。NHK惯例会在每年11月下旬举行有关本年度红白歌合战的记者会，首次出场的歌手将在记者会中露面，同时也会公开其他相关信息。
队伍归属
通常来说，主唱的性别将决定组合的队伍归属。节目早期由于严格要求“红组＝女队”、“白组＝男队”，许多混合性别组成的组合不能登台表演，其最多只能安排组合中一人单独参与红组或白组。1968年中第一次允许混合性别组合登场，登场的组合是。由于其主唱Pinky是女性，组合被划归到红组嘉宾行列。其他的分组标准如下：
原则上，同一名歌手不能以不同组合（或组合和独唱）的形式在同一场红白歌合战上多次出场。随着2000年代后半期同一成员属于多个组合的情况越来越多，伴随这些组合人气逐渐上升（如早安少女组、放浪兄弟、AKB48等大型演艺组合），节目也开始接受同一成员代表不同组合出场的情况。
歌手甄选
基本上，出场歌手均会在过去一年中有发行过热门歌曲的歌手或组合中进行甄选。NHK在每年的11月左右举行的遴选会议上确定出场名单并随即对社会公布，在名单公布的同时会举行记者会。自第61回（2010年）以来，出场嘉宾的见面会还会安排流媒体直播。
节目组在第64回（2014年）的记者会上就甄选标准做了“今年的活跃程度、舆论的支持程度、节目的演出和策划需求”三点说明，以此为中心，以下5个方面的数据被用作歌手甄选的参考标准：
从第23回（1972年）开始，作为选拔出场歌手的参考，NHK还会邀请部分人士参与“意见倾听会”（ご意见を伺う会），此意见征求制度一直持续到（1987年）。原则上包括调查问卷在内的调查结果均不对外公布。
此外，也有案例表示以下方面也可能是NHK甄选歌手的参考条件：
对NHK的贡献
对NHK的贡献也是决定歌手是否能够出场的重要参考因素，其中的贡献例如演唱晨间剧主题曲、出演大河剧、提供NHK对奥运会残奥会以及FIFA世界杯的转播时的主题曲、提供（初中部分）的作品等。
同公司的出场上限
早期NHK往往对于每家经纪公司或唱片公司的出场人数都有一定限制。关于此说法，节目组方面虽然没有公开回应， 但杰尼斯事务所的总裁强尼·喜多川曾表示NHK限制其公司仅能出场两组歌手”。
1980年代，在NHK红白歌合战仍是一部制的时候，节目组并没有限制同公司出场歌手人数的概念。但是在（1974年）上，当时处于全盛时代的在当时出场的50组红白歌手中占据了10组。因此在随后的红白上，节目组对于特定公司的出场公司人数上限做了调整。对于渡边制作这样的大型公司来说，旗下歌手的出场数量开始受到限制。由于这个原因，一些热门歌曲的演唱者往往因为同公司出场人数的限制而无法获得出场机会。杰尼斯事务所自（1997年）以来，出场歌手一直被限定在SMAP和TOKIO两组上 。之后，随着岚和NYC boys也在第60回（2009年）上登场，NHK红白歌合战对于杰尼斯事务所出场歌手的上限有所放宽。
是否牵扯丑闻
由于NHK是公共广播，日本法律规定其不能透过播放商业广告获得收入，唯一的收入来源是面向公众收取的收视费（受信料）。因此表演嘉宾的出场费用相当于也由公众支付，所以形象是否干净决定了歌手是否能够入围候选名单乃至被选中，甚至歌手最终是否能登台表演也与此有较大关联。实际上，因为牵扯到丑闻而落选、被辞退，甚至表演被临时取消的案例也曾有发生。
特别是日本娱乐圈内部的“黑色交际”，日本许多暴力团、极道团体等反社会集团可能跟部分艺人之间有合作关系。这种关系会对出场歌手形象产生不良影响，即便是当红艺人也不例外。曾经长期担任红组压轴歌手的美空云雀由于1973年其弟的逮捕案中被发现与山口组第三代组长有勾连，导致她在NHK的民意调查中支持率降低，从而失去了再次出场的机会。在第37回（1986年）上，由于媒体曝光了北岛三郎、山本让二出席暴力团主办的宴席，导致NHK在12月29日临时辞退了两人的演出。2011年，日本各都道府县为消除暴力团影响而制定的《暴力团排除条例》开始实施，NHK方面也于次年公布了“排除暴力团”的方针。
政治因素
NHK作为公共广播，某程度而言也作为国营媒体，故政治上的影响也有可能决定某些国籍的歌手是否能出场红白歌合战。例如韩国歌手自2000年代起逐渐在日本有一定影响力，NHK也会不时邀请来自韩国的歌手登场NHK红白歌合战。随着KPOP在日本流行起来，NHK红白歌合战上也有多组韩国偶像团体出场。然而，2012年的第63回NHK红白歌合战中却没有一个韩国偶像团体出场。有人认为这是受到日韩关系的恶化（特别是独岛主权争议）的影响，民众对于日本音乐节目上有韩国歌手出现而感到不满，NHK为了淡化有关的争议而做出的决定。虽然节目组强调是在调查中认为KPOP的影响力在日本影响力下降，从而没有选出任何韩国歌手，并且否定了和政治有关的因素；而NHK的放送总局长石田研一也曾经在记者会上否定了没有韩国歌手出场是因为「政治和文化的排他」所造成的说法，但实际上之后几年的红白歌合战，几乎没有任何韩国偶像团体可以出场。这一局面直到2017年的第68回NHK红白歌合战，韩国偶像团体TWICE初次出场后才被打破。
随着1975年中在日本电视台节目《笑点》出道的在红白中登场，日本各家商业电视台（民放）节目中的企划歌手及组合（由该节目的主持人或主要固定嘉宾出任）等嘉宾也纷纷登上了红白歌合战的舞台。NHK方面也开始放宽对这类合作的限制。同时，民放在确认获得出场名额后也会进行大规模的宣传，例如在红白歌合战播出结束后播出红白后台景象与演出时的场景，与红白方面积极合作。
于红白登台对于歌手是极高的荣誉，所以受邀歌手大多数都会答应登台。然而，一些歌手时常受邀但全部拒绝而从未登台的，有B'z、松山千春、小田和正、SPITZ等。曾登台但之后开始拒绝的，有南方之星、GLAY等。如果有相当名气的歌手长年拒绝但是终于答应登台，往往能造成极高收视率，如1994年的、2002及2014年的中岛美雪、2005及2011年的松任谷由实、2008年的MrChildren、2016年的宇多田光与2017年的安室奈美惠（后于2018年隐退）等。
节目虽然取名“歌合战”，但实际上也有一些纯演奏类团体登场，例如的、的女子十二乐坊等。
嘉宾审查员
节目组通常会邀请约10位本年度代表名人（包括来自演艺界、体育界、音乐界、文化界、媒体界的人士等等）作为当届红白的嘉宾审查员（ゲスト审查员）。在节目进行过程中，他们不仅要承担评审的任务，在演出过程中还要与台上的主持人进行对话。而在2004年的第55回前，他们被则称为“特别审查员”（特别审查员），而NHK节目制作局局长称为“审查委员长”（审查委员长）。
嘉宾审查员的公开通常较迟，为12月下旬。最初被选为审查员的，大多为下一年大河剧主演者（偶尔为本年度大河剧或晨间剧）。除去已被选作主持人或歌手的，剩余者构成了嘉宾审查员的主要部分。此外，本年度体育界相关人士也会收到邀请，如日本职棒总冠军赛获胜队伍教练或主力选手、大相扑中优秀力士（包括本年度的新横纲、新大关等）。自1984年第35回，江上由美（洛杉矶奥运会日本女子排球铜牌获得者）在节目中出场后，奥林匹克运动会中奖牌获得者也会被邀请到节目中。
其他出场者
除了上述主要出演者外，还有许多名人会作为嘉宾出场。他们按照类型包括：为与歌手或歌曲相关的人物的应援者、参与演奏的乐曲提供者等、会场外转播负责者（大多为NHK播音员）等。此外，在节目过程中也常常能看到本年度走红的年轻搞笑艺人的段子。以上出演节目的主要嘉宾在节目开始前均会对外公开。
特别的，遇上年末最后一天（大晦日）为祝祭日时，有时民放的播音员也会特例出场：
自2010年的第61回开始，“红白幕后谈话”（红白ウラトーク）开始。会场中设置特殊席位，在放送过程中有专门的主持人在此进行对谈，此部分最后会作为节目副音轨进行放送。除了专门的主持人（艺人和NHK播音员）外，歌手、嘉宾也会在此轮流出演。
节目流程
正式演出前
红白两组的主持人（尤其是本身为艺人身份的），在正式节目放送前将会参与到与NHK相关的工作中。各大电视情报杂志往往在年末期刊的封面上使用主持人的形象。此外，由两组主持人作为封面的电话卡、借书卡也会开始发售。
出场歌手的曲目和演唱顺序原则上会全部公开，曲目列表在12月中旬公开，演唱顺序则约为12月下旬公开。
12月下旬开始，有关红白的宣传电视节目会大幅增加，主持人会邀请往年红白的演出者或相关人员作为嘉宾参与节目。同样也会有以回放往届红白名场面为主的节目，例如BS2频道放送的《记忆中的红白》（思い出の红白歌合戦）、《由您选择的印象中的红白·感动的红白》（あなたが选ぶ思い出の红白・感动の红白）。
连接会场和休息室的信道的角落会摆放神棚。在彩排开始前会由神主主持，工作人员参与其中，共同祈祷演出的顺利进行。歌手在彩排间隙同样会来此处祈祷表演成功。
台本在正式表演开始前两天完成，之后便开始彩排。为确保预定流程的正常进行，台本以秒为单位记录了包括主持人和歌手的对话在内的所有演出内容，因此往往会像旧式电话簿一样厚。近年的红白台本也有放开限制的趋势，在台本中留出了大量的空白，为制作人员和主持人留下了一定程度自由发挥的空间。
在彩排的过程中，主持人将与出场歌手进行面谈。面谈过程中，主持人将对歌手正式表演曲目的内容进行说明，出场歌手则会对自己的歌曲进行介绍或分享自己在演唱时的轶事等等。面谈的内容亦有可能在之后的正式表演中公开。此外，在新年到来之前仍会有媒体记者在NHK大厅趁着彩排或正式节目的间隙对出演者进行采访，内容常常包括对过去一年的总结和来年的抱负。
NHK原则上不负责当晚出场歌手的晚餐，通常由出场歌手自行解决饮食问题。
正式表演中
随着节目的开场曲的开始，所有出场歌手慢慢向着舞台方向走去，开始入场。红白两组的主持人在开场时往往身着以所属队伍颜色为主的传统装束登场，总主持人则通常身着西方风格的服饰（男性为西服，女性为女士礼服）。台上的主持人各自介绍完毕后，立即开始介绍第一位出场歌手的歌曲。
原则上，红白两方以组为单位进行对决，而一方每组的演唱时间通常为三分钟左右。本届歌会上最受关注的歌手最长可以获得近10分钟的演唱时间，相应的关注度较弱的歌手演唱时间会缩短至最少2分钟。2000年后的红白歌会渐渐淡化这种对战的形式：例如在对战的过程中引入特别小环节、按照表演顺序本应该对决的歌手与实际对决的歌手有差异等情况也曾出现。或是实际演出时也曾出现本应单独出场的歌手或组合在同一组内出场，最终导致红白两队出场者数目不对等的情况。
约4组表演完后，总主持人通常会开始介绍嘉宾审查员、审查方法以及除NHK综合频道以外的红白收看途径，如广播等。出演者的站位方面，主持人一般站在舞台侧边的花道旁，介绍接下来的歌曲并和其他出演者进行对话。从台下观众的视角来看，红组站在舞台的左侧，白组站在舞台的右侧。节目前期各组的歌手入场过程也在两组的花道分开进行，但因为NHK大厅通向休息室的信道位于舞台左侧，这之后对于白组而言，除了最初和最后的约2组外，其他表演者均从舞台左侧入场。
出场歌手包括使用实况转播在内，一般均为现场表演，很少出现影像出演的情况。
节目组在舞台转换和台上演出方面往往会下很大力气，在这方面往往会消耗较多的预算和时间，例如使用3D技术和其他较先进的影像技术。歌手方面，以“华丽服装”作为亮点的小林幸子、美川宪一等歌手往往会自费较多在自身的服装上。
在歌曲的间隙，节目往往也有回顾本年度的热门话题或流行趋势的环节。歌手自愿参加。目前此环节大多以NHK人气节目（如电视剧）的回顾为主，也会有与奥林匹克等体育活动相关的表演。为今年内去世的名人安排的追悼企划同样常见。此外，在歌手出场的舞台上亦会有其他歌手为其“应援”。特别在2010年之后， 根据收视率情况，人气较高的流行音乐歌手也会在其他歌手表演时出场（例如在演歌歌手演出时作为伴舞）。会场内的观众往往配有小道具，如荧光棒、LED灯、AR系统、假花、面具等等。根据节目组工作人员的指示，观众使用特定道具加入到舞台上下的表演之中。在舞台上演出的歌手有时也会拿到类似的道具，并在表演时使用。
在今年宣布引退/休业的歌手或决定解散/停止活动的乐队和团体，作为「LAST STAGE」（ラストステージ；最后舞台）的出场通常会受到较多关注。另一方面，对于那些选择“年内停止活动”的歌手而言，也常会选择国民关注度较高的红白作为自己最后的舞台。
两组最后出场的歌手往往以「压轴」（トリ）称之（而最后一个出场的会被称为「大トリ」）。因其往往会受到特别关注，节目组一般会选择著名老牌歌手来担当压轴的部分。相应的此时很少会选择年轻歌手，选择年轻歌手时往往会称之为「大提拔」（大抜擢）。在压轴组表演时，之前出场的歌手均会站在两旁的花道旁为其捧场。因节目早期演歌较为流行，相当长的一段时间内均由演歌歌手垄断压轴的位置。红白两组首次同时出现流行音乐歌手作为压轴出场的年份需要追溯至第29回（1978年），因当年演歌销量不佳，由山口百惠和泽田研二担任压轴歌手位置。这之后虽然演歌人气逐年下滑，但因为演歌歌手地位较高，这一惯例仍未改变。从2000年代后半代开始，流行音乐歌手担任压轴的情况增加，演歌歌手出场的情况相应减少。流行音乐歌手方面，也大多选择诸如DREAMS COME TRUE、高桥真梨子、松田圣子等老牌歌手作为压轴。
最后的胜负由会场内的嘉宾审查员、观众以及场外使用双向电视放送系统或手机的观众投票决定。审查方式和结果的发表方式根据年份不同可能有较大差异。在过去的很长一段时间内均是等到所有歌手演唱结束后，由在场的审查员和观众进行投票决定胜负。21世纪以来，场外的一般观众也加入到投票的环节之中。由于统计来自场外观众的投票需要时间，在距第一部结束还有两组左右时，总主持人便会提醒观众开始投票。在这两组表演的过程中统计票数，插播新闻前即发表第一部投票结果。最终投票时，屏幕将回放本届红白所有歌手的表演现场（一组数秒，回放总长不超过3分钟）。统计过程中，包括审查员和嘉宾在内的所有出演者都将站在台上，等待结果的发表。
最终优胜队伍公开的一瞬间，电视画面将出现「○组胜利！！」（○组优胜!!）的标语，同时大厅从天花板内向外喷出大量纸雪花（节目初期也曾使用过向舞台上投掷彩带、从天花板中落下纸带、气球等等庆祝方式）；2000年代开始节目不再向舞台喷洒纸雪花，台上对结果稍事评论后便开始尾声合唱环节（下述）。此外，获胜队伍的主持人（或本队队长）将被授予优胜旗帜，其上挂有历年优胜队伍颜色的条带。而优胜旗授予者每年都会变化，通常为本届红白嘉宾审查员中的一位。
自第4回以来，最后一首歌均为台下观众和台上出演者（包括出场歌手、主持人、嘉宾审查员、嘉宾、合唱团等）一起合唱《萤之光》。届时，将有作为指挥的音乐家出场：
《萤之光》大合唱持续25~30秒，歌曲结束后从舞台的左右两侧会向观众席上方发射礼炮型烟花。
日本当地时间23时45分，节目结束。
演出结束后
演出结束后主持人和出场歌手在NHK广播电视中心一楼食堂举行结束宴。
1月2日左右发表的收视率成为各家媒体关注的焦点。由于同时会发表以分钟为单位的实时收视率，最高收视率点花落谁家更是媒体关注的重心。
节目历史
最初－1960年代
前身：红白音乐对抗
1945年，第二次世界大战结束，一档以《红白音乐对抗》为名的非公开广播节目开始在年末的最后一天（大晦日；下文称除夕）播放。导演希望能打造一档“与新时代相符的音乐节目”，基于剑道中“红白对抗”的概念和娱乐的三要素“速度、性别、运动”，一档预定名为“红白音乐合战”的节目诞生了 。但后来因收到了来自当时驻日盟军总司令 “战败国要什幺‘合战’！”的批评，这一想法不得不大幅改造：将原定的“战”改称为“对抗”，不进行胜负判定，相应的也不再设置审查员，也没有所谓的“应援团”之类的队伍存在。除了节目名，一些表演歌曲和名称也进行了类似的改编：川田正子演唱的《机车噗噗》（汽车ポッポ）原本是一首战时童谣《士兵叔叔的机车》；由近藤作词、与富原薫合作的歌曲《士兵 士兵 万岁》也相应地改编为《铁桥 铁桥 高兴吗》（鉄桥だ 鉄桥だ 楽しいな）。此外，当时大受欢迎的歌曲《苹果之歌》（リンゴの呗）由当年的新人和老牌歌手们合作表演。由于年末的深夜没有电车，赶不上末班电车的歌手只能在NHK东京放送会馆音乐部房间中的椅子上将就着就寝。节目取名“音乐”对抗，除了演唱歌曲以外，也可以在现场单独表演由木琴、曼陀林、尺八演奏的乐曲。
虽然第1回节目的播出获得了大量好评，制作组却没有计划在次年同一时间继续播出相同的节目。1946年之后，跨年节目《红白音乐对抗》不再播出。但由于《红白音乐对抗》播出时带来的反响仍给制作组留下了极深的印象，约五年后的1951年，节目组按照最初“红白歌合战”的想法，制作了另一档在正月（1月3日）播出的节目《第1回NHK红白歌合战》，登场者为没有参加前一年除夕《明星祭》的歌手，节目播出之后仍然得到了极高的评价。因此，节目组决定在次年继续制作该节目，但仍然没有将其打造为长寿节目的想法。
黎明期的摸索
在第1回播出前，出场歌手的曲目和表演顺序均没有公开，而出场歌手名单则记载在当年1月1日的报纸《每日新闻》上。节目采用对抗赛的形式，红组队长为渡边滨子，白组队长为藤山一郎，由队长一边观察对手的表现，一边决定本组由谁来唱哪一首歌。从第3回开始，节目为了体现“运动”这一概念，由一线的体育播音员担任实况播音员。
演出时间方面，第1~3回（1953年1月2日）均选在正月播出。第4回改在与第3回同一年的12月31日播出，因而在一年内播出了两回“红白歌合战”。以此次为契机，在除夕夜播出跨年音乐节目的传统正式固定下来。关于为何仍决定在除夕播出，一说当时可用于举行活动的大型场馆仅在年末年初开放，而年初的剧场大多用于公演；制作组方面也认为在这一天更容易邀请到顶尖歌手。在正月举行的年份，在舞台上方还会装饰有“谨贺新年”的横幅。
自第4回开始，相比过去完全依托于广播转播，节目加入了视觉上的演出并通过电视进行转播。选手在节目中需要进行宣誓并进行优胜旗的返还与授予。本回红组打破了白组自开播以来的连胜记录，首次败北的白组出场歌手异口同声地认为“电视太可怕了。这次输给了（红组女子组的）着装” ，红组歌手重视着装的倾向在节目后来引入彩色电视信号后得到了更得到了进一步的加强。此外，虽然第4回起NHK开始尝试使用电视信号播出节目，但由于当时一般家庭中电视的普及程度远远不足，并且以制作电视节目为内核又会增加节目预算上的压力，NHK仍主要以广播信号转播的形式播出红白歌合战，制作时也更偏向于广播听众的视角。
（摄于2016年），自第24回以来每年的红白歌合战均在此举行。
节目举行场地上，前三回均选在现已拆卸的NHK东京放送会馆。自1953年12月的第4回实行广播·电视同时转播以后，场地开始在东京宝冢剧场、日本剧场、、、新宿KOMA剧场等地中每年轮流更换，直到1973年第24回以后才固定在NHK大厅举行。
早期红白歌合战的音视频与照片资料目前仍在补全中。NHK在官网公开了第1回节目时长一分半的录音，在公开档案馆中亦可听到第6~10回的录音，但录音方面的来源并没有找到明确的文献或网页记载。另一方面，现存的节目照片中最早的则是第2回，这之后的节目也均有保存照片资料。视频影像方面，由于早期录制节目使用的录像带（2英尺VTR）在当时非常昂贵，节目播出后便会再次覆盖录制其他节目。因此，现存的完整节目影像最早的为1963年的第14回。第13回仅留下了广播音频和当时新闻中极其有限的影像。这之后NHK方面仅保存了第21回，由（前红白主持人）的妻子提供了使用家中录像机录制的16~20回、第22回节目。从第23回（1972年）起，NHK保存节目录像的意识增强，这之后至今的完整节目的录像均有很好地保留。
甄选模式的发展
出场歌手方面，截止到第8回（1957年），出场歌手均限制为独唱歌手。但是自第9回中、、、以组合为单位出场后，节目中组合的出场也变得愈发常见。并且截止到第13回，对于以组合形式出场的选手，其对手也一定为组合。直到第7回（1956年），战前和战时派歌手仍常年登台表演。第8回中作为战前派歌手“第一人”一般存在的藤山一郎选择将舞台交给后辈，谢绝了当年的演出。这之后，每年都有战前派的歌手从出演名单中离开。在这新旧交替的过程中，诸如、桥幸夫、平尾昌章（现：平尾昌晃）之类的新人歌手不断地出现在舞台的中央。第13回中，一直以来连续出场的、、等战前、战时派歌手落选，舞台上的面孔均为战后派歌手的时代正式到来。第11回（1960年）左右，节目组会避免同一家唱片公司的歌手相互对决。虽然这之后的节目中也常常出现同场较量，但在压轴环节仍然贯彻这一原则。
本年代内有部分人气歌手因某些原因而无法登上红白舞台：
收视率黄金期的到来
Video Research从第13回（1962年）开始进行电视收视率的调查，并在当年测得了本节目804%的收视率。而在次回（第14回）中，测得的收视率更是达到了814%，这是红白史上同时也是该公司自开始统计以来测得的最高电视收视率。
1969年TBS电视台《日本唱片大奖》开播，由于其直播时间为同日的19:00~21:00，与红白歌合战播出时间重合，常常会出现歌手在《唱片大奖》和红白会场之间移动的现象。届时将有警察保障歌手的转移车辆经过通往NHK大厅的路口均为绿灯，以及时到达另一个会场。
详细请参考以下页面：
1970年代－1990年代
音乐风格的变迁
1970年代以、新御三家、为代表的偶像为首，、摇滚乐组合开始盛行，红白舞台上表演的曲目种类随之发生了改变。
小林幸子的服装属于1980年代的红白歌合战的一大关注点。小林的演出服装自1980年代中期开始趋于华丽，如第36回（1985年）中《夫妇しぐれ》的服装十二单，次年第37回中的《别离》的克娄巴特拉七世的装扮，以及第42回《冬化粧》中如鸟类飞翔的设计。1990年代后，美川宪一与小林幸子之间常常形成服装上的直接对决局面，小林使用大规模的机械装置构建舞台，美川则在歌曲间奏中展示在监修下的幻想世界。由于二人的服装需要花费大量的预算，往往在节目播出前早已开始制作。此外，在除夕演出当天的体育报纸、私营电视台的资讯类节目（情报番组、ワイドショー）也会在演出前介绍二人今年的服装。除了小林和美川，其他歌手也同样以较为华丽的服装出演过。
1990年代由小室哲哉作词作曲的音乐称霸日本音乐市场，受此影响红白中也有大量小室系的歌手出场。此外，节目中与迪士尼进行合作表演的情况也变得常见。除了邀请歌手出演，节目本身也开始制作原创音乐或主题曲，并由出场歌手进行大合唱。这之后所表演的歌曲也可能会发行相关CD，或是在节目中播出，或是做为课题曲。从第45回（1994年）到第48回，在当年中的优胜者将获得本年度红白的出场权，直至1998年《歌曲大赛》停播。
1990年代后半视觉系乐队潮流兴起，X JAPAN、LUNA SEA、GLAY、L'Arc~en~Ciel等登上舞台。
技术发展与舞台改造
第29回（1978年）节目开始使用立体声播出。第32回后节目中几乎所有歌曲都加入了歌词的显示。
从第32回（1981年）开始，歌手不再通过观众席后方入场，而改为从舞台后方直接入场。节目组重新设置了舞台布景，使得在节目过程中也可以更换大规模的舞台装置。第34~36回（1983~1985年）中改为使用授予奖杯的方式表示胜负，获胜队伍选出代表授予金奖杯，落败队伍选出代表授予银奖杯。从第42回（1991年）开始，入场式、优胜旗返还、主持人和歌手代表进行宣誓的流程被省略，一直以来在乐池（オーケストラピット）中进行现场演奏的管弦乐队也被取消。第53回（2002年）中管弦乐团再次出现在舞台上，但是规模已经大幅减小。第63回（2012年）中更是取消了除乐队以外所有的现场演奏，一律改为使用事前准备好的录音。
第41回（1990年）中首次出现在NHK大厅以外的场所进行演出并进行实况转播。1990年柏林墙倒塌，歌手长渕刚在德国柏林连续表演了三首曲目并由电视转播，单人表演时间达到不寻常的15分钟。此事件导致同一回作为压轴的森进一的演出时间缩短，并引起了NHK大厅内观众和其他出场歌手的严重不满。转播演出直到2002年才重新出现，并在之后逐渐发展为一个固定环节。
第42回（1991年）后，节目组尝试在开幕式过程中加入大规模的演出。此外，本届红白被分成了三部：21:00新闻前属于第一部，21:00 - 22:30属于第二部，22:30直到结束为第三部。由于第二部和第三部之间并无类似插播新闻之类的硬性中断，Video Research在进行收视率调查时将后两部合为“第二部”进行统计，这之后包括NHK官方资料在内也采用相同的方式记录本届红白的出场和出场曲。
1995年NHK开播70周年，台标更新，红白中的台标随之改变。
收视率的变动与世纪末的展望
从1970年代到1980年代前半，红白歌会的收视率仍维持在较高水平。第34回（1983年）连续9年担任白组主持人的离开红白舞台，由他所主持的最后一回（第33回）仍达到了699%的收视率。第35回作为都春美隐退前的最后舞台，收视率更是达到可以与1960年代相匹敌的781%。但这之后，节目的收视率便不断大幅下滑。。1988年后节目分两部播出，图表中的数据为两部收视率之平均值。
第40回（1989年）开始红白分为两部播出。同一年昭和天皇驾崩，为了响应“回顾昭和”的呼声和逐渐多样化的观众的音乐喜好，节目组开始注重歌曲选择上的完整性以保证收视率的稳定。在这种对策下，收视率降低的趋势得到了一定的控制，1990年代节目第一部收视率基本稳定在50%左右。虽然1989年一度出现过停播红白歌合战的意见，并以预定的“Asia音乐祭”（アジア音楽祭）将之取代，但因NHK内部人事变动，该计划并未实现。此外，从本回开始第一部表演也加入到最终评分范围中。受此影响，同时播出的TBS电视台《日本唱片大奖》与红白形成了直接的竞争局面。因两档节目间争夺歌手而导致歌手在不同演出场地间大规模移动的现象也变得十分严重。在向NHK方面表示不满的同时，《日本唱片大奖》收视率长期受红白影响，1995年起从30%左右逐年降低。直到2006年TBS将《日本唱片大奖》的播出日期提前了一天，两档节目相互竞争的局面就此结束。
1999年第50回节目结束后，照例播出跨年节目。但在23:58:30左右电视画面再次切换回刚刚结束红白后的NHK大厅，开始播出由红白出演者和现场观众共同进行世纪倒计时的画面。这之后，由本回原创歌曲的表演者们进行大合唱。通常，NHK综合频道播出的《辞旧迎新》与其他私营电视台同时播出的跨年倒计时节目相比均较为简朴，但本回例外。
详细请参考以下页面：
1972年 1985年 1986年 1991年
2000年代－
演歌时代的落幕
21世纪下，音乐销量不再与歌手人气相对等，过去常年出现在舞台上的歌手人气开始下滑。部分传统演歌歌手根据当年的业绩与收入来决定是否参加当年的红白歌合战，而新兴的流行音乐歌手则并不需要通过出演红白来提升人气，有些甚至拒绝了红白出演的邀请。第54回（2003年），男子偶像组合SMAP担任最终压轴并上台表演歌曲《世界上唯一的花》，象征着演歌歌手占据节目压轴演出格局的瓦解。
2000年代后半，由于杰尼斯事务所几乎垄断了男性偶像市场，红白歌合战中几乎每年都由该公司的艺人担任白组主持人。另一方面，红组主持人多由NHK当家戏剧节目（如大河剧、晨间剧）的女主角担任。而过去的“辞退组”，如第59回（2008年）中MrChildren、第60回中等开始在红白首次出演，这之后每年都有过去拒绝红白出演的歌手接受了节目组的邀请并出场。
2010年代后，流行音乐歌手大量登上红白舞台。相对的演歌业则不断萎缩，演歌歌手出场也变得困难，而过去出演的歌手接连以“勇退”的形式离开红白歌会的舞台。但在中坚歌手和新人歌手的支持下，演歌在此时期的红白中仍占有一定席位。
传奇的结束与变革的尝试
随着国民生活方式的多样化，以及当时与卫星频道BS进行同步广播的原因，通过地面电视信号播出的红白歌合战收视率逐渐降至40%左右。过去，民营电视台在除夕当天往往避免自家的节目与红白歌合战正面对抗，随着红白人气的走低，部分电视台也开始尝试制作与红白相对抗的节目。2003年，TBS电视台播出的日本搏斗技节目《K-1 PREMIUM 2003 Dynamite!!》中鲍伯·萨普与曙太郎进行对决时，该节目的实时收视率连续4分钟反超同时播出的第54回红白歌合战，自开始统计收视率以来尚属首次。同一时间，正在红白舞台上表演的是歌手长渕刚。
为了改变红白歌合战的颓势，制作组方面也在对节目进行改革：（2010年）出场服装的小林幸子。实际出场时小林则是站在一直扇动着翅膀的大型仙鹤模型之上。
2004年，某NHK制作人曝出贪污近5000万日元的制作费的丑闻，社会对NHK方面的批判声变得异常猛烈，而对红白“应有的状态”的讨论则再次拿上台面。当年的第55回，为了体现出场歌手选择的透明性，节目组以“希望红白出场的歌手”为题进行了社会调查，但最终收到了来自歌手方面的不满。次年第56回，节目组以“想在红白上听到的歌曲”为题向观众进行问卷调查，但是收到了大量的有组织的投票（组织票），此方式仅此一年便废止了。
CG技术在2010年代后迅速发展，随之节目演出中也大量使用了视频特效技术。随着在NHK大厅以外进行转播演出的方式重回舞台，每年都有歌手通过转播的方式的进行表演。其中在NHK方面帮助下准备的舞台在规模上往往更胜一筹。第63回（2012年）现场演奏被全部取消，取而代之的是全体通过类似卡拉OK的方式进行演出。直到4年后第67回中，现场演奏才重回舞台。
对红白基础模式的反思
2000年代中期开始，对于节目胜负模式、审查方法、男女对抗形式等红白基础模式上的意见不断涌现。相关意见诸如“白组歌手的歌迷有组织地投票，总是让白组赢”，“私营电视台的音乐节目中歌手可以合作表演，对于区分男女的红白来说，表演受到了限制”，“还有必要将男女区分竞争吗”，“在LGBT平权的社会下，也有像生物股长和AAA这样性别构成多样化的组合，以‘红组’‘白组’这种男女相区分的形式竞争胜负难道不应是时代的错误吗”等。
其他
2019年4月29日，NHK综合播出，该节目利用两部共175分钟的时间，在新年号令和到来之际回顾了平成年代的所有红白歌合战。
2021年3月至2022年6月，因NHK大厅需要进行耐震施工，预定在2021年播出的第72回将改为在东京国际论坛举行。这是本节目自开始固定在NHK大厅播出的第23回以来，时隔近半个世纪再次更换场地。
详细请参考以下页面：
2006年 2007年 2008年 2009年 2010年 2011年 2012年 2013年 2014年 2015年 2016年 2017年 2018年 2019年 2020年 2021年 2022年
各回概要
截止最新一回NHK红白歌合战，红、白二组的对战成绩为：红组 33 － 40 白组。从第56回（2005年）到第61回（2010年）的白组6连胜是迄今红白歌合战史上的最长连胜纪录。
※收视率数据由Video Research提供，数据来源为关东地区家庭用户实时收视率。由于第1回至第12回在Video Research成立前举行，故而没有相关收视率数据。
历代压轴歌手及曲目
"（ ）内为歌手出场次数、[ ] 内为歌手压轴次数、浅黄色标记为总压轴。"
历代开场歌手及曲目
"（ ）内为歌手出场次数、[ ] 内为歌手开场次数、浅蓝色标记为总开场。"
节目备注
收视率
每年的红白均以极高的电视收视率而闻名。自Video Search有相关收视率记录（1962年的第13回）以来，收视率最高的为第14回（1963年），为814%（关东地区家庭实时收视率，下同）。因此，在1980年代前半时红白一度被称为“怪物节目”。目前最低收视率则是2021年第72回，为343%。几乎每年红白的收视率都会碾压民营电视台的人气节目，称霸收视率年榜的首位。因次年1月3日前后一般还会发表以分钟为单位的收视率，哪个歌手的部分收视率最高往往成为一时的话题。
“红白效果”
在红白出演过的歌手次年年初的CD销量多少会受到红白的影响（一般为销量再度提升），这个现象俗称“红白效果”（红白効果）。甚至还出现过因红白上的演出而再度受到关注，歌曲再次拢聚人气，结果在次年红白再度出演的情况。“红白效果”不仅会创造出热门歌曲，甚至还会创造出脍炙人口的“名曲”。
演出时的自由发挥
早期在表演过程中为了配合台本，节目组常常使用纸条的形式提示主持人，而后期则在大厅一层的墙壁上上设置了电子显示屏用以提示台本。对主持人而言，收到临时提示信息的情况并不罕见，内容往往包括提醒主持人尽快进入下一个出演者环节等等。
舞台上也曾有过主持人在直播时出错的情况。在1980年之前NHK方面严禁主持人有任何的错误或即兴发挥，后来此项限制才逐渐放开。对担任总主持人经验丰富的曾说道，“NHK大厅里住着魔鬼”，“就是这些魔鬼在折磨着主持人前辈们” 。这之后的红白大幅降低了此要求，即允许主持人即兴表演、自由发挥。例如1970年第21回中，白组主持人建议白组歌手桥幸夫演唱《いつでも梦を》，而不是原定的《俺たちの花》。
另一方面，也有艺人利用这种“通融规则”，在正式表演时加入原定安排之外的部分的情况。而其中对节目产生较大影响（多为不良影响）的，这之后甚至会受到NHK的封杀。下面列出了一些案例：
（上述所有情况均可能通过银行账户扣款支付收视费。）
综上，未缴纳收视费或欠费者无法参与申请。寄来的申请明信片上必须准确填写收视费缴纳人的姓名。NHK其他的公开放送节目亦采用同样的征集规则。
渊男建，高句丽人，渊盖苏文次子。唐高宗干封元年（666年），与弟渊男产逐兄渊男生而自立为莫离支。因避讳唐高祖李渊名而改称泉男建。
总章元年（668年），唐朝联合新罗的军队讨伐高句丽，大将薛仁贵率军击败渊男建。渊男建逃回平壤。高丽王高藏命令渊男产率领首领九十八人持白旗向唐军统帅李𪟝投降。但渊男建拒绝投降，紧闭城门坚守，还多次出兵向唐朝军队出击，不过屡战屡败。
后来他将一部份军事任务交由僧人信诚，，让唐朝军队攻入，城焚，渊男建欲死，不成。被俘后随高藏、同时被灭的百济国王扶余丰等一同被献俘于唐都长安太庙。
唐高宗认为男建是举兵抗唐的元凶，令杀之，后因渊男产的求情改判流放至黔州（今重庆市彭水县）。渊男建死于流放地。
张柏芝（英文名：Cecilia Cheung Pak-chi，），本名张栢芝，香港女演员及女歌手。凭《忘不了》电影得到第二十三届香港电影金像奖最佳女主角。
生平
早年生活
张柏芝出生在香港油麻地和佐敦之间的八文楼，其父张仁勇为上海人，其母则是中英混血儿。张柏芝因此遗传到妈妈的英国血统，幼年时头发自然金色。
其父张仁勇，又名「胡须勇」，乃三合会联英社之前坐馆，活跃于佐敦。其母嫁给张仁勇前曾经有过一段婚姻，育有一女，婚后又为张仁勇生有一女两子。张柏芝有一孪生弟张豪龙和一弟张柏文。张柏芝9岁时，父母离婚，张柏芝在13岁时随母亲前往澳洲墨尔本就读中学，其后就读于当地的。她在澳洲生活前，于沙田成长，曾就读培基小学和东华三院黄凤翎中学。完成中一年级后便到澳洲升学。
入行经过
1998年年中，刚从澳洲RMIT Holmes College毕业回港的张柏芝在经理人朱永龙的安排下，接拍了阳光柠檬茶广告，当这个不同版本的广告片播出后，迅速风靡华人世界，各大小杂志、电影公司及广告公司都惊为天人。其中周星驰于1999年贺岁片《喜剧之王》钦定张柏芝为电影女主角，并成为当时贺岁片中的票房冠军，成绩达3000万港元。此后，张柏芝接拍了由嘉禾电影公司出品，马楚成执导的爱情文艺片《星愿》并担演女主角，与任贤齐及苏永康合力演出。此片于1999年8月暑假黄金档期上演，而票房更冲破1000万，更为张柏芝奠定影坛的地位。唱片方面，张柏芝于1999年6月9日正式签约环球唱片公司，而首张EP《任何天气》于1999年7月26日推出后反应极之热烈，短短数周便售出白金数量。
经理人合约纠纷
张柏芝凭《喜剧之王》走红后，旋即与经纪人朱永龙爆发骂战。张柏芝以「经纪人条件太苛刻」为由要求解除与朱永龙的合约，朱永龙反驳她只是因为贪婪片酬才诬告他，而且朱永棠爆料张柏芝的家人把朱家当成「提款机」，甚至会半夜索财；他亦向法院递交了诉状，要求张柏芝赔偿损失。此事曾在港台两地闹得满城风雨，引起很多讨论。
原本张柏芝在1998年与朱永龙签订经理人合约，他从中可以抽取张柏芝工作酬金的3成作为佣金。1999年张柏芝凭借电影《喜剧之王》一炮而红，此后朱永龙与张柏芝双方却闹至告上法庭，直至2004年2月，双方才达成协议为官司画上句号，但是这次和解并不能弥补经理人朱永龙在她身上花的钱和打官司时的律师费。
诉讼期间，朱永龙遭3名歹徒疯狂狂砍，送医急救途中三度呼吸暂停，靠着顽强意志才惊险保命。当记者询问张柏芝此事是否有「不为人知的内情真相」，张柏芝却不闻不问拒绝回应。
电影历程
尖沙咀星光大道的手印
张柏芝刚出道不久，便接拍周星驰执导的电影《喜剧之王》，在电影的初次演出引起广泛好评，之后的爱情文艺电影《星愿》又为她大放异彩，更为她带来金像奖新人奖，她又趁势出唱片，歌影两双栖发展，之后主力拍电影。2001年张柏芝首度进军国际，接拍韩国电影《白兰》，该片不但为她带来大钟奖提名，在韩国知名度大增。2004年张柏芝先后凭《大只佬》及《忘不了》两部电影横扫香港各大电影颁奖礼女主角奖项，《忘不了》一片更为她带来首个金像奖影后荣衔。2006年张栢芝更应邀为香港星光大道打手印，成为被表扬的电影工作者之一，也是香港星光大道目前为止，获邀打手印的演员中最年轻的一位。
复出
2010年6月，张柏芝随同周星驰的《长江七号爱地球》片组参加上海国际电影节，并表示演唱该片主题曲，其后张柏芝携长子Lucas一起演唱该主题曲。同年10月，她以千万片酬接拍黄百鸣的贺岁电影《最强囍事》，合作演员有甄子丹、刘嘉玲、古天乐与熊黛林，此片成为她复出的首部电影。 之后，张柏芝亦加入了马楚成导演的《极速天使》进行客串演出；此后；10月，张柏芝携长子Lucas加入王晶新片《无价之宝》。
张柏芝复出后，她所拍摄的多部电影如《无价之宝》、《影子爱人》均票房失利，《河东狮吼2》在香港票房非常差劲，因而被称为「票房毒药」。随后张柏芝将精力放在了综艺节目上，她先后参加了《妈妈咪呀》《最强大脑》《明星到我家》《女神的新衣》《星跳水立方》等娱乐节目。
2015年3月17日，中国星公司老板向华强在香港接受访问，痛斥在电影《3D封神榜》反串饰演「哪吒」一角的张柏芝，称其失场（失约拍摄片场）、神经质变本加厉，并决定对张柏芝「永不录用”。
工作室
2011年，张柏芝签约新东家新亚洲娱乐集团，为期八年，不仅拥有经纪全约，还成立「张柏芝工作室」。进军唱片制作、视频投资等幕后工作。2013年张柏芝宣布息影一年并定居新加坡，在当地从事义工等公益事业。2014年张柏芝全面投入电影工作，5月14日，张柏芝带病录制节目《妈妈咪呀》，9月25日，张柏芝发布MV《该是时候》。2014年10月。张柏芝与女星黄圣依、秋瓷炫、李金铭一起参加江苏卫视实景真人秀《明星到我家》。在2014年11月28日《明星到我家》节目中，播出她在香港的第二个家、位于南区鸭脷洲可看到海景的港湾工贸中心（一座工厂大厦）。2022年起，张柏芝全新推出粤语歌曲。
个人生活
张柏芝绯闻甚多，较知名的有与朱永棠、歌星陈晓东拍拖，并与谢霆锋交往、结婚，又有陈小春及荷兰籍男友，不过早期惟一公开承认过的男友只有陈晓东，这对恋情被称作「东芝恋」。
2000年6月，张柏芝与陈晓东的恋情因一起去医院探望好友陈奕迅而曝光，张柏芝受访曾表示，两人是在她的坚持下公开恋情。
2001年张柏芝与陈晓东分手，多数传媒报导写两人恋情时间为一年，不过陈晓东在《康熙来了》承认两人谈了两年多的恋爱。香港传媒对于分手原因有诸多揣测，张柏芝当时的经纪人向太当时更为保护张柏芝而将分手原因怪罪陈晓东，数度在传媒上指责陈，张柏芝则在2005年于鲁豫有约谈起这段感情，表示两人最终是因媒体过度关注导致过大压力才使恋情而无疾而终，并称「太多人有太多意见.因为这样的压力，让我失去一个很好的男人」，说若当时「没有公开的话，我想现在还在一起」。陈晓东则在分手后将事业重心转往中国大陆，但两人均称「分手后仍是永远的知己」。
2006年9月29日，谢霆锋在香港国际机场向传媒公开表示已与张柏芝在菲律宾秘密结婚，但双方父母均不知情。
2007年，剖腹分娩，诞下6磅8安士儿子。
2008年，张柏芝与陈冠希的不雅照事件门，可谓使二人的事业形象遭受了毁灭性的打击。事件发生后张柏芝主动与陈冠希在飞机上合影。
2010年，再剖腹诞下7磅7安士小儿子。
2011年8月中旬，张柏芝与谢霆锋公开宣布离婚，两人日后共同抚养两个儿子。
2018年11月18日，张柏芝通过工作室发表声明，指产下第三胎儿子。其𡥧子出世纸被记者曝光，跟随母亲姓氏，惟其出生登记并未填写父亲的名字。
渊男产，唐高句丽人，渊盖苏文子。干封元年，与其二兄渊男建逐长兄渊男生，拥男建为莫离支。
生平
渊男产生于唐太宗贞观十三年（639年），家族世为高句丽莫离支，早年任中里小兄，18岁授中里大兄，21岁升中里大活，23岁迁中里位头大兄，累迁为中军主活。协助二兄渊男建逐走长兄渊男生，拥男建为莫离支。30岁时为太大莫离支（太大兄）。
唐干封元年（666年）11月，唐高宗任命司空、英国公李世𪟝为辽东道行军大总管，率裨将郭待封等征讨高句丽。干封二年(667年)2月，李世𪟝渡过辽河进军至新城下，随后攻下新城，渊男产持白幡率首领九十八人出降。
总章元年（668年）9月，李世𪟝移师于平壤城南，在高句丽下捉兵总管僧信诚的内应下，11月破城而入，渊男建自杀，但未死，与宝藏王一起被俘。12月，高句丽君臣被押回京师献俘于含元宫，宝藏王被授予司平太常伯；渊男产因先降，授司宰少卿（从四品上），仍加金紫光禄大夫员外置同正员（正三品）；渊男建顽抗唐军，被处死刑。但在渊男生求情下，免死配流黔州（在今重庆彭水县）；渊男生归附唐朝则授右卫大将军，封为卞国公。
696年或697年，泉男产封辽阳郡开国公，又迁营缮监大匠员外置同正员。圣历二年（699年），授上护军（勋官，视正三品）。
大足元年三月廿七日（701年5月9日），卒于府第，年六十三，以其年四月廿三日，葬于洛阳县平阴乡某所。
和熹皇后，名邓，为东汉和帝第二位皇后、东汉女政治家，南阳新野人（今河南省新野县）。
幼年时期
邓绥是汉光武帝时太傅邓禹的孙女，邓禹为南阳豪族，随光武帝起事，为东汉初年的功臣；其父邓训，曾为护羌校尉，抚边有功。其母阴氏是阴丽华堂弟的女儿。邓绥为训之次女，有长姊邓燕、娣邓容。
邓绥虚岁五岁时，邓绥的奶奶亲自为其剪发。邓绥的奶奶年事已高，眼睛不太好，不小心弄伤了邓绥的额头，邓绥忍痛不言，后来对身边的人表示之所以忍住是因为担心讲出来后会让老人家难过。虚岁六岁时，邓绥的兄长喜欢摸邓绥的头发，邓绥表示，「身体发肤，受之父母，不敢毁伤，孝之始也，你们为什么要弄我的头发？」同年邓绥开始阅读《史籀篇》，七岁时学习《论语》，十二岁通《诗》、《论语》，常和诸兄互相讨论。邓绥不喜学习家务，其母屡屡反对，认为女子需学习女工操持家务，但邓绥还是违背了母亲的意愿，除了在白天学女工之外，晚上仍读经书；邓父则对她读书较为支持，常常与她谈论政事。同时姐姐邓燕由于去世过早，只留下刚刚出生的女儿。十二岁的邓绥亲自养育。
入选掖庭
邓绥身高七尺二寸（约166米）。永元四年（92年），邓绥本来要入宫，恰逢邓训去世，邓绥为守孝，入宫事宜推迟。十五岁（95年）时，因容貌娇艳，被选入掖庭宫中，次年即升为贵人。她入宫之后，对待皇后阴氏甚谦谨，如在在宴会之时，嫔妃们多打扮艳丽，只有她素服不装饰，看起来清丽脱俗，而且平时衣服不敢与A|zh-hant:阴后; zh-hans:阴后同色。 此外，她晋见皇上时不敢与阴后并坐立，走路也表现谦卑的姿态，说话也不敢先于阴后；阴后被疏远之时，就常托病不受皇上召见。但也因她如此地敬慎恭谨，更加受到皇帝的喜爱。阴皇后身材矮小，有时举止不雅，时常遭到身边人嘲笑，邓绥却从不提及。她虽身为贵人且深受宠爱，但身为皇帝妾室，仍然谨守其位分，尊奉阴皇后而不敢逾越。
然而阴皇后因失宠而嫉恨邓绥，开始试图加害她。一次皇帝重病，阴皇后暗地里对近侍说：“等我掌权了，一定不让邓家有一个人存活。”邓绥风闻，想要饮药自尽以保全家族，内侍赵玉等人坚持劝阻，最后骗她皇帝病愈遣使通报，于是邓绥才放弃自尽；第二天和帝竟然真的病瘉了，邓氏才逃过一劫。
进位皇后
永元十四年（102年）阴后因为被告行巫蛊之事为皇帝所废；邓绥还试图想方设法保住阴后后位。此时和帝对邓绥十分宠幸，希望立她为后。邓绥知道后坚持称病，并深居简出。和帝愈发认为她有德行，立她为皇后。邓绥成为皇后之后，和帝曾想要加封邓氏一族；然而被邓绥坚持推辞了，终和帝一朝，邓绥兄长邓骘仅仅官居虎贲中郎将。同时邓绥还要求地方不要向皇后进贡奢华之物，只需进贡纸墨就行。和帝为了让邓绥能够充分学习智识，特聘班固与班超之妹班昭为皇后之师，教导邓绥。邓绥也博览群书，但法家的申不害和韩非写的书不看。
摄政太后
汉和帝在元兴元年（105年）去世，使邓皇后更得以进入政治权力的中心。她早在为贵人之时，因和帝子多夭死，即常为他选进才人，希望能广其后嗣；和帝之后所生数子则多秘养于民间，这虽是为刘家的继嗣着想，但也为邓氏提供了之后拥立新帝并掌权的机会。和帝去世，邓后虽无子，但迎回了养于民间、甫生百日的和帝幼子殇帝即位，邓后被尊为皇太后，因殇帝年幼，故她临朝听政。 她屡次以皇太后的名义下诏书，并自称为朕； 因此虽然她在诏书中自称「权佐助听政」，但事实上她已成为国家实质上的领袖。
此时宫中丢失了一箧大珠，邓绥本想派人审讯，但担心会有无罪之人被迫认罪，于是亲自对宫人察言观色，盗窃者受不了心理压力坦白认罪。此时汉和帝曾经的宫人吉成的侍从怨恨吉成，便诬陷吉成对太后搞巫蛊，吉成被审讯时被迫认罪。邓绥认为吉成曾经是汉和帝的身边人，自己对吉成很好，吉成平时也没有对自己口出恶言，现在这种情况不合人情，于是亲自召见吉成核实情况，发现果然吉成是被诬陷。
邓绥在此期间还下达了一些诏令，例如邓绥认为鬼神不会应验，祭祀它们不会有效，要求有关部门的人裁撤不符合典法礼仪的祠官；邓绥还下诏赦免马援家族、窦宪家族被禁锢的人士；同时邓绥要求宫内减少制作奢靡之物，伙食从简，节省开支，地方的贡品减少一半规模；又释放数百名宫人回到民间。
不及一年，殇帝亦死，邓太后与兄长车骑将军邓骘以和帝长子平原王刘胜有个疾为由，先以年十三岁的章帝之孙刘祜为和帝之后嗣，再立他为帝，是为安帝。 这种立姪不立子的安排，引起了一些大臣的不满，如司空周章谋立平原王刘胜，但事败自杀。安帝即位后，邓太后继续临朝，一直到她死为止，共摄政达十六年之久。
安帝即位初期，邓太后要求司隶校尉、河南尹、南阳太守，如果邓家亲属或宾客犯罪，不要包庇。此时邓太后还赦免汉和帝废后阴氏的族人，并归还财产。
永初二年（108年），京城洛阳干旱，邓太后来到洛阳寺了解有关冤狱的情况。有一名未犯法但被刑讯逼供被迫自证有罪囚犯被太后审问时，害怕监狱里面的官吏会惩罚，不敢申诉，等到犯人离开时，该犯人擡头似乎想要申诉。邓太后发现不对劲，立刻把该名囚犯喊来询问具体情况，认定为无辜受冤枉之人，并立即下令收押洛阳令。邓太后一行人在回到宫中途中，天降大雨。
永初三年秋（109年），邓太后生病，并决定在年底款待即将解甲归田的皇宫卫兵的宴会时取消演习弹奏音乐这一环节，并减少举办傩时一半的男巫人数。
邓太后白天处理政事，晚上阅读经书，总担心自己所读的经书内容有错漏，于是下令刘珍等五十余人，来到东观校勘书籍。又下令宫中近臣来东观学习儒家经典，学成后教授宫人。
永初四年（110年），邓太后的母亲新野君患上重病，太后亲自服侍，直至十月新野君去世。当时又干旱已久，太后接连三天来到监狱考察冤狱情况。
元初六年（119年），邓太后开设邸第，下诏汉和帝之弟济北、河闲王的年龄虚五岁以上子女共四十余人以及邓氏近亲子孙三十余人来到邸第学习经书，邓太后亲自督促。
安帝年纪渐长，早已成年， 而邓太后却迟迟不肯还政于皇帝，这不仅使得一些大臣之中有人不满，连邓太后的堂兄越骑校尉邓康也畏惧不安，于是以自称生病为由不来上朝。太后派宫内邓康家曾经的奴婢前去慰问，由于当时宫人权力很大，被人称为「中大人」，邓康家先前的奴婢来邓康家时也自称中大人。邓康大怒，说「你是从我家出来的，竟然也敢自称中大人」奴婢也大怒，回到宫中后对邓太后说邓康是谎称有病，还出言不逊。邓太后于是接解除邓康职务。此外当时诸多天灾，不少人以灾异比附人事，认为是太后摄政所致；又如杜根、成翊世均曾上书希望已年长的安帝亲政，太后不仅不听，杜根因此被扑杀于殿中，幸被救未死，而成翊世也因此而获罪。
邓太后在永宁二年（121年）4月5日（或17日）去世，4月30日，合葬汉慎陵。之后安帝终于获得亲政。由于长期安帝无法执政，对邓家非常反感，不久邓家即被诬告，邓骘兄弟等人自杀。而安帝打倒外戚之后，信用宦官及乳母王圣等一干人等，东汉的政治也开始日渐衰败。
胡茵梦（Terry Hu，），台湾模特儿、演员、作家、翻译家、环保运动人士，因为《茵梦湖》一剧，又称胡因梦。或称胡因子、胡茵子，本名是胡因因，生于台湾台中市，籍贯奉天省奉天府（辽宁省沈阳市），满族，瓜尔佳氏，曾有“台湾第一美人”的称号。1970年代至1980年代家喻户晓的电影巨星，与林青霞、林凤娇、胡慧中等齐名；后来成为英语书籍翻译，尤其是心理学、哲学、环保等主题；也曾因和台湾著名作家李敖的爱情故事而名噪一时，胡茵梦与李敖的婚姻关系仅维持了一百多天，被戏称为「百日维姻」。
生平简述
父亲是前中华民国立法委员胡赓年。1953年，璩诗方诞下一女胡因梦。
胡因梦从出生至12岁前都在台中市度过，小学之后搬到附近的存信巷（从育才街到存信巷那一带的村落当时称为新北里，住的是当年的立、监委与国大代表），后居于西区存中街。
初三时，父亲很少回家，因此父母分居，但月薪仍交母亲璩诗方作为家用。母女俩一直生活在一起，胡因梦所有收入，全交母亲管理，由她负责投资。虽衣食无忧，但璩诗方的不安全感让她仍省吃俭用，而且已到无法令人理解的程度，每个月只给胡因梦2万元新台币零用，多年来都未调整过。
中学、高中时，就读台北市卫理女中。1971年考进辅仁大学德语系，1973年辅仁大学德语系肄业。在其离开辅大时，校内曾有言“从此辅大没有春天”。
之后到美国西东大学攻读大众传播系，并且也就读过纽约的模特儿学校。
1975年，回到台湾，踏入影视圈，即获得出演机会，成为电影《梅花》的女主角，也在电视演出、电视主持、文学写作等方面发展，成为1970、80年代家喻户晓的电影巨星，与林青霞、林凤娇、胡慧中等影坛天后齐名。后来成为英语书籍翻译，尤其是心理学、哲学、环保等主题；也曾因和台湾著名作家李敖的闪电结婚与分手而名噪一时。
在多项发展的过程中，她也曾再次前往纽约，至HB工作室（Herbert Berghof Studio）进修演艺课程。此后她停止演艺事业，时年35岁，15年的电影生涯中总共拍摄了42部电影。接着她转往从事心灵探索方面的翻译及写作，同时也推动「新时代」的意识革命以及倡导环境保护。
2003年，曾参加第40届金马奖颁奖典礼。
感情生活
胡茵梦于1980年5月6日与李敖闪电结婚，8月28日即宣布离婚，婚姻关系只维持了一百多天。1994年11月25日，未婚生下女儿胡洁生，成为未婚妈妈，且她至今都拒绝公开女儿父亲的身分。
宇宙1号是一枚航天器，用于太阳帆技术试验。
概述
2001年7月20日，行星学会发射了“宇宙1号”航天器。这是一颗亚轨道航天器，是世界上首次使用太阳帆作为航天飞行动力装置的航天器，也称太阳帆飞船。这颗航天器没有能与第三级火箭分离，最终坠毁了。研制人员准备了3年时间，现在第二个“宇宙1号”太阳帆也已经发射。新的“宇宙1号”比原来设计更大，由8个三角帆组成，可以环绕地球轨道运行，于2005年6月21日从一艘位于巴伦支海的俄罗斯潜艇K-496上发射 。发射后与地球失去联络。之前有报道说发射失败，行星协会在保持一阵沉默后也承认这次实验未能成功将太空船送入轨道。
近地小行星（near-Earth asteroids，NEAs）指的是轨道与地球轨道相交的小行星。这类小行星可能会带来撞击地球的危险。同时，它们也是相对容易使用探测器进行探测的天体。事实上，访问一些近地小行星所需的推进剂比访问月球还少。NASA的会合-舒梅克号已经访问过爱神星，日本的隼鸟号和隼鸟二号分别在2005年和2018成功登陆糸川小行星和龙宫小行星，并带回土壤样本返抵地球。美国的欧西里斯号（OSIRIS-REx）也在2020年登陆班努小行星，现已携带采集的样本，正在返航回来。
目前已知的大小4千米的近地小行星已有数百个。可能还存在成千上万个直径大于1千米的近地小行星，数量估计超过2000个。
天文学家相信它们只能在轨道上存在一千万至一亿年。它们要幺最终与内行星碰撞要幺就是在接近行星时被弹出太阳系。该过程可能会消耗大量小行星，但似乎小行星来源仍然在不断补给。
NEA的分类
一些近地小行星拥有很高的偏心轨道导致它们实质成为"短期"彗星，因为它们会在太阳附近完全消耗挥发，事实上只有很少近地小行星展现出类似彗星那样昏暗的彗尾。这些近地小行星可能来自柯伊伯带，柯伊伯带在海王星外围，那里是彗星的来源地之一。其余的近地小行星可能是在木星的引力交互作用下衍生自小行星带。
三类近地小行星：
值得一提的是阿波罗小行星与阿登型小行星的轨道会穿越地球轨道，这使得它们可能产生撞击的威胁，而阿莫尔型小行星不穿越地球轨道但却可能距离地球非常近。
有时也使用阿周那型小行星指称那些几乎完全与地球轨道重合的小行星。"近地小行星"的称谓要比"近地天体"来的严格
NEA的威胁
阿尔瓦雷茨假设通常为人们所接受，该假设认为白垩纪-第三纪灭绝事件是一个大型小行星或彗星撞击的后果，该假设也认为在将来地球也存在与穿越地球轨道的小行星相撞的可能性。
与地球撞击的危险在1994年7月16日发生舒梅克利维9号彗星撞击木星后为人们所重视，人们开始担心相比很小的撞击也会给地球造成灾难性的后果。虽然可以肯定的是，木星要比地球大得多而且质量也大得多，所以木星遭受的撞击也更多，但是这次撞击事件也向人类提供了一个了解这类事件如何发生以及所造成的灾难性后果的很好窗口。
事实上在1989年3月23日，直径1000呎的阿波罗小行星4581（阿斯克勒庇奥斯）(1989 FC) 在地球表面650000公里穿过-与地球撞击只相差6小时。如果撞击发生，其将会产生有记录以来最大的灾难。
大约不到100万年地球就会与直径1千米的小行星撞击，5千米天体的大撞击大约1000万年发生一次。小型的撞击(仍然潜在着危险)每月会发生好几次。
虽然过去有错误的警报发布，但事实上确实有一些小行星对地球上的生命明确造成威胁。
历史上的撞击事件
普遍被接受的阿尔瓦雷斯假说，解释白垩纪-第三纪灭绝事件是大型天体的撞击事件的结果，提高了穿越地球轨道的其它天体和与地球碰撞可能性的认知。
1908年通古斯事件
现在普遍认为1908年6月30日一颗石质小行星在通古斯的上空爆炸，释放出超过1,000万吨TNT的能量。据估计爆炸发生在85公里的高度上，物体的直径在45-70米之间。
1979年船帆座事件
在1979年9月22日，在南大西洋和印度洋交界处附近记录道一次可能是低能量的核子试爆事件，但起初被认为可能是外星物体撞击的事件。此一事件称为船帆座事件，是由美国为监测核子试爆而设置在轨道上的船帆座卫星记录到的。这次警报导致多个组织进行了数年的调查，但最终还是不能确定爆炸是来自核武，或是非核武。
2002年东地中海事件
在2002年6月6日，一颗估计直径10米的小天体撞击了地球。这次撞击发生在地中海，介于希腊和利比亚之间，大约在东经21°北纬34°处的半空中发生爆炸。释放的能量估计在26,000吨黄色炸药，相当于一个小型的核子武器。
"主条目：
2008年苏丹事件
在2008年10月6日，科学家计算出一颗近地小行星，，将在10月7日0246UTC (当地时间5:46) ，发现后不久撞击地球。这颗小行星如预期的抵达，这是第一次准确的预测到小行星撞击地球。之后，在地球表面上寻获数百颗的陨石碎片。因为此一事件发生在人烟稀少的地区，到目前为止，没有任何已经提出的报告提到它所造成的影响。
2009年印度尼西亚事件
在2009年10月8日，一颗巨大的火球出现在印尼的波尼附近天空中。这个天体被认为是一颗直径10米的小行星，估计这颗火球释放出的能量为5万吨黄色炸药，或是2倍于广岛原子弹。没有人员伤亡的报导。
2013年俄罗斯流星事件
在2013年2月15日，当流星体在俄罗斯的车里雅宾斯克上空爆炸后，造成了当地1,200人的受伤。当地日出之际有着挥之不去越来越明亮的条纹出现，几秒钟之后，该物体达到最大强度，巨大的震波爆炸使得玻璃碎裂，并遮盖过汽车防盗器的警报声，随后还伴随着几次小爆炸。
外部链接
There is an excellent article in the October 2003 issue of Scientific American regarding NEA's and long term strategies for protecting Earth from them
万州区位于重庆市东北部、长江上游地区、三峡库区腹地，属于渝东北生态涵养发展区，是重庆市最大的市辖区，也是重庆万开云板块的内核城市。万州面积3,457平方公里，总人口175万，距离重庆主城区约374公里，是重庆市仅次于涪陵区的第三大城市，是重庆规划的位于三峡库区的区域性中心城市。
万州区东临湖北利川市，南和西南接石柱土家族自治县、忠县和梁平区，西临四川开江县，西北和东北接开州区、云阳县。至2017年，总人口17414万人，城区面积100平方公里，城市建成区面积6962平方公里，城区人口85万人，城镇化率6545%。
万州区位独特，历为渝东北、川东、鄂西、陕南、黔东、湘西的重要物资集散地，距重庆主城区和湖北宜昌200公里以上，是200公里半径范围内城市人口唯一超过80万的中心城市。交通便利，长江黄金水道穿境而过，是拥有机场、铁路、高速公路、深水港码头、高速铁路和海关口岸，国际保税物流的一个交通枢纽的城市。
自然地理
万州区位于四川盆地东缘，区内山丘起伏，最高点普子乡沙坪峰（海拔1762米），最低点黄柏乡处长江边（海拔106米），低山、丘陵面积约占四分之一，低中山和山间平地面积约占四分之一，极少平坝和台地，且零星散布。境内河流纵横，河流、溪涧切割深，落差大，高低悬殊，呈枝状分布，均属长江水系。
气候
万州区属中亚热带季风气候，最冷月均温73℃，四季分明，冬暖、多雾;夏热，多伏旱;春早，气温回升快而不稳定，秋长，阴雨绵绵，以及日照充足，雨量充沛，天气温和，无霜期长，霜雪稀少。境内多年平均气温 177 ° C ，最高年平均气温 190 ℃ ( 1982 年)，最低年平均气温 176 ℃( 1974 年)。
行政区划
万州区现辖11个街道、29个镇、10个乡、2个民族乡：。
现任领导
中共重庆市委常委、中共重庆市万州区委书记
中共重庆市万州区委副书记，重庆市万州区政府区长、党组书记
中共重庆市万州区委副书记
中共重庆市万州区委常委、区纪委书记，重庆市万州区监察委员会主任
中共重庆市万州区委常委、办公室主任
中共重庆市万州区委常委
中共重庆市万州区委常委，重庆市万州区政府副区长、党组副书记
重庆市万州区政府副区长、党组成员
重庆市万州区政府副区长、九三学社重庆市万州区委会主委
重庆市万州区政府副区长、党组成员
重庆市万州区政府区长助理
人口
万州区累计搬迁安置三峡移民263万人，是全市人口最多、移民任务最重的区县。
2018年，万州区常住人口16475万人，比上年增加117万人，其中城镇常住人口11018万人，占全区常住人口比重（常住人口城镇化率）为6688%，比上年末提高165个百分点。年末全区户籍总人口1739万人，其中，城镇人口7301万人；乡村人口10089万人。全年出生人口15414人，人口出生率886‰；死亡人口12358人，人口死亡率711‰；人口自然增长率176‰。
经济
2018年，万州区地区生产总值98258亿元，比上年增长01%。按产业分，第一产业增加值7442亿元，增长49%；第二产业增加值34496亿元，下降125%；第三产业增加值56319亿元，增长53%。三次产业结构比为76∶35.1∶573。按常住人口计算，全区人均地区生产总值为59853元（9045美元）。
全年添加就业人员431万人，城镇登记失业率293%。
教育
2018年，万州区有学校467所，其中，普通高等学校7所，普通中等职业学校7所，普通中学57所，小学106所，幼儿园289所，特殊学校1所。普通高等学校在校生663万人，中等职业学校在校生129万人，普通高中在校生374万人，普通初中在校生435万人，普通小学在校生902万人，学前教育在校生393万人，特殊教育在校生397人。高中阶段毛入学率91%，初中阶段毛入学率11121%，小学学龄儿童入学率100%。
高等学校
重庆三峡学院（本科）、重庆三峡医药高等专科学校（专科）、重庆安全技术职业学院（专科）、重庆三峡职业学院（专科）、重庆幼儿师范高等专科学校（专科）、重庆信息技术职业学院（专科）、重庆科技职业学院（专科）
交通
万州区位于长江上游，交通发达。
高速公路
目前万州区内已建成高速公路有有（已于2003年全线建成通车）、万云高速、万开高速、、宜万高速、万达高速、万忠（南线）高速，汉渝高速已列入国家规划，还有万忠（北线）高速正在规划中。
铁路
万州城区内有高铁站万州北站、火车站万州站等。
目前区内有达万铁路（已于2002年建成通车，2010年完成电气化改造）、宜万铁路（已于2010年12月正式通车）、郑渝高速铁路，已于2016年11月28日建成通车），在建、待建的还有渝西客运专线、万黔铁路，规划中的有西安到万州的高铁、万州到达州的高铁。
万奉铁路：正启动前期工作，已提交规划报告，准备开工。
航空
万州五桥机场于2003年竣工，为4C级民用机场，设计年吞吐量50万人次，可全天起降波音737、757客机。已于2016年进行扩能改造，预计2019年完成。
港口
万州区拥有“黄金水道”长江83公里航道，正在建设的长江上游最大深水港——万州港，并安装了集装箱码头设备，年吞吐量1500万吨以上，可常年通行万吨级船队。万州港货物年吞吐量在重庆市内居第2位，仅次于重庆港。
万州正在打造新田港、完善桐子园作业区等完善港口设施。同时，将与港口连接的疏港公路，疏港铁路支线创建起来，构成对外信道。有关部门加大万州的港口建设力度，进行集疏运体系的改造和新田港港口的大力建设。对新田港将投资3个亿，建成后形成4千万吨的吞吐量，100万标箱。这使万州的港口的能力可以成倍的增加。
桥梁
城区有万州长江大桥、万州长江二桥、万安大桥、万州大桥、石宝嘴大桥、驸马长江大桥。目前万州长江三桥已修建完成。
轨道交通
万州城市轨道交通正在有序推进线网规划编制工作。
名胜旅游
万州旅游资源丰富。著名的有：鬼斧神工的绮丽景观“亚洲第一瀑”——万州大瀑布、潭獐峡峡谷型风景区；清新自然的铁峰山国家森林公园、王二包森林公园；蕴藏深厚人文积淀的太白岩、天生城、都枥山；幽深莫测的盐井龙洞；具有独特民俗民风的罗田古镇、悦君山寨堡文化、恒合高山土家族乡村旅游区等等。
2016年，接待国内游客100841万人次，增长31%；国内旅游总收入5159亿元，增长32%。接待入境游客377万人次，国际旅游外汇收入62404万美元，下降92%。国家AAA级及以上景区达7处。成功承办第七届中国长江三峡国际旅游节，“平湖万州”被央视评为“新三峡”十大旅游新景观。
特产
万州红桔：中国地理标志产品。
布袋戏（又称布袋木偶戏、笼底戏、掌中戏），是一种起源于闽南泉州，后在台湾发扬光大的一种用布偶戏剧，布偶的头是用木头或是木材雕刻成中空的人头，除了偶头、戏偶手掌与人偶足部这些之外，布袋戏偶身之躯干与四肢都是用布料做出的服装；演出时，将手套入戏偶的服装中进行操偶表演。中华民国布袋戏学者吴明德先生在《布袋戏名称考述》中以布袋戏之“布袋”是因早期单人式布袋戏--肩担戏演出时收放道具的大型布袋，此布袋在演出时可当作屏蔽主演身躯的布帷，可以让观众留下鲜明的视觉印象，遂称之为“布袋戏”，而掌中戏此称呼是业界布袋戏团普遍的共识，只有“掌中戏”这个称呼才能强调主演的双掌能够“十指弄成百万兵”“十指摇动古今事由”，将布袋戏的表演提升到艺术的层次，所以“掌中戏”这称呼就成为布袋戏从业人员的最爱，而民间俗称“布袋戏”。
布袋戏在台湾的发展最为蓬勃。布袋戏在台湾所扮演的角色，与宗教信仰、天地神人关系、经济时令与文化价值的传承相当紧密的关联性。
戏偶
布袋戏偶基本结构包括身架、服饰、盔帽(头饰)，身架包括了头、布身、手（文手或武手，木制），实心的布腿、鞋（靴子，木制），传统戏偶高度约30cm，而戏偶头尺寸以头盔偶的头围(头部圆周)尺寸来分类，传统尺寸戏偶头分为头号、二号、三号等3种，头围分别大约台寸5寸半、4寸半、3寸半，金光戏偶尺寸则有台寸6寸半、7寸、7寸半及大于7寸半等尺寸。
若就角色分类来看，布袋戏戏偶种类有「生」、「旦」、「花脸」、「丑」、「杂」、「北」等分类。就传统布袋戏而言，一个布袋戏班形成，起码要准备包含此六大类戏偶约60个布袋戏偶；而这就是一般布袋戏艺人所俗称的一担笼。
传统偶头为木制，其制作流程为砍劈大样、雕刻粗胚、磨平、糊棉纸、打土底、粉底、上面漆、绘五官及花脸、上亮光漆、装发须、头上装饰等。不管哪角色，早期传统布袋戏的偶头木材多产自中国泉州。1920年代以后，台湾台中万里公司研发以花园头模型灌制的的赛璐珞偶头，虽广为流传，但正式演出仍以泉州进口的木制唐山头为主。战后台湾布袋戏蓬勃发展，在地戏偶雕刻师应运而生，随着两岸交通中断，以及剑侠戏、金光戏的兴盛，内台戏的应运而生，都导致戏偶尺寸变大，传统尺寸戏偶因此没落。唐山头中以涂门头和花园头最有名。涂门头的主要制造商是泉州涂门街的周冕号，台湾俗称涂头，以花脸见长。花园头主要来自泉州环山花园头村（现泉州市丰泽区清源街道环山社区），该偶头主要的师傅为泉州江加走，以生、旦等素面粉彩见长。泉州产的偶头木材多采用银杏木和香梓木，台湾制的偶头以梧桐木为主，少数使用公母柴；现代戏团使用的偶头材质以进口的印尼白木为主，收藏用偶及传统尺寸戏偶则普遍采用樟木。
现代电视布袋戏木偶大小约高74～80公分左右，身架及服饰比例拉长，身材比例相当接近真人，约七～九头身，头围则多为台寸7寸。偶头（如图1）用樟木或白木雕刻而成，偶头内有特殊设备，可以用绳子拉动控制眼睛睁眨，还有用中指扳动嘴唇的控制，也有特殊的偶眼可以左右动。
在戏偶的本体方面，手可分为木制手以及现在广为使用的橡胶手，该手内置铁丝，并可轻易扭成特定的手型（如图2） 另外，现代戏偶的脚（如图3）已普遍为塑料制，装设有膝、踝二关节，鞋子还可以穿脱交换。至於戏偶本身穿的服装，近代偶身所着衣服变得更华丽且多层次，甚至已与真人的穿着相仿。除此之外，操控布袋戏，让其自由表演的设备为天地同；其设备设于左右手：右手控制偶头和戏偶右手，左手持一棒控制戏偶左手（如图4）
剧本
布袋戏的传承着重在布偶操纵技巧与后场音乐搭配，对于演出的剧本大多是以口耳相传，或是剧团请说戏先生来跟团员们讲述故事内容，再由团员上场发挥，因此布袋戏的传统剧本保存极少。而这些口述剧本，则以三国演义、西游记、封神榜、水浒传等小说为主体。
1945年之后，漳泉两地布袋戏渐趋式微，惟台湾布袋戏仍继续发展。台湾布袋戏师傅黄海岱广招徒弟，为了方便传艺，便将写下了剧本纲要传授给徒弟，黄海岱著名的布袋戏剧本有《五虎战青龙》、《大唐五虎将》、《三门街》、《昆岛逸史》、《秘道遗书》。1960年代黄俊雄著名布袋戏作品《云州大儒侠》剧本，就来自黄海岱的《忠孝义勇传》。
在台湾之外，位于漳州的漳州木偶剧团1980年代经济开放后，也出现专替儿童所写的儿童剧剧本。这种创新的剧本，台湾的新生代及业余布袋戏剧团亦尝试过。不过，大纲式剧本的纲要写法与即兴式的闽南语典雅口白仍为布袋戏剧本的主要特色。
历史
布袋戏的历史相当久远，中国古籍《武林旧事》、《东京梦华录》中记载，宋朝宫廷宴会的礼单上就有掌中木偶戏。不过，就现今被人熟悉的传统布袋戏而言，该戏种起源，一般来说都是来自泉州的一则传奇故事。
依据中华民国戏剧学家吕诉上在其著作《台湾电影戏剧史》的记载：传说中国明朝期间，有位屡试不中的秀才梁炳麟，在福建仙游县九鲤湖一座仙公庙 祈求高中后，做了一个梦，梦中有一位老人在他手上写下「功名归掌上」后离去。梁秀才醒后非常高兴，认为是及第的吉兆，不料当次应试又名落孙山。失落之余，他开始向邻居学习悬丝傀儡戏，并发展出直接套在手中的人偶，凭着他的文学修养，出口成章，又能引用各种稗官野史，马上吸引了许多人来看他的表演，布袋戏从当地开始风行起来，梁秀才的名声也跟着水涨船高，此时他才领悟了「功名归掌上」这句话的意涵。除此之外，尚有秀才名为孙巧仁，或梁炳麟、孙巧仁二人同行等不同说法。
17世纪中期，布袋戏已于闽南相当受欢迎，亦出现了类似肩担戏的模式。所谓肩担戏乃是由演出者肩担戏箱与舞台。至一定地点后，搭好简易舞台后，操纵者躲在布帏下，肩上简易木架舞台即成为人偶演出戏台。至18世纪，根据演出场所的不同，布袋戏不但出现职业团体，还视演出地点，分为室外演出的野台戏与在剧院演出的内台戏。其中，野台戏是戏班应邀请到各地演出，后来渐渐变成以迎神谢神祝寿为目的的表演，布景较为简单，演出人员也较少。
内台戏表演方式虽出现于18世纪，但是真正盛行是在20世纪后。内台戏布景较为华丽，操控木偶与音乐演出者也较野台戏多。然而受到电影、电视等新式娱乐型态的冲击之下，1980年代之后，除了少许特例之外，内台戏于闽南及台湾各地已相当少见。
另外一方面，内台戏亦于1960年代之后，发展出了电视布袋戏的此一支流。此种将戏台架设于电视摄影棚，于棚内演出的布袋戏戏种，不但于战后盛行于台湾，造成轰动。演出媒介及场所甚至还延展到1990年代之后的有线电视、录像带、电影、光盘及多媒体。
流派
一般来说，学者都大约认定布袋戏起源为福建泉州，时间则约为17世纪左右。而之后发展上，布袋戏流派因发展区域横跨闽南与台湾，加上不同时期的各自发展与交流融合，实在很难区分。若最粗概的以表演型态来论，约可分传统布袋戏与现代布袋戏。在此分类下，即使在布袋戏仍十分流行的台湾，传统布袋戏的观众远低于现代布袋戏。而普遍所称的布袋戏流派，实际往往就是指传统布袋戏的流派。
在传统布袋戏流派而言，若简单的以音乐型态区分，使用南管者称南管布袋戏或南派布袋戏，使用广东潮调者称作潮调布袋戏，使用北管的称北管布袋戏或北派布袋戏，沈明正好彩头古书布袋戏剧中，都会加入北管音乐曲牌【风入松】，是著名代表作。20世纪后，台湾又有改良自漳州，并于北管文武场加入京剧锣鼓点节奏所形成的外江布袋戏和采用歌仔戏唱法的歌仔调布袋戏；后者并从台湾传至漳泉两地。
另一方面，若使用活跃地区再加以细分，在福建地区，擅长文戏的南派布袋戏盛行泉州，音乐采南管乐，唱腔采用南调，表演上则用唐代流传下来的梨园戏做派。漳州布袋戏除了邻近广东潮州的少数县属于潮调布袋戏外，其余各地均归属于音乐使用北管乐的北派布袋戏。 不过，于1980年代之后，随着布袋戏不断发展和文化不断融合，该地区的南北两派的界限已较不明显。
在台湾，传统布袋戏相同地盛行南管布袋戏及北管布袋戏。其中，南管布袋戏与泉州流行之剧种同名且型态类似，只不过传来台湾之后，有其别名为白字。该戏种相同擅长文戏；戏团发源地盛行于台湾中部之台西、麦寮、褒忠、东势、四湖等地，而盛行地点偏重台湾北部。另一者为北管派，于台湾又称乱弹，来自漳州，擅长武戏，戏团成员多来自云林县的斗六、西螺、斗南、虎尾、古坑与二仑，而戏种盛行地点为台湾中南部。
各地发展
虽说文献对于梁炳麟是否发明布袋戏，并无肯定记载。但可以确定：布袋戏的发明与泉州傀儡戏文化息息相关。泉州傀儡戏影响了泉州布袋戏的发明，泉州布袋戏又传到漳州跟台湾，中国大陆、台湾都公认泉州是布袋戏的发源地。然而在发源地的泉州，布袋戏只是木偶艺术的支流之一，较晚发展的漳州与台湾，布袋戏已凌驾发源地，1950年后的台湾，在传承与改良及观众接受度上，已经明显领先漳、泉两地。
中国大陆
福建泉州
早在清朝乾隆年间（1730年代至1760年代），相关文献就显示布袋戏于泉州地区相当流行。此阶段泉州布袋戏的戏偶、戏箱、身段、唱做仍不脱傀儡戏戏曲模式。演出也多是谢神、迎送神、祝寿喜庆宴会、动土。直至过不久的1798年，泉州产生了全中国首个职业布袋戏团－金永成偶戏团，布袋戏就此分离于提线木偶自行发展，这种源自傀儡调、使用泉州南音音乐作为唱腔、擅长文戏、讲究细致偶头与身段，并以泉州闽南语口白演出的布袋戏，也就是南派布袋戏。
19世纪是泉州南派布袋戏全盛时期，也同时是将其文化输出至中国其余各地的黄金年代。除从傀儡戏沿习更多戏种之外，泉州重视操偶技巧的梨园布袋戏也是让19世纪泉州布袋戏深受欢迎的原因之一。 20世纪初，比梨园布袋戏更重视唱工的笼底戏流行于泉州。 虽然此戏码身段细致，唱工讲究，但因剧本完全照本宣科毫无创意，加上诗韵饶舌，终究无法深入民间。经过一段时间沉淀改善后，这些戏码转成较活泼的章回小说。更在大幅度改善南管音乐后，泉州布袋戏配合其音乐的打点，过门做出极适合布袋戏文戏的戏码。不过，因为其进步幅度无法与台湾布袋戏与漳州布袋戏相比，至此已失去龙头的地位。
20世纪中，泉州布袋戏获得过支持和扶植，后在文化大革命中深受伤害。1980年代后，情况得以改善。这期间，最出名的泉州布袋戏职业剧团当属成立于1953年的晋江掌中木偶剧团。该团结合南管与梨园改革了本有南派布袋戏的风格，除在闽南颇有盛名外，也多次以《白龙公主》、《五里长虹》等戏剧于中国大陆其他省份及世界各地参与相关文艺活动。另外，泉州地区的布袋戏木偶头制作也相当具有代表性。从清代「西来意」等六家佛妆店的兼营，到后来涂门街「周冕号」黄良司、黄才司两兄弟的专业作坊，与花园头村江加走一门的崛起，其作品迄今仍颇受好评。2006年5月20日，泉州布袋戏被列进中华人民共和国国务院公布的第1批国家级非物质文化遗产名录。
福建漳州
漳州布袋戏的戏箱总数、偶头、戏码与泉州相似，应属同源。不过后场音乐却与泉州不同的，使用锣鼓、唢呐等北管音乐。因为唢呐音调高亢，锣鼓喧天，不适合与文戏剧情配合，因此，该地布袋戏戏码通常为加入武打场面的的武戏。此种用北管音乐且擅长武戏的漳州布袋戏称为北管布袋戏或北派布袋戏。除此，漳州的云霄、诏安、东山、平和等县，因与邻近广东潮州，不同于漳州其他地方的流行用潮州曲乐作为后场配乐，特别称为潮调布袋戏，不过该布袋戏仅于漳州小区域流行。
20世纪初，漳州地方士绅蓝汝汉自上海引进京剧而大力推广，同时间，漳州艺人杨胜将京剧唱腔与演出身段引进布袋戏演出中。就音乐方面，此阶段的漳州布袋戏则大量加入京剧腔调，芗剧（歌仔戏）回传入闽南后，用芗剧音乐演唱，或京、芗杂用。身段方面，则重视偶戏肢体细节。1900年至1930年，漳州北派布袋戏与流行于台湾和泉州南管（派）布袋戏或者泉州梨园布袋戏分庭抗礼，并在影响力与风格上，略胜一筹。不只此，该类型布袋戏还于日后传入台湾，即李天禄之外江布袋戏。
1950年代，中国共产党取得中国大陆政权后，以国家力量大量培养布袋戏传承人员，漳州布袋戏也进入了一个新的发展时期。1959年3月，漳州龙溪成立「龙溪专区木偶剧团」， 该剧团集中多位布袋木偶戏艺术家，形成了民间艺术向专业艺术迈进的雏形。1960年代开始，漳州布袋戏向全国范围传播，并走向世界。1960年，于罗马尼亚举行的第2届国际木偶傀儡戏节，漳州布袋戏演出的《大名府》、《雷万春打虎》两戏码，凭借精彩表演获得表演一等奖，并荣获两枚金质奖章。惟因布袋戏发展与这时期中国无法允许的宗教信仰（例如谢神）息息相关，加上文化大革命十多年内对该艺术的禁止，让漳州布袋戏发展停滞不前。
1980年代文革结束与中国大陆实行经济开放政策后，在政府扶持下，成立了漳州木偶艺术学校，漳州最大的职业布袋戏团漳州木偶剧团重新活跃起来。同时，在国家一级演员庄陈华与新生代接班人洪惠君、吴光亮等人努力下，漳州木偶剧团借由儿童木偶剧《森林的故事》等新戏码重新活跃起来。
现今包含漳州布偶戏团的多家职业漳州布袋戏剧团也如同台湾的传统布袋戏一样，面临着从未有过的挑战。没有台湾金光戏演出舞台的漳州布袋戏面对着市场萎缩，观众层面越来越窄，创新剧目资金缺乏等问题。为突破困境，漳州木偶剧团积极寻找新的发展途径，通过送戏下乡，组织演员到外地演出，开拓演出市场，同时通过各种形式，以台湾金光戏行销模式为效法对象，来提高漳州布袋戏的知名度。
2006年5月20日，漳州布袋戏与泉州布袋戏同时被列进中华人民共和国国务院公布的第1批国家级非物质文化遗产名录。
浙南温州
单档布袋戏这个剧种通行在以中国浙江南部温州的平阳县和苍南县（民国时的浙江平阳县，清帝国温州府平阳县）为内核的浙南闽语区，2008年6月7日被列进中华人民共和国国务院公布的第1批国家级非物质文化遗产扩展项目名录。
台湾
1750年代，布袋戏随着闽南移民进入台湾，其剧本以古书和演义小说为主。至1920年代，多采以清末民初新着的武侠小说为题，表现手法重视各种奇特剑招与武功的展现为主。在日治时期，因台湾总督府推行皇民化运动，布袋戏也因为皇民化运动而有了改变。到了战后的1950年代，「金光戏」开始在中南部各地的野台戏中开始发展。除了在剧情上仍延续剑侠戏时期的武侠内容外，也开始新创剧情及主角。另外，在戏剧之外的表现手法上，金光戏采用华丽的布景、金光闪闪的戏服，并以灯光或其他特效来增加武打的效果。1960年代初期，电影戏院演出布袋戏相当普遍，野台金光布袋戏也仍是乡村地区的重要娱乐之一，并且已经与中国大陆之布袋戏有很大的不同。然而，在2022年台北电影节将最佳动画视频颁给「素还真」布袋戏电影，以此可以将布袋戏划分为动画形式的一种。
印尼
在印尼，布袋戏一度流行于苏门答腊与爪哇的华人社群。特别是在爪哇，华人原乡文化与当地文化融合产生了「土生布袋戏」（Peranakan Potehi），现今则称「瓦样布袋戏」（Wayang Potehi，Wayang在爪哇语意指「戏偶」）。瓦样布袋戏主要被保存于中爪哇（三宝垄）与东爪哇的华人社群中；其对白语言大多为印尼语和爪哇语。
著名剧团
长子黄俊卿剧团在台中国立美术馆展示的布袋戏舞台
台湾
黄海岱出生于日治时期台中县斗六辨务署西螺堡（今云林县西螺镇），父黄马，母程扁。11岁起，由父亲送至学堂读汉学，创建起文学、音乐的基础。1915年进入父亲黄马的「锦春园」布袋戏班，与其弟程晟（随母姓）跟随父亲学习布袋戏，18岁时黄海岱已可独当一面。1926年娶妻钟罔市，翌年得子黄俊卿。
1998年黄海岱荣获成为教育部民族艺术艺师，成为全国最高龄之布袋戏宗师。1999年更获得全球中华文艺艺术薪传奖、终身奉献奖。2000年获颁中华民国行政院文化奖，2002年荣获国家文艺奖，同年国立台北艺术大学颁发名誉博士学位给黄海岱。2006年获选为探索频道《台湾人物志2》的六名主角之一。2007年云林县议会决议将黄海岱的生日1月2日定为「云林布袋戏日」。[19]
黄海岱于2007年1月16日因感冒住院，其后并发肺积水造成呼吸衰竭在2月1日进入加护病房。但因药石罔效，遂遵照民间习俗于2月10日11时30分送返家中，弥留至翌日凌晨0时15分过世，享嵩寿106岁。其告别式于3月10日云林县国立虎尾科技大学体育馆举行，公祭当天同时举办追悼会，总统陈水扁特别颁发「褒扬令」，各界代表亦以覆盖国旗仪式，表扬黄海岱对台湾布袋戏的贡献。[20]
而现今台湾布袋戏团经由国立传统艺术中心的推动，在数字化的时代中，亦有关于布袋戏系统性的数字网络建置网站－布袋戏主题知识网。台湾的布袋戏发展已经走向文化产业期，其中不乏中央补助团体和地方杰出团体，诸如：「五洲小桃源」、「桃兴阁」、「磐宇木偶剧团」、「兴洲园」等等戏班，让台湾布袋戏的展现更加有力。
注释
[19] 李静芳，《黄海岱１０７岁 生日订为布袋戏节》，自田时报，2006-12-29
[20]于志旭，《布袋戏大师黄海岱公祭 惊动万教》，2007/3/11 互联网档案馆的存盘，存档日期2016-03-03
外部链接
[国立传统艺术中心:布袋戏主题知识网| ]"""


def keep_chinese_english_punctuations(text):
    # Regular expression pattern
    pattern = re.compile(
        r'[^'  # Start of the negated character class
        '\u2000-\u206F'  # General Punctuation
        '\u3000-\u30ff'  # CJK Symbols and Punctuation, Hiragana, Katakana, Bopomofo, Hangul Compatibility Jamo, Kanbun, Halfwidth Katakana
        '\u3400-\u4dbf'  # CJK Unified Ideographs Extension A
        '\u4e00-\u9fff'  # CJK Unified Ideographs
        '\uFF00-\uFFEF'  # Full-width punctuation, Full-width letters and numbers
        '\uf900-\ufaff'  # CJK Compatibility Ideographs
        '\uff01-\uff9f'  # Halfwidth Katakana and Hangul, Full-width punctuation, Full-width letters and numbers
        '\U00020000-\U0002A6DF'  # CJK Unified Ideographs Extension B
        'a-zA-Z'  # English characters
        '\u0020-\u007F'  # Basic Latin characters and common English punctuation
        '\u0080-\u00ff'  # Latin-1 supplements, including signs like ¥
        '\n\r'  # New lines and carriage returns
        ']'  # End of the negated character class
    )
    # Substituting characters not matching the pattern with empty string
    new_text = re.sub(pattern, '', text)
    return new_text


# text = "你好，world! 这是一个测试文本.\nThis is a test text, 包含中文，English, 和各种符号！？“”……——（）【】[]@#*&$^\nNew Line Here！@#¥%……&*（）——+·～"
text = "Unable to generate font metrics for 城县的节肢动物，是双尾目铗𧈢科伟铗𧈢属唯一的一种。发现时是当时世Unable to generate font metrics for  皇太子朱翊𫟸，幼殇，追赠（世宗赠裕世子"
cleaned_text = keep_chinese_english_punctuations(text)
cleaned_text

'Unable to generate font metrics for 城县的节肢动物，是双尾目铗𧈢科伟铗𧈢属唯一的一种。发现时是当时世Unable to generate font metrics for  皇太子朱翊，幼殇，追赠（世宗赠裕世子'

In [16]:
import sys

# build a table mapping all non-printable characters to None
NOPRINT_TRANS_TABLE = {
    i: None for i in range(0, sys.maxunicode + 1) if not chr(i).isprintable()
}

def make_printable(s):
    """Replace non-printable characters in a string."""

    # the translate method on str removes characters
    # that map to None from the string
    return s.translate(NOPRINT_TRANS_TABLE)

text = """Unable to generate font metrics for 城县的节肢动物，是双尾目铗𧈢科伟铗𧈢属唯一的一种。发现时是当时世
Unable to generate font metrics for  皇太子朱翊𫟸，幼殇，追赠（世宗赠裕世子"""


make_printable(text)

'Unable to generate font metrics for 城县的节肢动物，是双尾目铗𧈢科伟铗𧈢属唯一的一种。发现时是当时世Unable to generate font metrics for  皇太子朱翊𫟸，幼殇，追赠（世宗赠裕世子'